In [0]:
# Define the endpoint names
endpoints_to_keep_alive = [
    # # "gwb_boltz_endpoint",    
    # # "gwb_rfdiffusion_inpainting_endpoint",
    # # "gwb_proteinmpnn_endpoint",
    # # "gwb_esmfold_endpoint", 
    # "boltz",    
    # "rfdiffusion_inpainting",
    # "proteinmpnn",
    # "esmfold", 
    "scimilarity_search_nearest",
    "scimilarity_get_embedding",
    "scimilarity_gene_order"

]

prefix = "gwb_scn_"
postfix = "_endpoint"


In [0]:
import requests
import time
import requests
import json
from concurrent.futures import ThreadPoolExecutor

# Define the Databricks instance and token
db_host = spark.conf.get("spark.databricks.workspaceUrl")
print(f"Databricks Host: {db_host}")
db_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

In [0]:
# Function to start an endpoint
def start_endpoint(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}/config:start"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        print(f"Successfully started endpoint: {endpoint_name}")
    else:
        print(f"Failed to start endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")


# Function to stop an endpoint
def stop_endpoint(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}/config:stop"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, headers=headers)
    if response.status_code == 200:
        print(f"Successfully stopped endpoint: {endpoint_name}")
    else:
        print(f"Failed to stop endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")


# Function to check the status of an endpoint
def check_endpoint_status(databricks_instance, endpoint_name, token):
    url = f"https://{databricks_instance}/api/2.0/serving-endpoints/{endpoint_name}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        status = response.json().get("state", {}).get("ready", "Unknown")
        print(f"Endpoint: {endpoint_name}, Status: {status}")
        return status
    else:
        print(f"Failed to check status of endpoint: {endpoint_name}, Status Code: {response.status_code}, Response: {response.text}")
        return "Unknown"


# Function to wait until all endpoints are ready
def wait_until_endpoints_ready(databricks_instance, endpoints, token, check_interval=60):
    all_ready = False
    while not all_ready:
        all_ready = True
        for endpoint in endpoints:
            status = check_endpoint_status(databricks_instance, endpoint, token)
            if status != "READY":
                all_ready = False
                print(f"Endpoint {endpoint} is not ready yet. Checking again in {check_interval} seconds...")
                break
        if not all_ready:
            time.sleep(check_interval)
    print("All endpoints are ready!")


# Function to send a single request
def send_request(databricks_instance, endpoint, payload, token):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    url = f"https://{databricks_instance}/serving-endpoints/{endpoint}/invocations"
    try:
        # Ensure the payload data is JSON serializable
        json_payload = json.dumps(payload["data"])
        response = requests.post(
            url,
            headers=headers,
            data=json_payload
        )
        if response.status_code == 200:
            print(f"Endpoint {endpoint} is awake and responded successfully.")
        else:
            print(f"Failed to wake up the endpoint {endpoint}. Probably timed out")
    except (TypeError, ValueError) as e:
        print(f"Error serializing JSON for endpoint {endpoint}: {e}")
    except Exception as e:
        print(f"Error sending request to endpoint {endpoint}: {e}. Probably timed out")

# Function to send inference inputs to the endpoints in parallel
def send_inference_inputs_parallel(databricks_instance, request_dict, token):
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(send_request, databricks_instance, endpoint, payload, token)
            for endpoint, payload in request_dict.items()
        ]
        # Optionally, you can wait for all futures to complete if needed
        for future in futures:
            future.result()

In [0]:
request_dict= {
               "esmfold": {"data":{
                                    "inputs": [
                                        "MADVQLQESGGGSVQAGGSLRLSCVASGVTSTRPCIGWFRQAPGKEREGVAVVNFRGDSTYITDSVKGRFTISRDEDSDTVYLQMNSLKPEDTATYYCAADVNRGGFCYIEDWYFSYWGQGTQVTVSSAAAHHHHHH"
                                    ]
                                    }
                           },
               "boltz": {"data":{
                                "dataframe_split": {
                                    "columns": [
                                    "input",
                                    "msa",
                                    "use_msa_server"
                                    ],
                                    "data": [
                                    [
                                        "protein_A:GTGAMWLTKLVLNPASRAARRDLANPYEMHRTLSKAVSRALEEGRERLLWRLEPARGLEPPVVLVQTLTEPDWSVLDEGYAQVFPPKPFHPALKPGQRLRFRLRANPAKRLAATGKRVALKTPAEKVAWLERRLEEGGFRLLEGERGPWVQILQDTFLEVRRKKDGEEAGKLLQVQAVLFEGRLEVVDPERALATLRRGVGPGKALGLGLLSVAP;rna_B:UCCCCACGCGUGUGGGGAU",
                                        "no_msa",
                                        "True"
                                    ]
                                    ]
                                }
                                }
                         },
               "rfdiffusion_inpainting": {"data":{
                                            "dataframe_split": {
                                                "columns": [
                                                "pdb",
                                                "start_idx",
                                                "end_idx"
                                                ],
                                                "data": [
                                                [
                                                    "ATOM      1  N   GLN A   1     -17.226 -15.172  28.982  1.00 38.93           N  \nATOM      2  CA  GLN A   1     -17.260 -14.625  27.604  1.00 32.79           C  \nATOM      3  C   GLN A   1     -15.882 -14.067  27.242  1.00 24.71           C  \nATOM      4  O   GLN A   1     -15.808 -13.217  26.371  1.00 25.47           O  \nATOM      5  CB  GLN A   1     -18.337 -13.541  27.483  1.00 35.07           C  \nATOM      6  CG  GLN A   1     -18.631 -12.751  28.769  1.00 42.43           C  \nATOM      7  CD  GLN A   1     -17.438 -12.283  29.581  1.00 48.26           C  \nATOM      8  OE1 GLN A   1     -16.675 -13.077  30.128  1.00 56.79           O  \nATOM      9  NE2 GLN A   1     -17.292 -10.974  29.727  1.00 52.41           N  \nATOM     10  H   GLN A   1     -18.074 -15.289  29.293  1.00 36.90           H  \nATOM     11  HA  GLN A   1     -17.476 -15.359  26.984  1.00 32.50           H  \nATOM     12  HB2 GLN A   1     -18.068 -12.912  26.782  1.00 36.05           H  \nATOM     13  HB3 GLN A   1     -19.166 -13.971  27.185  1.00 36.09           H  \nATOM     14  HG2 GLN A   1     -19.162 -11.962  28.527  1.00 41.83           H  \nATOM     15  HG3 GLN A   1     -19.195 -13.307  29.348  1.00 41.80           H  \nATOM     16 HE21 GLN A   1     -17.811 -10.403  29.158  0.00 55.54           H  \nATOM     17 HE22 GLN A   1     -16.524 -10.494  29.981  0.00 55.54           H  \nATOM     18  N   VAL A   2     -14.824 -14.538  27.916  1.00 23.25           N  \nATOM     19  CA  VAL A   2     -13.459 -14.253  27.488  1.00 20.98           C  \nATOM     20  C   VAL A   2     -13.231 -14.992  26.176  1.00 20.26           C  \nATOM     21  O   VAL A   2     -13.425 -16.208  26.090  1.00 21.47           O  \nATOM     22  CB  VAL A   2     -12.415 -14.659  28.537  1.00 22.35           C  \nATOM     23  CG1 VAL A   2     -10.993 -14.526  28.001  1.00 23.09           C  \nATOM     24  CG2 VAL A   2     -12.598 -13.887  29.839  1.00 23.40           C  \nATOM     25  H   VAL A   2     -14.949 -15.058  28.657  1.00 23.16           H  \nATOM     26  HA  VAL A   2     -13.381 -13.298  27.328  1.00 21.49           H  \nATOM     27  HB  VAL A   2     -12.566 -15.614  28.738  1.00 22.41           H  \nATOM     28 HG11 VAL A   2     -10.936 -13.748  27.420  1.00 22.85           H  \nATOM     29 HG12 VAL A   2     -10.759 -15.325  27.497  1.00 22.84           H  \nATOM     30 HG13 VAL A   2     -10.374 -14.421  28.744  1.00 22.85           H  \nATOM     31 HG21 VAL A   2     -11.916 -14.159  30.478  1.00 23.07           H  \nATOM     32 HG22 VAL A   2     -13.479 -14.074  30.208  1.00 23.07           H  \nATOM     33 HG23 VAL A   2     -12.516 -12.933  29.666  1.00 23.07           H  \nATOM     34  N   GLN A   3     -12.823 -14.246  25.149  1.00 17.28           N  \nATOM     35  CA  GLN A   3     -12.496 -14.836  23.867  1.00 16.74           C  \nATOM     36  C   GLN A   3     -11.245 -14.182  23.310  1.00 13.78           C  \nATOM     37  O   GLN A   3     -11.027 -12.968  23.427  1.00 14.33           O  \nATOM     38  CB  GLN A   3     -13.613 -14.684  22.834  1.00 21.69           C  \nATOM     39  CG  GLN A   3     -14.828 -15.528  23.139  1.00 27.81           C  \nATOM     40  CD  GLN A   3     -15.812 -15.532  21.989  1.00 32.40           C  \nATOM     41  OE1 GLN A   3     -15.548 -15.022  20.892  1.00 37.83           O  \nATOM     42  NE2 GLN A   3     -16.961 -16.147  22.235  1.00 39.39           N  \nATOM     43  H   GLN A   3     -12.747 -13.341  25.249  1.00 17.82           H  \nATOM     44  HA  GLN A   3     -12.318 -15.795  24.001  1.00 17.05           H  \nATOM     45  HB2 GLN A   3     -13.877 -13.742  22.795  1.00 21.63           H  \nATOM     46  HB3 GLN A   3     -13.259 -14.934  21.954  1.00 21.66           H  \nATOM     47  HG2 GLN A   3     -14.546 -16.449  23.325  1.00 27.18           H  \nATOM     48  HG3 GLN A   3     -15.272 -15.180  23.942  1.00 27.18           H  \nATOM     49 HE21 GLN A   3     -17.086 -16.659  23.028  0.00 39.46           H  \nATOM     50 HE22 GLN A   3     -17.615 -16.185  21.672  0.00 39.46           H  \nATOM     51  N   LEU A   4     -10.395 -15.020  22.688  1.00 14.08           N  \nATOM     52  CA  LEU A   4      -9.291 -14.616  21.851  1.00 13.08           C  \nATOM     53  C   LEU A   4      -9.490 -15.227  20.473  1.00 12.61           C  \nATOM     54  O   LEU A   4      -9.887 -16.386  20.341  1.00 13.93           O  \nATOM     55  CB  LEU A   4      -8.000 -15.179  22.449  1.00 13.33           C  \nATOM     56  CG  LEU A   4      -7.635 -14.708  23.836  1.00 13.21           C  \nATOM     57  CD1 LEU A   4      -6.406 -15.465  24.332  1.00 13.72           C  \nATOM     58  CD2 LEU A   4      -7.398 -13.231  23.850  1.00 15.00           C  \nATOM     59  H   LEU A   4     -10.526 -15.917  22.806  1.00 13.78           H  \nATOM     60  HA  LEU A   4      -9.253 -13.634  21.788  1.00 13.23           H  \nATOM     61  HB2 LEU A   4      -8.074 -16.154  22.467  1.00 13.25           H  \nATOM     62  HB3 LEU A   4      -7.264 -14.954  21.847  1.00 13.26           H  \nATOM     63  HG  LEU A   4      -8.390 -14.909  24.443  1.00 13.66           H  \nATOM     64 HD11 LEU A   4      -6.192 -15.179  25.238  1.00 13.56           H  \nATOM     65 HD12 LEU A   4      -5.651 -15.277  23.748  1.00 13.56           H  \nATOM     66 HD13 LEU A   4      -6.589 -16.420  24.329  1.00 13.56           H  \nATOM     67 HD21 LEU A   4      -8.248 -12.764  23.772  1.00 14.42           H  \nATOM     68 HD22 LEU A   4      -6.824 -12.987  23.103  1.00 14.42           H  \nATOM     69 HD23 LEU A   4      -6.966 -12.979  24.685  1.00 14.42           H  \nATOM     70  N   GLN A   5      -9.252 -14.439  19.435  1.00 13.56           N  \nATOM     71  CA  GLN A   5      -9.414 -14.946  18.087  1.00 12.34           C  \nATOM     72  C   GLN A   5      -8.341 -14.432  17.148  1.00 12.58           C  \nATOM     73  O   GLN A   5      -8.237 -13.228  16.876  1.00 11.23           O  \nATOM     74  CB  GLN A   5     -10.804 -14.547  17.585  1.00 13.13           C  \nATOM     75  CG  GLN A   5     -11.112 -15.222  16.240  1.00 16.22           C  \nATOM     76  CD  GLN A   5     -12.539 -15.050  15.788  1.00 19.51           C  \nATOM     77  OE1 GLN A   5     -13.331 -14.348  16.423  1.00 21.01           O  \nATOM     78  NE2 GLN A   5     -12.860 -15.713  14.682  1.00 20.99           N  \nATOM     79  H   GLN A   5      -8.984 -13.574  19.561  1.00 13.04           H  \nATOM     80  HA  GLN A   5      -9.362 -15.927  18.116  1.00 12.77           H  \nATOM     81  HB2 GLN A   5     -11.473 -14.813  18.249  1.00 13.61           H  \nATOM     82  HB3 GLN A   5     -10.840 -13.573  17.481  1.00 13.60           H  \nATOM     83  HG2 GLN A   5     -10.518 -14.852  15.553  1.00 16.11           H  \nATOM     84  HG3 GLN A   5     -10.919 -16.179  16.314  1.00 16.10           H  \nATOM     85 HE21 GLN A   5     -12.293 -16.139  14.268  0.00 20.98           H  \nATOM     86 HE22 GLN A   5     -13.732 -15.637  14.408  0.00 20.98           H  \nATOM     87  N   GLU A   6      -7.528 -15.371  16.651  1.00 11.77           N  \nATOM     88  CA  GLU A   6      -6.432 -15.076  15.769  1.00 12.16           C  \nATOM     89  C   GLU A   6      -6.911 -15.049  14.319  1.00 12.35           C  \nATOM     90  O   GLU A   6      -7.820 -15.804  13.905  1.00 14.21           O  \nATOM     91  CB  GLU A   6      -5.316 -16.134  15.838  1.00 12.65           C  \nATOM     92  CG  GLU A   6      -4.798 -16.439  17.220  1.00 13.65           C  \nATOM     93  CD  GLU A   6      -5.569 -17.505  17.979  1.00 15.08           C  \nATOM     94  OE1 GLU A   6      -6.734 -17.793  17.621  1.00 13.90           O  \nATOM     95  OE2 GLU A   6      -5.010 -18.055  18.957  1.00 12.50           O  \nATOM     96  H   GLU A   6      -7.683 -16.245  16.870  1.00 12.06           H  \nATOM     97  HA  GLU A   6      -6.062 -14.199  16.008  1.00 12.21           H  \nATOM     98  HB2 GLU A   6      -5.652 -16.964  15.439  1.00 12.76           H  \nATOM     99  HB3 GLU A   6      -4.567 -15.824  15.287  1.00 12.76           H  \nATOM    100  HG2 GLU A   6      -3.862 -16.727  17.150  1.00 13.73           H  \nATOM    101  HG3 GLU A   6      -4.811 -15.614  17.748  1.00 13.74           H  \nATOM    102  N   SER A   7      -6.225 -14.195  13.556  1.00 11.35           N  \nATOM    103  CA  SER A   7      -6.395 -14.095  12.105  1.00 12.63           C  \nATOM    104  C   SER A   7      -5.064 -13.840  11.444  1.00 11.83           C  \nATOM    105  O   SER A   7      -4.101 -13.350  12.056  1.00 12.82           O  \nATOM    106  CB  SER A   7      -7.338 -12.954  11.753  1.00 16.26           C  \nATOM    107  OG  SER A   7      -8.643 -13.221  12.156  1.00 23.48           O  \nATOM    108  H   SER A   7      -5.617 -13.642  13.956  1.00 11.87           H  \nATOM    109  HA  SER A   7      -6.769 -14.951  11.762  1.00 12.83           H  \nATOM    110  HB2 SER A   7      -7.026 -12.128  12.188  1.00 16.72           H  \nATOM    111  HB3 SER A   7      -7.323 -12.809  10.780  1.00 16.72           H  \nATOM    112  HG  SER A   7      -8.721 -12.932  13.032  0.00 15.64           H  \nATOM    113  N   GLY A   8      -5.017 -14.109  10.126  1.00 11.95           N  \nATOM    114  CA  GLY A   8      -3.907 -13.688   9.313  1.00 11.91           C  \nATOM    115  C   GLY A   8      -3.107 -14.830   8.689  1.00 12.55           C  \nATOM    116  O   GLY A   8      -2.263 -14.560   7.830  1.00 14.68           O  \nATOM    117  H   GLY A   8      -5.702 -14.570   9.734  1.00 11.91           H  \nATOM    118  HA2 GLY A   8      -4.254 -13.110   8.588  1.00 12.07           H  \nATOM    119  HA3 GLY A   8      -3.298 -13.135   9.862  1.00 12.07           H  \nATOM    120  N   GLY A   9      -3.308 -16.061   9.153  1.00 12.33           N  \nATOM    121  CA  GLY A   9      -2.503 -17.150   8.621  1.00 12.93           C  \nATOM    122  C   GLY A   9      -2.676 -17.324   7.116  1.00 13.95           C  \nATOM    123  O   GLY A   9      -3.757 -17.113   6.583  1.00 16.07           O  \nATOM    124  H   GLY A   9      -3.937 -16.231   9.794  1.00 12.52           H  \nATOM    125  HA2 GLY A   9      -1.550 -16.973   8.825  1.00 13.03           H  \nATOM    126  HA3 GLY A   9      -2.762 -17.987   9.078  1.00 13.02           H  \nATOM    127  N   ASP A  10      -1.599 -17.724   6.473  1.00 13.09           N  \nATOM    128  CA  ASP A  10      -1.630 -17.905   5.029  1.00 15.15           C  \nATOM    129  C   ASP A  10      -0.461 -18.787   4.641  1.00 16.17           C  \nATOM    130  O   ASP A  10       0.415 -19.066   5.440  1.00 14.37           O  \nATOM    131  CB  ASP A  10      -1.526 -16.541   4.335  1.00 17.18           C  \nATOM    132  CG  ASP A  10      -2.105 -16.457   2.926  1.00 22.14           C  \nATOM    133  OD1 ASP A  10      -2.504 -17.498   2.376  1.00 23.78           O  \nATOM    134  OD2 ASP A  10      -2.150 -15.335   2.409  1.00 32.81           O  \nATOM    135  H   ASP A  10      -0.825 -17.891   6.929  1.00 13.76           H  \nATOM    136  HA  ASP A  10      -2.474 -18.347   4.773  1.00 15.29           H  \nATOM    137  HB2 ASP A  10      -1.985 -15.873   4.885  1.00 17.69           H  \nATOM    138  HB3 ASP A  10      -0.581 -16.287   4.285  1.00 17.70           H  \nATOM    139  N   LEU A  11      -0.431 -19.107   3.337  1.00 16.69           N  \nATOM    140  CA  LEU A  11       0.719 -19.728   2.717  1.00 18.44           C  \nATOM    141  C   LEU A  11       1.647 -18.608   2.267  1.00 19.12           C  \nATOM    142  O   LEU A  11       1.211 -17.683   1.552  1.00 23.99           O  \nATOM    143  CB  LEU A  11       0.255 -20.558   1.515  1.00 17.53           C  \nATOM    144  CG  LEU A  11       1.379 -21.217   0.725  1.00 22.50           C  \nATOM    145  CD1 LEU A  11       2.046 -22.307   1.539  1.00 21.37           C  \nATOM    146  CD2 LEU A  11       0.854 -21.797  -0.582  1.00 25.73           C  \nATOM    147  H   LEU A  11      -1.168 -18.943   2.822  1.00 16.97           H  \nATOM    148  HA  LEU A  11       1.175 -20.300   3.372  1.00 18.04           H  \nATOM    149  HB2 LEU A  11      -0.353 -21.254   1.835  1.00 18.82           H  \nATOM    150  HB3 LEU A  11      -0.250 -19.977   0.913  1.00 18.81           H  \nATOM    151  HG  LEU A  11       2.057 -20.530   0.509  1.00 21.79           H  \nATOM    152 HD11 LEU A  11       2.695 -22.776   0.986  1.00 21.71           H  \nATOM    153 HD12 LEU A  11       1.373 -22.936   1.853  1.00 21.70           H  \nATOM    154 HD13 LEU A  11       2.500 -21.911   2.302  1.00 21.67           H  \nATOM    155 HD21 LEU A  11       0.973 -21.147  -1.296  1.00 24.68           H  \nATOM    156 HD22 LEU A  11      -0.092 -22.007  -0.490  1.00 24.67           H  \nATOM    157 HD23 LEU A  11       1.345 -22.609  -0.801  1.00 24.67           H  \nATOM    158  N   VAL A  12       2.908 -18.684   2.674  1.00 18.11           N  \nATOM    159  CA AVAL A  12       3.930 -17.659   2.482  0.50 17.87           C  \nATOM    160  CA BVAL A  12       3.872 -17.659   2.366  0.50 17.53           C  \nATOM    161  C   VAL A  12       5.148 -18.311   1.838  1.00 18.53           C  \nATOM    162  O   VAL A  12       5.395 -19.499   2.035  1.00 20.54           O  \nATOM    163  CB AVAL A  12       4.408 -17.014   3.808  0.50 20.16           C  \nATOM    164  CB BVAL A  12       4.063 -16.785   3.617  0.50 19.48           C  \nATOM    165  CG1AVAL A  12       5.331 -15.815   3.585  0.50 22.15           C  \nATOM    166  CG1BVAL A  12       2.810 -15.965   3.860  0.50 20.01           C  \nATOM    167  CG2AVAL A  12       3.261 -16.610   4.701  0.50 20.18           C  \nATOM    168  CG2BVAL A  12       4.397 -17.618   4.838  0.50 19.58           C  \nATOM    169  H   VAL A  12       3.166 -19.435   3.126  1.00 18.15           H  \nATOM    170  HA AVAL A  12       3.587 -16.966   1.891  0.50 18.47           H  \nATOM    171  HA BVAL A  12       3.506 -17.099   1.659  0.50 18.20           H  \nATOM    172  HB AVAL A  12       4.929 -17.701   4.293  0.50 20.06           H  \nATOM    173  HB BVAL A  12       4.814 -16.165   3.446  0.50 19.21           H  \nATOM    174 HG11AVAL A  12       5.209 -15.475   2.685  0.50 21.52           H  \nATOM    175 HG11BVAL A  12       2.136 -16.520   4.291  0.50 19.84           H  \nATOM    176 HG12AVAL A  12       6.256 -16.089   3.707  0.50 21.51           H  \nATOM    177 HG12BVAL A  12       2.465 -15.638   3.011  0.50 19.84           H  \nATOM    178 HG13AVAL A  12       5.113 -15.116   4.225  0.50 21.54           H  \nATOM    179 HG13BVAL A  12       3.022 -15.209   4.435  0.50 19.84           H  \nATOM    180 HG21AVAL A  12       3.567 -15.944   5.340  0.50 20.18           H  \nATOM    181 HG21BVAL A  12       4.349 -17.060   5.632  0.50 19.58           H  \nATOM    182 HG22AVAL A  12       2.932 -17.390   5.179  0.50 20.18           H  \nATOM    183 HG22BVAL A  12       5.296 -17.979   4.749  0.50 19.55           H  \nATOM    184 HG23AVAL A  12       2.545 -16.235   4.160  0.50 20.18           H  \nATOM    185 HG23BVAL A  12       3.762 -18.348   4.922  0.50 19.54           H  \nATOM    186  N   GLN A  13       5.927 -17.533   1.093  1.00 19.02           N  \nATOM    187  CA  GLN A  13       7.172 -18.030   0.540  1.00 19.92           C  \nATOM    188  C   GLN A  13       8.316 -17.782   1.514  1.00 19.10           C  \nATOM    189  O   GLN A  13       8.288 -16.832   2.301  1.00 17.54           O  \nATOM    190  CB  GLN A  13       7.436 -17.330  -0.800  1.00 24.55           C  \nATOM    191  CG  GLN A  13       6.326 -17.562  -1.816  1.00 25.29           C  \nATOM    192  CD  GLN A  13       6.140 -19.053  -2.003  1.00 29.61           C  \nATOM    193  OE1 GLN A  13       7.104 -19.766  -2.251  1.00 31.57           O  \nATOM    194  NE2 GLN A  13       4.917 -19.537  -1.812  1.00 28.85           N  \nATOM    195  H   GLN A  13       5.675 -16.674   0.912  1.00 19.12           H  \nATOM    196  HA  GLN A  13       7.085 -18.996   0.387  1.00 20.41           H  \nATOM    197  HB2 GLN A  13       7.530 -16.368  -0.638  1.00 23.53           H  \nATOM    198  HB3 GLN A  13       8.284 -17.659  -1.165  1.00 23.54           H  \nATOM    199  HG2 GLN A  13       5.490 -17.159  -1.498  1.00 26.06           H  \nATOM    200  HG3 GLN A  13       6.566 -17.148  -2.673  1.00 26.07           H  \nATOM    201 HE21 GLN A  13       4.306 -18.984  -1.560  0.00 29.60           H  \nATOM    202 HE22 GLN A  13       4.801 -20.400  -1.867  0.00 29.60           H  \nATOM    203  N   PRO A  14       9.397 -18.590   1.461  1.00 21.25           N  \nATOM    204  CA  PRO A  14      10.609 -18.350   2.237  1.00 21.54           C  \nATOM    205  C   PRO A  14      11.141 -16.953   1.983  1.00 20.15           C  \nATOM    206  O   PRO A  14      11.130 -16.504   0.832  1.00 21.44           O  \nATOM    207  CB  PRO A  14      11.661 -19.332   1.716  1.00 22.26           C  \nATOM    208  CG  PRO A  14      10.822 -20.426   1.070  1.00 25.45           C  \nATOM    209  CD  PRO A  14       9.542 -19.770   0.590  1.00 23.44           C  \nATOM    210  HA  PRO A  14      10.438 -18.503   3.200  1.00 21.33           H  \nATOM    211  HB2 PRO A  14      12.252 -18.908   1.057  1.00 22.79           H  \nATOM    212  HB3 PRO A  14      12.204 -19.692   2.449  1.00 22.79           H  \nATOM    213  HG2 PRO A  14      11.303 -20.827   0.317  1.00 24.18           H  \nATOM    214  HG3 PRO A  14      10.621 -21.131   1.719  1.00 24.18           H  \nATOM    215  HD2 PRO A  14       9.613 -19.506  -0.346  1.00 23.35           H  \nATOM    216  HD3 PRO A  14       8.783 -20.374   0.691  1.00 23.34           H  \nATOM    217  N   GLY A  15      11.517 -16.278   3.073  1.00 19.24           N  \nATOM    218  CA  GLY A  15      11.954 -14.893   3.014  1.00 18.29           C  \nATOM    219  C   GLY A  15      10.834 -13.856   3.115  1.00 17.43           C  \nATOM    220  O   GLY A  15      11.114 -12.650   3.215  1.00 22.25           O  \nATOM    221  H   GLY A  15      11.505 -16.697   3.885  1.00 19.23           H  \nATOM    222  HA2 GLY A  15      12.597 -14.738   3.750  1.00 18.30           H  \nATOM    223  HA3 GLY A  15      12.442 -14.752   2.164  1.00 18.31           H  \nATOM    224  N   GLY A  16       9.574 -14.311   3.062  1.00 18.38           N  \nATOM    225  CA  GLY A  16       8.422 -13.439   3.060  1.00 17.16           C  \nATOM    226  C   GLY A  16       8.049 -12.928   4.447  1.00 16.31           C  \nATOM    227  O   GLY A  16       8.675 -13.317   5.417  1.00 18.08           O  \nATOM    228  H   GLY A  16       9.429 -15.213   3.035  1.00 17.87           H  \nATOM    229  HA2 GLY A  16       8.607 -12.667   2.468  1.00 17.24           H  \nATOM    230  HA3 GLY A  16       7.652 -13.931   2.679  1.00 17.23           H  \nATOM    231  N   SER A  17       7.036 -12.057   4.445  1.00 17.54           N  \nATOM    232  CA  SER A  17       6.480 -11.403   5.637  1.00 18.43           C  \nATOM    233  C   SER A  17       4.997 -11.745   5.774  1.00 18.58           C  \nATOM    234  O   SER A  17       4.262 -11.985   4.804  1.00 18.82           O  \nATOM    235  CB  SER A  17       6.695  -9.915   5.558  1.00 19.11           C  \nATOM    236  OG  SER A  17       8.083  -9.610   5.666  1.00 24.79           O  \nATOM    237  H   SER A  17       6.659 -11.852   3.638  1.00 17.44           H  \nATOM    238  HA  SER A  17       6.958 -11.752   6.434  1.00 18.41           H  \nATOM    239  HB2 SER A  17       6.349  -9.577   4.701  1.00 20.15           H  \nATOM    240  HB3 SER A  17       6.203  -9.472   6.286  1.00 20.15           H  \nATOM    241  HG  SER A  17       8.409  -9.658   4.826  0.00 16.15           H  \nATOM    242  N   LEU A  18       4.492 -11.688   7.019  1.00 15.83           N  \nATOM    243  CA  LEU A  18       3.081 -11.929   7.278  1.00 15.52           C  \nATOM    244  C   LEU A  18       2.747 -11.298   8.625  1.00 14.83           C  \nATOM    245  O   LEU A  18       3.621 -11.323   9.472  1.00 17.15           O  \nATOM    246  CB  LEU A  18       2.807 -13.429   7.326  1.00 16.74           C  \nATOM    247  CG  LEU A  18       1.345 -13.835   7.379  1.00 16.99           C  \nATOM    248  CD1 LEU A  18       0.616 -13.492   6.086  1.00 19.13           C  \nATOM    249  CD2 LEU A  18       1.231 -15.326   7.669  1.00 17.46           C  \nATOM    250  H   LEU A  18       5.044 -11.495   7.721  1.00 16.37           H  \nATOM    251  HA  LEU A  18       2.542 -11.504   6.571  1.00 15.69           H  \nATOM    252  HB2 LEU A  18       3.212 -13.840   6.537  1.00 16.51           H  \nATOM    253  HB3 LEU A  18       3.258 -13.797   8.112  1.00 16.50           H  \nATOM    254  HG  LEU A  18       0.911 -13.343   8.119  1.00 17.41           H  \nATOM    255 HD11 LEU A  18      -0.241 -13.953   6.063  1.00 18.43           H  \nATOM    256 HD12 LEU A  18       1.154 -13.772   5.326  1.00 18.43           H  \nATOM    257 HD13 LEU A  18       0.468 -12.532   6.042  1.00 18.43           H  \nATOM    258 HD21 LEU A  18       1.675 -15.529   8.511  1.00 17.31           H  \nATOM    259 HD22 LEU A  18       1.652 -15.830   6.952  1.00 17.32           H  \nATOM    260 HD23 LEU A  18       0.292 -15.575   7.731  1.00 17.32           H  \nATOM    261  N   LYS A  19       1.591 -10.668   8.735  1.00 13.26           N  \nATOM    262  CA  LYS A  19       1.161 -10.082   9.996  1.00 14.43           C  \nATOM    263  C   LYS A  19      -0.062 -10.824  10.520  1.00 13.91           C  \nATOM    264  O   LYS A  19      -1.084 -10.936   9.841  1.00 15.38           O  \nATOM    265  CB  LYS A  19       0.850  -8.591   9.821  1.00 18.90           C  \nATOM    266  CG  LYS A  19       0.471  -7.876  11.115  1.00 23.93           C  \nATOM    267  CD  LYS A  19      -0.072  -6.504  10.754  1.00 28.35           C  \nATOM    268  CE  LYS A  19      -1.155  -5.999  11.674  1.00 34.91           C  \nATOM    269  NZ  LYS A  19      -1.830  -4.811  11.078  1.00 38.99           N  \nATOM    270  H   LYS A  19       1.044 -10.596   8.006  1.00 13.90           H  \nATOM    271  HA  LYS A  19       1.890 -10.175  10.647  1.00 14.83           H  \nATOM    272  HB2 LYS A  19       1.636  -8.151   9.437  1.00 18.77           H  \nATOM    273  HB3 LYS A  19       0.112  -8.502   9.183  1.00 18.79           H  \nATOM    274  HG2 LYS A  19      -0.210  -8.390  11.597  1.00 23.57           H  \nATOM    275  HG3 LYS A  19       1.263  -7.783  11.688  1.00 23.56           H  \nATOM    276  HD2 LYS A  19       0.669  -5.861  10.761  1.00 28.60           H  \nATOM    277  HD3 LYS A  19      -0.429  -6.533   9.841  1.00 28.57           H  \nATOM    278  HE2 LYS A  19      -1.815  -6.701  11.827  1.00 34.07           H  \nATOM    279  HE3 LYS A  19      -0.767  -5.750  12.534  1.00 34.06           H  \nATOM    280  HZ1 LYS A  19      -2.256  -5.050  10.314  1.00 37.66           H  \nATOM    281  HZ2 LYS A  19      -1.216  -4.171  10.888  1.00 37.68           H  \nATOM    282  HZ3 LYS A  19      -2.437  -4.476  11.663  1.00 37.68           H  \nATOM    283  N   LEU A  20       0.024 -11.338  11.765  1.00 11.06           N  \nATOM    284  CA  LEU A  20      -1.075 -11.968  12.445  1.00 11.39           C  \nATOM    285  C   LEU A  20      -1.664 -10.995  13.450  1.00 11.67           C  \nATOM    286  O   LEU A  20      -0.919 -10.156  13.966  1.00 11.99           O  \nATOM    287  CB  LEU A  20      -0.594 -13.216  13.210  1.00 11.94           C  \nATOM    288  CG  LEU A  20       0.212 -14.245  12.442  1.00 12.42           C  \nATOM    289  CD1 LEU A  20       0.531 -15.430  13.312  1.00 12.23           C  \nATOM    290  CD2 LEU A  20      -0.433 -14.692  11.157  1.00 13.19           C  \nATOM    291  H   LEU A  20       0.824 -11.284  12.203  1.00 11.77           H  \nATOM    292  HA  LEU A  20      -1.763 -12.219  11.788  1.00 11.50           H  \nATOM    293  HB2 LEU A  20      -0.053 -12.914  13.967  1.00 11.92           H  \nATOM    294  HB3 LEU A  20      -1.381 -13.664  13.579  1.00 11.92           H  \nATOM    295  HG  LEU A  20       1.073 -13.821  12.203  1.00 12.43           H  \nATOM    296 HD11 LEU A  20       1.181 -15.997  12.863  1.00 12.29           H  \nATOM    297 HD12 LEU A  20      -0.282 -15.938  13.479  1.00 12.29           H  \nATOM    298 HD13 LEU A  20       0.899 -15.121  14.158  1.00 12.29           H  \nATOM    299 HD21 LEU A  20      -0.392 -13.972  10.504  1.00 12.94           H  \nATOM    300 HD22 LEU A  20      -1.363 -14.925  11.324  1.00 12.95           H  \nATOM    301 HD23 LEU A  20       0.037 -15.470  10.810  1.00 12.94           H  \nATOM    302  N   SER A  21      -2.970 -11.155  13.693  1.00 11.71           N  \nATOM    303  CA  SER A  21      -3.732 -10.362  14.650  1.00 12.40           C  \nATOM    304  C   SER A  21      -4.428 -11.324  15.590  1.00 12.34           C  \nATOM    305  O   SER A  21      -4.814 -12.427  15.231  1.00 12.78           O  \nATOM    306  CB  SER A  21      -4.785  -9.460  13.968  1.00 14.51           C  \nATOM    307  OG  SER A  21      -4.202  -8.582  13.034  1.00 18.36           O  \nATOM    308  H   SER A  21      -3.416 -11.806  13.231  1.00 11.86           H  \nATOM    309  HA  SER A  21      -3.104  -9.796  15.171  1.00 12.62           H  \nATOM    310  HB2 SER A  21      -5.448 -10.026  13.512  1.00 14.78           H  \nATOM    311  HB3 SER A  21      -5.255  -8.937  14.656  1.00 14.78           H  \nATOM    312  HG  SER A  21      -3.574  -8.808  12.318  0.00 10.33           H  \nATOM    313  N   CYS A  22      -4.586 -10.935  16.863  1.00 10.77           N  \nATOM    314  CA  CYS A  22      -5.412 -11.617  17.802  1.00 12.35           C  \nATOM    315  C   CYS A  22      -6.289 -10.585  18.495  1.00 12.59           C  \nATOM    316  O   CYS A  22      -5.817  -9.670  19.165  1.00 13.77           O  \nATOM    317  CB  CYS A  22      -4.497 -12.269  18.800  1.00 13.37           C  \nATOM    318  SG  CYS A  22      -5.315 -12.842  20.323  1.00 20.83           S  \nATOM    319  H   CYS A  22      -4.137 -10.191  17.147  1.00 11.49           H  \nATOM    320  HA  CYS A  22      -5.959 -12.294  17.346  1.00 12.26           H  \nATOM    321  HB2 CYS A  22      -4.064 -13.038  18.376  1.00 14.59           H  \nATOM    322  HB3 CYS A  22      -3.796 -11.634  19.048  1.00 14.57           H  \nATOM    323  N   ALA A  23      -7.571 -10.787  18.349  1.00 11.33           N  \nATOM    324  CA  ALA A  23      -8.579  -9.863  18.890  1.00 12.09           C  \nATOM    325  C   ALA A  23      -9.197 -10.434  20.153  1.00 12.46           C  \nATOM    326  O   ALA A  23      -9.765 -11.517  20.152  1.00 12.73           O  \nATOM    327  CB  ALA A  23      -9.627  -9.636  17.835  1.00 13.30           C  \nATOM    328  H   ALA A  23      -7.858 -11.528  17.898  1.00 11.80           H  \nATOM    329  HA  ALA A  23      -8.147  -9.003  19.103  1.00 12.23           H  \nATOM    330  HB1 ALA A  23     -10.268  -8.976  18.149  1.00 12.92           H  \nATOM    331  HB2 ALA A  23      -9.205  -9.315  17.021  1.00 12.91           H  \nATOM    332  HB3 ALA A  23     -10.089 -10.472  17.651  1.00 12.91           H  \nATOM    333  N   ALA A  24      -9.062  -9.675  21.252  1.00 11.32           N  \nATOM    334  CA  ALA A  24      -9.562 -10.076  22.556  1.00 13.21           C  \nATOM    335  C   ALA A  24     -10.925  -9.454  22.805  1.00 13.46           C  \nATOM    336  O   ALA A  24     -11.168  -8.332  22.389  1.00 15.14           O  \nATOM    337  CB  ALA A  24      -8.548  -9.671  23.608  1.00 13.03           C  \nATOM    338  H   ALA A  24      -8.640  -8.868  21.180  1.00 12.01           H  \nATOM    339  HA  ALA A  24      -9.655 -11.055  22.569  1.00 12.85           H  \nATOM    340  HB1 ALA A  24      -8.898  -9.873  24.492  1.00 13.09           H  \nATOM    341  HB2 ALA A  24      -7.721 -10.162  23.468  1.00 13.09           H  \nATOM    342  HB3 ALA A  24      -8.373  -8.717  23.541  1.00 13.08           H  \nATOM    343  N   SER A  25     -11.721 -10.177  23.587  1.00 13.71           N  \nATOM    344  CA  SER A  25     -12.995  -9.643  24.057  1.00 14.07           C  \nATOM    345  C   SER A  25     -13.318 -10.275  25.401  1.00 14.41           C  \nATOM    346  O   SER A  25     -12.840 -11.353  25.783  1.00 16.26           O  \nATOM    347  CB  SER A  25     -14.073  -9.812  23.013  1.00 15.24           C  \nATOM    348  OG  SER A  25     -14.212 -11.158  22.632  1.00 18.68           O  \nATOM    349  H   SER A  25     -11.474 -11.021  23.835  1.00 13.73           H  \nATOM    350  HA  SER A  25     -12.871  -8.668  24.210  1.00 14.28           H  \nATOM    351  HB2 SER A  25     -14.928  -9.483  23.372  1.00 15.69           H  \nATOM    352  HB3 SER A  25     -13.847  -9.274  22.225  1.00 15.69           H  \nATOM    353  HG  SER A  25     -14.640 -11.709  23.172  0.00 18.54           H  \nATOM    354  N   GLY A  26     -14.140  -9.550  26.188  1.00 15.41           N  \nATOM    355  CA  GLY A  26     -14.730 -10.036  27.426  1.00 15.20           C  \nATOM    356  C   GLY A  26     -13.897  -9.871  28.683  1.00 15.47           C  \nATOM    357  O   GLY A  26     -14.171 -10.523  29.690  1.00 16.86           O  \nATOM    358  H   GLY A  26     -14.341  -8.697  25.928  1.00 15.11           H  \nATOM    359  HA2 GLY A  26     -15.593  -9.568  27.558  1.00 15.31           H  \nATOM    360  HA3 GLY A  26     -14.940 -10.995  27.313  1.00 15.33           H  \nATOM    361  N   PHE A  27     -12.834  -9.062  28.614  1.00 13.13           N  \nATOM    362  CA  PHE A  27     -11.991  -8.741  29.759  1.00 13.31           C  \nATOM    363  C   PHE A  27     -11.230  -7.444  29.474  1.00 14.08           C  \nATOM    364  O   PHE A  27     -11.219  -6.924  28.350  1.00 13.98           O  \nATOM    365  CB  PHE A  27     -11.012  -9.875  30.086  1.00 13.93           C  \nATOM    366  CG  PHE A  27      -9.959 -10.187  29.055  1.00 13.82           C  \nATOM    367  CD1 PHE A  27     -10.291 -10.913  27.907  1.00 15.55           C  \nATOM    368  CD2 PHE A  27      -8.634  -9.787  29.228  1.00 13.45           C  \nATOM    369  CE1 PHE A  27      -9.314 -11.233  26.982  1.00 14.34           C  \nATOM    370  CE2 PHE A  27      -7.662 -10.135  28.310  1.00 14.33           C  \nATOM    371  CZ  PHE A  27      -8.002 -10.840  27.182  1.00 14.63           C  \nATOM    372  H   PHE A  27     -12.621  -8.689  27.807  1.00 13.70           H  \nATOM    373  HA  PHE A  27     -12.574  -8.591  30.546  1.00 13.54           H  \nATOM    374  HB2 PHE A  27     -10.562  -9.650  30.927  1.00 13.76           H  \nATOM    375  HB3 PHE A  27     -11.536 -10.688  30.245  1.00 13.76           H  \nATOM    376  HD1 PHE A  27     -11.180 -11.196  27.773  1.00 14.79           H  \nATOM    377  HD2 PHE A  27      -8.391  -9.317  30.010  1.00 13.80           H  \nATOM    378  HE1 PHE A  27      -9.545 -11.721  26.207  1.00 14.64           H  \nATOM    379  HE2 PHE A  27      -6.771  -9.850  28.437  1.00 14.20           H  \nATOM    380  HZ  PHE A  27      -7.343 -11.049  26.539  1.00 14.47           H  \nATOM    381  N   THR A  28     -10.628  -6.864  30.515  1.00 13.50           N  \nATOM    382  CA  THR A  28      -9.795  -5.690  30.374  1.00 11.99           C  \nATOM    383  C   THR A  28      -8.430  -6.097  29.819  1.00 11.91           C  \nATOM    384  O   THR A  28      -7.602  -6.633  30.537  1.00 11.62           O  \nATOM    385  CB  THR A  28      -9.607  -4.930  31.688  1.00 12.96           C  \nATOM    386  OG1 THR A  28     -10.923  -4.678  32.215  1.00 14.52           O  \nATOM    387  CG2 THR A  28      -8.836  -3.634  31.530  1.00 13.25           C  \nATOM    388  H   THR A  28     -10.746  -7.219  31.349  1.00 13.26           H  \nATOM    389  HA  THR A  28     -10.228  -5.086  29.728  1.00 12.44           H  \nATOM    390  HB  THR A  28      -9.120  -5.514  32.317  1.00 13.10           H  \nATOM    391  HG1 THR A  28     -11.368  -4.184  31.640  0.00 14.50           H  \nATOM    392 HG21 THR A  28      -9.091  -3.014  32.236  1.00 13.16           H  \nATOM    393 HG22 THR A  28      -9.038  -3.238  30.663  1.00 13.15           H  \nATOM    394 HG23 THR A  28      -7.880  -3.815  31.588  1.00 13.16           H  \nATOM    395  N   PHE A  29      -8.284  -5.922  28.505  1.00 11.77           N  \nATOM    396  CA  PHE A  29      -7.128  -6.480  27.796  1.00 12.43           C  \nATOM    397  C   PHE A  29      -5.832  -5.942  28.387  1.00 12.03           C  \nATOM    398  O   PHE A  29      -4.878  -6.701  28.590  1.00 11.51           O  \nATOM    399  CB  PHE A  29      -7.280  -6.129  26.321  1.00 12.20           C  \nATOM    400  CG  PHE A  29      -6.187  -6.605  25.398  1.00 11.14           C  \nATOM    401  CD1 PHE A  29      -5.928  -7.969  25.194  1.00 11.24           C  \nATOM    402  CD2 PHE A  29      -5.431  -5.683  24.697  1.00 10.08           C  \nATOM    403  CE1 PHE A  29      -4.970  -8.340  24.249  1.00 10.81           C  \nATOM    404  CE2 PHE A  29      -4.499  -6.066  23.751  1.00 10.53           C  \nATOM    405  CZ  PHE A  29      -4.252  -7.400  23.574  1.00 11.21           C  \nATOM    406  H   PHE A  29      -8.908  -5.448  28.034  1.00 11.96           H  \nATOM    407  HA  PHE A  29      -7.137  -7.465  27.895  1.00 12.19           H  \nATOM    408  HB2 PHE A  29      -8.131  -6.502  26.009  1.00 12.00           H  \nATOM    409  HB3 PHE A  29      -7.346  -5.154  26.249  1.00 12.00           H  \nATOM    410  HD1 PHE A  29      -6.439  -8.624  25.642  1.00 11.09           H  \nATOM    411  HD2 PHE A  29      -5.621  -4.765  24.805  1.00 10.50           H  \nATOM    412  HE1 PHE A  29      -4.782  -9.255  24.114  1.00 10.99           H  \nATOM    413  HE2 PHE A  29      -3.985  -5.418  23.296  1.00 10.62           H  \nATOM    414  HZ  PHE A  29      -3.583  -7.673  22.967  1.00 10.95           H  \nATOM    415  N   SER A  30      -5.774  -4.630  28.660  1.00 12.76           N  \nATOM    416  CA  SER A  30      -4.609  -3.958  29.200  1.00 11.40           C  \nATOM    417  C   SER A  30      -4.168  -4.443  30.570  1.00 12.13           C  \nATOM    418  O   SER A  30      -3.129  -3.983  31.009  1.00 14.98           O  \nATOM    419  CB  SER A  30      -4.844  -2.439  29.240  1.00 12.95           C  \nATOM    420  OG  SER A  30      -5.931  -2.130  30.078  1.00 15.08           O  \nATOM    421  H   SER A  30      -6.516  -4.124  28.491  1.00 12.26           H  \nATOM    422  HA  SER A  30      -3.857  -4.129  28.571  1.00 12.08           H  \nATOM    423  HB2 SER A  30      -4.034  -1.990  29.572  1.00 13.03           H  \nATOM    424  HB3 SER A  30      -5.025  -2.112  28.331  1.00 13.04           H  \nATOM    425  HG  SER A  30      -5.541  -2.021  30.849  0.00 15.25           H  \nATOM    426  N   SER A  31      -4.926  -5.300  31.234  1.00 12.75           N  \nATOM    427  CA  SER A  31      -4.561  -5.791  32.547  1.00 12.88           C  \nATOM    428  C   SER A  31      -3.697  -7.044  32.450  1.00 10.91           C  \nATOM    429  O   SER A  31      -3.303  -7.541  33.501  1.00 14.99           O  \nATOM    430  CB  SER A  31      -5.723  -6.046  33.405  1.00 14.78           C  \nATOM    431  OG  SER A  31      -6.402  -4.839  33.715  1.00 17.53           O  \nATOM    432  H   SER A  31      -5.714  -5.582  30.868  1.00 12.63           H  \nATOM    433  HA  SER A  31      -4.014  -5.086  32.987  1.00 12.79           H  \nATOM    434  HB2 SER A  31      -6.339  -6.660  32.946  1.00 14.90           H  \nATOM    435  HB3 SER A  31      -5.428  -6.477  34.240  1.00 14.89           H  \nATOM    436  HG  SER A  31      -6.963  -4.595  33.085  0.00 17.51           H  \nATOM    437  N   TYR A  32      -3.499  -7.586  31.229  1.00 10.63           N  \nATOM    438  CA  TYR A  32      -2.855  -8.886  31.101  1.00  9.89           C  \nATOM    439  C   TYR A  32      -1.723  -8.864  30.099  1.00  9.47           C  \nATOM    440  O   TYR A  32      -1.821  -8.317  28.996  1.00  9.33           O  \nATOM    441  CB  TYR A  32      -3.857  -9.948  30.641  1.00  9.43           C  \nATOM    442  CG  TYR A  32      -4.931 -10.212  31.645  1.00 10.89           C  \nATOM    443  CD1 TYR A  32      -4.798 -11.230  32.555  1.00 11.60           C  \nATOM    444  CD2 TYR A  32      -6.044  -9.424  31.668  1.00 11.28           C  \nATOM    445  CE1 TYR A  32      -5.792 -11.504  33.451  1.00 13.73           C  \nATOM    446  CE2 TYR A  32      -7.043  -9.668  32.602  1.00 12.68           C  \nATOM    447  CZ  TYR A  32      -6.891 -10.686  33.505  1.00 15.03           C  \nATOM    448  OH  TYR A  32      -7.876 -10.992  34.448  1.00 20.37           O  \nATOM    449  H   TYR A  32      -3.768  -7.146  30.475  1.00 10.53           H  \nATOM    450  HA  TYR A  32      -2.492  -9.153  31.983  1.00  9.86           H  \nATOM    451  HB2 TYR A  32      -4.268  -9.650  29.803  1.00  9.87           H  \nATOM    452  HB3 TYR A  32      -3.373 -10.781  30.462  1.00  9.87           H  \nATOM    453  HD1 TYR A  32      -4.037 -11.783  32.526  1.00 11.98           H  \nATOM    454  HD2 TYR A  32      -6.133  -8.713  31.056  1.00 11.63           H  \nATOM    455  HE1 TYR A  32      -5.688 -12.199  34.079  1.00 13.38           H  \nATOM    456  HE2 TYR A  32      -7.814  -9.125  32.626  1.00 12.82           H  \nATOM    457  HH  TYR A  32      -7.796 -11.634  35.005  0.00 20.58           H  \nATOM    458  N   GLY A  33      -0.661  -9.576  30.469  1.00  8.48           N  \nATOM    459  CA  GLY A  33       0.378  -9.920  29.523  1.00  9.02           C  \nATOM    460  C   GLY A  33      -0.157 -10.977  28.543  1.00  8.19           C  \nATOM    461  O   GLY A  33      -1.140 -11.652  28.811  1.00  9.93           O  \nATOM    462  H   GLY A  33      -0.578  -9.854  31.336  1.00  8.83           H  \nATOM    463  HA2 GLY A  33       0.655  -9.110  29.027  1.00  8.69           H  \nATOM    464  HA3 GLY A  33       1.161 -10.281  30.010  1.00  8.69           H  \nATOM    465  N   MET A  34       0.442 -11.018  27.356  1.00  8.92           N  \nATOM    466  CA  MET A  34      -0.006 -11.861  26.254  1.00  9.35           C  \nATOM    467  C   MET A  34       1.204 -12.477  25.584  1.00  8.92           C  \nATOM    468  O   MET A  34       2.298 -11.903  25.556  1.00  9.61           O  \nATOM    469  CB  MET A  34      -0.774 -11.038  25.225  1.00  9.75           C  \nATOM    470  CG  MET A  34      -2.072 -10.389  25.693  1.00 11.01           C  \nATOM    471  SD  MET A  34      -3.399 -11.527  26.121  1.00 12.82           S  \nATOM    472  CE  MET A  34      -3.641 -12.330  24.571  1.00 14.29           C  \nATOM    473  H   MET A  34       1.175 -10.489  27.222  1.00  8.84           H  \nATOM    474  HA  MET A  34      -0.579 -12.579  26.609  1.00  9.26           H  \nATOM    475  HB2 MET A  34      -0.180 -10.333  24.899  1.00  9.94           H  \nATOM    476  HB3 MET A  34      -0.976 -11.617  24.463  1.00  9.95           H  \nATOM    477  HG2 MET A  34      -1.878  -9.834  26.479  1.00 11.08           H  \nATOM    478  HG3 MET A  34      -2.396  -9.792  24.986  1.00 11.09           H  \nATOM    479  HE1 MET A  34      -3.560 -11.684  23.859  1.00 13.82           H  \nATOM    480  HE2 MET A  34      -2.975 -13.019  24.458  1.00 13.82           H  \nATOM    481  HE3 MET A  34      -4.521 -12.726  24.548  1.00 13.82           H  \nATOM    482  N   SER A  35       0.982 -13.615  24.906  1.00  8.91           N  \nATOM    483  CA  SER A  35       2.023 -14.295  24.153  1.00  8.94           C  \nATOM    484  C   SER A  35       1.477 -14.868  22.851  1.00  9.00           C  \nATOM    485  O   SER A  35       0.287 -15.147  22.718  1.00  9.78           O  \nATOM    486  CB  SER A  35       2.653 -15.416  24.928  1.00  8.96           C  \nATOM    487  OG  SER A  35       3.051 -14.980  26.229  1.00 10.43           O  \nATOM    488  H   SER A  35       0.143 -13.979  24.917  1.00  8.92           H  \nATOM    489  HA  SER A  35       2.727 -13.631  23.928  1.00  8.95           H  \nATOM    490  HB2 SER A  35       2.009 -16.156  25.013  1.00  9.28           H  \nATOM    491  HB3 SER A  35       3.439 -15.749  24.439  1.00  9.29           H  \nATOM    492  HG  SER A  35       2.626 -14.914  26.782  0.00 10.70           H  \nATOM    493  N   TRP A  36       2.403 -15.112  21.938  1.00  8.76           N  \nATOM    494  CA  TRP A  36       2.179 -15.967  20.789  1.00  8.36           C  \nATOM    495  C   TRP A  36       2.946 -17.273  21.017  1.00  8.26           C  \nATOM    496  O   TRP A  36       4.115 -17.232  21.352  1.00  9.21           O  \nATOM    497  CB  TRP A  36       2.686 -15.311  19.489  1.00  9.89           C  \nATOM    498  CG  TRP A  36       1.854 -14.154  19.026  1.00 10.44           C  \nATOM    499  CD1 TRP A  36       2.105 -12.842  19.252  1.00 10.82           C  \nATOM    500  CD2 TRP A  36       0.623 -14.211  18.271  1.00 10.39           C  \nATOM    501  NE1 TRP A  36       1.112 -12.080  18.711  1.00 10.86           N  \nATOM    502  CE2 TRP A  36       0.207 -12.882  18.062  1.00 10.35           C  \nATOM    503  CE3 TRP A  36      -0.123 -15.254  17.704  1.00 11.48           C  \nATOM    504  CZ2 TRP A  36      -0.925 -12.563  17.321  1.00 11.38           C  \nATOM    505  CZ3 TRP A  36      -1.258 -14.937  16.999  1.00 11.72           C  \nATOM    506  CH2 TRP A  36      -1.640 -13.609  16.799  1.00 10.73           C  \nATOM    507  H   TRP A  36       3.225 -14.724  22.033  1.00  8.72           H  \nATOM    508  HA  TRP A  36       1.217 -16.148  20.703  1.00  8.70           H  \nATOM    509  HB2 TRP A  36       3.605 -15.006  19.636  1.00  9.63           H  \nATOM    510  HB3 TRP A  36       2.705 -15.993  18.786  1.00  9.63           H  \nATOM    511  HD1 TRP A  36       2.826 -12.503  19.758  1.00 10.76           H  \nATOM    512  HE1 TRP A  36       1.093 -11.207  18.723  1.00 10.76           H  \nATOM    513  HE3 TRP A  36       0.143 -16.153  17.816  1.00 11.21           H  \nATOM    514  HZ2 TRP A  36      -1.206 -11.671  17.216  1.00 10.97           H  \nATOM    515  HZ3 TRP A  36      -1.774 -15.629  16.615  1.00 11.34           H  \nATOM    516  HH2 TRP A  36      -2.431 -13.427  16.323  1.00 11.07           H  \nATOM    517  N   VAL A  37       2.238 -18.407  20.880  1.00  8.61           N  \nATOM    518  CA  VAL A  37       2.778 -19.760  21.019  1.00  9.12           C  \nATOM    519  C   VAL A  37       2.409 -20.544  19.756  1.00  9.52           C  \nATOM    520  O   VAL A  37       1.233 -20.540  19.387  1.00 11.21           O  \nATOM    521  CB  VAL A  37       2.189 -20.441  22.263  1.00  9.67           C  \nATOM    522  CG1 VAL A  37       2.732 -21.841  22.436  1.00 10.95           C  \nATOM    523  CG2 VAL A  37       2.435 -19.587  23.521  1.00 10.66           C  \nATOM    524  H   VAL A  37       1.348 -18.328  20.683  1.00  8.65           H  \nATOM    525  HA  VAL A  37       3.746 -19.713  21.103  1.00  9.20           H  \nATOM    526  HB  VAL A  37       1.212 -20.510  22.130  1.00  9.97           H  \nATOM    527 HG11 VAL A  37       3.670 -21.861  22.177  1.00 10.53           H  \nATOM    528 HG12 VAL A  37       2.229 -22.456  21.875  1.00 10.53           H  \nATOM    529 HG13 VAL A  37       2.648 -22.111  23.367  1.00 10.53           H  \nATOM    530 HG21 VAL A  37       2.199 -20.099  24.314  1.00 10.34           H  \nATOM    531 HG22 VAL A  37       1.887 -18.784  23.481  1.00 10.34           H  \nATOM    532 HG23 VAL A  37       3.374 -19.336  23.565  1.00 10.34           H  \nATOM    533  N   ARG A  38       3.399 -21.180  19.151  1.00  9.75           N  \nATOM    534  CA  ARG A  38       3.126 -21.914  17.918  1.00  9.34           C  \nATOM    535  C   ARG A  38       3.260 -23.405  18.185  1.00  9.00           C  \nATOM    536  O   ARG A  38       4.009 -23.838  19.037  1.00 10.68           O  \nATOM    537  CB  ARG A  38       4.066 -21.471  16.796  1.00  9.50           C  \nATOM    538  CG  ARG A  38       5.518 -21.835  16.911  1.00  9.68           C  \nATOM    539  CD  ARG A  38       6.301 -21.287  15.741  1.00 10.36           C  \nATOM    540  NE  ARG A  38       7.697 -21.525  15.911  1.00 11.78           N  \nATOM    541  CZ  ARG A  38       8.647 -20.911  15.192  1.00 12.60           C  \nATOM    542  NH1 ARG A  38       8.321 -20.006  14.284  1.00 13.35           N  \nATOM    543  NH2 ARG A  38       9.922 -21.161  15.445  1.00 15.47           N  \nATOM    544  H   ARG A  38       4.246 -21.167  19.494  1.00  9.60           H  \nATOM    545  HA  ARG A  38       2.203 -21.725  17.644  1.00  9.39           H  \nATOM    546  HB2 ARG A  38       3.726 -21.843  15.957  1.00  9.51           H  \nATOM    547  HB3 ARG A  38       4.005 -20.495  16.724  1.00  9.51           H  \nATOM    548  HG2 ARG A  38       5.884 -21.471  17.745  1.00  9.79           H  \nATOM    549  HG3 ARG A  38       5.614 -22.811  16.933  1.00  9.79           H  \nATOM    550  HD2 ARG A  38       5.995 -21.714  14.913  1.00 10.51           H  \nATOM    551  HD3 ARG A  38       6.140 -20.322  15.665  1.00 10.53           H  \nATOM    552  HE  ARG A  38       7.948 -22.103  16.515  1.00 11.61           H  \nATOM    553 HH11 ARG A  38       7.459 -19.833  14.197  0.00 12.98           H  \nATOM    554 HH12 ARG A  38       8.928 -19.582  13.860  0.00 12.98           H  \nATOM    555 HH21 ARG A  38      10.446 -21.454  14.803  1.00 14.52           H  \nATOM    556 HH22 ARG A  38      10.241 -21.033  16.255  1.00 14.54           H  \nATOM    557  N   GLN A  39       2.488 -24.169  17.413  1.00  8.88           N  \nATOM    558  CA  GLN A  39       2.527 -25.630  17.469  1.00 10.02           C  \nATOM    559  C   GLN A  39       2.982 -26.116  16.094  1.00 10.43           C  \nATOM    560  O   GLN A  39       2.269 -25.940  15.099  1.00 10.97           O  \nATOM    561  CB  GLN A  39       1.167 -26.182  17.823  1.00 10.99           C  \nATOM    562  CG  GLN A  39       1.236 -27.694  18.038  1.00 11.94           C  \nATOM    563  CD  GLN A  39       0.095 -28.200  18.844  1.00 13.72           C  \nATOM    564  OE1 GLN A  39      -1.044 -27.799  18.600  1.00 17.74           O  \nATOM    565  NE2 GLN A  39       0.402 -29.051  19.833  1.00 12.29           N  \nATOM    566  H   GLN A  39       1.914 -23.770  16.824  1.00  9.16           H  \nATOM    567  HA  GLN A  39       3.177 -25.907  18.148  1.00 10.05           H  \nATOM    568  HB2 GLN A  39       0.847 -25.747  18.641  1.00 10.97           H  \nATOM    569  HB3 GLN A  39       0.538 -25.980  17.099  1.00 10.96           H  \nATOM    570  HG2 GLN A  39       1.237 -28.144  17.166  1.00 12.09           H  \nATOM    571  HG3 GLN A  39       2.075 -27.919  18.493  1.00 12.09           H  \nATOM    572 HE21 GLN A  39       1.221 -29.296  19.990  0.00 12.31           H  \nATOM    573 HE22 GLN A  39      -0.258 -29.395  20.371  0.00 12.31           H  \nATOM    574  N   THR A  40       4.141 -26.758  16.098  1.00 11.33           N  \nATOM    575  CA  THR A  40       4.807 -27.201  14.875  1.00 11.57           C  \nATOM    576  C   THR A  40       4.197 -28.545  14.469  1.00 12.36           C  \nATOM    577  O   THR A  40       3.442 -29.176  15.179  1.00 11.71           O  \nATOM    578  CB  THR A  40       6.326 -27.180  15.033  1.00 13.25           C  \nATOM    579  OG1 THR A  40       6.666 -27.703  16.317  1.00 15.17           O  \nATOM    580  CG2 THR A  40       6.877 -25.779  14.893  1.00 15.34           C  \nATOM    581  H   THR A  40       4.545 -26.935  16.898  1.00 11.16           H  \nATOM    582  HA  THR A  40       4.578 -26.549  14.174  1.00 11.98           H  \nATOM    583  HB  THR A  40       6.724 -27.753  14.338  1.00 13.62           H  \nATOM    584  HG1 THR A  40       6.292 -27.230  16.922  0.00 14.81           H  \nATOM    585 HG21 THR A  40       7.836 -25.788  15.061  1.00 14.67           H  \nATOM    586 HG22 THR A  40       6.441 -25.192  15.537  1.00 14.67           H  \nATOM    587 HG23 THR A  40       6.709 -25.451  13.992  1.00 14.67           H  \nATOM    588  N   PRO A  41       4.491 -29.012  13.242  1.00 12.84           N  \nATOM    589  CA  PRO A  41       3.868 -30.258  12.798  1.00 12.59           C  \nATOM    590  C   PRO A  41       4.205 -31.499  13.625  1.00 12.93           C  \nATOM    591  O   PRO A  41       3.392 -32.426  13.574  1.00 14.87           O  \nATOM    592  CB  PRO A  41       4.387 -30.397  11.356  1.00 12.94           C  \nATOM    593  CG  PRO A  41       4.698 -28.991  10.907  1.00 13.08           C  \nATOM    594  CD  PRO A  41       5.191 -28.295  12.168  1.00 13.28           C  \nATOM    595  HA  PRO A  41       2.886 -30.134  12.774  1.00 12.77           H  \nATOM    596  HB2 PRO A  41       5.193 -30.955  11.328  1.00 12.89           H  \nATOM    597  HB3 PRO A  41       3.705 -30.801  10.779  1.00 12.89           H  \nATOM    598  HG2 PRO A  41       5.390 -28.990  10.215  1.00 13.09           H  \nATOM    599  HG3 PRO A  41       3.897 -28.551  10.557  1.00 13.09           H  \nATOM    600  HD2 PRO A  41       6.158 -28.380  12.260  1.00 13.13           H  \nATOM    601  HD3 PRO A  41       4.953 -27.351  12.161  1.00 13.12           H  \nATOM    602  N   ASP A  42       5.293 -31.459  14.381  1.00 13.04           N  \nATOM    603  CA  ASP A  42       5.668 -32.525  15.323  1.00 12.72           C  \nATOM    604  C   ASP A  42       4.936 -32.415  16.665  1.00 13.15           C  \nATOM    605  O   ASP A  42       5.222 -33.186  17.585  1.00 13.70           O  \nATOM    606  CB  ASP A  42       7.179 -32.493  15.568  1.00 13.60           C  \nATOM    607  CG  ASP A  42       7.690 -31.220  16.243  1.00 14.54           C  \nATOM    608  OD1 ASP A  42       6.849 -30.304  16.473  1.00 15.04           O  \nATOM    609  OD2 ASP A  42       8.931 -31.087  16.434  1.00 19.08           O  \nATOM    610  H   ASP A  42       5.851 -30.740  14.302  1.00 12.94           H  \nATOM    611  HA  ASP A  42       5.441 -33.393  14.913  1.00 13.03           H  \nATOM    612  HB2 ASP A  42       7.425 -33.257  16.129  1.00 13.60           H  \nATOM    613  HB3 ASP A  42       7.639 -32.588  14.709  1.00 13.60           H  \nATOM    614  N   LYS A  43       3.968 -31.486  16.742  1.00 12.70           N  \nATOM    615  CA  LYS A  43       3.072 -31.252  17.879  1.00 12.90           C  \nATOM    616  C   LYS A  43       3.788 -30.564  19.043  1.00 11.47           C  \nATOM    617  O   LYS A  43       3.146 -30.349  20.074  1.00 12.98           O  \nATOM    618  CB  LYS A  43       2.328 -32.499  18.348  1.00 16.12           C  \nATOM    619  CG  LYS A  43       1.533 -33.160  17.220  1.00 20.21           C  \nATOM    620  CD  LYS A  43       0.471 -32.291  16.677  1.00 27.26           C  \nATOM    621  CE  LYS A  43      -0.612 -33.067  15.950  1.00 34.99           C  \nATOM    622  NZ  LYS A  43      -1.361 -32.173  15.037  1.00 43.00           N  \nATOM    623  H   LYS A  43       3.863 -30.936  16.021  1.00 12.85           H  \nATOM    624  HA  LYS A  43       2.388 -30.619  17.561  1.00 13.16           H  \nATOM    625  HB2 LYS A  43       2.973 -33.143  18.706  1.00 16.12           H  \nATOM    626  HB3 LYS A  43       1.711 -32.251  19.068  1.00 16.14           H  \nATOM    627  HG2 LYS A  43       2.150 -33.403  16.497  1.00 20.55           H  \nATOM    628  HG3 LYS A  43       1.129 -33.986  17.561  1.00 20.59           H  \nATOM    629  HD2 LYS A  43       0.061 -31.787  17.412  1.00 26.86           H  \nATOM    630  HD3 LYS A  43       0.871 -31.647  16.055  1.00 26.87           H  \nATOM    631  HE2 LYS A  43      -0.212 -33.791  15.433  1.00 34.59           H  \nATOM    632  HE3 LYS A  43      -1.231 -33.458  16.596  1.00 34.62           H  \nATOM    633  HZ1 LYS A  43      -0.814 -31.882  14.374  1.00 40.30           H  \nATOM    634  HZ2 LYS A  43      -1.676 -31.457  15.498  1.00 40.31           H  \nATOM    635  HZ3 LYS A  43      -2.059 -32.621  14.672  1.00 40.33           H  \nATOM    636  N   ARG A  44       5.033 -30.143  18.864  1.00 12.59           N  \nATOM    637  CA  ARG A  44       5.694 -29.343  19.905  1.00 12.84           C  \nATOM    638  C   ARG A  44       5.036 -27.970  19.973  1.00 12.35           C  \nATOM    639  O   ARG A  44       4.628 -27.419  18.968  1.00 12.44           O  \nATOM    640  CB  ARG A  44       7.192 -29.163  19.680  1.00 14.46           C  \nATOM    641  CG  ARG A  44       8.031 -30.431  19.859  1.00 18.47           C  \nATOM    642  CD  ARG A  44       9.495 -30.128  19.678  1.00 23.21           C  \nATOM    643  NE  ARG A  44       9.688 -29.038  18.703  1.00 38.40           N  \nATOM    644  CZ  ARG A  44      10.197 -29.144  17.472  1.00 38.76           C  \nATOM    645  NH1 ARG A  44      10.997 -30.145  17.152  1.00 57.03           N  \nATOM    646  NH2 ARG A  44       9.919 -28.233  16.561  1.00 43.89           N  \nATOM    647  H   ARG A  44       5.487 -30.336  18.095  1.00 12.38           H  \nATOM    648  HA  ARG A  44       5.561 -29.797  20.768  1.00 12.98           H  \nATOM    649  HB2 ARG A  44       7.323 -28.826  18.769  1.00 14.98           H  \nATOM    650  HB3 ARG A  44       7.520 -28.480  20.302  1.00 14.94           H  \nATOM    651  HG2 ARG A  44       7.884 -30.800  20.757  1.00 18.40           H  \nATOM    652  HG3 ARG A  44       7.754 -31.102  19.202  1.00 18.40           H  \nATOM    653  HD2 ARG A  44       9.882 -29.864  20.541  1.00 24.83           H  \nATOM    654  HD3 ARG A  44       9.964 -30.925  19.369  1.00 24.88           H  \nATOM    655  HE  ARG A  44       9.473 -28.234  18.965  1.00 33.89           H  \nATOM    656 HH11 ARG A  44      11.109 -30.557  17.690  0.00 22.19           H  \nATOM    657 HH12 ARG A  44      11.528 -29.784  16.440  0.00 22.19           H  \nATOM    658 HH21 ARG A  44      10.558 -27.924  16.042  1.00 42.24           H  \nATOM    659 HH22 ARG A  44       9.096 -27.938  16.474  1.00 42.24           H  \nATOM    660  N   LEU A  45       5.043 -27.393  21.195  1.00 10.83           N  \nATOM    661  CA  LEU A  45       4.624 -26.032  21.472  1.00 10.33           C  \nATOM    662  C   LEU A  45       5.852 -25.209  21.797  1.00 10.69           C  \nATOM    663  O   LEU A  45       6.746 -25.627  22.524  1.00 12.37           O  \nATOM    664  CB  LEU A  45       3.647 -26.017  22.641  1.00 10.14           C  \nATOM    665  CG  LEU A  45       2.282 -26.665  22.378  1.00 10.83           C  \nATOM    666  CD1 LEU A  45       1.585 -27.098  23.657  1.00 11.62           C  \nATOM    667  CD2 LEU A  45       1.389 -25.736  21.579  1.00 11.26           C  \nATOM    668  H   LEU A  45       5.334 -27.897  21.900  1.00 11.06           H  \nATOM    669  HA  LEU A  45       4.189 -25.665  20.671  1.00 10.46           H  \nATOM    670  HB2 LEU A  45       4.067 -26.477  23.395  1.00 10.35           H  \nATOM    671  HB3 LEU A  45       3.502 -25.087  22.907  1.00 10.35           H  \nATOM    672  HG  LEU A  45       2.435 -27.476  21.833  1.00 10.92           H  \nATOM    673 HD11 LEU A  45       0.744 -27.535  23.437  1.00 11.37           H  \nATOM    674 HD12 LEU A  45       1.408 -26.317  24.211  1.00 11.37           H  \nATOM    675 HD13 LEU A  45       2.154 -27.718  24.144  1.00 11.37           H  \nATOM    676 HD21 LEU A  45       1.855 -25.450  20.774  1.00 11.12           H  \nATOM    677 HD22 LEU A  45       1.166 -24.957  22.118  1.00 11.13           H  \nATOM    678 HD23 LEU A  45       0.572 -26.203  21.332  1.00 11.13           H  \nATOM    679  N   GLU A  46       5.896 -24.016  21.206  1.00 10.42           N  \nATOM    680  CA  GLU A  46       7.059 -23.148  21.319  1.00 11.27           C  \nATOM    681  C   GLU A  46       6.531 -21.713  21.500  1.00 10.86           C  \nATOM    682  O   GLU A  46       5.854 -21.184  20.620  1.00 10.90           O  \nATOM    683  CB  GLU A  46       7.972 -23.228  20.081  1.00 15.04           C  \nATOM    684  CG  GLU A  46       9.234 -22.367  20.167  1.00 18.81           C  \nATOM    685  CD  GLU A  46      10.095 -22.320  18.905  1.00 26.44           C  \nATOM    686  OE1 GLU A  46      11.227 -21.770  18.983  1.00 31.70           O  \nATOM    687  OE2 GLU A  46       9.600 -22.790  17.847  1.00 28.77           O  \nATOM    688  H   GLU A  46       5.180 -23.729  20.716  1.00 10.68           H  \nATOM    689  HA  GLU A  46       7.574 -23.404  22.118  1.00 11.65           H  \nATOM    690  HB2 GLU A  46       8.236 -24.163  19.950  1.00 14.81           H  \nATOM    691  HB3 GLU A  46       7.454 -22.952  19.297  1.00 14.83           H  \nATOM    692  HG2 GLU A  46       8.975 -21.450  20.391  1.00 19.31           H  \nATOM    693  HG3 GLU A  46       9.790 -22.700  20.904  1.00 19.34           H  \nATOM    694  N   LEU A  47       6.953 -21.049  22.562  1.00 10.32           N  \nATOM    695  CA  LEU A  47       6.701 -19.622  22.664  1.00 11.29           C  \nATOM    696  C   LEU A  47       7.525 -18.899  21.624  1.00 11.10           C  \nATOM    697  O   LEU A  47       8.708 -19.157  21.525  1.00 12.64           O  \nATOM    698  CB  LEU A  47       7.017 -19.176  24.084  1.00 11.94           C  \nATOM    699  CG  LEU A  47       6.629 -17.725  24.360  1.00 13.93           C  \nATOM    700  CD1 LEU A  47       6.044 -17.523  25.718  1.00 16.99           C  \nATOM    701  CD2 LEU A  47       7.879 -16.934  24.204  1.00 15.25           C  \nATOM    702  H   LEU A  47       7.400 -21.479  23.234  1.00 10.68           H  \nATOM    703  HA  LEU A  47       5.747 -19.461  22.480  1.00 11.18           H  \nATOM    704  HB2 LEU A  47       6.537 -19.757  24.708  1.00 12.24           H  \nATOM    705  HB3 LEU A  47       7.975 -19.290  24.244  1.00 12.24           H  \nATOM    706  HG  LEU A  47       5.975 -17.430  23.679  1.00 14.28           H  \nATOM    707 HD11 LEU A  47       5.756 -16.598  25.813  1.00 15.98           H  \nATOM    708 HD12 LEU A  47       6.713 -17.725  26.394  1.00 15.97           H  \nATOM    709 HD13 LEU A  47       5.279 -18.113  25.833  1.00 15.95           H  \nATOM    710 HD21 LEU A  47       8.277 -17.115  23.336  1.00 14.81           H  \nATOM    711 HD22 LEU A  47       8.507 -17.178  24.906  1.00 14.82           H  \nATOM    712 HD23 LEU A  47       7.672 -15.986  24.273  1.00 14.83           H  \nATOM    713  N   VAL A  48       6.908 -17.973  20.881  1.00 10.14           N  \nATOM    714  CA  VAL A  48       7.711 -17.190  19.945  1.00 11.91           C  \nATOM    715  C   VAL A  48       7.768 -15.713  20.284  1.00 10.24           C  \nATOM    716  O   VAL A  48       8.631 -15.034  19.743  1.00 11.00           O  \nATOM    717  CB  VAL A  48       7.311 -17.393  18.494  1.00 15.21           C  \nATOM    718  CG1 VAL A  48       7.587 -18.814  18.044  1.00 17.84           C  \nATOM    719  CG2 VAL A  48       5.887 -17.000  18.315  1.00 15.00           C  \nATOM    720  H   VAL A  48       6.007 -17.832  20.947  1.00 10.78           H  \nATOM    721  HA  VAL A  48       8.624 -17.518  20.023  1.00 11.78           H  \nATOM    722  HB  VAL A  48       7.866 -16.787  17.943  1.00 14.90           H  \nATOM    723 HG11 VAL A  48       7.027 -19.429  18.548  1.00 16.98           H  \nATOM    724 HG12 VAL A  48       8.523 -19.028  18.198  1.00 16.96           H  \nATOM    725 HG13 VAL A  48       7.387 -18.898  17.095  1.00 16.96           H  \nATOM    726 HG21 VAL A  48       5.677 -16.968  17.366  1.00 15.05           H  \nATOM    727 HG22 VAL A  48       5.736 -16.123  18.708  1.00 15.04           H  \nATOM    728 HG23 VAL A  48       5.312 -17.653  18.750  1.00 15.03           H  \nATOM    729  N   ALA A  49       6.841 -15.210  21.116  1.00  9.14           N  \nATOM    730  CA  ALA A  49       6.932 -13.808  21.516  1.00  9.36           C  \nATOM    731  C   ALA A  49       6.048 -13.579  22.715  1.00  8.38           C  \nATOM    732  O   ALA A  49       5.005 -14.199  22.869  1.00  9.69           O  \nATOM    733  CB  ALA A  49       6.533 -12.912  20.382  1.00  8.98           C  \nATOM    734  H   ALA A  49       6.153 -15.726  21.426  1.00  9.44           H  \nATOM    735  HA  ALA A  49       7.863 -13.612  21.768  1.00  9.06           H  \nATOM    736  HB1 ALA A  49       6.372 -12.015  20.719  1.00  9.10           H  \nATOM    737  HB2 ALA A  49       7.246 -12.883  19.722  1.00  9.11           H  \nATOM    738  HB3 ALA A  49       5.723 -13.252  19.967  1.00  9.10           H  \nATOM    739  N   THR A  50       6.443 -12.609  23.577  1.00  8.32           N  \nATOM    740  CA  THR A  50       5.650 -12.292  24.748  1.00  9.02           C  \nATOM    741  C   THR A  50       5.785 -10.796  25.033  1.00  8.62           C  \nATOM    742  O   THR A  50       6.761 -10.147  24.635  1.00  8.87           O  \nATOM    743  CB  THR A  50       6.051 -13.194  25.912  1.00 11.53           C  \nATOM    744  OG1 THR A  50       4.990 -13.071  26.864  1.00 14.20           O  \nATOM    745  CG2 THR A  50       7.381 -12.816  26.503  1.00 11.75           C  \nATOM    746  H   THR A  50       7.218 -12.149  23.424  1.00  8.50           H  \nATOM    747  HA  THR A  50       4.707 -12.476  24.530  1.00  9.24           H  \nATOM    748  HB  THR A  50       6.093 -14.126  25.590  1.00 11.48           H  \nATOM    749  HG1 THR A  50       4.938 -12.290  27.133  0.00 14.58           H  \nATOM    750 HG21 THR A  50       7.758 -13.580  26.974  1.00 11.68           H  \nATOM    751 HG22 THR A  50       7.263 -12.077  27.127  1.00 11.67           H  \nATOM    752 HG23 THR A  50       7.986 -12.544  25.791  1.00 11.66           H  \nATOM    753  N   ILE A  51       4.707 -10.246  25.593  1.00  9.26           N  \nATOM    754  CA  ILE A  51       4.608  -8.804  25.787  1.00  9.21           C  \nATOM    755  C   ILE A  51       3.941  -8.553  27.136  1.00  9.00           C  \nATOM    756  O   ILE A  51       3.025  -9.272  27.538  1.00 10.17           O  \nATOM    757  CB  ILE A  51       3.866  -8.117  24.624  1.00  9.21           C  \nATOM    758  CG1 ILE A  51       3.989  -6.578  24.690  1.00 10.36           C  \nATOM    759  CG2 ILE A  51       2.444  -8.586  24.471  1.00 10.59           C  \nATOM    760  CD1 ILE A  51       3.736  -5.932  23.365  1.00 10.75           C  \nATOM    761  H   ILE A  51       4.015 -10.778  25.863  1.00  9.09           H  \nATOM    762  HA  ILE A  51       5.509  -8.440  25.824  1.00  9.18           H  \nATOM    763  HB  ILE A  51       4.338  -8.393  23.800  1.00  9.68           H  \nATOM    764 HG12 ILE A  51       3.344  -6.234  25.343  1.00 10.17           H  \nATOM    765 HG13 ILE A  51       4.890  -6.340  24.994  1.00 10.16           H  \nATOM    766 HG21 ILE A  51       1.906  -8.248  25.208  1.00 10.15           H  \nATOM    767 HG22 ILE A  51       2.418  -9.558  24.473  1.00 10.15           H  \nATOM    768 HG23 ILE A  51       2.081  -8.257  23.631  1.00 10.16           H  \nATOM    769 HD11 ILE A  51       2.798  -6.033  23.127  1.00 10.63           H  \nATOM    770 HD12 ILE A  51       4.288  -6.354  22.684  1.00 10.63           H  \nATOM    771 HD13 ILE A  51       3.955  -4.985  23.416  1.00 10.63           H  \nATOM    772  N   ASN A  52       4.356  -7.504  27.839  1.00  8.78           N  \nATOM    773  CA  ASN A  52       3.702  -7.118  29.062  1.00  9.32           C  \nATOM    774  C   ASN A  52       2.380  -6.385  28.797  1.00  9.82           C  \nATOM    775  O   ASN A  52       2.026  -6.055  27.664  1.00 10.85           O  \nATOM    776  CB  ASN A  52       4.654  -6.318  29.952  1.00  9.94           C  \nATOM    777  CG  ASN A  52       4.987  -4.945  29.428  1.00 11.59           C  \nATOM    778  OD1 ASN A  52       4.733  -4.614  28.275  1.00 10.40           O  \nATOM    779  ND2 ASN A  52       5.635  -4.118  30.278  1.00 13.43           N  \nATOM    780  H   ASN A  52       5.069  -7.015  27.544  1.00  8.96           H  \nATOM    781  HA  ASN A  52       3.488  -7.945  29.554  1.00  9.42           H  \nATOM    782  HB2 ASN A  52       4.251  -6.228  30.840  1.00 10.15           H  \nATOM    783  HB3 ASN A  52       5.485  -6.826  30.053  1.00 10.15           H  \nATOM    784 HD21 ASN A  52       5.821  -4.376  31.061  0.00 13.41           H  \nATOM    785 HD22 ASN A  52       5.836  -3.301  29.971  0.00 13.41           H  \nATOM    786  N   SER A  53       1.684  -6.077  29.868  1.00 10.24           N  \nATOM    787  CA  SER A  53       0.308  -5.561  29.789  1.00 10.12           C  \nATOM    788  C   SER A  53       0.254  -4.200  29.104  1.00 10.45           C  \nATOM    789  O   SER A  53      -0.620  -3.947  28.247  1.00 11.31           O  \nATOM    790  CB  SER A  53      -0.352  -5.516  31.133  1.00 12.16           C  \nATOM    791  OG  SER A  53       0.213  -4.565  32.017  1.00 15.94           O  \nATOM    792  H   SER A  53       2.058  -6.193  30.693  1.00 10.12           H  \nATOM    793  HA  SER A  53      -0.209  -6.196  29.224  1.00 10.56           H  \nATOM    794  HB2 SER A  53      -1.303  -5.313  31.011  1.00 12.42           H  \nATOM    795  HB3 SER A  53      -0.290  -6.408  31.545  1.00 12.41           H  \nATOM    796  HG  SER A  53      -0.074  -3.779  31.870  0.00  9.21           H  \nATOM    797  N   ASN A  54       1.219  -3.327  29.418  1.00 10.90           N  \nATOM    798  CA  ASN A  54       1.174  -1.994  28.820  1.00 12.08           C  \nATOM    799  C   ASN A  54       1.847  -1.897  27.459  1.00 12.41           C  \nATOM    800  O   ASN A  54       1.768  -0.858  26.805  1.00 14.12           O  \nATOM    801  CB  ASN A  54       1.767  -0.967  29.779  1.00 12.73           C  \nATOM    802  CG  ASN A  54       3.253  -1.087  30.045  1.00 14.72           C  \nATOM    803  OD1 ASN A  54       3.992  -1.570  29.216  1.00 15.77           O  \nATOM    804  ND2 ASN A  54       3.733  -0.535  31.160  1.00 17.97           N  \nATOM    805  H   ASN A  54       1.891  -3.548  29.996  1.00 11.06           H  \nATOM    806  HA  ASN A  54       0.225  -1.753  28.702  1.00 12.02           H  \nATOM    807  HB2 ASN A  54       1.589  -0.074  29.419  1.00 13.01           H  \nATOM    808  HB3 ASN A  54       1.294  -1.038  30.634  1.00 13.01           H  \nATOM    809 HD21 ASN A  54       3.175  -0.122  31.685  0.00 18.26           H  \nATOM    810 HD22 ASN A  54       4.585  -0.590  31.317  0.00 18.26           H  \nATOM    811  N   GLY A  55       2.368  -3.013  26.933  1.00 10.52           N  \nATOM    812  CA  GLY A  55       2.933  -3.058  25.600  1.00 10.47           C  \nATOM    813  C   GLY A  55       4.321  -2.479  25.421  1.00 11.59           C  \nATOM    814  O   GLY A  55       4.766  -2.392  24.271  1.00 13.11           O  \nATOM    815  H   GLY A  55       2.381  -3.777  27.433  1.00 10.92           H  \nATOM    816  HA2 GLY A  55       2.950  -4.002  25.309  1.00 10.74           H  \nATOM    817  HA3 GLY A  55       2.315  -2.585  24.987  1.00 10.74           H  \nATOM    818  N   GLY A  56       4.961  -2.108  26.531  1.00 10.20           N  \nATOM    819  CA  GLY A  56       6.244  -1.458  26.428  1.00 11.37           C  \nATOM    820  C   GLY A  56       7.431  -2.386  26.468  1.00 12.10           C  \nATOM    821  O   GLY A  56       8.514  -1.924  26.107  1.00 14.84           O  \nATOM    822  H   GLY A  56       4.604  -2.272  27.357  1.00 10.79           H  \nATOM    823  HA2 GLY A  56       6.271  -0.945  25.587  1.00 11.25           H  \nATOM    824  HA3 GLY A  56       6.324  -0.808  27.170  1.00 11.25           H  \nATOM    825  N   SER A  57       7.245  -3.644  26.874  1.00 11.50           N  \nATOM    826  CA  SER A  57       8.391  -4.527  26.976  1.00 11.48           C  \nATOM    827  C   SER A  57       8.038  -5.851  26.314  1.00 10.69           C  \nATOM    828  O   SER A  57       6.934  -6.356  26.517  1.00 10.09           O  \nATOM    829  CB  SER A  57       8.737  -4.757  28.392  1.00 12.25           C  \nATOM    830  OG  SER A  57       9.163  -3.538  29.036  1.00 16.77           O  \nATOM    831  H   SER A  57       6.413  -3.954  27.089  1.00 11.63           H  \nATOM    832  HA  SER A  57       9.163  -4.118  26.505  1.00 11.47           H  \nATOM    833  HB2 SER A  57       7.953  -5.119  28.863  1.00 12.99           H  \nATOM    834  HB3 SER A  57       9.460  -5.423  28.445  1.00 12.99           H  \nATOM    835  HG  SER A  57       8.474  -3.138  29.374  0.00 17.03           H  \nATOM    836  N   THR A  58       9.026  -6.378  25.611  1.00 11.13           N  \nATOM    837  CA  THR A  58       8.844  -7.564  24.780  1.00 11.45           C  \nATOM    838  C   THR A  58      10.013  -8.514  24.915  1.00 10.72           C  \nATOM    839  O   THR A  58      11.118  -8.109  25.248  1.00 11.37           O  \nATOM    840  CB  THR A  58       8.711  -7.189  23.296  1.00 12.51           C  \nATOM    841  OG1 THR A  58       9.932  -6.540  22.903  1.00 14.07           O  \nATOM    842  CG2 THR A  58       7.505  -6.322  23.010  1.00 13.74           C  \nATOM    843  H   THR A  58       9.854  -5.994  25.646  1.00 11.10           H  \nATOM    844  HA  THR A  58       8.024  -8.025  25.067  1.00 11.43           H  \nATOM    845  HB  THR A  58       8.622  -8.025  22.779  1.00 12.80           H  \nATOM    846  HG1 THR A  58      10.043  -5.808  23.414  0.00 14.05           H  \nATOM    847 HG21 THR A  58       7.402  -6.216  22.047  1.00 13.35           H  \nATOM    848 HG22 THR A  58       7.628  -5.448  23.420  1.00 13.35           H  \nATOM    849 HG23 THR A  58       6.707  -6.743  23.376  1.00 13.35           H  \nATOM    850  N   PHE A  59       9.738  -9.808  24.658  1.00  9.80           N  \nATOM    851  CA  PHE A  59      10.750 -10.851  24.700  1.00 10.11           C  \nATOM    852  C   PHE A  59      10.515 -11.850  23.577  1.00 10.49           C  \nATOM    853  O   PHE A  59       9.380 -12.231  23.326  1.00 10.05           O  \nATOM    854  CB  PHE A  59      10.770 -11.553  26.059  1.00 11.04           C  \nATOM    855  CG  PHE A  59      11.692 -12.740  26.081  1.00 11.71           C  \nATOM    856  CD1 PHE A  59      13.051 -12.587  26.320  1.00 13.32           C  \nATOM    857  CD2 PHE A  59      11.200 -14.000  25.740  1.00 13.31           C  \nATOM    858  CE1 PHE A  59      13.883 -13.707  26.256  1.00 14.47           C  \nATOM    859  CE2 PHE A  59      12.035 -15.101  25.685  1.00 13.80           C  \nATOM    860  CZ  PHE A  59      13.366 -14.941  25.942  1.00 15.19           C  \nATOM    861  H   PHE A  59       8.880 -10.043  24.447  1.00 10.09           H  \nATOM    862  HA  PHE A  59      11.634 -10.431  24.556  1.00 10.31           H  \nATOM    863  HB2 PHE A  59      11.057 -10.911  26.740  1.00 10.97           H  \nATOM    864  HB3 PHE A  59       9.864 -11.844  26.280  1.00 10.97           H  \nATOM    865  HD1 PHE A  59      13.406 -11.738  26.526  1.00 13.22           H  \nATOM    866  HD2 PHE A  59      10.281 -14.103  25.552  1.00 13.08           H  \nATOM    867  HE1 PHE A  59      14.804 -13.618  26.443  1.00 14.24           H  \nATOM    868  HE2 PHE A  59      11.688 -15.952  25.476  1.00 13.92           H  \nATOM    869  HZ  PHE A  59      13.936 -15.693  25.922  1.00 14.54           H  \nATOM    870  N   TYR A  60      11.622 -12.302  22.975  1.00 10.17           N  \nATOM    871  CA  TYR A  60      11.620 -13.298  21.908  1.00 10.92           C  \nATOM    872  C   TYR A  60      12.721 -14.298  22.174  1.00 12.12           C  \nATOM    873  O   TYR A  60      13.830 -13.897  22.500  1.00 13.73           O  \nATOM    874  CB  TYR A  60      11.965 -12.688  20.558  1.00 11.95           C  \nATOM    875  CG  TYR A  60      11.134 -11.481  20.264  1.00 11.87           C  \nATOM    876  CD1 TYR A  60       9.902 -11.576  19.643  1.00 12.55           C  \nATOM    877  CD2 TYR A  60      11.576 -10.222  20.588  1.00 11.58           C  \nATOM    878  CE1 TYR A  60       9.131 -10.458  19.416  1.00 12.07           C  \nATOM    879  CE2 TYR A  60      10.820  -9.089  20.405  1.00 12.32           C  \nATOM    880  CZ  TYR A  60       9.605  -9.195  19.741  1.00 11.87           C  \nATOM    881  OH  TYR A  60       8.865  -8.071  19.521  1.00 12.63           O  \nATOM    882  H   TYR A  60      12.429 -11.969  23.247  1.00 10.42           H  \nATOM    883  HA  TYR A  60      10.743 -13.754  21.866  1.00 11.19           H  \nATOM    884  HB2 TYR A  60      12.913 -12.437  20.552  1.00 11.68           H  \nATOM    885  HB3 TYR A  60      11.821 -13.360  19.859  1.00 11.68           H  \nATOM    886  HD1 TYR A  60       9.568 -12.424  19.402  1.00 12.25           H  \nATOM    887  HD2 TYR A  60      12.405 -10.139  21.029  1.00 11.85           H  \nATOM    888  HE1 TYR A  60       8.299 -10.545  18.983  1.00 12.13           H  \nATOM    889  HE2 TYR A  60      11.168  -8.241  20.627  1.00 12.05           H  \nATOM    890  HH  TYR A  60       8.087  -8.125  19.119  0.00 12.60           H  \nATOM    891  N   PRO A  61      12.460 -15.601  21.963  1.00 12.32           N  \nATOM    892  CA  PRO A  61      13.535 -16.610  21.987  1.00 13.54           C  \nATOM    893  C   PRO A  61      14.461 -16.390  20.792  1.00 13.81           C  \nATOM    894  O   PRO A  61      14.117 -15.755  19.810  1.00 15.29           O  \nATOM    895  CB  PRO A  61      12.735 -17.912  21.855  1.00 13.93           C  \nATOM    896  CG  PRO A  61      11.574 -17.543  21.024  1.00 15.62           C  \nATOM    897  CD  PRO A  61      11.164 -16.174  21.550  1.00 14.30           C  \nATOM    898  HA  PRO A  61      14.027 -16.581  22.845  1.00 13.43           H  \nATOM    899  HB2 PRO A  61      13.269 -18.611  21.419  1.00 14.21           H  \nATOM    900  HB3 PRO A  61      12.446 -18.237  22.735  1.00 14.22           H  \nATOM    901  HG2 PRO A  61      11.822 -17.493  20.078  1.00 14.89           H  \nATOM    902  HG3 PRO A  61      10.848 -18.190  21.129  1.00 14.88           H  \nATOM    903  HD2 PRO A  61      10.747 -15.640  20.851  1.00 14.07           H  \nATOM    904  HD3 PRO A  61      10.554 -16.251  22.307  1.00 14.09           H  \nATOM    905  N   ASP A  62      15.665 -16.966  20.895  1.00 19.46           N  \nATOM    906  CA  ASP A  62      16.676 -16.901  19.841  1.00 23.01           C  \nATOM    907  C   ASP A  62      16.204 -17.353  18.451  1.00 21.81           C  \nATOM    908  O   ASP A  62      16.659 -16.784  17.465  1.00 25.69           O  \nATOM    909  CB  ASP A  62      17.859 -17.757  20.293  1.00 25.12           C  \nATOM    910  CG  ASP A  62      18.574 -17.133  21.486  1.00 30.13           C  \nATOM    911  OD1 ASP A  62      18.240 -15.958  21.845  1.00 33.99           O  \nATOM    912  OD2 ASP A  62      19.454 -17.804  22.057  1.00 39.18           O  \nATOM    913  H   ASP A  62      15.863 -17.428  21.659  1.00 18.68           H  \nATOM    914  HA  ASP A  62      16.982 -15.966  19.770  1.00 22.47           H  \nATOM    915  HB2 ASP A  62      17.542 -18.649  20.543  1.00 25.67           H  \nATOM    916  HB3 ASP A  62      18.497 -17.850  19.556  1.00 25.68           H  \nATOM    917  N   SER A  63      15.284 -18.328  18.369  1.00 24.24           N  \nATOM    918  CA  SER A  63      14.788 -18.866  17.098  1.00 24.55           C  \nATOM    919  C   SER A  63      14.176 -17.780  16.207  1.00 26.53           C  \nATOM    920  O   SER A  63      14.232 -17.878  14.982  1.00 27.78           O  \nATOM    921  CB  SER A  63      13.770 -19.986  17.329  1.00 21.36           C  \nATOM    922  OG  SER A  63      12.788 -19.607  18.302  1.00 28.17           O  \nATOM    923  H   SER A  63      14.954 -18.682  19.145  1.00 23.67           H  \nATOM    924  HA  SER A  63      15.565 -19.253  16.613  1.00 24.21           H  \nATOM    925  HB2 SER A  63      13.322 -20.200  16.479  1.00 23.58           H  \nATOM    926  HB3 SER A  63      14.240 -20.794  17.637  1.00 23.57           H  \nATOM    927  HG  SER A  63      12.142 -19.184  17.749  0.00 27.63           H  \nATOM    928  N   VAL A  64      13.595 -16.718  16.784  1.00 20.23           N  \nATOM    929  CA  VAL A  64      12.813 -15.771  15.992  1.00 20.28           C  \nATOM    930  C   VAL A  64      13.243 -14.326  16.233  1.00 18.37           C  \nATOM    931  O   VAL A  64      12.744 -13.423  15.567  1.00 18.62           O  \nATOM    932  CB  VAL A  64      11.302 -15.885  16.232  1.00 20.40           C  \nATOM    933  CG1 VAL A  64      10.785 -17.297  16.041  1.00 23.65           C  \nATOM    934  CG2 VAL A  64      10.923 -15.363  17.621  1.00 20.76           C  \nATOM    935  H   VAL A  64      13.648 -16.605  17.689  1.00 21.57           H  \nATOM    936  HA  VAL A  64      12.974 -15.968  15.053  1.00 19.94           H  \nATOM    937  HB  VAL A  64      10.855 -15.310  15.564  1.00 21.02           H  \nATOM    938 HG11 VAL A  64      11.144 -17.873  16.739  1.00 22.59           H  \nATOM    939 HG12 VAL A  64      11.065 -17.631  15.171  1.00 22.58           H  \nATOM    940 HG13 VAL A  64       9.814 -17.298  16.090  1.00 22.59           H  \nATOM    941 HG21 VAL A  64      10.135 -15.836  17.940  1.00 20.63           H  \nATOM    942 HG22 VAL A  64      10.728 -14.411  17.570  1.00 20.64           H  \nATOM    943 HG23 VAL A  64      11.659 -15.511  18.240  1.00 20.64           H  \nATOM    944  N   LYS A  65      14.115 -14.114  17.234  1.00 19.70           N  \nATOM    945  CA  LYS A  65      14.544 -12.769  17.580  1.00 21.37           C  \nATOM    946  C   LYS A  65      15.198 -12.096  16.366  1.00 21.32           C  \nATOM    947  O   LYS A  65      15.990 -12.699  15.661  1.00 23.24           O  \nATOM    948  CB  LYS A  65      15.454 -12.860  18.789  1.00 22.72           C  \nATOM    949  CG  LYS A  65      15.872 -11.514  19.317  1.00 24.75           C  \nATOM    950  CD  LYS A  65      16.836 -11.644  20.478  1.00 28.51           C  \nATOM    951  CE  LYS A  65      17.169 -10.295  21.061  1.00 33.52           C  \nATOM    952  NZ  LYS A  65      18.152 -10.444  22.157  1.00 39.59           N  \nATOM    953  H   LYS A  65      14.453 -14.819  17.706  1.00 19.78           H  \nATOM    954  HA  LYS A  65      13.745 -12.251  17.830  1.00 21.28           H  \nATOM    955  HB2 LYS A  65      14.989 -13.348  19.499  1.00 22.83           H  \nATOM    956  HB3 LYS A  65      16.256 -13.368  18.544  1.00 22.85           H  \nATOM    957  HG2 LYS A  65      16.300 -11.002  18.598  1.00 25.07           H  \nATOM    958  HG3 LYS A  65      15.075 -11.022  19.610  1.00 25.08           H  \nATOM    959  HD2 LYS A  65      16.434 -12.207  21.174  1.00 28.65           H  \nATOM    960  HD3 LYS A  65      17.661 -12.077  20.170  1.00 28.65           H  \nATOM    961  HE2 LYS A  65      17.543  -9.718  20.368  1.00 33.55           H  \nATOM    962  HE3 LYS A  65      16.360  -9.875  21.408  1.00 33.55           H  \nATOM    963  HZ1 LYS A  65      18.900 -10.852  21.846  1.00 37.59           H  \nATOM    964  HZ2 LYS A  65      17.796 -10.946  22.824  1.00 37.61           H  \nATOM    965  HZ3 LYS A  65      18.373  -9.628  22.485  1.00 37.61           H  \nATOM    966  N   GLY A  66      14.803 -10.861  16.108  1.00 20.68           N  \nATOM    967  CA  GLY A  66      15.219 -10.127  14.929  1.00 22.76           C  \nATOM    968  C   GLY A  66      14.313 -10.332  13.721  1.00 23.01           C  \nATOM    969  O   GLY A  66      14.415  -9.572  12.779  1.00 24.81           O  \nATOM    970  H   GLY A  66      14.245 -10.445  16.701  1.00 21.30           H  \nATOM    971  HA2 GLY A  66      15.244  -9.163  15.151  1.00 22.31           H  \nATOM    972  HA3 GLY A  66      16.141 -10.400  14.691  1.00 22.31           H  \nATOM    973  N   ARG A  67      13.484 -11.386  13.691  1.00 18.80           N  \nATOM    974  CA  ARG A  67      12.593 -11.626  12.581  1.00 20.35           C  \nATOM    975  C   ARG A  67      11.132 -11.309  12.940  1.00 16.83           C  \nATOM    976  O   ARG A  67      10.360 -10.947  12.063  1.00 18.92           O  \nATOM    977  CB  ARG A  67      12.681 -13.076  12.121  1.00 18.15           C  \nATOM    978  CG  ARG A  67      14.067 -13.477  11.624  1.00 19.26           C  \nATOM    979  CD  ARG A  67      14.123 -14.859  11.003  1.00 17.72           C  \nATOM    980  NE  ARG A  67      13.703 -15.903  11.890  1.00 17.51           N  \nATOM    981  CZ  ARG A  67      12.630 -16.682  11.688  1.00 15.49           C  \nATOM    982  NH1 ARG A  67      11.817 -16.466  10.658  1.00 15.84           N  \nATOM    983  NH2 ARG A  67      12.338 -17.592  12.599  1.00 17.07           N  \nATOM    984  H   ARG A  67      13.489 -11.982  14.384  1.00 20.08           H  \nATOM    985  HA  ARG A  67      12.862 -11.045  11.833  1.00 18.93           H  \nATOM    986  HB2 ARG A  67      12.433 -13.658  12.870  1.00 18.90           H  \nATOM    987  HB3 ARG A  67      12.033 -13.218  11.402  1.00 18.90           H  \nATOM    988  HG2 ARG A  67      14.368 -12.822  10.959  1.00 18.62           H  \nATOM    989  HG3 ARG A  67      14.696 -13.446  12.376  1.00 18.62           H  \nATOM    990  HD2 ARG A  67      13.555 -14.871  10.204  1.00 18.03           H  \nATOM    991  HD3 ARG A  67      15.043 -15.039  10.714  1.00 18.05           H  \nATOM    992  HE  ARG A  67      14.176 -16.042  12.609  1.00 17.06           H  \nATOM    993 HH11 ARG A  67      12.026 -15.854  10.091  0.00 15.70           H  \nATOM    994 HH12 ARG A  67      11.129 -16.996  10.568  0.00 15.70           H  \nATOM    995 HH21 ARG A  67      11.501 -17.725  12.831  1.00 16.58           H  \nATOM    996 HH22 ARG A  67      12.981 -18.063  12.969  1.00 16.57           H  \nATOM    997  N   PHE A  68      10.755 -11.494  14.211  1.00 15.31           N  \nATOM    998  CA  PHE A  68       9.358 -11.331  14.611  1.00 15.75           C  \nATOM    999  C   PHE A  68       9.227 -10.135  15.551  1.00 14.33           C  \nATOM   1000  O   PHE A  68      10.130  -9.872  16.333  1.00 14.71           O  \nATOM   1001  CB  PHE A  68       8.833 -12.538  15.408  1.00 15.88           C  \nATOM   1002  CG  PHE A  68       8.610 -13.835  14.641  1.00 17.25           C  \nATOM   1003  CD1 PHE A  68       9.035 -13.981  13.327  1.00 17.83           C  \nATOM   1004  CD2 PHE A  68       7.949 -14.896  15.222  1.00 18.79           C  \nATOM   1005  CE1 PHE A  68       8.838 -15.162  12.609  1.00 19.91           C  \nATOM   1006  CE2 PHE A  68       7.785 -16.087  14.523  1.00 19.60           C  \nATOM   1007  CZ  PHE A  68       8.198 -16.215  13.218  1.00 18.21           C  \nATOM   1008  H   PHE A  68      11.371 -11.729  14.843  1.00 15.79           H  \nATOM   1009  HA  PHE A  68       8.794 -11.189  13.814  1.00 15.42           H  \nATOM   1010  HB2 PHE A  68       9.466 -12.716  16.134  1.00 16.16           H  \nATOM   1011  HB3 PHE A  68       7.983 -12.279  15.822  1.00 16.16           H  \nATOM   1012  HD1 PHE A  68       9.471 -13.264  12.905  1.00 18.16           H  \nATOM   1013  HD2 PHE A  68       7.664 -14.835  16.120  1.00 18.57           H  \nATOM   1014  HE1 PHE A  68       9.142 -15.237  11.718  1.00 18.94           H  \nATOM   1015  HE2 PHE A  68       7.333 -16.805  14.937  1.00 19.00           H  \nATOM   1016  HZ  PHE A  68       8.059 -17.024  12.751  1.00 18.85           H  \nATOM   1017  N   THR A  69       8.117  -9.402  15.450  1.00 13.19           N  \nATOM   1018  CA  THR A  69       7.867  -8.282  16.359  1.00 13.02           C  \nATOM   1019  C   THR A  69       6.467  -8.446  16.920  1.00 11.93           C  \nATOM   1020  O   THR A  69       5.489  -8.553  16.187  1.00 11.49           O  \nATOM   1021  CB  THR A  69       7.944  -6.925  15.636  1.00 15.84           C  \nATOM   1022  OG1 THR A  69       9.222  -6.893  14.998  1.00 18.67           O  \nATOM   1023  CG2 THR A  69       7.748  -5.745  16.570  1.00 16.57           C  \nATOM   1024  H   THR A  69       7.499  -9.598  14.805  1.00 13.41           H  \nATOM   1025  HA  THR A  69       8.522  -8.304  17.090  1.00 13.32           H  \nATOM   1026  HB  THR A  69       7.241  -6.903  14.944  1.00 15.86           H  \nATOM   1027  HG1 THR A  69       9.865  -6.986  15.532  0.00 19.33           H  \nATOM   1028 HG21 THR A  69       8.221  -4.970  16.218  1.00 16.34           H  \nATOM   1029 HG22 THR A  69       8.099  -5.966  17.452  1.00 16.33           H  \nATOM   1030 HG23 THR A  69       6.799  -5.540  16.641  1.00 16.33           H  \nATOM   1031  N   ILE A  70       6.346  -8.455  18.243  1.00 10.88           N  \nATOM   1032  CA  ILE A  70       5.051  -8.472  18.905  1.00 10.75           C  \nATOM   1033  C   ILE A  70       4.729  -7.040  19.342  1.00 12.25           C  \nATOM   1034  O   ILE A  70       5.608  -6.298  19.787  1.00 12.41           O  \nATOM   1035  CB  ILE A  70       5.029  -9.444  20.111  1.00 10.45           C  \nATOM   1036  CG1 ILE A  70       3.639  -9.682  20.686  1.00 10.54           C  \nATOM   1037  CG2 ILE A  70       6.038  -9.066  21.210  1.00 11.56           C  \nATOM   1038  CD1 ILE A  70       3.581 -10.744  21.786  1.00 10.58           C  \nATOM   1039  H   ILE A  70       7.101  -8.442  18.757  1.00 11.10           H  \nATOM   1040  HA  ILE A  70       4.380  -8.764  18.266  1.00 10.99           H  \nATOM   1041  HB  ILE A  70       5.326 -10.317  19.755  1.00 10.73           H  \nATOM   1042 HG12 ILE A  70       3.301  -8.837  21.051  1.00 10.53           H  \nATOM   1043 HG13 ILE A  70       3.041  -9.955  19.958  1.00 10.53           H  \nATOM   1044 HG21 ILE A  70       5.650  -8.383  21.787  1.00 11.21           H  \nATOM   1045 HG22 ILE A  70       6.853  -8.723  20.807  1.00 11.21           H  \nATOM   1046 HG23 ILE A  70       6.250  -9.855  21.742  1.00 11.21           H  \nATOM   1047 HD11 ILE A  70       3.883 -10.359  22.627  1.00 10.56           H  \nATOM   1048 HD12 ILE A  70       4.157 -11.491  21.548  1.00 10.56           H  \nATOM   1049 HD13 ILE A  70       2.666 -11.060  21.886  1.00 10.56           H  \nATOM   1050  N   SER A  71       3.458  -6.711  19.203  1.00 10.65           N  \nATOM   1051  CA  SER A  71       2.951  -5.428  19.652  1.00 10.90           C  \nATOM   1052  C   SER A  71       1.478  -5.567  20.061  1.00 11.50           C  \nATOM   1053  O   SER A  71       0.813  -6.554  19.763  1.00 10.78           O  \nATOM   1054  CB  SER A  71       3.122  -4.385  18.552  1.00 13.65           C  \nATOM   1055  OG  SER A  71       2.324  -4.711  17.435  1.00 15.02           O  \nATOM   1056  H   SER A  71       2.878  -7.302  18.815  1.00 11.07           H  \nATOM   1057  HA  SER A  71       3.474  -5.142  20.450  1.00 11.45           H  \nATOM   1058  HB2 SER A  71       2.862  -3.500  18.896  1.00 13.24           H  \nATOM   1059  HB3 SER A  71       4.067  -4.344  18.282  1.00 13.24           H  \nATOM   1060  HG  SER A  71       1.464  -4.560  17.392  0.00 15.13           H  \nATOM   1061  N   ARG A  72       0.940  -4.518  20.702  1.00 10.93           N  \nATOM   1062  CA  ARG A  72      -0.441  -4.527  21.094  1.00 12.05           C  \nATOM   1063  C   ARG A  72      -0.977  -3.105  20.954  1.00 13.96           C  \nATOM   1064  O   ARG A  72      -0.244  -2.143  21.165  1.00 15.15           O  \nATOM   1065  CB  ARG A  72      -0.644  -5.117  22.493  1.00 11.34           C  \nATOM   1066  CG  ARG A  72      -0.056  -4.285  23.624  1.00 11.51           C  \nATOM   1067  CD  ARG A  72       0.026  -5.098  24.895  1.00 10.71           C  \nATOM   1068  NE  ARG A  72      -1.259  -5.511  25.401  1.00 10.33           N  \nATOM   1069  CZ  ARG A  72      -1.473  -6.433  26.313  1.00  9.40           C  \nATOM   1070  NH1 ARG A  72      -0.450  -7.192  26.745  1.00  9.05           N  \nATOM   1071  NH2 ARG A  72      -2.711  -6.641  26.721  1.00  9.81           N  \nATOM   1072  H   ARG A  72       1.455  -3.789  20.896  1.00 11.32           H  \nATOM   1073  HA  ARG A  72      -0.928  -5.095  20.454  1.00 12.03           H  \nATOM   1074  HB2 ARG A  72      -1.606  -5.223  22.649  1.00 11.54           H  \nATOM   1075  HB3 ARG A  72      -0.238  -6.009  22.515  1.00 11.54           H  \nATOM   1076  HG2 ARG A  72       0.842  -3.979  23.376  1.00 11.27           H  \nATOM   1077  HG3 ARG A  72      -0.618  -3.496  23.780  1.00 11.28           H  \nATOM   1078  HD2 ARG A  72       0.573  -5.895  24.726  1.00 10.80           H  \nATOM   1079  HD3 ARG A  72       0.482  -4.568  25.582  1.00 10.80           H  \nATOM   1080  HE  ARG A  72      -1.962  -5.112  25.072  1.00 10.19           H  \nATOM   1081 HH11 ARG A  72       0.349  -7.000  26.462  0.00  9.04           H  \nATOM   1082 HH12 ARG A  72      -0.607  -7.797  27.348  0.00  9.04           H  \nATOM   1083 HH21 ARG A  72      -3.021  -7.462  26.777  1.00  9.68           H  \nATOM   1084 HH22 ARG A  72      -3.222  -5.959  26.936  1.00  9.68           H  \nATOM   1085  N   ASP A  73      -2.240  -3.030  20.591  1.00 14.72           N  \nATOM   1086  CA  ASP A  73      -2.975  -1.775  20.594  1.00 17.98           C  \nATOM   1087  C   ASP A  73      -4.084  -1.934  21.604  1.00 14.78           C  \nATOM   1088  O   ASP A  73      -5.070  -2.645  21.424  1.00 15.56           O  \nATOM   1089  CB  ASP A  73      -3.462  -1.450  19.193  1.00 22.38           C  \nATOM   1090  CG  ASP A  73      -2.692  -0.230  18.654  1.00 45.04           C  \nATOM   1091  OD1 ASP A  73      -3.393   0.811  18.386  1.00 59.02           O  \nATOM   1092  OD2 ASP A  73      -1.393  -0.296  18.542  1.00 59.39           O  \nATOM   1093  H   ASP A  73      -2.675  -3.790  20.327  1.00 15.22           H  \nATOM   1094  HA  ASP A  73      -2.373  -1.054  20.894  1.00 17.40           H  \nATOM   1095  HB2 ASP A  73      -3.315  -2.220  18.600  1.00 25.16           H  \nATOM   1096  HB3 ASP A  73      -4.425  -1.243  19.212  1.00 25.22           H  \nATOM   1097  N   ASN A  74      -3.849  -1.425  22.822  1.00 14.30           N  \nATOM   1098  CA  ASN A  74      -4.751  -1.690  23.908  1.00 12.12           C  \nATOM   1099  C   ASN A  74      -6.124  -1.030  23.722  1.00 14.36           C  \nATOM   1100  O   ASN A  74      -7.093  -1.535  24.250  1.00 16.96           O  \nATOM   1101  CB  ASN A  74      -4.138  -1.282  25.262  1.00 11.47           C  \nATOM   1102  CG  ASN A  74      -3.161  -2.324  25.790  1.00 13.69           C  \nATOM   1103  OD1 ASN A  74      -3.344  -3.489  25.464  1.00 15.24           O  \nATOM   1104  ND2 ASN A  74      -2.216  -1.939  26.637  1.00 12.20           N  \nATOM   1105  H   ASN A  74      -3.112  -0.904  22.969  1.00 13.87           H  \nATOM   1106  HA  ASN A  74      -4.895  -2.664  23.940  1.00 12.79           H  \nATOM   1107  HB2 ASN A  74      -3.672  -0.428  25.157  1.00 12.12           H  \nATOM   1108  HB3 ASN A  74      -4.858  -1.158  25.914  1.00 12.12           H  \nATOM   1109 HD21 ASN A  74      -2.143  -1.046  26.792  0.00 12.22           H  \nATOM   1110 HD22 ASN A  74      -1.637  -2.485  26.921  0.00 12.22           H  \nATOM   1111  N   ALA A  75      -6.170   0.024  22.917  1.00 15.69           N  \nATOM   1112  CA  ALA A  75      -7.435   0.674  22.627  1.00 19.55           C  \nATOM   1113  C   ALA A  75      -8.301  -0.220  21.748  1.00 19.20           C  \nATOM   1114  O   ALA A  75      -9.526  -0.160  21.880  1.00 21.49           O  \nATOM   1115  CB  ALA A  75      -7.185   2.043  22.031  1.00 18.03           C  \nATOM   1116  H   ALA A  75      -5.402   0.358  22.551  1.00 16.18           H  \nATOM   1117  HA  ALA A  75      -7.910   0.800  23.482  1.00 18.39           H  \nATOM   1118  HB1 ALA A  75      -7.995   2.362  21.598  1.00 18.49           H  \nATOM   1119  HB2 ALA A  75      -6.930   2.663  22.734  1.00 18.49           H  \nATOM   1120  HB3 ALA A  75      -6.470   1.988  21.374  1.00 18.48           H  \nATOM   1121  N   LYS A  76      -7.668  -0.996  20.862  1.00 17.30           N  \nATOM   1122  CA  LYS A  76      -8.399  -1.900  19.973  1.00 17.35           C  \nATOM   1123  C   LYS A  76      -8.446  -3.345  20.498  1.00 15.13           C  \nATOM   1124  O   LYS A  76      -8.987  -4.244  19.858  1.00 15.55           O  \nATOM   1125  CB  LYS A  76      -7.735  -1.898  18.595  1.00 22.78           C  \nATOM   1126  CG  LYS A  76      -8.090  -0.702  17.721  1.00 27.80           C  \nATOM   1127  CD  LYS A  76      -7.636  -0.883  16.279  1.00 31.95           C  \nATOM   1128  CE  LYS A  76      -8.332   0.071  15.331  1.00 35.79           C  \nATOM   1129  NZ  LYS A  76      -8.160   1.483  15.758  1.00 42.62           N  \nATOM   1130  H   LYS A  76      -6.757  -0.961  20.802  1.00 17.76           H  \nATOM   1131  HA  LYS A  76      -9.322  -1.570  19.879  1.00 17.82           H  \nATOM   1132  HB2 LYS A  76      -6.763  -1.917  18.717  1.00 22.37           H  \nATOM   1133  HB3 LYS A  76      -7.993  -2.716  18.121  1.00 22.37           H  \nATOM   1134  HG2 LYS A  76      -9.062  -0.571  17.739  1.00 27.40           H  \nATOM   1135  HG3 LYS A  76      -7.669   0.102  18.093  1.00 27.41           H  \nATOM   1136  HD2 LYS A  76      -6.667  -0.736  16.226  1.00 31.76           H  \nATOM   1137  HD3 LYS A  76      -7.819  -1.805  15.996  1.00 31.73           H  \nATOM   1138  HE2 LYS A  76      -7.967  -0.036  14.432  1.00 36.31           H  \nATOM   1139  HE3 LYS A  76      -9.285  -0.138  15.298  1.00 36.31           H  \nATOM   1140  HZ1 LYS A  76      -7.341   1.595  16.133  1.00 40.34           H  \nATOM   1141  HZ2 LYS A  76      -8.798   1.702  16.365  1.00 40.35           H  \nATOM   1142  HZ3 LYS A  76      -8.234   2.032  15.040  1.00 40.37           H  \nATOM   1143  N   ASN A  77      -7.936  -3.561  21.719  1.00 13.51           N  \nATOM   1144  CA  ASN A  77      -7.933  -4.869  22.344  1.00 13.71           C  \nATOM   1145  C   ASN A  77      -7.372  -5.931  21.410  1.00 12.38           C  \nATOM   1146  O   ASN A  77      -7.893  -7.041  21.292  1.00 13.83           O  \nATOM   1147  CB  ASN A  77      -9.341  -5.262  22.791  1.00 13.76           C  \nATOM   1148  CG  ASN A  77      -9.853  -4.239  23.805  1.00 15.11           C  \nATOM   1149  OD1 ASN A  77      -9.133  -3.804  24.694  1.00 17.20           O  \nATOM   1150  ND2 ASN A  77     -11.111  -3.848  23.666  1.00 18.21           N  \nATOM   1151  H   ASN A  77      -7.572  -2.861  22.179  1.00 13.95           H  \nATOM   1152  HA  ASN A  77      -7.365  -4.825  23.145  1.00 13.42           H  \nATOM   1153  HB2 ASN A  77      -9.939  -5.288  22.016  1.00 14.07           H  \nATOM   1154  HB3 ASN A  77      -9.320  -6.150  23.202  1.00 14.06           H  \nATOM   1155 HD21 ASN A  77     -11.602  -4.232  22.958  0.00 18.54           H  \nATOM   1156 HD22 ASN A  77     -11.474  -3.287  24.155  0.00 18.54           H  \nATOM   1157  N   THR A  78      -6.280  -5.571  20.768  1.00 12.14           N  \nATOM   1158  CA  THR A  78      -5.694  -6.472  19.780  1.00 12.87           C  \nATOM   1159  C   THR A  78      -4.197  -6.624  19.995  1.00 10.79           C  \nATOM   1160  O   THR A  78      -3.476  -5.668  20.266  1.00 11.46           O  \nATOM   1161  CB  THR A  78      -5.955  -5.942  18.365  1.00 14.21           C  \nATOM   1162  OG1 THR A  78      -7.371  -5.868  18.238  1.00 14.47           O  \nATOM   1163  CG2 THR A  78      -5.433  -6.841  17.270  1.00 14.16           C  \nATOM   1164  H   THR A  78      -5.880  -4.767  20.938  1.00 12.37           H  \nATOM   1165  HA  THR A  78      -6.120  -7.352  19.871  1.00 12.54           H  \nATOM   1166  HB  THR A  78      -5.563  -5.040  18.276  1.00 13.98           H  \nATOM   1167  HG1 THR A  78      -7.754  -6.598  18.356  0.00 14.16           H  \nATOM   1168 HG21 THR A  78      -5.801  -6.557  16.414  1.00 14.17           H  \nATOM   1169 HG22 THR A  78      -5.700  -7.760  17.448  1.00 14.17           H  \nATOM   1170 HG23 THR A  78      -4.461  -6.786  17.237  1.00 14.17           H  \nATOM   1171  N   LEU A  79      -3.735  -7.876  19.798  1.00 11.53           N  \nATOM   1172  CA  LEU A  79      -2.332  -8.273  19.801  1.00 11.96           C  \nATOM   1173  C   LEU A  79      -1.894  -8.512  18.362  1.00 11.60           C  \nATOM   1174  O   LEU A  79      -2.678  -9.076  17.616  1.00 11.93           O  \nATOM   1175  CB  LEU A  79      -2.207  -9.552  20.654  1.00 11.74           C  \nATOM   1176  CG  LEU A  79      -0.796 -10.068  20.880  1.00 12.54           C  \nATOM   1177  CD1 LEU A  79      -0.123  -9.185  21.903  1.00 14.37           C  \nATOM   1178  CD2 LEU A  79      -0.824 -11.488  21.373  1.00 13.27           C  \nATOM   1179  H   LEU A  79      -4.354  -8.532  19.651  1.00 11.45           H  \nATOM   1180  HA  LEU A  79      -1.791  -7.550  20.193  1.00 11.77           H  \nATOM   1181  HB2 LEU A  79      -2.616  -9.379  21.525  1.00 11.98           H  \nATOM   1182  HB3 LEU A  79      -2.727 -10.257  20.223  1.00 11.98           H  \nATOM   1183  HG  LEU A  79      -0.291 -10.026  20.031  1.00 12.84           H  \nATOM   1184 HD11 LEU A  79       0.787  -9.497  22.051  1.00 13.76           H  \nATOM   1185 HD12 LEU A  79      -0.618  -9.222  22.739  1.00 13.77           H  \nATOM   1186 HD13 LEU A  79      -0.101  -8.269  21.579  1.00 13.76           H  \nATOM   1187 HD21 LEU A  79      -1.116 -12.076  20.655  1.00 13.04           H  \nATOM   1188 HD22 LEU A  79      -1.442 -11.560  22.121  1.00 13.04           H  \nATOM   1189 HD23 LEU A  79       0.067 -11.750  21.663  1.00 13.05           H  \nATOM   1190  N   TYR A  80      -0.650  -8.207  18.022  1.00 10.97           N  \nATOM   1191  CA  TYR A  80      -0.138  -8.425  16.679  1.00 12.16           C  \nATOM   1192  C   TYR A  80       1.159  -9.214  16.731  1.00 11.91           C  \nATOM   1193  O   TYR A  80       1.938  -9.117  17.687  1.00 11.69           O  \nATOM   1194  CB  TYR A  80       0.123  -7.091  15.958  1.00 13.39           C  \nATOM   1195  CG  TYR A  80      -1.084  -6.199  15.901  1.00 14.28           C  \nATOM   1196  CD1 TYR A  80      -2.086  -6.403  14.955  1.00 15.50           C  \nATOM   1197  CD2 TYR A  80      -1.202  -5.149  16.795  1.00 16.38           C  \nATOM   1198  CE1 TYR A  80      -3.191  -5.568  14.898  1.00 16.12           C  \nATOM   1199  CE2 TYR A  80      -2.319  -4.341  16.764  1.00 17.28           C  \nATOM   1200  CZ  TYR A  80      -3.315  -4.547  15.815  1.00 18.26           C  \nATOM   1201  OH  TYR A  80      -4.431  -3.734  15.700  1.00 21.43           O  \nATOM   1202  H   TYR A  80      -0.091  -7.849  18.650  1.00 11.39           H  \nATOM   1203  HA  TYR A  80      -0.803  -8.943  16.161  1.00 12.09           H  \nATOM   1204  HB2 TYR A  80       0.847  -6.621  16.421  1.00 13.30           H  \nATOM   1205  HB3 TYR A  80       0.422  -7.282  15.045  1.00 13.29           H  \nATOM   1206  HD1 TYR A  80      -2.004  -7.107  14.333  1.00 15.47           H  \nATOM   1207  HD2 TYR A  80      -0.539  -5.010  17.451  1.00 16.09           H  \nATOM   1208  HE1 TYR A  80      -3.866  -5.713  14.256  1.00 16.44           H  \nATOM   1209  HE2 TYR A  80      -2.394  -3.625  17.369  1.00 17.20           H  \nATOM   1210  HH  TYR A  80      -5.063  -3.860  15.124  0.00 21.14           H  \nATOM   1211  N   LEU A  81       1.434  -9.936  15.629  1.00 11.51           N  \nATOM   1212  CA  LEU A  81       2.720 -10.563  15.384  1.00 11.44           C  \nATOM   1213  C   LEU A  81       3.105 -10.221  13.954  1.00 11.05           C  \nATOM   1214  O   LEU A  81       2.491 -10.715  13.012  1.00 12.82           O  \nATOM   1215  CB  LEU A  81       2.680 -12.072  15.589  1.00 11.54           C  \nATOM   1216  CG  LEU A  81       4.073 -12.740  15.577  1.00 11.65           C  \nATOM   1217  CD1 LEU A  81       4.910 -12.322  16.773  1.00 13.16           C  \nATOM   1218  CD2 LEU A  81       3.961 -14.248  15.511  1.00 11.88           C  \nATOM   1219  H   LEU A  81       0.777 -10.042  15.002  1.00 11.59           H  \nATOM   1220  HA  LEU A  81       3.382 -10.167  15.994  1.00 11.41           H  \nATOM   1221  HB2 LEU A  81       2.250 -12.259  16.446  1.00 11.54           H  \nATOM   1222  HB3 LEU A  81       2.131 -12.472  14.886  1.00 11.55           H  \nATOM   1223  HG  LEU A  81       4.544 -12.440  14.762  1.00 11.95           H  \nATOM   1224 HD11 LEU A  81       5.635 -12.958  16.901  1.00 12.67           H  \nATOM   1225 HD12 LEU A  81       4.353 -12.303  17.570  1.00 12.67           H  \nATOM   1226 HD13 LEU A  81       5.281 -11.436  16.616  1.00 12.67           H  \nATOM   1227 HD21 LEU A  81       3.391 -14.499  14.764  1.00 11.81           H  \nATOM   1228 HD22 LEU A  81       3.574 -14.582  16.339  1.00 11.81           H  \nATOM   1229 HD23 LEU A  81       4.845 -14.635  15.388  1.00 11.81           H  \nATOM   1230  N   GLN A  82       4.139  -9.407  13.805  1.00 12.72           N  \nATOM   1231  CA  GLN A  82       4.728  -9.193  12.498  1.00 13.74           C  \nATOM   1232  C   GLN A  82       5.885 -10.152  12.292  1.00 13.04           C  \nATOM   1233  O   GLN A  82       6.860 -10.173  13.060  1.00 15.91           O  \nATOM   1234  CB  GLN A  82       5.253  -7.765  12.373  1.00 15.83           C  \nATOM   1235  CG  GLN A  82       5.880  -7.484  10.997  1.00 16.16           C  \nATOM   1236  CD  GLN A  82       4.894  -7.474   9.861  1.00 18.87           C  \nATOM   1237  OE1 GLN A  82       3.893  -6.755   9.883  1.00 23.81           O  \nATOM   1238  NE2 GLN A  82       5.121  -8.372   8.900  1.00 21.53           N  \nATOM   1239  H   GLN A  82       4.487  -8.970  14.528  1.00 12.53           H  \nATOM   1240  HA  GLN A  82       4.045  -9.349  11.807  1.00 13.78           H  \nATOM   1241  HB2 GLN A  82       4.510  -7.143  12.523  1.00 15.39           H  \nATOM   1242  HB3 GLN A  82       5.922  -7.612  13.071  1.00 15.39           H  \nATOM   1243  HG2 GLN A  82       6.329  -6.612  11.029  1.00 16.68           H  \nATOM   1244  HG3 GLN A  82       6.566  -8.160  10.814  1.00 16.68           H  \nATOM   1245 HE21 GLN A  82       5.823  -8.881   8.946  0.00 21.92           H  \nATOM   1246 HE22 GLN A  82       4.532  -8.442   8.251  0.00 21.92           H  \nATOM   1247  N   MET A  83       5.782 -10.946  11.215  1.00 14.37           N  \nATOM   1248  CA  MET A  83       6.733 -11.977  10.918  1.00 16.62           C  \nATOM   1249  C   MET A  83       7.488 -11.528   9.676  1.00 18.11           C  \nATOM   1250  O   MET A  83       6.879 -10.999   8.749  1.00 19.23           O  \nATOM   1251  CB  MET A  83       5.983 -13.273  10.617  1.00 18.73           C  \nATOM   1252  CG  MET A  83       4.971 -13.698  11.666  1.00 19.84           C  \nATOM   1253  SD  MET A  83       3.883 -15.135  11.158  1.00 23.75           S  \nATOM   1254  CE  MET A  83       4.999 -16.319  11.856  1.00 23.86           C  \nATOM   1255  H   MET A  83       5.079 -10.828  10.643  1.00 14.53           H  \nATOM   1256  HA  MET A  83       7.354 -12.095  11.673  1.00 16.81           H  \nATOM   1257  HB2 MET A  83       5.520 -13.169   9.761  1.00 18.45           H  \nATOM   1258  HB3 MET A  83       6.639 -13.990  10.509  1.00 18.46           H  \nATOM   1259  HG2 MET A  83       5.450 -13.943  12.486  1.00 20.43           H  \nATOM   1260  HG3 MET A  83       4.393 -12.934  11.878  1.00 20.41           H  \nATOM   1261  HE1 MET A  83       5.898 -16.121  11.568  1.00 23.78           H  \nATOM   1262  HE2 MET A  83       4.952 -16.276  12.819  1.00 23.77           H  \nATOM   1263  HE3 MET A  83       4.753 -17.203  11.560  1.00 23.77           H  \nATOM   1264  N   SER A  84       8.782 -11.739   9.667  1.00 18.64           N  \nATOM   1265  CA  SER A  84       9.582 -11.395   8.488  1.00 18.79           C  \nATOM   1266  C   SER A  84      10.646 -12.466   8.317  1.00 18.86           C  \nATOM   1267  O   SER A  84      10.895 -13.279   9.213  1.00 18.47           O  \nATOM   1268  CB  SER A  84      10.189 -10.016   8.623  1.00 18.67           C  \nATOM   1269  OG  SER A  84      11.177 -10.012   9.639  1.00 23.26           O  \nATOM   1270  H   SER A  84       9.194 -12.101  10.398  1.00 18.54           H  \nATOM   1271  HA  SER A  84       8.992 -11.409   7.688  1.00 18.75           H  \nATOM   1272  HB2 SER A  84      10.595  -9.751   7.767  1.00 19.69           H  \nATOM   1273  HB3 SER A  84       9.484  -9.367   8.845  1.00 19.69           H  \nATOM   1274  HG  SER A  84      11.910 -10.197   9.375  0.00 22.96           H  \nATOM   1275  N   SER A  85      11.294 -12.476   7.144  1.00 18.22           N  \nATOM   1276  CA  SER A  85      12.331 -13.436   6.860  1.00 18.58           C  \nATOM   1277  C   SER A  85      11.864 -14.859   7.168  1.00 16.51           C  \nATOM   1278  O   SER A  85      12.606 -15.618   7.790  1.00 19.98           O  \nATOM   1279  CB  SER A  85      13.624 -13.169   7.598  1.00 23.93           C  \nATOM   1280  OG  SER A  85      13.934 -11.720   7.554  1.00 41.19           O  \nATOM   1281  H   SER A  85      11.076 -11.860   6.505  1.00 18.45           H  \nATOM   1282  HA  SER A  85      12.521 -13.387   5.884  1.00 19.00           H  \nATOM   1283  HB2 SER A  85      13.517 -13.488   8.591  1.00 25.63           H  \nATOM   1284  HB3 SER A  85      14.383 -13.730   7.143  1.00 25.70           H  \nATOM   1285  N   LEU A  86      10.665 -15.219   6.718  1.00 15.71           N  \nATOM   1286  CA  LEU A  86      10.110 -16.498   7.109  1.00 16.83           C  \nATOM   1287  C   LEU A  86      10.945 -17.658   6.575  1.00 18.77           C  \nATOM   1288  O   LEU A  86      11.521 -17.601   5.479  1.00 19.08           O  \nATOM   1289  CB  LEU A  86       8.657 -16.573   6.660  1.00 19.13           C  \nATOM   1290  CG  LEU A  86       7.696 -15.745   7.521  1.00 23.01           C  \nATOM   1291  CD1 LEU A  86       6.428 -15.408   6.749  1.00 27.97           C  \nATOM   1292  CD2 LEU A  86       7.359 -16.513   8.771  1.00 24.93           C  \nATOM   1293  H   LEU A  86      10.192 -14.669   6.162  1.00 16.18           H  \nATOM   1294  HA  LEU A  86      10.134 -16.538   8.092  1.00 17.42           H  \nATOM   1295  HB2 LEU A  86       8.600 -16.262   5.735  1.00 19.41           H  \nATOM   1296  HB3 LEU A  86       8.372 -17.508   6.678  1.00 19.41           H  \nATOM   1297  HG  LEU A  86       8.145 -14.902   7.778  1.00 23.41           H  \nATOM   1298 HD11 LEU A  86       5.907 -14.755   7.248  1.00 26.31           H  \nATOM   1299 HD12 LEU A  86       5.903 -16.215   6.625  1.00 26.28           H  \nATOM   1300 HD13 LEU A  86       6.665 -15.038   5.882  1.00 26.24           H  \nATOM   1301 HD21 LEU A  86       8.153 -16.593   9.328  1.00 24.29           H  \nATOM   1302 HD22 LEU A  86       7.040 -17.401   8.532  1.00 24.30           H  \nATOM   1303 HD23 LEU A  86       6.665 -16.042   9.264  1.00 24.31           H  \nATOM   1304  N   LYS A  87      10.965 -18.728   7.357  1.00 19.15           N  \nATOM   1305  CA  LYS A  87      11.761 -19.909   7.124  1.00 19.83           C  \nATOM   1306  C   LYS A  87      10.855 -21.124   7.254  1.00 18.42           C  \nATOM   1307  O   LYS A  87       9.760 -21.051   7.830  1.00 16.06           O  \nATOM   1308  CB  LYS A  87      12.828 -20.041   8.223  1.00 24.20           C  \nATOM   1309  CG  LYS A  87      13.761 -18.865   8.415  1.00 30.23           C  \nATOM   1310  CD  LYS A  87      14.849 -19.167   9.444  1.00 33.13           C  \nATOM   1311  CE  LYS A  87      15.806 -18.000   9.577  1.00 38.98           C  \nATOM   1312  NZ  LYS A  87      16.906 -18.312  10.525  1.00 42.85           N  \nATOM   1313  H   LYS A  87      10.432 -18.719   8.100  1.00 19.20           H  \nATOM   1314  HA  LYS A  87      12.177 -19.878   6.232  1.00 20.17           H  \nATOM   1315  HB2 LYS A  87      12.370 -20.207   9.074  1.00 24.32           H  \nATOM   1316  HB3 LYS A  87      13.372 -20.832   8.025  1.00 24.33           H  \nATOM   1317  HG2 LYS A  87      14.180 -18.644   7.555  1.00 29.30           H  \nATOM   1318  HG3 LYS A  87      13.241 -18.088   8.711  1.00 29.27           H  \nATOM   1319  HD2 LYS A  87      14.434 -19.347  10.314  1.00 33.69           H  \nATOM   1320  HD3 LYS A  87      15.347 -19.966   9.168  1.00 33.69           H  \nATOM   1321  HE2 LYS A  87      16.188 -17.788   8.704  1.00 38.33           H  \nATOM   1322  HE3 LYS A  87      15.323 -17.215   9.896  1.00 38.23           H  \nATOM   1323  HZ1 LYS A  87      17.311 -19.086  10.282  1.00 41.62           H  \nATOM   1324  HZ2 LYS A  87      16.572 -18.402  11.364  1.00 41.62           H  \nATOM   1325  HZ3 LYS A  87      17.517 -17.642  10.518  1.00 41.62           H  \nATOM   1326  N   SER A  88      11.343 -22.264   6.726  1.00 22.38           N  \nATOM   1327  CA  SER A  88      10.644 -23.535   6.787  1.00 24.20           C  \nATOM   1328  C   SER A  88      10.239 -23.875   8.210  1.00 20.03           C  \nATOM   1329  O   SER A  88       9.125 -24.354   8.440  1.00 22.15           O  \nATOM   1330  CB  SER A  88      11.510 -24.669   6.210  1.00 25.87           C  \nATOM   1331  OG  SER A  88      12.191 -24.238   5.044  1.00 31.77           O  \nATOM   1332  H   SER A  88      12.155 -22.238   6.308  1.00 21.77           H  \nATOM   1333  HA  SER A  88       9.817 -23.458   6.238  1.00 23.29           H  \nATOM   1334  HB2 SER A  88      12.165 -24.957   6.886  1.00 26.71           H  \nATOM   1335  HB3 SER A  88      10.937 -25.439   5.990  1.00 26.73           H  \nATOM   1336  HG  SER A  88      12.552 -23.765   4.889  0.00 31.13           H  \nATOM   1337  N   GLU A  89      11.108 -23.544   9.172  1.00 21.68           N  \nATOM   1338  CA  GLU A  89      10.887 -23.914  10.562  1.00 22.22           C  \nATOM   1339  C   GLU A  89       9.711 -23.135  11.152  1.00 19.37           C  \nATOM   1340  O   GLU A  89       9.300 -23.404  12.283  1.00 19.36           O  \nATOM   1341  CB  GLU A  89      12.142 -23.692  11.416  1.00 27.39           C  \nATOM   1342  CG  GLU A  89      13.390 -24.380  10.894  1.00 33.04           C  \nATOM   1343  CD  GLU A  89      14.280 -23.509  10.004  1.00 39.34           C  \nATOM   1344  OE1 GLU A  89      13.992 -23.366   8.779  1.00 30.43           O  \nATOM   1345  OE2 GLU A  89      15.249 -22.919  10.559  1.00 52.56           O  \nATOM   1346  H   GLU A  89      11.860 -23.073   8.957  1.00 21.47           H  \nATOM   1347  HA  GLU A  89      10.663 -24.873  10.586  1.00 22.42           H  \nATOM   1348  HB2 GLU A  89      12.312 -22.728  11.472  1.00 27.25           H  \nATOM   1349  HB3 GLU A  89      11.960 -24.016  12.323  1.00 27.25           H  \nATOM   1350  HG2 GLU A  89      13.924 -24.688  11.657  1.00 32.94           H  \nATOM   1351  HG3 GLU A  89      13.123 -25.173  10.382  1.00 32.91           H  \nATOM   1352  N   ASP A  90       9.219 -22.099  10.456  1.00 14.90           N  \nATOM   1353  CA  ASP A  90       8.143 -21.267  10.982  1.00 14.89           C  \nATOM   1354  C   ASP A  90       6.761 -21.840  10.674  1.00 13.49           C  \nATOM   1355  O   ASP A  90       5.748 -21.339  11.180  1.00 14.71           O  \nATOM   1356  CB  ASP A  90       8.215 -19.817  10.505  1.00 14.80           C  \nATOM   1357  CG  ASP A  90       9.502 -19.117  10.896  1.00 16.06           C  \nATOM   1358  OD1 ASP A  90       9.989 -19.307  12.053  1.00 15.71           O  \nATOM   1359  OD2 ASP A  90      10.026 -18.352  10.044  1.00 20.46           O  \nATOM   1360  H   ASP A  90       9.572 -21.891   9.640  1.00 15.89           H  \nATOM   1361  HA  ASP A  90       8.241 -21.247  11.961  1.00 14.61           H  \nATOM   1362  HB2 ASP A  90       8.134 -19.796   9.529  1.00 15.12           H  \nATOM   1363  HB3 ASP A  90       7.465 -19.316  10.885  1.00 15.11           H  \nATOM   1364  N   THR A  91       6.647 -22.907   9.866  1.00 12.39           N  \nATOM   1365  CA  THR A  91       5.356 -23.490   9.565  1.00 11.30           C  \nATOM   1366  C   THR A  91       4.746 -24.095  10.836  1.00 10.76           C  \nATOM   1367  O   THR A  91       5.360 -24.932  11.503  1.00 12.43           O  \nATOM   1368  CB  THR A  91       5.466 -24.624   8.537  1.00 12.78           C  \nATOM   1369  OG1 THR A  91       5.890 -24.026   7.317  1.00 13.19           O  \nATOM   1370  CG2 THR A  91       4.224 -25.429   8.300  1.00 12.62           C  \nATOM   1371  H   THR A  91       7.395 -23.275   9.490  1.00 12.39           H  \nATOM   1372  HA  THR A  91       4.763 -22.787   9.216  1.00 11.67           H  \nATOM   1373  HB  THR A  91       6.175 -25.237   8.844  1.00 12.53           H  \nATOM   1374  HG1 THR A  91       5.334 -23.388   6.994  0.00 13.17           H  \nATOM   1375 HG21 THR A  91       4.245 -25.802   7.400  1.00 12.67           H  \nATOM   1376 HG22 THR A  91       3.441 -24.858   8.395  1.00 12.66           H  \nATOM   1377 HG23 THR A  91       4.177 -26.155   8.948  1.00 12.66           H  \nATOM   1378  N   ALA A  92       3.566 -23.631  11.208  1.00 10.04           N  \nATOM   1379  CA  ALA A  92       2.978 -23.994  12.477  1.00  9.57           C  \nATOM   1380  C   ALA A  92       1.573 -23.434  12.577  1.00  9.31           C  \nATOM   1381  O   ALA A  92       1.186 -22.514  11.836  1.00 10.91           O  \nATOM   1382  CB  ALA A  92       3.831 -23.492  13.634  1.00  9.91           C  \nATOM   1383  H   ALA A  92       3.107 -23.068  10.653  1.00 10.10           H  \nATOM   1384  HA  ALA A  92       2.925 -24.978  12.526  1.00  9.68           H  \nATOM   1385  HB1 ALA A  92       3.298 -23.470  14.446  1.00  9.80           H  \nATOM   1386  HB2 ALA A  92       4.588 -24.087  13.764  1.00  9.81           H  \nATOM   1387  HB3 ALA A  92       4.154 -22.597  13.436  1.00  9.81           H  \nATOM   1388  N   MET A  93       0.802 -23.943  13.562  1.00  9.56           N  \nATOM   1389  CA  MET A  93      -0.376 -23.250  14.058  1.00  9.97           C  \nATOM   1390  C   MET A  93       0.062 -22.197  15.092  1.00  9.46           C  \nATOM   1391  O   MET A  93       0.775 -22.528  16.023  1.00 10.98           O  \nATOM   1392  CB  MET A  93      -1.364 -24.195  14.735  1.00 12.36           C  \nATOM   1393  CG  MET A  93      -2.578 -23.506  15.178  1.00 15.27           C  \nATOM   1394  SD  MET A  93      -3.780 -23.190  13.883  1.00 20.77           S  \nATOM   1395  CE  MET A  93      -4.400 -24.874  13.648  1.00 17.95           C  \nATOM   1396  H   MET A  93       1.025 -24.749  13.930  1.00  9.60           H  \nATOM   1397  HA  MET A  93      -0.817 -22.796  13.304  1.00 10.21           H  \nATOM   1398  HB2 MET A  93      -1.605 -24.904  14.108  1.00 12.38           H  \nATOM   1399  HB3 MET A  93      -0.929 -24.609  15.507  1.00 12.35           H  \nATOM   1400  HG2 MET A  93      -3.010 -24.048  15.872  1.00 15.62           H  \nATOM   1401  HG3 MET A  93      -2.331 -22.650  15.587  1.00 15.58           H  \nATOM   1402  HE1 MET A  93      -3.752 -25.389  13.152  1.00 18.76           H  \nATOM   1403  HE2 MET A  93      -4.547 -25.285  14.509  1.00 18.76           H  \nATOM   1404  HE3 MET A  93      -5.231 -24.845  13.159  1.00 18.77           H  \nATOM   1405  N   TYR A  94      -0.333 -20.949  14.878  1.00  9.79           N  \nATOM   1406  CA  TYR A  94      -0.007 -19.873  15.796  1.00 10.44           C  \nATOM   1407  C   TYR A  94      -1.205 -19.536  16.658  1.00  9.60           C  \nATOM   1408  O   TYR A  94      -2.289 -19.171  16.186  1.00 11.80           O  \nATOM   1409  CB  TYR A  94       0.422 -18.628  15.009  1.00 10.26           C  \nATOM   1410  CG  TYR A  94       1.805 -18.764  14.456  1.00  9.54           C  \nATOM   1411  CD1 TYR A  94       2.068 -19.614  13.396  1.00  9.98           C  \nATOM   1412  CD2 TYR A  94       2.906 -18.119  15.014  1.00 10.32           C  \nATOM   1413  CE1 TYR A  94       3.353 -19.823  12.935  1.00  9.52           C  \nATOM   1414  CE2 TYR A  94       4.203 -18.319  14.576  1.00 10.64           C  \nATOM   1415  CZ  TYR A  94       4.441 -19.173  13.497  1.00  9.57           C  \nATOM   1416  OH  TYR A  94       5.724 -19.434  13.014  1.00 10.57           O  \nATOM   1417  H   TYR A  94      -0.827 -20.756  14.134  1.00  9.86           H  \nATOM   1418  HA  TYR A  94       0.742 -20.158  16.379  1.00 10.11           H  \nATOM   1419  HB2 TYR A  94      -0.209 -18.483  14.274  1.00 10.13           H  \nATOM   1420  HB3 TYR A  94       0.386 -17.849  15.602  1.00 10.13           H  \nATOM   1421  HD1 TYR A  94       1.357 -20.091  13.001  1.00  9.80           H  \nATOM   1422  HD2 TYR A  94       2.766 -17.554  15.756  1.00 10.18           H  \nATOM   1423  HE1 TYR A  94       3.494 -20.413  12.214  1.00  9.70           H  \nATOM   1424  HE2 TYR A  94       4.919 -17.861  14.984  1.00 10.27           H  \nATOM   1425  HH  TYR A  94       5.860 -19.940  12.387  0.00 10.30           H  \nATOM   1426  N   TYR A  95      -0.964 -19.589  17.977  1.00  9.82           N  \nATOM   1427  CA  TYR A  95      -1.944 -19.256  18.992  1.00 10.10           C  \nATOM   1428  C   TYR A  95      -1.547 -17.994  19.727  1.00 10.93           C  \nATOM   1429  O   TYR A  95      -0.384 -17.748  20.029  1.00 10.75           O  \nATOM   1430  CB  TYR A  95      -1.991 -20.362  20.044  1.00 10.98           C  \nATOM   1431  CG  TYR A  95      -2.526 -21.696  19.575  1.00 10.78           C  \nATOM   1432  CD1 TYR A  95      -3.891 -21.885  19.458  1.00 11.38           C  \nATOM   1433  CD2 TYR A  95      -1.689 -22.761  19.355  1.00 12.76           C  \nATOM   1434  CE1 TYR A  95      -4.396 -23.112  19.037  1.00 12.28           C  \nATOM   1435  CE2 TYR A  95      -2.187 -24.017  19.016  1.00 12.74           C  \nATOM   1436  CZ  TYR A  95      -3.541 -24.172  18.837  1.00 12.54           C  \nATOM   1437  OH  TYR A  95      -4.050 -25.436  18.459  1.00 15.96           O  \nATOM   1438  H   TYR A  95      -0.134 -19.852  18.256  1.00  9.83           H  \nATOM   1439  HA  TYR A  95      -2.836 -19.142  18.581  1.00 10.37           H  \nATOM   1440  HB2 TYR A  95      -1.085 -20.493  20.391  1.00 10.73           H  \nATOM   1441  HB3 TYR A  95      -2.548 -20.050  20.788  1.00 10.72           H  \nATOM   1442  HD1 TYR A  95      -4.477 -21.159  19.590  1.00 11.54           H  \nATOM   1443  HD2 TYR A  95      -0.761 -22.654  19.487  1.00 12.26           H  \nATOM   1444  HE1 TYR A  95      -5.327 -23.235  18.948  1.00 12.14           H  \nATOM   1445  HE2 TYR A  95      -1.599 -24.734  18.844  1.00 12.65           H  \nATOM   1446  HH  TYR A  95      -4.880 -25.480  18.336  0.00 15.80           H  \nATOM   1447  N   CYS A  96      -2.544 -17.184  20.035  1.00 10.71           N  \nATOM   1448  CA  CYS A  96      -2.319 -16.214  21.085  1.00 12.51           C  \nATOM   1449  C   CYS A  96      -2.905 -16.792  22.369  1.00 10.93           C  \nATOM   1450  O   CYS A  96      -3.888 -17.537  22.395  1.00 11.56           O  \nATOM   1451  CB  CYS A  96      -2.847 -14.861  20.695  1.00 17.84           C  \nATOM   1452  SG  CYS A  96      -4.640 -14.852  20.503  1.00 25.06           S  \nATOM   1453  H   CYS A  96      -3.350 -17.223  19.607  1.00 11.16           H  \nATOM   1454  HA  CYS A  96      -1.348 -16.129  21.206  1.00 12.65           H  \nATOM   1455  HB2 CYS A  96      -2.600 -14.206  21.380  1.00 17.77           H  \nATOM   1456  HB3 CYS A  96      -2.438 -14.585  19.849  1.00 17.77           H  \nATOM   1457  N   ALA A  97      -2.279 -16.370  23.465  1.00  9.87           N  \nATOM   1458  CA  ALA A  97      -2.630 -16.875  24.765  1.00  9.00           C  \nATOM   1459  C   ALA A  97      -2.440 -15.785  25.804  1.00  9.55           C  \nATOM   1460  O   ALA A  97      -1.536 -14.953  25.726  1.00 10.13           O  \nATOM   1461  CB  ALA A  97      -1.789 -18.101  25.079  1.00  9.88           C  \nATOM   1462  H   ALA A  97      -1.615 -15.746  23.397  1.00  9.90           H  \nATOM   1463  HA  ALA A  97      -3.579 -17.135  24.753  1.00  9.44           H  \nATOM   1464  HB1 ALA A  97      -2.022 -18.438  25.960  1.00  9.60           H  \nATOM   1465  HB2 ALA A  97      -1.958 -18.790  24.414  1.00  9.60           H  \nATOM   1466  HB3 ALA A  97      -0.847 -17.862  25.063  1.00  9.60           H  \nATOM   1467  N   ARG A  98      -3.349 -15.788  26.764  1.00  9.72           N  \nATOM   1468  CA  ARG A  98      -3.354 -14.833  27.853  1.00  9.74           C  \nATOM   1469  C   ARG A  98      -2.527 -15.356  29.021  1.00  9.89           C  \nATOM   1470  O   ARG A  98      -2.740 -16.472  29.430  1.00 10.37           O  \nATOM   1471  CB  ARG A  98      -4.770 -14.573  28.319  1.00 11.41           C  \nATOM   1472  CG  ARG A  98      -4.847 -13.427  29.315  1.00 11.60           C  \nATOM   1473  CD  ARG A  98      -6.246 -13.069  29.771  1.00 11.71           C  \nATOM   1474  NE  ARG A  98      -6.794 -13.960  30.723  1.00 13.03           N  \nATOM   1475  CZ  ARG A  98      -7.896 -13.709  31.415  1.00 17.79           C  \nATOM   1476  NH1 ARG A  98      -8.666 -12.673  31.109  1.00 17.53           N  \nATOM   1477  NH2 ARG A  98      -8.202 -14.508  32.403  1.00 19.21           N  \nATOM   1478  H   ARG A  98      -4.006 -16.423  26.745  1.00  9.69           H  \nATOM   1479  HA  ARG A  98      -2.962 -13.989  27.536  1.00 10.06           H  \nATOM   1480  HB2 ARG A  98      -5.327 -14.361  27.541  1.00 11.04           H  \nATOM   1481  HB3 ARG A  98      -5.123 -15.387  28.736  1.00 11.03           H  \nATOM   1482  HG2 ARG A  98      -4.316 -13.658  30.106  1.00 11.58           H  \nATOM   1483  HG3 ARG A  98      -4.439 -12.631  28.911  1.00 11.58           H  \nATOM   1484  HD2 ARG A  98      -6.231 -12.167  30.155  1.00 12.00           H  \nATOM   1485  HD3 ARG A  98      -6.836 -13.044  28.987  1.00 11.98           H  \nATOM   1486  HE  ARG A  98      -6.387 -14.719  30.860  1.00 13.70           H  \nATOM   1487 HH11 ARG A  98      -8.412 -12.156  30.431  0.00 18.07           H  \nATOM   1488 HH12 ARG A  98      -9.420 -12.570  31.502  0.00 18.07           H  \nATOM   1489 HH21 ARG A  98      -9.045 -14.676  32.583  1.00 18.75           H  \nATOM   1490 HH22 ARG A  98      -7.565 -14.870  32.888  1.00 18.72           H  \nATOM   1491  N   ARG A  99      -1.627 -14.514  29.557  1.00  9.10           N  \nATOM   1492  CA  ARG A  99      -0.859 -14.821  30.776  1.00 10.22           C  \nATOM   1493  C   ARG A  99      -1.615 -14.448  32.036  1.00 10.44           C  \nATOM   1494  O   ARG A  99      -2.178 -13.337  32.112  1.00 12.08           O  \nATOM   1495  CB  ARG A  99       0.440 -14.036  30.737  1.00  9.58           C  \nATOM   1496  CG  ARG A  99       1.352 -14.478  29.600  1.00 11.32           C  \nATOM   1497  CD  ARG A  99       2.562 -13.583  29.330  1.00 11.78           C  \nATOM   1498  NE  ARG A  99       3.334 -13.370  30.536  1.00 12.85           N  \nATOM   1499  CZ  ARG A  99       4.383 -12.588  30.615  1.00 14.57           C  \nATOM   1500  NH1 ARG A  99       4.783 -11.903  29.545  1.00 17.59           N  \nATOM   1501  NH2 ARG A  99       5.041 -12.523  31.763  1.00 15.41           N  \nATOM   1502  H   ARG A  99      -1.483 -13.710  29.147  1.00  9.54           H  \nATOM   1503  HA  ARG A  99      -0.654 -15.782  30.790  1.00  9.92           H  \nATOM   1504  HB2 ARG A  99       0.233 -13.083  30.632  1.00 10.12           H  \nATOM   1505  HB3 ARG A  99       0.908 -14.155  31.590  1.00 10.13           H  \nATOM   1506  HG2 ARG A  99       1.677 -15.382  29.795  1.00 10.98           H  \nATOM   1507  HG3 ARG A  99       0.823 -14.532  28.776  1.00 10.98           H  \nATOM   1508  HD2 ARG A  99       3.132 -14.002  28.650  1.00 11.91           H  \nATOM   1509  HD3 ARG A  99       2.257 -12.719  28.981  1.00 11.92           H  \nATOM   1510  HE  ARG A  99       3.094 -13.797  31.257  1.00 12.95           H  \nATOM   1511 HH11 ARG A  99       4.353 -11.999  28.778  0.00 17.37           H  \nATOM   1512 HH12 ARG A  99       5.503 -11.397  29.585  0.00 17.37           H  \nATOM   1513 HH21 ARG A  99       5.274 -11.741  32.091  1.00 15.14           H  \nATOM   1514 HH22 ARG A  99       5.242 -13.262  32.194  1.00 15.14           H  \nATOM   1515  N   ARG A 100      -1.714 -15.381  32.980  1.00 10.86           N  \nATOM   1516  CA  ARG A 100      -2.315 -15.109  34.274  1.00 11.47           C  \nATOM   1517  C   ARG A 100      -1.247 -15.388  35.316  1.00 11.19           C  \nATOM   1518  O   ARG A 100      -0.608 -16.422  35.299  1.00 11.99           O  \nATOM   1519  CB  ARG A 100      -3.571 -15.963  34.533  1.00 12.96           C  \nATOM   1520  CG  ARG A 100      -4.360 -15.427  35.742  1.00 15.14           C  \nATOM   1521  CD  ARG A 100      -5.624 -16.164  36.117  1.00 19.05           C  \nATOM   1522  NE  ARG A 100      -5.301 -17.536  36.549  1.00 21.68           N  \nATOM   1523  CZ  ARG A 100      -6.102 -18.613  36.453  1.00 23.34           C  \nATOM   1524  NH1 ARG A 100      -7.112 -18.647  35.596  1.00 26.38           N  \nATOM   1525  NH2 ARG A 100      -5.868 -19.648  37.243  1.00 24.63           N  \nATOM   1526  H   ARG A 100      -1.396 -16.221  32.813  1.00 10.90           H  \nATOM   1527  HA  ARG A 100      -2.562 -14.157  34.316  1.00 11.56           H  \nATOM   1528  HB2 ARG A 100      -4.141 -15.945  33.738  1.00 13.07           H  \nATOM   1529  HB3 ARG A 100      -3.302 -16.890  34.703  1.00 13.06           H  \nATOM   1530  HG2 ARG A 100      -3.767 -15.428  36.523  1.00 15.41           H  \nATOM   1531  HG3 ARG A 100      -4.597 -14.492  35.564  1.00 15.41           H  \nATOM   1532  HD2 ARG A 100      -6.078 -15.689  36.845  1.00 18.60           H  \nATOM   1533  HD3 ARG A 100      -6.229 -16.194  35.347  1.00 18.61           H  \nATOM   1534  HE  ARG A 100      -4.517 -17.662  36.907  1.00 21.32           H  \nATOM   1535 HH11 ARG A 100      -7.321 -17.897  35.231  0.00 26.33           H  \nATOM   1536 HH12 ARG A 100      -7.590 -19.356  35.599  0.00 26.33           H  \nATOM   1537 HH21 ARG A 100      -6.542 -20.092  37.592  1.00 24.19           H  \nATOM   1538 HH22 ARG A 100      -5.041 -19.889  37.418  1.00 24.09           H  \nATOM   1539  N   ASN A 101      -1.033 -14.416  36.210  1.00 11.07           N  \nATOM   1540  CA  ASN A 101      -0.077 -14.568  37.289  1.00 10.37           C  \nATOM   1541  C   ASN A 101      -0.820 -14.474  38.622  1.00 11.74           C  \nATOM   1542  O   ASN A 101      -0.900 -13.388  39.225  1.00 13.54           O  \nATOM   1543  CB  ASN A 101       0.998 -13.501  37.177  1.00 11.41           C  \nATOM   1544  CG  ASN A 101       2.159 -13.690  38.115  1.00 13.08           C  \nATOM   1545  OD1 ASN A 101       2.149 -14.560  38.986  1.00 13.56           O  \nATOM   1546  ND2 ASN A 101       3.192 -12.870  37.960  1.00 15.46           N  \nATOM   1547  H   ASN A 101      -1.499 -13.632  36.144  1.00 10.93           H  \nATOM   1548  HA  ASN A 101       0.354 -15.451  37.220  1.00 10.96           H  \nATOM   1549  HB2 ASN A 101       1.335 -13.496  36.258  1.00 11.52           H  \nATOM   1550  HB3 ASN A 101       0.592 -12.627  37.349  1.00 11.53           H  \nATOM   1551 HD21 ASN A 101       3.237 -12.322  37.294  0.00 15.45           H  \nATOM   1552 HD22 ASN A 101       3.968 -13.032  38.436  0.00 15.45           H  \nATOM   1553  N   TYR A 102      -1.430 -15.587  39.041  1.00 10.88           N  \nATOM   1554  CA  TYR A 102      -2.390 -15.589  40.140  1.00 11.89           C  \nATOM   1555  C   TYR A 102      -2.290 -16.909  40.869  1.00 12.05           C  \nATOM   1556  O   TYR A 102      -2.196 -17.959  40.235  1.00 12.37           O  \nATOM   1557  CB  TYR A 102      -3.840 -15.391  39.660  1.00 12.39           C  \nATOM   1558  CG  TYR A 102      -4.825 -15.356  40.793  1.00 14.55           C  \nATOM   1559  CD1 TYR A 102      -5.048 -14.147  41.423  1.00 15.65           C  \nATOM   1560  CD2 TYR A 102      -5.375 -16.506  41.327  1.00 16.31           C  \nATOM   1561  CE1 TYR A 102      -5.844 -14.075  42.558  1.00 19.21           C  \nATOM   1562  CE2 TYR A 102      -6.157 -16.462  42.482  1.00 19.46           C  \nATOM   1563  CZ  TYR A 102      -6.429 -15.228  43.055  1.00 21.50           C  \nATOM   1564  OH  TYR A 102      -7.253 -15.111  44.190  1.00 24.37           O  \nATOM   1565  H   TYR A 102      -1.242 -16.378  38.624  1.00 11.31           H  \nATOM   1566  HA  TYR A 102      -2.163 -14.859  40.762  1.00 11.81           H  \nATOM   1567  HB2 TYR A 102      -3.894 -14.551  39.158  1.00 12.76           H  \nATOM   1568  HB3 TYR A 102      -4.073 -16.124  39.052  1.00 12.75           H  \nATOM   1569  HD1 TYR A 102      -4.646 -13.363  41.085  1.00 16.30           H  \nATOM   1570  HD2 TYR A 102      -5.179 -17.340  40.932  1.00 16.66           H  \nATOM   1571  HE1 TYR A 102      -6.029 -13.241  42.957  1.00 18.65           H  \nATOM   1572  HE2 TYR A 102      -6.571 -17.243  42.810  1.00 18.92           H  \nATOM   1573  HH  TYR A 102      -7.416 -14.368  44.519  0.00 24.19           H  \nATOM   1574  N   PRO A 103      -2.311 -16.932  42.203  1.00 11.66           N  \nATOM   1575  CA  PRO A 103      -2.156 -15.733  43.051  1.00 11.11           C  \nATOM   1576  C   PRO A 103      -0.851 -15.003  42.756  1.00 11.43           C  \nATOM   1577  O   PRO A 103       0.122 -15.575  42.294  1.00 11.99           O  \nATOM   1578  CB  PRO A 103      -2.118 -16.323  44.467  1.00 13.26           C  \nATOM   1579  CG  PRO A 103      -2.845 -17.617  44.343  1.00 13.43           C  \nATOM   1580  CD  PRO A 103      -2.511 -18.154  42.961  1.00 13.33           C  \nATOM   1581  HA  PRO A 103      -2.936 -15.132  42.948  1.00 11.67           H  \nATOM   1582  HB2 PRO A 103      -1.194 -16.470  44.764  1.00 12.75           H  \nATOM   1583  HB3 PRO A 103      -2.566 -15.730  45.107  1.00 12.75           H  \nATOM   1584  HG2 PRO A 103      -2.552 -18.245  45.036  1.00 13.37           H  \nATOM   1585  HG3 PRO A 103      -3.810 -17.480  44.434  1.00 13.37           H  \nATOM   1586  HD2 PRO A 103      -1.701 -18.697  42.976  1.00 12.94           H  \nATOM   1587  HD3 PRO A 103      -3.246 -18.681  42.597  1.00 12.94           H  \nATOM   1588  N   TYR A 104      -0.821 -13.724  43.116  1.00 10.70           N  \nATOM   1589  CA  TYR A 104       0.333 -12.870  42.906  1.00 11.72           C  \nATOM   1590  C   TYR A 104       1.112 -12.747  44.214  1.00 11.79           C  \nATOM   1591  O   TYR A 104       0.523 -12.509  45.273  1.00 14.45           O  \nATOM   1592  CB  TYR A 104      -0.062 -11.486  42.391  1.00 13.79           C  \nATOM   1593  CG  TYR A 104       1.099 -10.678  41.890  1.00 14.82           C  \nATOM   1594  CD1 TYR A 104       1.880  -9.971  42.771  1.00 15.97           C  \nATOM   1595  CD2 TYR A 104       1.447 -10.664  40.560  1.00 16.97           C  \nATOM   1596  CE1 TYR A 104       2.983  -9.252  42.354  1.00 18.58           C  \nATOM   1597  CE2 TYR A 104       2.482  -9.880  40.102  1.00 20.71           C  \nATOM   1598  CZ  TYR A 104       3.281  -9.196  41.003  1.00 20.65           C  \nATOM   1599  OH  TYR A 104       4.385  -8.464  40.590  1.00 25.35           O  \nATOM   1600  H   TYR A 104      -1.559 -13.364  43.516  1.00 11.10           H  \nATOM   1601  HA  TYR A 104       0.921 -13.295  42.232  1.00 11.89           H  \nATOM   1602  HB2 TYR A 104      -0.710 -11.597  41.665  1.00 13.51           H  \nATOM   1603  HB3 TYR A 104      -0.502 -10.997  43.117  1.00 13.50           H  \nATOM   1604  HD1 TYR A 104       1.680 -10.006  43.692  1.00 16.44           H  \nATOM   1605  HD2 TYR A 104       0.901 -11.114  39.936  1.00 17.31           H  \nATOM   1606  HE1 TYR A 104       3.498  -8.765  42.976  1.00 18.33           H  \nATOM   1607  HE2 TYR A 104       2.707  -9.885  39.186  1.00 19.54           H  \nATOM   1608  HH  TYR A 104       4.863  -8.043  41.120  0.00 25.65           H  \nATOM   1609  N   TYR A 105       2.416 -12.982  44.136  1.00 11.66           N  \nATOM   1610  CA  TYR A 105       3.278 -12.827  45.312  1.00 11.98           C  \nATOM   1611  C   TYR A 105       4.684 -12.567  44.797  1.00 11.40           C  \nATOM   1612  O   TYR A 105       4.946 -12.639  43.584  1.00 12.20           O  \nATOM   1613  CB  TYR A 105       3.193 -14.014  46.260  1.00 11.51           C  \nATOM   1614  CG  TYR A 105       3.469 -15.365  45.668  1.00 12.25           C  \nATOM   1615  CD1 TYR A 105       4.757 -15.777  45.414  1.00 13.79           C  \nATOM   1616  CD2 TYR A 105       2.419 -16.207  45.328  1.00 14.51           C  \nATOM   1617  CE1 TYR A 105       5.019 -16.994  44.788  1.00 14.65           C  \nATOM   1618  CE2 TYR A 105       2.677 -17.419  44.724  1.00 14.59           C  \nATOM   1619  CZ  TYR A 105       3.963 -17.780  44.429  1.00 16.04           C  \nATOM   1620  OH  TYR A 105       4.275 -19.023  43.880  1.00 22.02           O  \nATOM   1621  H   TYR A 105       2.787 -13.242  43.342  1.00 11.76           H  \nATOM   1622  HA  TYR A 105       2.984 -12.019  45.805  1.00 11.72           H  \nATOM   1623  HB2 TYR A 105       3.826 -13.863  46.992  1.00 11.79           H  \nATOM   1624  HB3 TYR A 105       2.294 -14.025  46.650  1.00 11.80           H  \nATOM   1625  HD1 TYR A 105       5.474 -15.201  45.616  1.00 13.71           H  \nATOM   1626  HD2 TYR A 105       1.528 -15.937  45.480  1.00 14.03           H  \nATOM   1627  HE1 TYR A 105       5.905 -17.268  44.614  1.00 14.73           H  \nATOM   1628  HE2 TYR A 105       1.963 -17.989  44.487  1.00 14.86           H  \nATOM   1629  HH  TYR A 105       5.058 -19.297  43.696  0.00 22.13           H  \nATOM   1630  N   TYR A 106       5.601 -12.185  45.703  1.00 11.57           N  \nATOM   1631  CA  TYR A 106       6.898 -11.786  45.234  1.00 10.40           C  \nATOM   1632  C   TYR A 106       7.557 -12.952  44.479  1.00 11.27           C  \nATOM   1633  O   TYR A 106       7.713 -14.041  45.032  1.00 12.36           O  \nATOM   1634  CB  TYR A 106       7.792 -11.396  46.433  1.00 10.75           C  \nATOM   1635  CG  TYR A 106       8.887 -10.435  46.083  1.00 11.71           C  \nATOM   1636  CD1 TYR A 106      10.042 -10.844  45.455  1.00 12.22           C  \nATOM   1637  CD2 TYR A 106       8.679  -9.102  46.316  1.00 14.27           C  \nATOM   1638  CE1 TYR A 106      11.000  -9.910  45.125  1.00 13.19           C  \nATOM   1639  CE2 TYR A 106       9.626  -8.152  46.000  1.00 14.17           C  \nATOM   1640  CZ  TYR A 106      10.766  -8.565  45.340  1.00 14.51           C  \nATOM   1641  OH  TYR A 106      11.669  -7.550  45.022  1.00 17.64           O  \nATOM   1642  H   TYR A 106       5.411 -12.178  46.597  1.00 11.25           H  \nATOM   1643  HA  TYR A 106       6.804 -11.010  44.625  1.00 10.86           H  \nATOM   1644  HB2 TYR A 106       7.226 -10.995  47.124  1.00 10.90           H  \nATOM   1645  HB3 TYR A 106       8.191 -12.210  46.806  1.00 10.89           H  \nATOM   1646  HD1 TYR A 106      10.188 -11.758  45.275  1.00 12.44           H  \nATOM   1647  HD2 TYR A 106       7.879  -8.828  46.729  1.00 13.58           H  \nATOM   1648  HE1 TYR A 106      11.786 -10.181  44.686  1.00 13.27           H  \nATOM   1649  HE2 TYR A 106       9.461  -7.236  46.151  1.00 14.17           H  \nATOM   1650  HH  TYR A 106      12.452  -7.793  44.649  0.00 17.30           H  \nATOM   1651  N   GLY A 107       7.968 -12.673  43.228  1.00 12.07           N  \nATOM   1652  CA  GLY A 107       8.622 -13.710  42.431  1.00 12.16           C  \nATOM   1653  C   GLY A 107       7.648 -14.645  41.718  1.00 13.27           C  \nATOM   1654  O   GLY A 107       8.099 -15.582  41.042  1.00 14.50           O  \nATOM   1655  H   GLY A 107       7.842 -11.841  42.870  1.00 11.89           H  \nATOM   1656  HA2 GLY A 107       9.199 -13.272  41.756  1.00 12.39           H  \nATOM   1657  HA3 GLY A 107       9.208 -14.244  43.022  1.00 12.40           H  \nATOM   1658  N   SER A 108       6.336 -14.557  41.996  1.00 11.67           N  \nATOM   1659  CA  SER A 108       5.391 -15.468  41.343  1.00 11.74           C  \nATOM   1660  C   SER A 108       5.495 -15.355  39.818  1.00 12.01           C  \nATOM   1661  O   SER A 108       5.632 -14.285  39.223  1.00 11.85           O  \nATOM   1662  CB  SER A 108       3.977 -15.263  41.795  1.00 11.67           C  \nATOM   1663  OG  SER A 108       3.424 -14.047  41.363  1.00 11.87           O  \nATOM   1664  H   SER A 108       6.022 -13.931  42.582  1.00 12.04           H  \nATOM   1665  HA  SER A 108       5.655 -16.394  41.594  1.00 11.77           H  \nATOM   1666  HB2 SER A 108       3.425 -16.004  41.457  1.00 11.74           H  \nATOM   1667  HB3 SER A 108       3.950 -15.295  42.777  1.00 11.74           H  \nATOM   1668  HG  SER A 108       3.202 -13.750  40.684  0.00 11.68           H  \nATOM   1669  N   ARG A 109       5.402 -16.532  39.161  1.00 12.48           N  \nATOM   1670  CA  ARG A 109       5.424 -16.596  37.689  1.00 12.86           C  \nATOM   1671  C   ARG A 109       4.078 -17.119  37.220  1.00 11.83           C  \nATOM   1672  O   ARG A 109       3.423 -17.914  37.904  1.00 12.03           O  \nATOM   1673  CB  ARG A 109       6.506 -17.533  37.135  1.00 13.61           C  \nATOM   1674  CG  ARG A 109       7.872 -17.361  37.764  1.00 16.01           C  \nATOM   1675  CD  ARG A 109       8.479 -16.078  37.472  1.00 17.55           C  \nATOM   1676  NE  ARG A 109       9.959 -16.238  37.529  1.00 20.59           N  \nATOM   1677  CZ  ARG A 109      10.690 -16.627  38.581  1.00 21.89           C  \nATOM   1678  NH1 ARG A 109      10.248 -16.605  39.846  1.00 20.26           N  \nATOM   1679  NH2 ARG A 109      11.926 -17.004  38.332  1.00 21.91           N  \nATOM   1680  H   ARG A 109       5.314 -17.308  39.634  1.00 12.46           H  \nATOM   1681  HA  ARG A 109       5.562 -15.691  37.329  1.00 12.71           H  \nATOM   1682  HB2 ARG A 109       6.214 -18.459  37.269  1.00 13.95           H  \nATOM   1683  HB3 ARG A 109       6.583 -17.381  36.170  1.00 13.96           H  \nATOM   1684  HG2 ARG A 109       7.794 -17.461  38.736  1.00 15.74           H  \nATOM   1685  HG3 ARG A 109       8.464 -18.073  37.438  1.00 15.76           H  \nATOM   1686  HD2 ARG A 109       8.211 -15.773  36.579  1.00 17.81           H  \nATOM   1687  HD3 ARG A 109       8.189 -15.412  38.132  1.00 17.83           H  \nATOM   1688  HE  ARG A 109      10.398 -16.057  36.798  1.00 20.12           H  \nATOM   1689 HH11 ARG A 109       9.402 -16.479  39.860  0.00 20.32           H  \nATOM   1690 HH12 ARG A 109      10.784 -16.844  40.400  0.00 20.32           H  \nATOM   1691 HH21 ARG A 109      12.585 -16.695  38.824  1.00 21.91           H  \nATOM   1692 HH22 ARG A 109      12.088 -17.564  37.674  1.00 21.91           H  \nATOM   1693  N   GLY A 110       3.676 -16.652  36.011  1.00 11.27           N  \nATOM   1694  CA  GLY A 110       2.385 -16.966  35.427  1.00 11.36           C  \nATOM   1695  C   GLY A 110       2.390 -18.207  34.545  1.00 10.46           C  \nATOM   1696  O   GLY A 110       3.330 -18.987  34.474  1.00 10.86           O  \nATOM   1697  H   GLY A 110       4.244 -16.115  35.538  1.00 11.43           H  \nATOM   1698  HA2 GLY A 110       1.730 -17.094  36.157  1.00 11.12           H  \nATOM   1699  HA3 GLY A 110       2.085 -16.191  34.891  1.00 11.10           H  \nATOM   1700  N   TYR A 111       1.261 -18.292  33.828  1.00 10.37           N  \nATOM   1701  CA  TYR A 111       0.892 -19.444  33.023  1.00 10.16           C  \nATOM   1702  C   TYR A 111      -0.187 -18.979  32.078  1.00 10.05           C  \nATOM   1703  O   TYR A 111      -0.674 -17.853  32.179  1.00 10.32           O  \nATOM   1704  CB  TYR A 111       0.483 -20.615  33.923  1.00 11.59           C  \nATOM   1705  CG  TYR A 111      -0.489 -20.186  34.988  1.00 10.82           C  \nATOM   1706  CD1 TYR A 111      -1.835 -20.036  34.779  1.00 11.16           C  \nATOM   1707  CD2 TYR A 111      -0.025 -19.833  36.241  1.00 11.76           C  \nATOM   1708  CE1 TYR A 111      -2.709 -19.585  35.753  1.00 12.08           C  \nATOM   1709  CE2 TYR A 111      -0.876 -19.362  37.233  1.00 11.56           C  \nATOM   1710  CZ  TYR A 111      -2.222 -19.219  37.000  1.00 11.86           C  \nATOM   1711  OH  TYR A 111      -3.181 -18.762  37.895  1.00 14.33           O  \nATOM   1712  H   TYR A 111       0.684 -17.583  33.848  1.00 10.35           H  \nATOM   1713  HA  TYR A 111       1.679 -19.718  32.487  1.00 10.44           H  \nATOM   1714  HB2 TYR A 111       0.074 -21.314  33.371  1.00 11.05           H  \nATOM   1715  HB3 TYR A 111       1.285 -20.987  34.345  1.00 11.06           H  \nATOM   1716  HD1 TYR A 111      -2.185 -20.253  33.930  1.00 11.35           H  \nATOM   1717  HD2 TYR A 111       0.899 -19.893  36.422  1.00 11.52           H  \nATOM   1718  HE1 TYR A 111      -3.627 -19.494  35.560  1.00 11.85           H  \nATOM   1719  HE2 TYR A 111      -0.524 -19.129  38.077  1.00 11.68           H  \nATOM   1720  HH  TYR A 111      -3.972 -18.719  37.706  0.00 14.16           H  \nATOM   1721  N   PHE A 112      -0.614 -19.841  31.128  1.00  9.59           N  \nATOM   1722  CA  PHE A 112      -1.604 -19.477  30.139  1.00 10.19           C  \nATOM   1723  C   PHE A 112      -2.978 -19.962  30.578  1.00 12.70           C  \nATOM   1724  O   PHE A 112      -3.208 -21.166  30.610  1.00 16.38           O  \nATOM   1725  CB  PHE A 112      -1.222 -20.044  28.778  1.00 10.30           C  \nATOM   1726  CG  PHE A 112       0.056 -19.503  28.217  1.00  9.39           C  \nATOM   1727  CD1 PHE A 112       0.184 -18.128  28.038  1.00 10.06           C  \nATOM   1728  CD2 PHE A 112       1.124 -20.328  27.887  1.00  9.95           C  \nATOM   1729  CE1 PHE A 112       1.335 -17.596  27.513  1.00 10.27           C  \nATOM   1730  CE2 PHE A 112       2.290 -19.778  27.370  1.00 10.47           C  \nATOM   1731  CZ  PHE A 112       2.368 -18.427  27.139  1.00 10.62           C  \nATOM   1732  H   PHE A 112      -0.264 -20.686  31.106  1.00  9.84           H  \nATOM   1733  HA  PHE A 112      -1.631 -18.493  30.070  1.00 10.53           H  \nATOM   1734  HB2 PHE A 112      -1.145 -21.018  28.859  1.00 10.06           H  \nATOM   1735  HB3 PHE A 112      -1.948 -19.853  28.148  1.00 10.06           H  \nATOM   1736  HD1 PHE A 112      -0.539 -17.561  28.251  1.00  9.97           H  \nATOM   1737  HD2 PHE A 112       1.053 -21.262  28.008  1.00  9.96           H  \nATOM   1738  HE1 PHE A 112       1.402 -16.665  27.375  1.00 10.29           H  \nATOM   1739  HE2 PHE A 112       3.007 -20.341  27.123  1.00 10.36           H  \nATOM   1740  HZ  PHE A 112       3.171 -18.055  26.812  1.00 10.48           H  \nATOM   1741  N   ASP A 113      -3.902 -19.058  30.796  1.00 10.95           N  \nATOM   1742  CA  ASP A 113      -5.254 -19.483  31.154  1.00 12.27           C  \nATOM   1743  C   ASP A 113      -6.286 -19.447  30.043  1.00 14.32           C  \nATOM   1744  O   ASP A 113      -7.319 -20.088  30.202  1.00 17.64           O  \nATOM   1745  CB  ASP A 113      -5.759 -18.708  32.366  1.00 14.06           C  \nATOM   1746  CG  ASP A 113      -6.063 -17.240  32.184  1.00 15.89           C  \nATOM   1747  OD1 ASP A 113      -5.560 -16.667  31.186  1.00 14.18           O  \nATOM   1748  OD2 ASP A 113      -6.829 -16.701  33.069  1.00 16.96           O  \nATOM   1749  H   ASP A 113      -3.711 -18.166  30.728  1.00 11.64           H  \nATOM   1750  HA  ASP A 113      -5.188 -20.422  31.446  1.00 12.70           H  \nATOM   1751  HB2 ASP A 113      -6.578 -19.140  32.683  1.00 14.01           H  \nATOM   1752  HB3 ASP A 113      -5.089 -18.783  33.076  1.00 14.01           H  \nATOM   1753  N   TYR A 114      -6.050 -18.732  28.948  1.00 13.14           N  \nATOM   1754  CA  TYR A 114      -6.957 -18.673  27.803  1.00 12.97           C  \nATOM   1755  C   TYR A 114      -6.123 -18.647  26.530  1.00 12.58           C  \nATOM   1756  O   TYR A 114      -5.048 -18.053  26.495  1.00 11.99           O  \nATOM   1757  CB  TYR A 114      -7.865 -17.435  27.833  1.00 14.79           C  \nATOM   1758  CG  TYR A 114      -9.049 -17.475  28.767  1.00 17.81           C  \nATOM   1759  CD1 TYR A 114     -10.252 -18.031  28.340  1.00 19.81           C  \nATOM   1760  CD2 TYR A 114      -8.958 -16.943  30.046  1.00 18.53           C  \nATOM   1761  CE1 TYR A 114     -11.352 -18.067  29.175  1.00 20.85           C  \nATOM   1762  CE2 TYR A 114     -10.059 -16.982  30.905  1.00 21.02           C  \nATOM   1763  CZ  TYR A 114     -11.249 -17.534  30.446  1.00 21.75           C  \nATOM   1764  OH  TYR A 114     -12.374 -17.553  31.244  1.00 27.50           O  \nATOM   1765  H   TYR A 114      -5.279 -18.242  28.909  1.00 13.36           H  \nATOM   1766  HA  TYR A 114      -7.522 -19.487  27.797  1.00 13.26           H  \nATOM   1767  HB2 TYR A 114      -7.312 -16.663  28.072  1.00 14.99           H  \nATOM   1768  HB3 TYR A 114      -8.198 -17.284  26.924  1.00 14.98           H  \nATOM   1769  HD1 TYR A 114     -10.319 -18.386  27.469  1.00 19.61           H  \nATOM   1770  HD2 TYR A 114      -8.146 -16.566  30.342  1.00 19.02           H  \nATOM   1771  HE1 TYR A 114     -12.168 -18.434  28.877  1.00 20.71           H  \nATOM   1772  HE2 TYR A 114     -10.007 -16.611  31.770  1.00 20.50           H  \nATOM   1773  HH  TYR A 114     -13.045 -17.863  30.971  0.00 27.73           H  \nATOM   1774  N   TRP A 115      -6.635 -19.320  25.496  1.00 12.61           N  \nATOM   1775  CA  TRP A 115      -6.008 -19.487  24.196  1.00 12.70           C  \nATOM   1776  C   TRP A 115      -7.044 -19.206  23.118  1.00 12.95           C  \nATOM   1777  O   TRP A 115      -8.241 -19.495  23.310  1.00 15.19           O  \nATOM   1778  CB  TRP A 115      -5.467 -20.899  23.992  1.00 11.94           C  \nATOM   1779  CG  TRP A 115      -4.472 -21.378  24.992  1.00 12.67           C  \nATOM   1780  CD1 TRP A 115      -4.654 -21.604  26.312  1.00 12.87           C  \nATOM   1781  CD2 TRP A 115      -3.131 -21.777  24.715  1.00 11.07           C  \nATOM   1782  NE1 TRP A 115      -3.516 -22.087  26.896  1.00 12.82           N  \nATOM   1783  CE2 TRP A 115      -2.555 -22.226  25.930  1.00 12.32           C  \nATOM   1784  CE3 TRP A 115      -2.342 -21.775  23.553  1.00 11.89           C  \nATOM   1785  CZ2 TRP A 115      -1.241 -22.679  25.977  1.00 11.62           C  \nATOM   1786  CZ3 TRP A 115      -1.053 -22.247  23.609  1.00 11.69           C  \nATOM   1787  CH2 TRP A 115      -0.502 -22.687  24.810  1.00 12.40           C  \nATOM   1788  H   TRP A 115      -7.453 -19.709  25.616  1.00 12.63           H  \nATOM   1789  HA  TRP A 115      -5.264 -18.846  24.116  1.00 12.58           H  \nATOM   1790  HB2 TRP A 115      -6.228 -21.515  23.992  1.00 12.29           H  \nATOM   1791  HB3 TRP A 115      -5.059 -20.939  23.103  1.00 12.29           H  \nATOM   1792  HD1 TRP A 115      -5.458 -21.436  26.780  1.00 12.71           H  \nATOM   1793  HE1 TRP A 115      -3.428 -22.302  27.742  1.00 12.62           H  \nATOM   1794  HE3 TRP A 115      -2.718 -21.514  22.727  1.00 11.72           H  \nATOM   1795  HZ2 TRP A 115      -0.859 -22.957  26.792  1.00 11.99           H  \nATOM   1796  HZ3 TRP A 115      -0.527 -22.259  22.825  1.00 11.93           H  \nATOM   1797  HH2 TRP A 115       0.395 -22.972  24.832  1.00 12.06           H  \nATOM   1798  N   GLY A 116      -6.584 -18.719  21.984  1.00 12.97           N  \nATOM   1799  CA  GLY A 116      -7.448 -18.640  20.830  1.00 11.97           C  \nATOM   1800  C   GLY A 116      -7.518 -20.003  20.120  1.00 13.07           C  \nATOM   1801  O   GLY A 116      -6.946 -21.003  20.569  1.00 13.78           O  \nATOM   1802  H   GLY A 116      -5.720 -18.429  21.920  1.00 12.72           H  \nATOM   1803  HA2 GLY A 116      -8.355 -18.369  21.117  1.00 12.46           H  \nATOM   1804  HA3 GLY A 116      -7.102 -17.956  20.210  1.00 12.46           H  \nATOM   1805  N   GLN A 117      -8.221 -20.023  18.992  1.00 12.38           N  \nATOM   1806  CA AGLN A 117      -8.386 -21.278  18.264  0.50 13.42           C  \nATOM   1807  CA BGLN A 117      -8.434 -21.217  18.186  0.50 12.74           C  \nATOM   1808  C   GLN A 117      -7.220 -21.544  17.320  1.00 13.86           C  \nATOM   1809  O   GLN A 117      -7.114 -22.683  16.825  1.00 13.91           O  \nATOM   1810  CB AGLN A 117      -9.709 -21.368  17.493  0.50 15.35           C  \nATOM   1811  CB BGLN A 117      -9.723 -21.037  17.360  0.50 13.39           C  \nATOM   1812  CG AGLN A 117     -10.941 -21.565  18.375  0.50 16.56           C  \nATOM   1813  CG BGLN A 117      -9.585 -20.362  16.005  0.50 13.89           C  \nATOM   1814  CD AGLN A 117     -10.974 -22.844  19.182  0.50 17.11           C  \nATOM   1815  CD BGLN A 117      -9.273 -18.885  16.046  0.50 13.58           C  \nATOM   1816  OE1AGLN A 117     -11.059 -23.950  18.650  0.50 22.53           O  \nATOM   1817  OE1BGLN A 117      -9.624 -18.192  17.004  0.50 13.97           O  \nATOM   1818  NE2AGLN A 117     -10.906 -22.695  20.490  0.50 18.20           N  \nATOM   1819  NE2BGLN A 117      -8.673 -18.384  14.986  0.50 10.97           N  \nATOM   1820  H   GLN A 117      -8.599 -19.249  18.687  1.00 12.78           H  \nATOM   1821  HA AGLN A 117      -8.391 -22.002  18.932  0.50 13.64           H  \nATOM   1822  HA BGLN A 117      -8.574 -21.972  18.803  0.50 13.00           H  \nATOM   1823  HB2AGLN A 117      -9.819 -20.546  16.972  0.50 15.14           H  \nATOM   1824  HB2BGLN A 117     -10.116 -21.925  17.224  0.50 13.35           H  \nATOM   1825  HB3AGLN A 117      -9.650 -22.115  16.860  0.50 15.13           H  \nATOM   1826  HB3BGLN A 117     -10.357 -20.521  17.900  0.50 13.35           H  \nATOM   1827  HG2AGLN A 117     -11.003 -20.807  18.994  0.50 16.39           H  \nATOM   1828  HG2BGLN A 117      -8.881 -20.815  15.496  0.50 13.70           H  \nATOM   1829  HG3AGLN A 117     -11.738 -21.540  17.805  0.50 16.39           H  \nATOM   1830  HG3BGLN A 117     -10.424 -20.486  15.511  0.50 13.70           H  \nATOM   1831 HE21AGLN A 117     -10.799 -21.926  20.739  0.00 30.03           H  \nATOM   1832 HE21BGLN A 117      -8.370 -18.879  14.444  0.00 30.03           H  \nATOM   1833 HE22AGLN A 117     -10.774 -23.453  20.845  0.00 30.03           H  \nATOM   1834 HE22BGLN A 117      -8.468 -17.515  15.131  0.00 30.03           H  \nATOM   1835  N   GLY A 118      -6.369 -20.534  17.080  1.00 12.48           N  \nATOM   1836  CA  GLY A 118      -5.170 -20.676  16.264  1.00 12.87           C  \nATOM   1837  C   GLY A 118      -5.413 -20.284  14.813  1.00 13.84           C  \nATOM   1838  O   GLY A 118      -6.548 -20.347  14.310  1.00 16.36           O  \nATOM   1839  H   GLY A 118      -6.555 -19.713  17.431  1.00 12.89           H  \nATOM   1840  HA2 GLY A 118      -4.460 -20.104  16.644  1.00 12.99           H  \nATOM   1841  HA3 GLY A 118      -4.859 -21.615  16.302  1.00 13.00           H  \nATOM   1842  N   THR A 119      -4.337 -19.835  14.192  1.00 12.72           N  \nATOM   1843  CA  THR A 119      -4.289 -19.501  12.771  1.00 11.83           C  \nATOM   1844  C   THR A 119      -3.061 -20.207  12.170  1.00 12.44           C  \nATOM   1845  O   THR A 119      -1.966 -20.165  12.706  1.00 12.43           O  \nATOM   1846  CB  THR A 119      -4.368 -17.981  12.566  1.00 12.16           C  \nATOM   1847  OG1 THR A 119      -4.625 -17.711  11.168  1.00 14.50           O  \nATOM   1848  CG2 THR A 119      -3.107 -17.237  12.920  1.00 12.68           C  \nATOM   1849  H   THR A 119      -3.572 -19.726  14.680  1.00 12.76           H  \nATOM   1850  HA  THR A 119      -5.089 -19.895  12.354  1.00 12.19           H  \nATOM   1851  HB  THR A 119      -5.114 -17.628  13.107  1.00 12.61           H  \nATOM   1852  HG1 THR A 119      -4.011 -18.023  10.682  0.00 14.11           H  \nATOM   1853 HG21 THR A 119      -3.309 -16.292  13.039  1.00 12.52           H  \nATOM   1854 HG22 THR A 119      -2.455 -17.340  12.203  1.00 12.52           H  \nATOM   1855 HG23 THR A 119      -2.738 -17.596  13.747  1.00 12.52           H  \nATOM   1856  N   THR A 120      -3.240 -20.850  10.995  1.00 14.03           N  \nATOM   1857  CA  THR A 120      -2.208 -21.657  10.379  1.00 14.94           C  \nATOM   1858  C   THR A 120      -1.330 -20.817   9.460  1.00 12.89           C  \nATOM   1859  O   THR A 120      -1.831 -20.083   8.603  1.00 15.15           O  \nATOM   1860  CB  THR A 120      -2.873 -22.765   9.559  1.00 18.33           C  \nATOM   1861  OG1 THR A 120      -3.496 -23.618  10.514  1.00 23.80           O  \nATOM   1862  CG2 THR A 120      -1.916 -23.581   8.727  1.00 20.77           C  \nATOM   1863  H   THR A 120      -4.043 -20.770  10.568  1.00 13.85           H  \nATOM   1864  HA  THR A 120      -1.655 -22.063  11.085  1.00 14.90           H  \nATOM   1865  HB  THR A 120      -3.560 -22.365   8.974  1.00 18.96           H  \nATOM   1866  HG1 THR A 120      -2.909 -23.934  11.119  0.00 23.67           H  \nATOM   1867 HG21 THR A 120      -2.289 -24.468   8.578  1.00 19.98           H  \nATOM   1868 HG22 THR A 120      -1.065 -23.663   9.194  1.00 19.95           H  \nATOM   1869 HG23 THR A 120      -1.774 -23.142   7.869  1.00 19.96           H  \nATOM   1870  N   VAL A 121      -0.027 -20.923   9.652  1.00 12.48           N  \nATOM   1871  CA  VAL A 121       0.964 -20.324   8.792  1.00 12.75           C  \nATOM   1872  C   VAL A 121       1.744 -21.437   8.124  1.00 12.96           C  \nATOM   1873  O   VAL A 121       2.290 -22.273   8.804  1.00 14.36           O  \nATOM   1874  CB  VAL A 121       1.912 -19.375   9.534  1.00 13.94           C  \nATOM   1875  CG1 VAL A 121       3.027 -18.893   8.639  1.00 16.63           C  \nATOM   1876  CG2 VAL A 121       1.118 -18.214  10.160  1.00 14.73           C  \nATOM   1877  H   VAL A 121       0.267 -21.402  10.372  1.00 12.65           H  \nATOM   1878  HA  VAL A 121       0.504 -19.816   8.101  1.00 12.96           H  \nATOM   1879  HB  VAL A 121       2.323 -19.888  10.272  1.00 14.30           H  \nATOM   1880 HG11 VAL A 121       2.690 -18.772   7.734  1.00 15.74           H  \nATOM   1881 HG12 VAL A 121       3.746 -19.548   8.632  1.00 15.74           H  \nATOM   1882 HG13 VAL A 121       3.368 -18.044   8.971  1.00 15.74           H  \nATOM   1883 HG21 VAL A 121       1.733 -17.593  10.588  1.00 14.48           H  \nATOM   1884 HG22 VAL A 121       0.499 -18.564  10.823  1.00 14.48           H  \nATOM   1885 HG23 VAL A 121       0.620 -17.749   9.465  1.00 14.48           H  \nATOM   1886  N   THR A 122       1.815 -21.430   6.776  1.00 12.99           N  \nATOM   1887  CA  THR A 122       2.579 -22.438   6.071  1.00 14.82           C  \nATOM   1888  C   THR A 122       3.651 -21.745   5.249  1.00 13.37           C  \nATOM   1889  O   THR A 122       3.299 -20.835   4.517  1.00 15.83           O  \nATOM   1890  CB  THR A 122       1.631 -23.207   5.157  1.00 15.25           C  \nATOM   1891  OG1 THR A 122       0.603 -23.848   5.929  1.00 19.71           O  \nATOM   1892  CG2 THR A 122       2.377 -24.272   4.371  1.00 16.72           C  \nATOM   1893  H   THR A 122       1.380 -20.784   6.297  1.00 13.39           H  \nATOM   1894  HA  THR A 122       2.994 -23.053   6.717  1.00 14.25           H  \nATOM   1895  HB  THR A 122       1.216 -22.571   4.527  1.00 16.21           H  \nATOM   1896  HG1 THR A 122       0.969 -24.347   6.543  0.00 19.30           H  \nATOM   1897 HG21 THR A 122       1.744 -24.932   4.036  1.00 16.26           H  \nATOM   1898 HG22 THR A 122       3.024 -24.711   4.951  1.00 16.25           H  \nATOM   1899 HG23 THR A 122       2.842 -23.861   3.622  1.00 16.27           H  \nATOM   1900  N   VAL A 123       4.908 -22.123   5.417  1.00 13.51           N  \nATOM   1901  CA  VAL A 123       5.993 -21.588   4.607  1.00 15.66           C  \nATOM   1902  C   VAL A 123       6.343 -22.633   3.549  1.00 18.16           C  \nATOM   1903  O   VAL A 123       6.584 -23.795   3.864  1.00 19.63           O  \nATOM   1904  CB  VAL A 123       7.207 -21.208   5.449  1.00 17.96           C  \nATOM   1905  CG1 VAL A 123       8.295 -20.596   4.577  1.00 19.86           C  \nATOM   1906  CG2 VAL A 123       6.797 -20.247   6.543  1.00 18.77           C  \nATOM   1907  H   VAL A 123       5.107 -22.740   6.062  1.00 13.96           H  \nATOM   1908  HA  VAL A 123       5.674 -20.788   4.158  1.00 16.10           H  \nATOM   1909  HB  VAL A 123       7.564 -22.030   5.867  1.00 17.99           H  \nATOM   1910 HG11 VAL A 123       7.892 -19.986   3.935  1.00 19.24           H  \nATOM   1911 HG12 VAL A 123       8.766 -21.302   4.101  1.00 19.25           H  \nATOM   1912 HG13 VAL A 123       8.925 -20.109   5.135  1.00 19.25           H  \nATOM   1913 HG21 VAL A 123       7.586 -19.817   6.914  1.00 18.52           H  \nATOM   1914 HG22 VAL A 123       6.335 -20.735   7.247  1.00 18.51           H  \nATOM   1915 HG23 VAL A 123       6.202 -19.571   6.176  1.00 18.51           H  \nATOM   1916  N   SER A 124       6.210 -22.229   2.289  1.00 19.33           N  \nATOM   1917  CA  SER A 124       6.387 -23.163   1.198  1.00 21.83           C  \nATOM   1918  C   SER A 124       7.861 -23.503   1.061  1.00 22.84           C  \nATOM   1919  O   SER A 124       8.726 -23.058   1.841  1.00 22.37           O  \nATOM   1920  CB  SER A 124       5.786 -22.578  -0.027  1.00 21.09           C  \nATOM   1921  OG  SER A 124       6.602 -21.539  -0.474  1.00 28.86           O  \nATOM   1922  H   SER A 124       6.010 -21.357   2.104  1.00 19.59           H  \nATOM   1923  HA  SER A 124       5.894 -23.999   1.421  1.00 21.36           H  \nATOM   1924  HB2 SER A 124       5.709 -23.268  -0.724  1.00 22.90           H  \nATOM   1925  HB3 SER A 124       4.886 -22.235   0.175  1.00 22.88           H  \nATOM   1926  HG  SER A 124       7.296 -21.877  -0.985  0.00 21.88           H  \nATOM   1927  N   SER A 125       8.126 -24.428   0.112  1.00 25.53           N  \nATOM   1928  CA  SER A 125       9.468 -24.960  -0.027  1.00 26.73           C  \nATOM   1929  C   SER A 125      10.312 -24.080  -0.952  1.00 24.03           C  \nATOM   1930  O   SER A 125      11.505 -24.124  -0.621  1.00 31.52           O  \nATOM   1931  CB  SER A 125       9.509 -26.396  -0.468  1.00 26.34           C  \nATOM   1932  OG  SER A 125       9.000 -26.543  -1.786  1.00 26.98           O  \nATOM   1933  H   SER A 125       7.465 -24.729  -0.442  1.00 25.12           H  \nATOM   1934  HA  SER A 125       9.889 -24.923   0.874  1.00 25.89           H  \nATOM   1935  HB2 SER A 125      10.439 -26.715  -0.438  1.00 26.58           H  \nATOM   1936  HB3 SER A 125       8.978 -26.940   0.154  1.00 26.58           H  \nATOM   1937  HG  SER A 125       9.700 -26.353  -2.415  0.00 26.88           H  \nATOM   1938  N   SER A 126      20.795 -22.792  26.104  1.00 39.02           N  \nATOM   1939  CA  SER A 126      20.502 -22.587  27.550  1.00 33.87           C  \nATOM   1940  C   SER A 126      18.993 -22.569  27.838  1.00 28.91           C  \nATOM   1941  O   SER A 126      18.593 -22.363  28.985  1.00 28.84           O  \nATOM   1942  CB  SER A 126      21.139 -21.318  28.018  1.00 35.68           C  \nATOM   1943  OG  SER A 126      20.634 -20.248  27.246  1.00 41.26           O  \nATOM   1944  H   SER A 126      21.695 -22.761  25.968  1.00 37.32           H  \nATOM   1945  HA  SER A 126      20.904 -23.344  28.054  1.00 34.10           H  \nATOM   1946  HB2 SER A 126      20.935 -21.171  28.970  1.00 36.44           H  \nATOM   1947  HB3 SER A 126      22.116 -21.376  27.913  1.00 36.47           H  \nATOM   1948  HG  SER A 126      19.739 -19.847  27.731  0.00 49.07           H  \nATOM   1949  N   ASP A 127      18.137 -22.842  26.840  1.00 25.82           N  \nATOM   1950  CA  ASP A 127      16.711 -22.951  27.135  1.00 23.42           C  \nATOM   1951  C   ASP A 127      16.496 -24.183  28.004  1.00 22.37           C  \nATOM   1952  O   ASP A 127      17.220 -25.174  27.894  1.00 23.82           O  \nATOM   1953  CB  ASP A 127      15.885 -23.065  25.855  1.00 27.51           C  \nATOM   1954  CG  ASP A 127      16.017 -21.876  24.911  1.00 30.90           C  \nATOM   1955  OD1 ASP A 127      16.641 -20.859  25.314  1.00 33.86           O  \nATOM   1956  OD2 ASP A 127      15.500 -21.965  23.781  1.00 36.74           O  \nATOM   1957  H   ASP A 127      18.431 -22.962  25.984  1.00 26.02           H  \nATOM   1958  HA  ASP A 127      16.425 -22.148  27.632  1.00 24.40           H  \nATOM   1959  HB2 ASP A 127      16.159 -23.871  25.372  1.00 27.22           H  \nATOM   1960  HB3 ASP A 127      14.940 -23.161  26.095  1.00 27.22           H  \nATOM   1961  N   ILE A 128      15.496 -24.131  28.887  1.00 16.98           N  \nATOM   1962  CA  ILE A 128      15.187 -25.247  29.757  1.00 16.59           C  \nATOM   1963  C   ILE A 128      14.446 -26.303  28.963  1.00 15.39           C  \nATOM   1964  O   ILE A 128      13.460 -26.023  28.299  1.00 15.75           O  \nATOM   1965  CB  ILE A 128      14.380 -24.846  30.989  1.00 16.73           C  \nATOM   1966  CG1 ILE A 128      15.182 -23.820  31.807  1.00 19.65           C  \nATOM   1967  CG2 ILE A 128      13.948 -26.061  31.792  1.00 15.78           C  \nATOM   1968  CD1 ILE A 128      14.560 -23.429  33.091  1.00 24.34           C  \nATOM   1969  H   ILE A 128      14.988 -23.374  28.945  1.00 18.05           H  \nATOM   1970  HA  ILE A 128      16.028 -25.630  30.066  1.00 16.47           H  \nATOM   1971  HB  ILE A 128      13.561 -24.393  30.670  1.00 17.03           H  \nATOM   1972 HG12 ILE A 128      16.070 -24.193  31.991  1.00 19.91           H  \nATOM   1973 HG13 ILE A 128      15.304 -23.014  31.263  1.00 19.94           H  \nATOM   1974 HG21 ILE A 128      14.732 -26.497  32.170  1.00 16.08           H  \nATOM   1975 HG22 ILE A 128      13.479 -26.687  31.215  1.00 16.08           H  \nATOM   1976 HG23 ILE A 128      13.355 -25.784  32.512  1.00 16.10           H  \nATOM   1977 HD11 ILE A 128      14.501 -24.204  33.675  1.00 22.76           H  \nATOM   1978 HD12 ILE A 128      13.667 -23.078  32.929  1.00 22.80           H  \nATOM   1979 HD13 ILE A 128      15.107 -22.750  33.517  1.00 22.82           H  \nATOM   1980  N   GLU A 129      15.020 -27.502  28.940  1.00 14.56           N  \nATOM   1981  CA  GLU A 129      14.441 -28.600  28.186  1.00 14.61           C  \nATOM   1982  C   GLU A 129      13.603 -29.473  29.118  1.00 13.64           C  \nATOM   1983  O   GLU A 129      14.022 -29.861  30.211  1.00 14.73           O  \nATOM   1984  CB  GLU A 129      15.579 -29.413  27.568  1.00 17.48           C  \nATOM   1985  CG  GLU A 129      15.094 -30.471  26.594  1.00 23.18           C  \nATOM   1986  CD  GLU A 129      16.172 -31.335  25.957  1.00 31.55           C  \nATOM   1987  OE1 GLU A 129      15.807 -32.336  25.305  1.00 36.94           O  \nATOM   1988  OE2 GLU A 129      17.368 -31.011  26.108  1.00 38.36           O  \nATOM   1989  H   GLU A 129      15.793 -27.646  29.407  1.00 14.78           H  \nATOM   1990  HA  GLU A 129      13.871 -28.239  27.469  1.00 14.91           H  \nATOM   1991  HB2 GLU A 129      16.186 -28.801  27.101  1.00 17.94           H  \nATOM   1992  HB3 GLU A 129      16.083 -29.848  28.288  1.00 17.91           H  \nATOM   1993  HG2 GLU A 129      14.468 -31.064  27.061  1.00 23.30           H  \nATOM   1994  HG3 GLU A 129      14.595 -30.028  25.876  1.00 23.32           H  \nATOM   1995  N   LEU A 130      12.355 -29.747  28.692  1.00 13.29           N  \nATOM   1996  CA  LEU A 130      11.403 -30.551  29.432  1.00 12.64           C  \nATOM   1997  C   LEU A 130      11.186 -31.837  28.646  1.00 13.09           C  \nATOM   1998  O   LEU A 130      10.810 -31.820  27.468  1.00 13.89           O  \nATOM   1999  CB  LEU A 130      10.058 -29.825  29.593  1.00 12.72           C  \nATOM   2000  CG  LEU A 130      10.068 -28.467  30.290  1.00 15.94           C  \nATOM   2001  CD1 LEU A 130       8.646 -28.041  30.630  1.00 13.49           C  \nATOM   2002  CD2 LEU A 130      10.982 -28.417  31.462  1.00 18.21           C  \nATOM   2003  H   LEU A 130      12.089 -29.407  27.886  1.00 13.22           H  \nATOM   2004  HA  LEU A 130      11.778 -30.765  30.315  1.00 12.89           H  \nATOM   2005  HB2 LEU A 130       9.674 -29.702  28.702  1.00 13.39           H  \nATOM   2006  HB3 LEU A 130       9.455 -30.415  30.087  1.00 13.41           H  \nATOM   2007  HG  LEU A 130      10.412 -27.812  29.633  1.00 15.17           H  \nATOM   2008 HD11 LEU A 130       8.661 -27.166  31.055  1.00 14.21           H  \nATOM   2009 HD12 LEU A 130       8.249 -28.689  31.237  1.00 14.21           H  \nATOM   2010 HD13 LEU A 130       8.117 -27.996  29.814  1.00 14.20           H  \nATOM   2011 HD21 LEU A 130      11.903 -28.369  31.155  1.00 17.45           H  \nATOM   2012 HD22 LEU A 130      10.863 -29.218  32.002  1.00 17.47           H  \nATOM   2013 HD23 LEU A 130      10.779 -27.632  32.000  1.00 17.45           H  \nATOM   2014  N   THR A 131      11.455 -32.932  29.313  1.00 13.41           N  \nATOM   2015  CA  THR A 131      11.344 -34.241  28.679  1.00 14.32           C  \nATOM   2016  C   THR A 131      10.221 -35.000  29.372  1.00 12.84           C  \nATOM   2017  O   THR A 131      10.280 -35.251  30.578  1.00 14.74           O  \nATOM   2018  CB  THR A 131      12.654 -35.025  28.818  1.00 14.30           C  \nATOM   2019  OG1 THR A 131      13.687 -34.267  28.209  1.00 18.03           O  \nATOM   2020  CG2 THR A 131      12.542 -36.368  28.142  1.00 17.83           C  \nATOM   2021  H   THR A 131      11.714 -32.883  30.188  1.00 13.54           H  \nATOM   2022  HA  THR A 131      11.129 -34.126  27.726  1.00 13.85           H  \nATOM   2023  HB  THR A 131      12.855 -35.151  29.776  1.00 15.59           H  \nATOM   2024  HG1 THR A 131      13.539 -34.107  27.404  0.00 18.21           H  \nATOM   2025 HG21 THR A 131      13.423 -36.657  27.844  1.00 16.66           H  \nATOM   2026 HG22 THR A 131      11.948 -36.299  27.373  1.00 16.64           H  \nATOM   2027 HG23 THR A 131      12.184 -37.020  28.770  1.00 16.65           H  \nATOM   2028  N   GLN A 132       9.189 -35.370  28.599  1.00 14.42           N  \nATOM   2029  CA  GLN A 132       8.073 -36.137  29.139  1.00 14.22           C  \nATOM   2030  C   GLN A 132       8.277 -37.635  28.968  1.00 17.03           C  \nATOM   2031  O   GLN A 132       8.856 -38.067  27.971  1.00 19.63           O  \nATOM   2032  CB  GLN A 132       6.755 -35.743  28.471  1.00 14.56           C  \nATOM   2033  CG  GLN A 132       6.420 -34.278  28.794  1.00 14.30           C  \nATOM   2034  CD  GLN A 132       5.090 -33.744  28.308  1.00 12.72           C  \nATOM   2035  OE1 GLN A 132       4.986 -32.606  27.820  1.00 14.39           O  \nATOM   2036  NE2 GLN A 132       4.060 -34.556  28.483  1.00 13.79           N  \nATOM   2037  H   GLN A 132       9.179 -35.139  27.715  1.00 13.99           H  \nATOM   2038  HA  GLN A 132       8.005 -35.940  30.098  1.00 14.82           H  \nATOM   2039  HB2 GLN A 132       6.835 -35.860  27.502  1.00 14.42           H  \nATOM   2040  HB3 GLN A 132       6.039 -36.327  28.798  1.00 14.42           H  \nATOM   2041  HG2 GLN A 132       6.454 -34.165  29.767  1.00 13.97           H  \nATOM   2042  HG3 GLN A 132       7.128 -33.713  28.417  1.00 13.97           H  \nATOM   2043 HE21 GLN A 132       4.201 -35.347  28.858  0.00 13.25           H  \nATOM   2044 HE22 GLN A 132       3.272 -34.320  28.204  0.00 13.25           H  \nATOM   2045  N   SER A 133       7.696 -38.382  29.910  1.00 15.30           N  \nATOM   2046  CA  SER A 133       7.703 -39.831  29.806  1.00 17.58           C  \nATOM   2047  C   SER A 133       6.436 -40.364  30.421  1.00 16.94           C  \nATOM   2048  O   SER A 133       5.939 -39.879  31.434  1.00 17.43           O  \nATOM   2049  CB  SER A 133       8.938 -40.446  30.419  1.00 19.46           C  \nATOM   2050  OG  SER A 133       9.090 -40.167  31.772  1.00 25.33           O  \nATOM   2051  H   SER A 133       7.290 -37.986  30.628  1.00 16.20           H  \nATOM   2052  HA  SER A 133       7.697 -40.058  28.842  1.00 17.33           H  \nATOM   2053  HB2 SER A 133       8.900 -41.421  30.297  1.00 20.22           H  \nATOM   2054  HB3 SER A 133       9.729 -40.117  29.935  1.00 20.22           H  \nATOM   2055  HG  SER A 133       8.516 -40.499  32.434  0.00 25.57           H  \nATOM   2056  N   PRO A 134       5.830 -41.417  29.834  1.00 16.61           N  \nATOM   2057  CA  PRO A 134       6.182 -41.927  28.518  1.00 15.85           C  \nATOM   2058  C   PRO A 134       5.669 -40.961  27.459  1.00 16.72           C  \nATOM   2059  O   PRO A 134       4.978 -40.017  27.820  1.00 19.28           O  \nATOM   2060  CB  PRO A 134       5.372 -43.233  28.481  1.00 15.61           C  \nATOM   2061  CG  PRO A 134       4.168 -42.906  29.263  1.00 15.93           C  \nATOM   2062  CD  PRO A 134       4.664 -42.050  30.419  1.00 16.38           C  \nATOM   2063  HA  PRO A 134       7.153 -42.100  28.438  1.00 16.11           H  \nATOM   2064  HB2 PRO A 134       5.139 -43.481  27.561  1.00 15.74           H  \nATOM   2065  HB3 PRO A 134       5.871 -43.970  28.894  1.00 15.74           H  \nATOM   2066  HG2 PRO A 134       3.522 -42.412  28.717  1.00 15.96           H  \nATOM   2067  HG3 PRO A 134       3.739 -43.721  29.596  1.00 15.97           H  \nATOM   2068  HD2 PRO A 134       3.997 -41.391  30.686  1.00 16.33           H  \nATOM   2069  HD3 PRO A 134       4.906 -42.596  31.190  1.00 16.33           H  \nATOM   2070  N   ALA A 135       5.990 -41.228  26.200  1.00 17.64           N  \nATOM   2071  CA  ALA A 135       5.504 -40.397  25.116  1.00 16.62           C  \nATOM   2072  C   ALA A 135       4.062 -40.742  24.778  1.00 16.92           C  \nATOM   2073  O   ALA A 135       3.353 -39.921  24.224  1.00 16.11           O  \nATOM   2074  CB  ALA A 135       6.364 -40.583  23.896  1.00 18.52           C  \nATOM   2075  H   ALA A 135       6.525 -41.945  26.008  1.00 17.18           H  \nATOM   2076  HA  ALA A 135       5.545 -39.453  25.396  1.00 17.22           H  \nATOM   2077  HB1 ALA A 135       6.276 -39.804  23.323  1.00 17.91           H  \nATOM   2078  HB2 ALA A 135       7.293 -40.682  24.164  1.00 17.91           H  \nATOM   2079  HB3 ALA A 135       6.080 -41.377  23.415  1.00 17.91           H  \nATOM   2080  N   SER A 136       3.684 -41.985  25.086  1.00 17.04           N  \nATOM   2081  CA  SER A 136       2.383 -42.525  24.756  1.00 16.42           C  \nATOM   2082  C   SER A 136       2.030 -43.531  25.832  1.00 15.81           C  \nATOM   2083  O   SER A 136       2.906 -44.309  26.236  1.00 17.38           O  \nATOM   2084  CB  SER A 136       2.378 -43.181  23.385  1.00 20.50           C  \nATOM   2085  OG  SER A 136       1.038 -43.487  23.049  1.00 28.75           O  \nATOM   2086  H   SER A 136       4.279 -42.531  25.514  1.00 16.86           H  \nATOM   2087  HA  SER A 136       1.718 -41.789  24.769  1.00 17.12           H  \nATOM   2088  HB2 SER A 136       2.762 -42.568  22.718  1.00 21.08           H  \nATOM   2089  HB3 SER A 136       2.918 -44.004  23.404  1.00 21.07           H  \nATOM   2090  HG  SER A 136       0.741 -42.794  22.575  0.00 20.76           H  \nATOM   2091  N   LEU A 137       0.774 -43.522  26.252  1.00 16.36           N  \nATOM   2092  CA  LEU A 137       0.365 -44.617  27.113  1.00 18.90           C  \nATOM   2093  C   LEU A 137      -1.105 -44.917  26.873  1.00 16.80           C  \nATOM   2094  O   LEU A 137      -1.888 -44.049  26.484  1.00 16.63           O  \nATOM   2095  CB  LEU A 137       0.712 -44.395  28.590  1.00 24.76           C  \nATOM   2096  CG  LEU A 137      -0.189 -43.455  29.371  1.00 25.70           C  \nATOM   2097  CD1 LEU A 137       0.123 -43.416  30.857  1.00 24.83           C  \nATOM   2098  CD2 LEU A 137      -0.048 -42.046  28.815  1.00 31.25           C  \nATOM   2099  H   LEU A 137       0.183 -42.865  26.017  1.00 16.82           H  \nATOM   2100  HA  LEU A 137       0.865 -45.410  26.812  1.00 18.92           H  \nATOM   2101  HB2 LEU A 137       0.703 -45.266  29.036  1.00 23.42           H  \nATOM   2102  HB3 LEU A 137       1.627 -44.054  28.639  1.00 23.39           H  \nATOM   2103  HG  LEU A 137      -1.127 -43.746  29.256  1.00 26.30           H  \nATOM   2104 HD11 LEU A 137      -0.571 -42.916  31.321  1.00 25.11           H  \nATOM   2105 HD12 LEU A 137       0.983 -42.982  30.997  1.00 25.11           H  \nATOM   2106 HD13 LEU A 137       0.157 -44.323  31.206  1.00 25.10           H  \nATOM   2107 HD21 LEU A 137      -0.601 -41.954  28.021  1.00 29.24           H  \nATOM   2108 HD22 LEU A 137       0.882 -41.877  28.583  1.00 29.35           H  \nATOM   2109 HD23 LEU A 137      -0.335 -41.401  29.485  1.00 29.36           H  \nATOM   2110  N   ALA A 138      -1.451 -46.198  27.086  1.00 15.60           N  \nATOM   2111  CA  ALA A 138      -2.815 -46.644  26.960  1.00 15.83           C  \nATOM   2112  C   ALA A 138      -3.200 -47.340  28.256  1.00 15.65           C  \nATOM   2113  O   ALA A 138      -2.367 -48.052  28.802  1.00 20.30           O  \nATOM   2114  CB  ALA A 138      -2.948 -47.580  25.797  1.00 16.60           C  \nATOM   2115  H   ALA A 138      -0.810 -46.807  27.318  1.00 15.94           H  \nATOM   2116  HA  ALA A 138      -3.399 -45.867  26.822  1.00 15.90           H  \nATOM   2117  HB1 ALA A 138      -3.881 -47.640  25.532  1.00 16.39           H  \nATOM   2118  HB2 ALA A 138      -2.421 -47.248  25.051  1.00 16.39           H  \nATOM   2119  HB3 ALA A 138      -2.627 -48.462  26.050  1.00 16.37           H  \nATOM   2120  N   VAL A 139      -4.324 -46.923  28.804  1.00 15.83           N  \nATOM   2121  CA  VAL A 139      -4.821 -47.520  30.035  1.00 16.42           C  \nATOM   2122  C   VAL A 139      -6.286 -47.893  29.829  1.00 16.31           C  \nATOM   2123  O   VAL A 139      -6.976 -47.403  28.956  1.00 18.36           O  \nATOM   2124  CB  VAL A 139      -4.642 -46.607  31.249  1.00 18.00           C  \nATOM   2125  CG1 VAL A 139      -3.206 -46.142  31.402  1.00 19.98           C  \nATOM   2126  CG2 VAL A 139      -5.614 -45.454  31.202  1.00 19.27           C  \nATOM   2127  H   VAL A 139      -4.804 -46.252  28.411  1.00 15.93           H  \nATOM   2128  HA  VAL A 139      -4.323 -48.341  30.199  1.00 16.58           H  \nATOM   2129  HB  VAL A 139      -4.863 -47.144  32.049  1.00 18.27           H  \nATOM   2130 HG11 VAL A 139      -3.011 -45.464  30.732  1.00 19.35           H  \nATOM   2131 HG12 VAL A 139      -2.606 -46.898  31.282  1.00 19.35           H  \nATOM   2132 HG13 VAL A 139      -3.079 -45.766  32.290  1.00 19.35           H  \nATOM   2133 HG21 VAL A 139      -5.268 -44.716  31.734  1.00 18.86           H  \nATOM   2134 HG22 VAL A 139      -6.472 -45.735  31.564  1.00 18.86           H  \nATOM   2135 HG23 VAL A 139      -5.731 -45.160  30.282  1.00 18.84           H  \nATOM   2136  N   SER A 140      -6.802 -48.739  30.730  1.00 15.59           N  \nATOM   2137  CA  SER A 140      -8.206 -49.076  30.708  1.00 15.83           C  \nATOM   2138  C   SER A 140      -8.990 -48.050  31.498  1.00 14.39           C  \nATOM   2139  O   SER A 140      -8.477 -47.435  32.438  1.00 14.81           O  \nATOM   2140  CB  SER A 140      -8.456 -50.503  31.265  1.00 16.15           C  \nATOM   2141  OG  SER A 140      -7.849 -51.432  30.418  1.00 18.46           O  \nATOM   2142  H   SER A 140      -6.263 -49.111  31.368  1.00 15.82           H  \nATOM   2143  HA  SER A 140      -8.519 -49.050  29.763  1.00 15.56           H  \nATOM   2144  HB2 SER A 140      -8.078 -50.577  32.171  1.00 16.59           H  \nATOM   2145  HB3 SER A 140      -9.424 -50.676  31.315  1.00 16.59           H  \nATOM   2146  HG  SER A 140      -6.973 -51.570  30.622  0.00 11.51           H  \nATOM   2147  N   LEU A 141     -10.279 -47.912  31.200  1.00 14.84           N  \nATOM   2148  CA  LEU A 141     -11.140 -47.046  31.954  1.00 16.62           C  \nATOM   2149  C   LEU A 141     -11.038 -47.360  33.452  1.00 16.09           C  \nATOM   2150  O   LEU A 141     -11.096 -48.534  33.833  1.00 16.75           O  \nATOM   2151  CB  LEU A 141     -12.576 -47.177  31.463  1.00 18.06           C  \nATOM   2152  CG  LEU A 141     -12.855 -46.625  30.064  1.00 19.39           C  \nATOM   2153  CD1 LEU A 141     -14.296 -46.936  29.689  1.00 21.44           C  \nATOM   2154  CD2 LEU A 141     -12.583 -45.132  30.002  1.00 22.18           C  \nATOM   2155  H   LEU A 141     -10.629 -48.382  30.498  1.00 15.16           H  \nATOM   2156  HA  LEU A 141     -10.835 -46.121  31.812  1.00 16.44           H  \nATOM   2157  HB2 LEU A 141     -12.818 -48.124  31.475  1.00 18.02           H  \nATOM   2158  HB3 LEU A 141     -13.160 -46.717  32.098  1.00 18.01           H  \nATOM   2159  HG  LEU A 141     -12.257 -47.080  29.421  1.00 19.99           H  \nATOM   2160 HD11 LEU A 141     -14.471 -46.622  28.785  1.00 20.78           H  \nATOM   2161 HD12 LEU A 141     -14.897 -46.488  30.310  1.00 20.78           H  \nATOM   2162 HD13 LEU A 141     -14.443 -47.897  29.733  1.00 20.78           H  \nATOM   2163 HD21 LEU A 141     -11.628 -44.979  29.900  1.00 21.26           H  \nATOM   2164 HD22 LEU A 141     -12.892 -44.710  30.823  1.00 21.26           H  \nATOM   2165 HD23 LEU A 141     -13.055 -44.747  29.243  1.00 21.27           H  \nATOM   2166  N   GLY A 142     -10.912 -46.312  34.286  1.00 16.60           N  \nATOM   2167  CA  GLY A 142     -10.878 -46.479  35.725  1.00 15.85           C  \nATOM   2168  C   GLY A 142      -9.477 -46.716  36.301  1.00 15.34           C  \nATOM   2169  O   GLY A 142      -9.345 -46.695  37.526  1.00 15.48           O  \nATOM   2170  H   GLY A 142     -10.845 -45.468  33.941  1.00 16.30           H  \nATOM   2171  HA2 GLY A 142     -11.264 -45.674  36.142  1.00 15.91           H  \nATOM   2172  HA3 GLY A 142     -11.459 -47.243  35.969  1.00 15.90           H  \nATOM   2173  N   GLN A 143      -8.463 -46.934  35.452  1.00 13.55           N  \nATOM   2174  CA  GLN A 143      -7.079 -47.038  35.862  1.00 13.67           C  \nATOM   2175  C   GLN A 143      -6.480 -45.642  36.067  1.00 15.59           C  \nATOM   2176  O   GLN A 143      -7.110 -44.647  35.729  1.00 16.20           O  \nATOM   2177  CB  GLN A 143      -6.216 -47.818  34.883  1.00 14.57           C  \nATOM   2178  CG  GLN A 143      -6.561 -49.301  34.819  1.00 14.12           C  \nATOM   2179  CD  GLN A 143      -5.605 -50.079  33.959  1.00 15.12           C  \nATOM   2180  OE1 GLN A 143      -5.148 -49.644  32.898  1.00 16.06           O  \nATOM   2181  NE2 GLN A 143      -5.300 -51.282  34.399  1.00 15.55           N  \nATOM   2182  H   GLN A 143      -8.651 -47.025  34.563  1.00 14.00           H  \nATOM   2183  HA  GLN A 143      -7.057 -47.507  36.726  1.00 14.17           H  \nATOM   2184  HB2 GLN A 143      -6.321 -47.425  33.992  1.00 14.25           H  \nATOM   2185  HB3 GLN A 143      -5.277 -47.720  35.144  1.00 14.26           H  \nATOM   2186  HG2 GLN A 143      -6.552 -49.673  35.726  1.00 14.46           H  \nATOM   2187  HG3 GLN A 143      -7.469 -49.405  34.462  1.00 14.46           H  \nATOM   2188 HE21 GLN A 143      -5.641 -51.566  35.130  0.00 16.16           H  \nATOM   2189 HE22 GLN A 143      -4.732 -51.756  33.914  0.00 16.16           H  \nATOM   2190  N   ARG A 144      -5.306 -45.621  36.698  1.00 15.63           N  \nATOM   2191  CA  ARG A 144      -4.562 -44.356  36.790  1.00 15.77           C  \nATOM   2192  C   ARG A 144      -3.597 -44.241  35.608  1.00 17.78           C  \nATOM   2193  O   ARG A 144      -3.006 -45.192  35.109  1.00 18.41           O  \nATOM   2194  CB  ARG A 144      -3.878 -44.184  38.138  1.00 19.94           C  \nATOM   2195  CG  ARG A 144      -2.566 -44.914  38.266  1.00 21.69           C  \nATOM   2196  CD  ARG A 144      -1.818 -44.504  39.538  1.00 24.20           C  \nATOM   2197  NE  ARG A 144      -2.682 -44.679  40.691  1.00 27.74           N  \nATOM   2198  CZ  ARG A 144      -2.405 -44.241  41.918  1.00 31.43           C  \nATOM   2199  NH1 ARG A 144      -1.435 -43.368  42.129  1.00 34.58           N  \nATOM   2200  NH2 ARG A 144      -3.117 -44.677  42.935  1.00 32.58           N  \nATOM   2201  H   ARG A 144      -4.961 -46.380  37.074  1.00 15.64           H  \nATOM   2202  HA  ARG A 144      -5.218 -43.631  36.705  1.00 16.84           H  \nATOM   2203  HB2 ARG A 144      -3.723 -43.228  38.289  1.00 19.25           H  \nATOM   2204  HB3 ARG A 144      -4.489 -44.502  38.835  1.00 19.27           H  \nATOM   2205  HG2 ARG A 144      -2.731 -45.881  38.287  1.00 21.82           H  \nATOM   2206  HG3 ARG A 144      -2.002 -44.718  37.489  1.00 21.81           H  \nATOM   2207  HD2 ARG A 144      -1.013 -45.056  39.641  1.00 24.36           H  \nATOM   2208  HD3 ARG A 144      -1.542 -43.565  39.471  1.00 24.37           H  \nATOM   2209  HE  ARG A 144      -3.425 -45.121  40.581  1.00 27.61           H  \nATOM   2210 HH11 ARG A 144      -0.937 -43.202  41.448  0.00 35.98           H  \nATOM   2211 HH12 ARG A 144      -1.338 -43.125  42.921  0.00 35.98           H  \nATOM   2212 HH21 ARG A 144      -3.353 -44.122  43.574  1.00 32.22           H  \nATOM   2213 HH22 ARG A 144      -3.354 -45.523  42.972  1.00 32.22           H  \nATOM   2214  N   ALA A 145      -3.458 -42.984  35.157  1.00 16.40           N  \nATOM   2215  CA  ALA A 145      -2.423 -42.617  34.217  1.00 16.34           C  \nATOM   2216  C   ALA A 145      -1.440 -41.666  34.907  1.00 14.64           C  \nATOM   2217  O   ALA A 145      -1.896 -40.729  35.549  1.00 16.47           O  \nATOM   2218  CB  ALA A 145      -3.034 -41.934  33.025  1.00 16.86           C  \nATOM   2219  H   ALA A 145      -4.033 -42.336  35.449  1.00 16.70           H  \nATOM   2220  HA  ALA A 145      -1.944 -43.426  33.922  1.00 16.12           H  \nATOM   2221  HB1 ALA A 145      -2.332 -41.658  32.412  1.00 16.70           H  \nATOM   2222  HB2 ALA A 145      -3.636 -42.547  32.572  1.00 16.70           H  \nATOM   2223  HB3 ALA A 145      -3.532 -41.152  33.319  1.00 16.70           H  \nATOM   2224  N   THR A 146      -0.161 -42.001  34.806  1.00 14.82           N  \nATOM   2225  CA  THR A 146       0.927 -41.279  35.447  1.00 14.85           C  \nATOM   2226  C   THR A 146       1.851 -40.770  34.355  1.00 14.74           C  \nATOM   2227  O   THR A 146       2.435 -41.544  33.596  1.00 17.81           O  \nATOM   2228  CB  THR A 146       1.663 -42.143  36.492  1.00 19.05           C  \nATOM   2229  OG1 THR A 146       0.692 -42.601  37.443  1.00 20.63           O  \nATOM   2230  CG2 THR A 146       2.806 -41.394  37.146  1.00 22.29           C  \nATOM   2231  H   THR A 146       0.054 -42.731  34.299  1.00 14.79           H  \nATOM   2232  HA  THR A 146       0.541 -40.502  35.912  1.00 15.54           H  \nATOM   2233  HB  THR A 146       2.036 -42.929  36.024  1.00 19.01           H  \nATOM   2234  HG1 THR A 146       0.315 -41.868  37.813  0.00 20.30           H  \nATOM   2235 HG21 THR A 146       2.853 -41.630  38.090  1.00 21.22           H  \nATOM   2236 HG22 THR A 146       2.659 -40.437  37.059  1.00 21.18           H  \nATOM   2237 HG23 THR A 146       3.644 -41.634  36.711  1.00 21.22           H  \nATOM   2238  N   ILE A 147       1.965 -39.427  34.221  1.00 14.04           N  \nATOM   2239  CA  ILE A 147       2.739 -38.779  33.184  1.00 14.46           C  \nATOM   2240  C   ILE A 147       3.829 -37.970  33.877  1.00 14.71           C  \nATOM   2241  O   ILE A 147       3.499 -37.162  34.725  1.00 15.75           O  \nATOM   2242  CB  ILE A 147       1.809 -37.869  32.362  1.00 15.37           C  \nATOM   2243  CG1 ILE A 147       0.735 -38.738  31.687  1.00 17.70           C  \nATOM   2244  CG2 ILE A 147       2.610 -37.030  31.357  1.00 17.83           C  \nATOM   2245  CD1 ILE A 147      -0.251 -37.999  30.824  1.00 18.95           C  \nATOM   2246  H   ILE A 147       1.528 -38.895  34.823  1.00 14.30           H  \nATOM   2247  HA  ILE A 147       3.141 -39.448  32.602  1.00 14.60           H  \nATOM   2248  HB  ILE A 147       1.354 -37.252  32.987  1.00 16.04           H  \nATOM   2249 HG12 ILE A 147       1.186 -39.412  31.135  1.00 17.39           H  \nATOM   2250 HG13 ILE A 147       0.236 -39.214  32.385  1.00 17.39           H  \nATOM   2251 HG21 ILE A 147       3.357 -37.553  31.017  1.00 17.02           H  \nATOM   2252 HG22 ILE A 147       2.950 -36.231  31.796  1.00 17.02           H  \nATOM   2253 HG23 ILE A 147       2.037 -36.768  30.616  1.00 17.04           H  \nATOM   2254 HD11 ILE A 147       0.081 -37.960  29.910  1.00 18.54           H  \nATOM   2255 HD12 ILE A 147      -0.366 -37.094  31.164  1.00 18.55           H  \nATOM   2256 HD13 ILE A 147      -1.107 -38.461  30.838  1.00 18.55           H  \nATOM   2257  N   SER A 148       5.073 -38.172  33.497  1.00 15.09           N  \nATOM   2258  CA  SER A 148       6.206 -37.555  34.149  1.00 16.04           C  \nATOM   2259  C   SER A 148       6.805 -36.483  33.252  1.00 14.57           C  \nATOM   2260  O   SER A 148       6.747 -36.572  32.013  1.00 14.95           O  \nATOM   2261  CB  SER A 148       7.247 -38.598  34.485  1.00 20.35           C  \nATOM   2262  OG  SER A 148       6.697 -39.475  35.439  1.00 24.59           O  \nATOM   2263  H   SER A 148       5.232 -38.726  32.787  1.00 15.21           H  \nATOM   2264  HA  SER A 148       5.895 -37.130  34.991  1.00 16.28           H  \nATOM   2265  HB2 SER A 148       7.497 -39.095  33.674  1.00 20.12           H  \nATOM   2266  HB3 SER A 148       8.051 -38.163  34.850  1.00 20.10           H  \nATOM   2267  HG  SER A 148       6.294 -40.132  35.017  0.00 24.32           H  \nATOM   2268  N   CYS A 149       7.391 -35.447  33.894  1.00 13.86           N  \nATOM   2269  CA  CYS A 149       8.090 -34.393  33.161  1.00 15.89           C  \nATOM   2270  C   CYS A 149       9.374 -34.104  33.916  1.00 16.49           C  \nATOM   2271  O   CYS A 149       9.316 -33.847  35.120  1.00 19.60           O  \nATOM   2272  CB  CYS A 149       7.179 -33.173  33.113  1.00 15.69           C  \nATOM   2273  SG  CYS A 149       7.889 -31.683  32.339  1.00 22.23           S  \nATOM   2274  H   CYS A 149       7.351 -35.399  34.806  1.00 14.50           H  \nATOM   2275  HA  CYS A 149       8.292 -34.699  32.248  1.00 15.55           H  \nATOM   2276  HB2 CYS A 149       6.367 -33.410  32.622  1.00 17.07           H  \nATOM   2277  HB3 CYS A 149       6.915 -32.944  34.027  1.00 17.09           H  \nATOM   2278  N   ARG A 150      10.510 -34.217  33.244  1.00 14.92           N  \nATOM   2279  CA  ARG A 150      11.798 -33.899  33.846  1.00 15.67           C  \nATOM   2280  C   ARG A 150      12.388 -32.661  33.181  1.00 15.89           C  \nATOM   2281  O   ARG A 150      12.464 -32.571  31.964  1.00 17.30           O  \nATOM   2282  CB  ARG A 150      12.743 -35.096  33.741  1.00 19.71           C  \nATOM   2283  CG  ARG A 150      14.077 -34.850  34.417  1.00 27.19           C  \nATOM   2284  CD  ARG A 150      14.980 -36.066  34.319  1.00 33.20           C  \nATOM   2285  NE  ARG A 150      15.186 -36.521  32.941  1.00 42.62           N  \nATOM   2286  CZ  ARG A 150      15.709 -35.794  31.949  1.00 46.79           C  \nATOM   2287  NH1 ARG A 150      15.801 -36.316  30.740  1.00 47.18           N  \nATOM   2288  NH2 ARG A 150      16.112 -34.546  32.150  1.00 47.17           N  \nATOM   2289  H   ARG A 150      10.491 -34.504  32.376  1.00 15.45           H  \nATOM   2290  HA  ARG A 150      11.652 -33.704  34.797  1.00 16.27           H  \nATOM   2291  HB2 ARG A 150      12.314 -35.876  34.152  1.00 20.16           H  \nATOM   2292  HB3 ARG A 150      12.895 -35.296  32.794  1.00 20.18           H  \nATOM   2293  HG2 ARG A 150      14.522 -34.083  34.000  1.00 26.44           H  \nATOM   2294  HG3 ARG A 150      13.928 -34.636  35.363  1.00 26.42           H  \nATOM   2295  HD2 ARG A 150      15.850 -35.851  34.717  1.00 33.60           H  \nATOM   2296  HD3 ARG A 150      14.587 -36.798  34.841  1.00 33.60           H  \nATOM   2297  HE  ARG A 150      14.946 -37.337  32.750  1.00 41.08           H  \nATOM   2298 HH11 ARG A 150      15.639 -37.049  30.566  0.00 46.88           H  \nATOM   2299 HH12 ARG A 150      16.139 -35.701  30.047  0.00 46.88           H  \nATOM   2300 HH21 ARG A 150      15.734 -33.887  31.707  1.00 47.05           H  \nATOM   2301 HH22 ARG A 150      16.755 -34.381  32.727  1.00 47.05           H  \nATOM   2302  N   ALA A 151      12.752 -31.669  34.016  1.00 14.53           N  \nATOM   2303  CA  ALA A 151      13.381 -30.455  33.522  1.00 15.09           C  \nATOM   2304  C   ALA A 151      14.903 -30.563  33.573  1.00 15.13           C  \nATOM   2305  O   ALA A 151      15.457 -31.114  34.506  1.00 17.80           O  \nATOM   2306  CB  ALA A 151      12.911 -29.290  34.338  1.00 14.74           C  \nATOM   2307  H   ALA A 151      12.605 -31.756  34.913  1.00 14.98           H  \nATOM   2308  HA  ALA A 151      13.107 -30.316  32.587  1.00 14.93           H  \nATOM   2309  HB1 ALA A 151      13.310 -28.472  33.997  1.00 14.85           H  \nATOM   2310  HB2 ALA A 151      11.943 -29.224  34.283  1.00 14.85           H  \nATOM   2311  HB3 ALA A 151      13.173 -29.415  35.265  1.00 14.85           H  \nATOM   2312  N   SER A 152      15.550 -29.922  32.603  1.00 14.65           N  \nATOM   2313  CA  SER A 152      17.017 -29.931  32.504  1.00 15.64           C  \nATOM   2314  C   SER A 152      17.684 -29.178  33.652  1.00 17.93           C  \nATOM   2315  O   SER A 152      18.843 -29.472  33.947  1.00 20.21           O  \nATOM   2316  CB  SER A 152      17.474 -29.458  31.172  1.00 16.85           C  \nATOM   2317  OG  SER A 152      17.089 -28.108  30.916  1.00 16.91           O  \nATOM   2318  H   SER A 152      15.072 -29.457  31.978  1.00 15.00           H  \nATOM   2319  HA  SER A 152      17.299 -30.882  32.585  1.00 16.08           H  \nATOM   2320  HB2 SER A 152      18.455 -29.530  31.124  1.00 16.57           H  \nATOM   2321  HB3 SER A 152      17.095 -30.041  30.475  1.00 16.57           H  \nATOM   2322  HG  SER A 152      17.612 -27.493  31.330  0.00 17.24           H  \nATOM   2323  N   GLU A 153      16.998 -28.196  34.258  1.00 16.19           N  \nATOM   2324  CA  GLU A 153      17.444 -27.527  35.473  1.00 18.95           C  \nATOM   2325  C   GLU A 153      16.239 -27.252  36.352  1.00 17.23           C  \nATOM   2326  O   GLU A 153      15.083 -27.357  35.929  1.00 17.36           O  \nATOM   2327  CB  GLU A 153      18.122 -26.206  35.153  1.00 24.66           C  \nATOM   2328  CG  GLU A 153      19.303 -26.260  34.204  1.00 33.07           C  \nATOM   2329  CD  GLU A 153      19.690 -24.884  33.676  1.00 40.88           C  \nATOM   2330  OE1 GLU A 153      18.923 -23.914  33.915  1.00 47.10           O  \nATOM   2331  OE2 GLU A 153      20.773 -24.776  33.049  1.00 56.67           O  \nATOM   2332  H   GLU A 153      16.211 -27.923  33.883  1.00 17.20           H  \nATOM   2333  HA  GLU A 153      18.072 -28.113  35.955  1.00 19.00           H  \nATOM   2334  HB2 GLU A 153      17.446 -25.608  34.772  1.00 24.88           H  \nATOM   2335  HB3 GLU A 153      18.423 -25.808  35.997  1.00 24.83           H  \nATOM   2336  HG2 GLU A 153      20.075 -26.650  34.667  1.00 32.41           H  \nATOM   2337  HG3 GLU A 153      19.085 -26.837  33.443  1.00 32.33           H  \nATOM   2338  N   SER A 154      16.455 -26.906  37.606  1.00 15.50           N  \nATOM   2339  CA  SER A 154      15.325 -26.662  38.492  1.00 14.51           C  \nATOM   2340  C   SER A 154      14.439 -25.517  37.989  1.00 13.53           C  \nATOM   2341  O   SER A 154      14.965 -24.498  37.515  1.00 16.10           O  \nATOM   2342  CB  SER A 154      15.789 -26.386  39.913  1.00 15.28           C  \nATOM   2343  OG  SER A 154      14.725 -25.978  40.793  1.00 16.48           O  \nATOM   2344  H   SER A 154      17.307 -26.819  37.925  1.00 15.65           H  \nATOM   2345  HA  SER A 154      14.775 -27.488  38.507  1.00 14.66           H  \nATOM   2346  HB2 SER A 154      16.208 -27.199  40.274  1.00 15.37           H  \nATOM   2347  HB3 SER A 154      16.475 -25.680  39.892  1.00 15.37           H  \nATOM   2348  HG  SER A 154      14.287 -26.661  41.077  0.00 16.44           H  \nATOM   2349  N   VAL A 155      13.130 -25.686  38.161  1.00 13.71           N  \nATOM   2350  CA  VAL A 155      12.190 -24.639  37.778  1.00 14.15           C  \nATOM   2351  C   VAL A 155      11.511 -24.061  39.011  1.00 14.40           C  \nATOM   2352  O   VAL A 155      10.391 -23.558  38.985  1.00 13.11           O  \nATOM   2353  CB  VAL A 155      11.202 -25.135  36.712  1.00 13.09           C  \nATOM   2354  CG1 VAL A 155      11.930 -25.476  35.424  1.00 14.33           C  \nATOM   2355  CG2 VAL A 155      10.404 -26.309  37.202  1.00 14.74           C  \nATOM   2356  H   VAL A 155      12.811 -26.458  38.531  1.00 13.78           H  \nATOM   2357  HA  VAL A 155      12.706 -23.919  37.374  1.00 13.91           H  \nATOM   2358  HB  VAL A 155      10.573 -24.396  36.520  1.00 13.82           H  \nATOM   2359 HG11 VAL A 155      12.535 -26.223  35.579  1.00 13.94           H  \nATOM   2360 HG12 VAL A 155      12.441 -24.704  35.124  1.00 13.94           H  \nATOM   2361 HG13 VAL A 155      11.283 -25.722  34.740  1.00 13.93           H  \nATOM   2362 HG21 VAL A 155       9.633 -26.440  36.624  1.00 14.22           H  \nATOM   2363 HG22 VAL A 155      10.101 -26.144  38.111  1.00 14.21           H  \nATOM   2364 HG23 VAL A 155      10.958 -27.108  37.186  1.00 14.22           H  \nATOM   2365  N   ASP A 156      12.218 -24.213  40.145  1.00 14.65           N  \nATOM   2366  CA  ASP A 156      11.766 -23.690  41.425  1.00 16.05           C  \nATOM   2367  C   ASP A 156      12.476 -22.375  41.720  1.00 13.89           C  \nATOM   2368  O   ASP A 156      13.639 -22.149  41.373  1.00 17.54           O  \nATOM   2369  CB  ASP A 156      12.025 -24.622  42.620  1.00 17.17           C  \nATOM   2370  CG  ASP A 156      11.186 -25.867  42.656  1.00 21.91           C  \nATOM   2371  OD1 ASP A 156      10.506 -26.152  41.653  1.00 20.87           O  \nATOM   2372  OD2 ASP A 156      11.204 -26.562  43.726  1.00 28.42           O  \nATOM   2373  H   ASP A 156      13.016 -24.655  40.118  1.00 14.90           H  \nATOM   2374  HA  ASP A 156      10.799 -23.524  41.366  1.00 15.54           H  \nATOM   2375  HB2 ASP A 156      12.966 -24.891  42.609  1.00 17.90           H  \nATOM   2376  HB3 ASP A 156      11.862 -24.125  43.448  1.00 17.89           H  \nATOM   2377  N   SER A 157      11.667 -21.485  42.273  1.00 15.59           N  \nATOM   2378  CA  SER A 157      12.163 -20.259  42.903  1.00 14.81           C  \nATOM   2379  C   SER A 157      11.121 -19.787  43.908  1.00 14.41           C  \nATOM   2380  O   SER A 157       9.956 -20.133  43.859  1.00 15.47           O  \nATOM   2381  CB  SER A 157      12.530 -19.248  41.889  1.00 17.73           C  \nATOM   2382  OG  SER A 157      11.423 -18.867  41.082  1.00 18.39           O  \nATOM   2383  H   SER A 157      10.765 -21.634  42.259  1.00 14.99           H  \nATOM   2384  HA  SER A 157      12.986 -20.496  43.410  1.00 15.40           H  \nATOM   2385  HB2 SER A 157      12.887 -18.451  42.344  1.00 17.14           H  \nATOM   2386  HB3 SER A 157      13.241 -19.609  41.313  1.00 17.14           H  \nATOM   2387  HG  SER A 157      10.722 -18.317  41.412  0.00 18.18           H  \nATOM   2388  N   TYR A 158      11.614 -19.059  44.944  1.00 15.53           N  \nATOM   2389  CA  TYR A 158      10.731 -18.504  45.979  1.00 16.36           C  \nATOM   2390  C   TYR A 158       9.889 -19.573  46.712  1.00 17.57           C  \nATOM   2391  O   TYR A 158       8.769 -19.306  47.092  1.00 22.23           O  \nATOM   2392  CB  TYR A 158       9.880 -17.372  45.385  1.00 16.18           C  \nATOM   2393  CG  TYR A 158      10.681 -16.281  44.729  1.00 15.88           C  \nATOM   2394  CD1 TYR A 158      11.197 -15.228  45.461  1.00 15.08           C  \nATOM   2395  CD2 TYR A 158      10.989 -16.335  43.386  1.00 16.18           C  \nATOM   2396  CE1 TYR A 158      11.991 -14.277  44.841  1.00 14.50           C  \nATOM   2397  CE2 TYR A 158      11.796 -15.417  42.755  1.00 14.07           C  \nATOM   2398  CZ  TYR A 158      12.290 -14.351  43.492  1.00 15.58           C  \nATOM   2399  OH  TYR A 158      13.070 -13.425  42.853  1.00 16.79           O  \nATOM   2400  H   TYR A 158      12.513 -18.907  45.003  1.00 15.45           H  \nATOM   2401  HA  TYR A 158      11.318 -18.091  46.662  1.00 16.38           H  \nATOM   2402  HB2 TYR A 158       9.265 -17.755  44.726  1.00 16.15           H  \nATOM   2403  HB3 TYR A 158       9.342 -16.979  46.104  1.00 16.16           H  \nATOM   2404  HD1 TYR A 158      11.020 -15.163  46.384  1.00 15.16           H  \nATOM   2405  HD2 TYR A 158      10.670 -17.066  42.884  1.00 15.58           H  \nATOM   2406  HE1 TYR A 158      12.331 -13.559  45.349  1.00 14.94           H  \nATOM   2407  HE2 TYR A 158      11.971 -15.484  41.831  1.00 14.94           H  \nATOM   2408  HH  TYR A 158      12.721 -12.803  42.623  0.00 16.57           H  \nATOM   2409  N   GLY A 159      10.395 -20.788  46.836  1.00 21.41           N  \nATOM   2410  CA  GLY A 159       9.705 -21.850  47.557  1.00 23.38           C  \nATOM   2411  C   GLY A 159       8.590 -22.528  46.754  1.00 25.59           C  \nATOM   2412  O   GLY A 159       7.701 -23.171  47.314  1.00 28.53           O  \nATOM   2413  H   GLY A 159      11.211 -20.972  46.468  1.00 20.82           H  \nATOM   2414  HA2 GLY A 159      10.369 -22.534  47.823  1.00 23.39           H  \nATOM   2415  HA3 GLY A 159       9.319 -21.473  48.388  1.00 23.38           H  \nATOM   2416  N   ASN A 160       8.535 -22.254  45.452  1.00 19.26           N  \nATOM   2417  CA  ASN A 160       7.433 -22.766  44.656  1.00 18.89           C  \nATOM   2418  C   ASN A 160       8.000 -23.274  43.339  1.00 15.38           C  \nATOM   2419  O   ASN A 160       9.126 -22.976  42.975  1.00 15.51           O  \nATOM   2420  CB  ASN A 160       6.335 -21.732  44.398  1.00 20.07           C  \nATOM   2421  CG  ASN A 160       5.569 -21.243  45.614  1.00 24.28           C  \nATOM   2422  OD1 ASN A 160       4.457 -21.689  45.880  1.00 33.39           O  \nATOM   2423  ND2 ASN A 160       6.109 -20.277  46.328  1.00 26.83           N  \nATOM   2424  H   ASN A 160       9.187 -21.757  45.050  1.00 20.50           H  \nATOM   2425  HA  ASN A 160       7.024 -23.526  45.133  1.00 18.46           H  \nATOM   2426  HB2 ASN A 160       6.743 -20.958  43.960  1.00 20.69           H  \nATOM   2427  HB3 ASN A 160       5.693 -22.119  43.768  1.00 20.66           H  \nATOM   2428 HD21 ASN A 160       6.971 -19.919  46.063  0.00 26.33           H  \nATOM   2429 HD22 ASN A 160       5.816 -19.984  47.065  0.00 26.33           H  \nATOM   2430  N   SER A 161       7.185 -24.085  42.640  1.00 15.62           N  \nATOM   2431  CA  SER A 161       7.603 -24.700  41.391  1.00 15.42           C  \nATOM   2432  C   SER A 161       6.792 -24.156  40.223  1.00 13.02           C  \nATOM   2433  O   SER A 161       5.570 -24.303  40.255  1.00 15.77           O  \nATOM   2434  CB  SER A 161       7.372 -26.190  41.445  1.00 17.52           C  \nATOM   2435  OG  SER A 161       8.199 -26.781  42.423  1.00 21.63           O  \nATOM   2436  H   SER A 161       6.344 -24.258  42.956  1.00 15.54           H  \nATOM   2437  HA  SER A 161       8.567 -24.518  41.237  1.00 15.33           H  \nATOM   2438  HB2 SER A 161       6.429 -26.369  41.659  1.00 17.85           H  \nATOM   2439  HB3 SER A 161       7.569 -26.583  40.565  1.00 17.83           H  \nATOM   2440  HG  SER A 161       7.767 -26.802  43.165  0.00 21.88           H  \nATOM   2441  N   PHE A 162       7.439 -23.567  39.229  1.00 12.16           N  \nATOM   2442  CA  PHE A 162       6.696 -22.922  38.142  1.00 12.30           C  \nATOM   2443  C   PHE A 162       6.610 -23.851  36.939  1.00 12.31           C  \nATOM   2444  O   PHE A 162       7.181 -23.591  35.894  1.00 12.31           O  \nATOM   2445  CB  PHE A 162       7.302 -21.535  37.875  1.00 12.43           C  \nATOM   2446  CG  PHE A 162       7.186 -20.659  39.086  1.00 13.04           C  \nATOM   2447  CD1 PHE A 162       5.939 -20.207  39.521  1.00 13.44           C  \nATOM   2448  CD2 PHE A 162       8.297 -20.439  39.891  1.00 14.25           C  \nATOM   2449  CE1 PHE A 162       5.796 -19.514  40.706  1.00 14.20           C  \nATOM   2450  CE2 PHE A 162       8.135 -19.706  41.064  1.00 13.95           C  \nATOM   2451  CZ  PHE A 162       6.896 -19.242  41.468  1.00 15.15           C  \nATOM   2452  H   PHE A 162       8.353 -23.560  39.213  1.00 12.41           H  \nATOM   2453  HA  PHE A 162       5.768 -22.777  38.459  1.00 12.31           H  \nATOM   2454  HB2 PHE A 162       8.246 -21.639  37.632  1.00 12.55           H  \nATOM   2455  HB3 PHE A 162       6.833 -21.119  37.123  1.00 12.54           H  \nATOM   2456  HD1 PHE A 162       5.174 -20.404  39.005  1.00 13.56           H  \nATOM   2457  HD2 PHE A 162       9.147 -20.756  39.634  1.00 13.89           H  \nATOM   2458  HE1 PHE A 162       4.950 -19.179  40.959  1.00 14.19           H  \nATOM   2459  HE2 PHE A 162       8.891 -19.519  41.595  1.00 14.25           H  \nATOM   2460  HZ  PHE A 162       6.813 -18.734  42.259  1.00 14.55           H  \nATOM   2461  N   MET A 163       5.937 -24.990  37.168  1.00 12.48           N  \nATOM   2462  CA  MET A 163       5.637 -26.029  36.190  1.00 13.06           C  \nATOM   2463  C   MET A 163       4.130 -26.166  36.095  1.00 12.54           C  \nATOM   2464  O   MET A 163       3.466 -26.230  37.112  1.00 14.21           O  \nATOM   2465  CB  MET A 163       6.271 -27.391  36.561  1.00 14.34           C  \nATOM   2466  CG  MET A 163       6.160 -28.483  35.388  1.00 16.87           C  \nATOM   2467  SD  MET A 163       7.150 -28.078  34.010  1.00 18.05           S  \nATOM   2468  CE  MET A 163       8.738 -28.755  34.486  1.00 19.32           C  \nATOM   2469  H   MET A 163       5.631 -25.124  38.018  1.00 12.57           H  \nATOM   2470  HA  MET A 163       5.981 -25.737  35.317  1.00 13.07           H  \nATOM   2471  HB2 MET A 163       7.214 -27.250  36.778  1.00 14.57           H  \nATOM   2472  HB3 MET A 163       5.828 -27.736  37.361  1.00 14.57           H  \nATOM   2473  HG2 MET A 163       6.440 -29.358  35.733  1.00 16.49           H  \nATOM   2474  HG3 MET A 163       5.226 -28.557  35.098  1.00 16.49           H  \nATOM   2475  HE1 MET A 163       9.038 -28.329  35.298  1.00 18.91           H  \nATOM   2476  HE2 MET A 163       8.651 -29.705  34.634  1.00 18.92           H  \nATOM   2477  HE3 MET A 163       9.379 -28.596  33.783  1.00 18.91           H  \nATOM   2478  N   HIS A 164       3.580 -26.216  34.860  1.00 10.28           N  \nATOM   2479  CA  HIS A 164       2.152 -26.243  34.645  1.00  9.88           C  \nATOM   2480  C   HIS A 164       1.817 -27.375  33.683  1.00 10.40           C  \nATOM   2481  O   HIS A 164       2.681 -27.844  32.952  1.00 10.30           O  \nATOM   2482  CB  HIS A 164       1.665 -24.899  34.094  1.00 10.20           C  \nATOM   2483  CG  HIS A 164       2.391 -23.750  34.689  1.00 10.63           C  \nATOM   2484  ND1 HIS A 164       2.190 -23.309  35.970  1.00 11.16           N  \nATOM   2485  CD2 HIS A 164       3.385 -23.026  34.137  1.00 11.76           C  \nATOM   2486  CE1 HIS A 164       3.054 -22.314  36.173  1.00 11.41           C  \nATOM   2487  NE2 HIS A 164       3.805 -22.122  35.091  1.00 11.90           N  \nATOM   2488  H   HIS A 164       4.131 -26.235  34.130  1.00 10.68           H  \nATOM   2489  HA  HIS A 164       1.707 -26.411  35.508  1.00 10.13           H  \nATOM   2490  HB2 HIS A 164       1.790 -24.886  33.123  1.00 10.22           H  \nATOM   2491  HB3 HIS A 164       0.708 -24.804  34.281  1.00 10.22           H  \nATOM   2492  HD1 HIS A 164       1.643 -23.670  36.529  0.00 11.44           H  \nATOM   2493  HD2 HIS A 164       3.740 -23.135  33.268  1.00 11.49           H  \nATOM   2494  HE1 HIS A 164       3.120 -21.814  36.970  1.00 11.45           H  \nATOM   2495  HE2 HIS A 164       4.402 -21.485  35.050  0.00 12.32           H  \nATOM   2496  N   TRP A 165       0.564 -27.843  33.710  1.00 10.01           N  \nATOM   2497  CA  TRP A 165       0.115 -28.926  32.826  1.00 11.42           C  \nATOM   2498  C   TRP A 165      -1.112 -28.516  32.034  1.00 10.44           C  \nATOM   2499  O   TRP A 165      -2.041 -27.878  32.579  1.00 11.35           O  \nATOM   2500  CB  TRP A 165      -0.231 -30.157  33.646  1.00 11.06           C  \nATOM   2501  CG  TRP A 165       0.915 -30.805  34.341  1.00 11.31           C  \nATOM   2502  CD1 TRP A 165       1.332 -30.635  35.641  1.00 12.63           C  \nATOM   2503  CD2 TRP A 165       1.776 -31.796  33.773  1.00 11.21           C  \nATOM   2504  NE1 TRP A 165       2.402 -31.445  35.893  1.00 12.13           N  \nATOM   2505  CE2 TRP A 165       2.683 -32.205  34.777  1.00 11.05           C  \nATOM   2506  CE3 TRP A 165       1.837 -32.388  32.507  1.00 11.52           C  \nATOM   2507  CZ2 TRP A 165       3.620 -33.203  34.565  1.00 12.16           C  \nATOM   2508  CZ3 TRP A 165       2.769 -33.391  32.304  1.00 12.89           C  \nATOM   2509  CH2 TRP A 165       3.647 -33.774  33.323  1.00 12.40           C  \nATOM   2510  H   TRP A 165      -0.036 -27.479  34.296  1.00 10.41           H  \nATOM   2511  HA  TRP A 165       0.845 -29.155  32.210  1.00 10.88           H  \nATOM   2512  HB2 TRP A 165      -0.899 -29.899  34.312  1.00 11.21           H  \nATOM   2513  HB3 TRP A 165      -0.648 -30.812  33.049  1.00 11.20           H  \nATOM   2514  HD1 TRP A 165       0.965 -30.021  36.259  1.00 12.15           H  \nATOM   2515  HE1 TRP A 165       2.810 -31.509  36.660  1.00 11.98           H  \nATOM   2516  HE3 TRP A 165       1.202 -32.173  31.843  1.00 11.77           H  \nATOM   2517  HZ2 TRP A 165       4.247 -33.435  35.229  1.00 11.94           H  \nATOM   2518  HZ3 TRP A 165       2.828 -33.808  31.459  1.00 12.36           H  \nATOM   2519  HH2 TRP A 165       4.311 -34.413  33.134  1.00 12.38           H  \nATOM   2520  N   TYR A 166      -1.102 -28.863  30.736  1.00 10.51           N  \nATOM   2521  CA  TYR A 166      -2.194 -28.577  29.822  1.00 10.53           C  \nATOM   2522  C   TYR A 166      -2.670 -29.870  29.172  1.00 10.84           C  \nATOM   2523  O   TYR A 166      -1.876 -30.801  28.999  1.00 11.56           O  \nATOM   2524  CB  TYR A 166      -1.730 -27.649  28.702  1.00 10.58           C  \nATOM   2525  CG  TYR A 166      -1.196 -26.310  29.158  1.00 10.08           C  \nATOM   2526  CD1 TYR A 166       0.112 -26.165  29.574  1.00 11.22           C  \nATOM   2527  CD2 TYR A 166      -2.033 -25.213  29.297  1.00 10.28           C  \nATOM   2528  CE1 TYR A 166       0.609 -24.939  30.005  1.00 10.31           C  \nATOM   2529  CE2 TYR A 166      -1.549 -24.019  29.801  1.00  9.46           C  \nATOM   2530  CZ  TYR A 166      -0.233 -23.855  30.130  1.00 10.50           C  \nATOM   2531  OH  TYR A 166       0.331 -22.676  30.628  1.00 10.50           O  \nATOM   2532  H   TYR A 166      -0.371 -29.303  30.408  1.00 10.50           H  \nATOM   2533  HA  TYR A 166      -2.941 -28.158  30.315  1.00 10.60           H  \nATOM   2534  HB2 TYR A 166      -1.030 -28.109  28.192  1.00 10.45           H  \nATOM   2535  HB3 TYR A 166      -2.486 -27.496  28.097  1.00 10.45           H  \nATOM   2536  HD1 TYR A 166       0.706 -26.891  29.480  1.00 10.68           H  \nATOM   2537  HD2 TYR A 166      -2.946 -25.290  29.075  1.00 10.08           H  \nATOM   2538  HE1 TYR A 166       1.510 -24.862  30.262  1.00 10.53           H  \nATOM   2539  HE2 TYR A 166      -2.128 -23.278  29.856  1.00  9.94           H  \nATOM   2540  HH  TYR A 166       1.027 -22.621  30.856  0.00 10.68           H  \nATOM   2541  N   GLN A 167      -3.938 -29.851  28.779  1.00 10.69           N  \nATOM   2542  CA  GLN A 167      -4.545 -30.918  27.997  1.00 11.38           C  \nATOM   2543  C   GLN A 167      -4.866 -30.396  26.614  1.00 12.40           C  \nATOM   2544  O   GLN A 167      -5.482 -29.343  26.496  1.00 13.78           O  \nATOM   2545  CB  GLN A 167      -5.864 -31.324  28.675  1.00 11.95           C  \nATOM   2546  CG  GLN A 167      -6.640 -32.460  28.011  1.00 12.17           C  \nATOM   2547  CD  GLN A 167      -8.017 -32.578  28.631  1.00 14.11           C  \nATOM   2548  OE1 GLN A 167      -8.853 -31.652  28.577  1.00 15.34           O  \nATOM   2549  NE2 GLN A 167      -8.277 -33.722  29.271  1.00 16.74           N  \nATOM   2550  H   GLN A 167      -4.457 -29.135  29.007  1.00 10.88           H  \nATOM   2551  HA  GLN A 167      -3.936 -31.689  27.941  1.00 11.54           H  \nATOM   2552  HB2 GLN A 167      -5.664 -31.583  29.596  1.00 11.87           H  \nATOM   2553  HB3 GLN A 167      -6.441 -30.533  28.713  1.00 11.88           H  \nATOM   2554  HG2 GLN A 167      -6.728 -32.286  27.050  1.00 12.55           H  \nATOM   2555  HG3 GLN A 167      -6.154 -33.304  28.127  1.00 12.56           H  \nATOM   2556 HE21 GLN A 167      -7.680 -34.366  29.248  0.00 17.30           H  \nATOM   2557 HE22 GLN A 167      -9.082 -33.854  29.589  0.00 17.30           H  \nATOM   2558  N   GLN A 168      -4.602 -31.185  25.571  1.00 10.95           N  \nATOM   2559  CA  GLN A 168      -4.971 -30.737  24.229  1.00 11.70           C  \nATOM   2560  C   GLN A 168      -5.701 -31.870  23.507  1.00 11.89           C  \nATOM   2561  O   GLN A 168      -5.149 -32.922  23.291  1.00 13.03           O  \nATOM   2562  CB  GLN A 168      -3.758 -30.260  23.443  1.00 11.50           C  \nATOM   2563  CG  GLN A 168      -4.174 -29.618  22.123  1.00 11.91           C  \nATOM   2564  CD  GLN A 168      -2.999 -29.109  21.343  1.00 13.15           C  \nATOM   2565  OE1 GLN A 168      -1.944 -29.731  21.333  1.00 16.00           O  \nATOM   2566  NE2 GLN A 168      -3.247 -28.005  20.612  1.00 14.00           N  \nATOM   2567  H   GLN A 168      -4.197 -31.996  25.685  1.00 11.46           H  \nATOM   2568  HA  GLN A 168      -5.593 -29.981  24.319  1.00 11.58           H  \nATOM   2569  HB2 GLN A 168      -3.263 -29.610  23.983  1.00 11.64           H  \nATOM   2570  HB3 GLN A 168      -3.172 -31.025  23.266  1.00 11.65           H  \nATOM   2571  HG2 GLN A 168      -4.656 -30.277  21.579  1.00 12.09           H  \nATOM   2572  HG3 GLN A 168      -4.785 -28.873  22.306  1.00 12.10           H  \nATOM   2573 HE21 GLN A 168      -4.038 -27.611  20.589  0.00 13.90           H  \nATOM   2574 HE22 GLN A 168      -2.601 -27.686  20.067  0.00 13.90           H  \nATOM   2575  N   LYS A 169      -6.888 -31.571  23.010  1.00 14.70           N  \nATOM   2576  CA  LYS A 169      -7.619 -32.459  22.115  1.00 17.80           C  \nATOM   2577  C   LYS A 169      -7.543 -31.996  20.677  1.00 19.13           C  \nATOM   2578  O   LYS A 169      -7.227 -30.856  20.396  1.00 17.70           O  \nATOM   2579  CB  LYS A 169      -9.068 -32.505  22.556  1.00 19.80           C  \nATOM   2580  CG  LYS A 169      -9.233 -33.113  23.932  1.00 21.56           C  \nATOM   2581  CD  LYS A 169     -10.627 -33.379  24.330  1.00 24.50           C  \nATOM   2582  CE  LYS A 169     -10.644 -33.928  25.731  1.00 23.79           C  \nATOM   2583  NZ  LYS A 169     -12.023 -34.296  26.096  1.00 24.43           N  \nATOM   2584  H   LYS A 169      -7.272 -30.773  23.237  1.00 14.62           H  \nATOM   2585  HA  LYS A 169      -7.235 -33.361  22.184  1.00 17.76           H  \nATOM   2586  HB2 LYS A 169      -9.427 -31.593  22.564  1.00 19.70           H  \nATOM   2587  HB3 LYS A 169      -9.583 -33.031  21.910  1.00 19.71           H  \nATOM   2588  HG2 LYS A 169      -8.732 -33.956  23.961  1.00 21.77           H  \nATOM   2589  HG3 LYS A 169      -8.830 -32.507  24.590  1.00 21.77           H  \nATOM   2590  HD2 LYS A 169     -11.148 -32.548  24.295  1.00 23.61           H  \nATOM   2591  HD3 LYS A 169     -11.030 -34.030  23.716  1.00 23.61           H  \nATOM   2592  HE2 LYS A 169     -10.072 -34.717  25.788  1.00 24.09           H  \nATOM   2593  HE3 LYS A 169     -10.310 -33.257  26.357  1.00 24.10           H  \nATOM   2594  HZ1 LYS A 169     -12.340 -34.918  25.517  1.00 24.23           H  \nATOM   2595  HZ2 LYS A 169     -12.556 -33.562  26.065  1.00 24.23           H  \nATOM   2596  HZ3 LYS A 169     -12.036 -34.637  26.935  1.00 24.24           H  \nATOM   2597  N   PRO A 170      -7.816 -32.873  19.677  1.00 19.22           N  \nATOM   2598  CA  PRO A 170      -7.694 -32.498  18.276  1.00 20.71           C  \nATOM   2599  C   PRO A 170      -8.573 -31.324  17.881  1.00 19.77           C  \nATOM   2600  O   PRO A 170      -9.718 -31.232  18.314  1.00 22.48           O  \nATOM   2601  CB  PRO A 170      -8.204 -33.762  17.550  1.00 22.96           C  \nATOM   2602  CG  PRO A 170      -7.865 -34.874  18.501  1.00 24.17           C  \nATOM   2603  CD  PRO A 170      -8.122 -34.301  19.870  1.00 23.16           C  \nATOM   2604  HA  PRO A 170      -6.746 -32.330  18.047  1.00 20.64           H  \nATOM   2605  HB2 PRO A 170      -9.172 -33.717  17.395  1.00 22.68           H  \nATOM   2606  HB3 PRO A 170      -7.747 -33.884  16.690  1.00 22.68           H  \nATOM   2607  HG2 PRO A 170      -8.432 -35.655  18.339  1.00 23.64           H  \nATOM   2608  HG3 PRO A 170      -6.927 -35.139  18.406  1.00 23.64           H  \nATOM   2609  HD2 PRO A 170      -9.050 -34.431  20.141  1.00 22.38           H  \nATOM   2610  HD3 PRO A 170      -7.533 -34.704  20.535  1.00 22.38           H  \nATOM   2611  N   GLY A 171      -7.977 -30.399  17.110  1.00 21.79           N  \nATOM   2612  CA  GLY A 171      -8.689 -29.245  16.600  1.00 22.46           C  \nATOM   2613  C   GLY A 171      -8.824 -28.130  17.621  1.00 23.46           C  \nATOM   2614  O   GLY A 171      -9.388 -27.099  17.296  1.00 26.97           O  \nATOM   2615  H   GLY A 171      -7.093 -30.496  16.898  1.00 21.44           H  \nATOM   2616  HA2 GLY A 171      -8.208 -28.901  15.806  1.00 22.53           H  \nATOM   2617  HA3 GLY A 171      -9.591 -29.529  16.307  1.00 22.52           H  \nATOM   2618  N   GLN A 172      -8.277 -28.318  18.839  1.00 18.73           N  \nATOM   2619  CA  GLN A 172      -8.480 -27.386  19.931  1.00 17.60           C  \nATOM   2620  C   GLN A 172      -7.149 -26.881  20.449  1.00 15.40           C  \nATOM   2621  O   GLN A 172      -6.137 -27.569  20.356  1.00 15.44           O  \nATOM   2622  CB  GLN A 172      -9.159 -28.046  21.115  1.00 18.65           C  \nATOM   2623  CG  GLN A 172     -10.509 -28.627  20.757  1.00 24.00           C  \nATOM   2624  CD  GLN A 172     -11.246 -29.175  21.955  1.00 29.28           C  \nATOM   2625  OE1 GLN A 172     -10.777 -29.139  23.097  1.00 35.00           O  \nATOM   2626  NE2 GLN A 172     -12.449 -29.673  21.695  1.00 33.56           N  \nATOM   2627  H   GLN A 172      -7.746 -29.046  18.985  1.00 19.48           H  \nATOM   2628  HA  GLN A 172      -9.023 -26.627  19.619  1.00 17.57           H  \nATOM   2629  HB2 GLN A 172      -8.580 -28.760  21.454  1.00 19.52           H  \nATOM   2630  HB3 GLN A 172      -9.271 -27.381  21.825  1.00 19.54           H  \nATOM   2631  HG2 GLN A 172     -11.057 -27.930  20.337  1.00 23.69           H  \nATOM   2632  HG3 GLN A 172     -10.386 -29.346  20.103  1.00 23.68           H  \nATOM   2633 HE21 GLN A 172     -12.752 -29.807  20.864  0.00 33.39           H  \nATOM   2634 HE22 GLN A 172     -12.931 -30.081  22.359  0.00 33.39           H  \nATOM   2635  N   PRO A 173      -7.084 -25.682  21.047  1.00 14.14           N  \nATOM   2636  CA  PRO A 173      -5.877 -25.292  21.741  1.00 14.37           C  \nATOM   2637  C   PRO A 173      -5.669 -26.079  23.037  1.00 13.84           C  \nATOM   2638  O   PRO A 173      -6.621 -26.659  23.556  1.00 14.58           O  \nATOM   2639  CB  PRO A 173      -6.134 -23.838  22.121  1.00 15.60           C  \nATOM   2640  CG  PRO A 173      -7.629 -23.761  22.254  1.00 17.28           C  \nATOM   2641  CD  PRO A 173      -8.138 -24.639  21.137  1.00 15.63           C  \nATOM   2642  HA  PRO A 173      -5.093 -25.362  21.140  1.00 14.45           H  \nATOM   2643  HB2 PRO A 173      -5.694 -23.609  22.968  1.00 15.66           H  \nATOM   2644  HB3 PRO A 173      -5.813 -23.230  21.423  1.00 15.66           H  \nATOM   2645  HG2 PRO A 173      -7.921 -24.100  23.125  1.00 16.48           H  \nATOM   2646  HG3 PRO A 173      -7.944 -22.841  22.148  1.00 16.47           H  \nATOM   2647  HD2 PRO A 173      -9.004 -25.028  21.360  1.00 15.64           H  \nATOM   2648  HD3 PRO A 173      -8.214 -24.141  20.302  1.00 15.63           H  \nATOM   2649  N   PRO A 174      -4.465 -26.009  23.633  1.00 13.36           N  \nATOM   2650  CA  PRO A 174      -4.255 -26.525  24.983  1.00 13.69           C  \nATOM   2651  C   PRO A 174      -5.191 -25.853  25.980  1.00 15.36           C  \nATOM   2652  O   PRO A 174      -5.569 -24.692  25.816  1.00 15.88           O  \nATOM   2653  CB  PRO A 174      -2.776 -26.163  25.256  1.00 14.93           C  \nATOM   2654  CG  PRO A 174      -2.149 -26.024  23.879  1.00 14.41           C  \nATOM   2655  CD  PRO A 174      -3.246 -25.351  23.111  1.00 14.61           C  \nATOM   2656  HA  PRO A 174      -4.373 -27.507  24.992  1.00 14.16           H  \nATOM   2657  HB2 PRO A 174      -2.708 -25.321  25.755  1.00 14.51           H  \nATOM   2658  HB3 PRO A 174      -2.333 -26.871  25.770  1.00 14.51           H  \nATOM   2659  HG2 PRO A 174      -1.343 -25.469  23.909  1.00 14.58           H  \nATOM   2660  HG3 PRO A 174      -1.930 -26.898  23.496  1.00 14.58           H  \nATOM   2661  HD2 PRO A 174      -3.264 -24.392  23.285  1.00 14.25           H  \nATOM   2662  HD3 PRO A 174      -3.146 -25.501  22.153  1.00 14.26           H  \nATOM   2663  N   LYS A 175      -5.510 -26.589  27.046  1.00 13.58           N  \nATOM   2664  CA  LYS A 175      -6.356 -26.138  28.135  1.00 13.85           C  \nATOM   2665  C   LYS A 175      -5.634 -26.393  29.453  1.00 13.29           C  \nATOM   2666  O   LYS A 175      -5.150 -27.490  29.751  1.00 12.66           O  \nATOM   2667  CB  LYS A 175      -7.683 -26.902  28.059  1.00 16.62           C  \nATOM   2668  CG  LYS A 175      -8.577 -26.779  29.277  1.00 20.14           C  \nATOM   2669  CD  LYS A 175      -9.770 -27.728  29.206  1.00 24.45           C  \nATOM   2670  CE  LYS A 175     -10.781 -27.491  30.310  1.00 29.77           C  \nATOM   2671  NZ  LYS A 175     -11.989 -28.339  30.108  1.00 34.14           N  \nATOM   2672  H   LYS A 175      -5.172 -27.437  27.096  1.00 14.04           H  \nATOM   2673  HA  LYS A 175      -6.527 -25.174  28.035  1.00 14.20           H  \nATOM   2674  HB2 LYS A 175      -8.180 -26.581  27.278  1.00 16.68           H  \nATOM   2675  HB3 LYS A 175      -7.485 -27.851  27.916  1.00 16.66           H  \nATOM   2676  HG2 LYS A 175      -8.056 -26.979  30.083  1.00 20.14           H  \nATOM   2677  HG3 LYS A 175      -8.901 -25.855  29.345  1.00 20.15           H  \nATOM   2678  HD2 LYS A 175     -10.216 -27.617  28.339  1.00 24.49           H  \nATOM   2679  HD3 LYS A 175      -9.448 -28.652  29.263  1.00 24.47           H  \nATOM   2680  HE2 LYS A 175     -10.382 -27.705  31.170  1.00 29.27           H  \nATOM   2681  HE3 LYS A 175     -11.046 -26.552  30.320  1.00 29.34           H  \nATOM   2682  HZ1 LYS A 175     -11.761 -29.215  30.160  1.00 32.73           H  \nATOM   2683  HZ2 LYS A 175     -12.347 -28.174  29.291  1.00 32.71           H  \nATOM   2684  HZ3 LYS A 175     -12.605 -28.153  30.746  1.00 32.73           H  \nATOM   2685  N   LEU A 176      -5.513 -25.324  30.267  1.00 13.71           N  \nATOM   2686  CA  LEU A 176      -4.828 -25.461  31.549  1.00 13.43           C  \nATOM   2687  C   LEU A 176      -5.535 -26.384  32.531  1.00 12.91           C  \nATOM   2688  O   LEU A 176      -6.743 -26.239  32.733  1.00 14.23           O  \nATOM   2689  CB  LEU A 176      -4.742 -24.068  32.151  1.00 13.79           C  \nATOM   2690  CG  LEU A 176      -4.072 -23.968  33.492  1.00 14.27           C  \nATOM   2691  CD1 LEU A 176      -2.582 -24.268  33.403  1.00 13.54           C  \nATOM   2692  CD2 LEU A 176      -4.333 -22.560  34.070  1.00 14.07           C  \nATOM   2693  H   LEU A 176      -5.858 -24.513  30.023  1.00 13.55           H  \nATOM   2694  HA  LEU A 176      -3.920 -25.803  31.382  1.00 13.44           H  \nATOM   2695  HB2 LEU A 176      -4.261 -23.493  31.523  1.00 13.82           H  \nATOM   2696  HB3 LEU A 176      -5.650 -23.715  32.235  1.00 13.82           H  \nATOM   2697  HG  LEU A 176      -4.484 -24.630  34.099  1.00 13.99           H  \nATOM   2698 HD11 LEU A 176      -2.157 -24.060  34.253  1.00 13.76           H  \nATOM   2699 HD12 LEU A 176      -2.183 -23.726  32.700  1.00 13.76           H  \nATOM   2700 HD13 LEU A 176      -2.452 -25.210  33.198  1.00 13.76           H  \nATOM   2701 HD21 LEU A 176      -5.286 -22.447  34.232  1.00 14.14           H  \nATOM   2702 HD22 LEU A 176      -4.030 -21.887  33.436  1.00 14.13           H  \nATOM   2703 HD23 LEU A 176      -3.848 -22.458  34.907  1.00 14.13           H  \nATOM   2704  N   LEU A 177      -4.786 -27.359  33.070  1.00 11.69           N  \nATOM   2705  CA  LEU A 177      -5.277 -28.291  34.088  1.00 12.66           C  \nATOM   2706  C   LEU A 177      -4.775 -27.945  35.490  1.00 14.59           C  \nATOM   2707  O   LEU A 177      -5.560 -27.909  36.451  1.00 14.86           O  \nATOM   2708  CB  LEU A 177      -4.793 -29.710  33.791  1.00 14.23           C  \nATOM   2709  CG  LEU A 177      -5.217 -30.296  32.466  1.00 15.27           C  \nATOM   2710  CD1 LEU A 177      -4.382 -31.548  32.263  1.00 14.65           C  \nATOM   2711  CD2 LEU A 177      -6.720 -30.628  32.452  1.00 15.63           C  \nATOM   2712  H   LEU A 177      -3.924 -27.456  32.779  1.00 12.20           H  \nATOM   2713  HA  LEU A 177      -6.260 -28.272  34.087  1.00 13.11           H  \nATOM   2714  HB2 LEU A 177      -3.816 -29.712  33.830  1.00 14.08           H  \nATOM   2715  HB3 LEU A 177      -5.115 -30.298  34.503  1.00 14.08           H  \nATOM   2716  HG  LEU A 177      -5.017 -29.651  31.744  1.00 15.01           H  \nATOM   2717 HD11 LEU A 177      -4.703 -32.029  31.483  1.00 14.81           H  \nATOM   2718 HD12 LEU A 177      -4.455 -32.119  33.048  1.00 14.83           H  \nATOM   2719 HD13 LEU A 177      -3.450 -31.300  32.129  1.00 14.83           H  \nATOM   2720 HD21 LEU A 177      -7.235 -29.804  32.500  1.00 15.52           H  \nATOM   2721 HD22 LEU A 177      -6.935 -31.189  33.218  1.00 15.51           H  \nATOM   2722 HD23 LEU A 177      -6.941 -31.101  31.631  1.00 15.52           H  \nATOM   2723  N   ILE A 178      -3.466 -27.706  35.558  1.00 12.52           N  \nATOM   2724  CA  ILE A 178      -2.705 -27.616  36.812  1.00 14.01           C  \nATOM   2725  C   ILE A 178      -1.732 -26.452  36.642  1.00 12.16           C  \nATOM   2726  O   ILE A 178      -0.988 -26.405  35.671  1.00 12.33           O  \nATOM   2727  CB  ILE A 178      -1.891 -28.883  37.163  1.00 15.26           C  \nATOM   2728  CG1 ILE A 178      -2.717 -30.174  37.308  1.00 18.94           C  \nATOM   2729  CG2 ILE A 178      -0.983 -28.636  38.370  1.00 16.83           C  \nATOM   2730  CD1 ILE A 178      -3.658 -30.191  38.410  1.00 20.88           C  \nATOM   2731  H   ILE A 178      -3.007 -27.598  34.776  1.00 13.32           H  \nATOM   2732  HA  ILE A 178      -3.315 -27.420  37.540  1.00 13.59           H  \nATOM   2733  HB  ILE A 178      -1.287 -29.036  36.396  1.00 15.92           H  \nATOM   2734 HG12 ILE A 178      -3.214 -30.319  36.475  1.00 18.41           H  \nATOM   2735 HG13 ILE A 178      -2.098 -30.927  37.418  1.00 18.41           H  \nATOM   2736 HG21 ILE A 178      -1.417 -28.020  38.985  1.00 16.32           H  \nATOM   2737 HG22 ILE A 178      -0.139 -28.255  38.072  1.00 16.32           H  \nATOM   2738 HG23 ILE A 178      -0.812 -29.478  38.826  1.00 16.34           H  \nATOM   2739 HD11 ILE A 178      -4.284 -29.452  38.315  1.00 20.23           H  \nATOM   2740 HD12 ILE A 178      -3.178 -30.101  39.252  1.00 20.25           H  \nATOM   2741 HD13 ILE A 178      -4.148 -31.031  38.407  1.00 20.25           H  \nATOM   2742  N   TYR A 179      -1.705 -25.536  37.615  1.00 13.18           N  \nATOM   2743  CA  TYR A 179      -0.716 -24.462  37.607  1.00 12.23           C  \nATOM   2744  C   TYR A 179       0.198 -24.573  38.824  1.00 11.67           C  \nATOM   2745  O   TYR A 179      -0.207 -25.020  39.895  1.00 11.77           O  \nATOM   2746  CB  TYR A 179      -1.373 -23.081  37.463  1.00 12.96           C  \nATOM   2747  CG  TYR A 179      -2.234 -22.609  38.601  1.00 15.56           C  \nATOM   2748  CD1 TYR A 179      -1.674 -21.879  39.630  1.00 15.13           C  \nATOM   2749  CD2 TYR A 179      -3.585 -22.826  38.606  1.00 16.72           C  \nATOM   2750  CE1 TYR A 179      -2.439 -21.408  40.679  1.00 17.93           C  \nATOM   2751  CE2 TYR A 179      -4.376 -22.358  39.642  1.00 21.10           C  \nATOM   2752  CZ  TYR A 179      -3.797 -21.634  40.665  1.00 19.20           C  \nATOM   2753  OH  TYR A 179      -4.635 -21.216  41.682  1.00 22.93           O  \nATOM   2754  H   TYR A 179      -2.312 -25.576  38.297  1.00 12.71           H  \nATOM   2755  HA  TYR A 179      -0.146 -24.593  36.808  1.00 12.43           H  \nATOM   2756  HB2 TYR A 179      -0.659 -22.423  37.325  1.00 13.35           H  \nATOM   2757  HB3 TYR A 179      -1.918 -23.092  36.649  1.00 13.35           H  \nATOM   2758  HD1 TYR A 179      -0.746 -21.715  39.628  1.00 16.03           H  \nATOM   2759  HD2 TYR A 179      -3.982 -23.309  37.900  1.00 17.37           H  \nATOM   2760  HE1 TYR A 179      -2.043 -20.920  41.382  1.00 17.50           H  \nATOM   2761  HE2 TYR A 179      -5.305 -22.523  39.645  1.00 19.30           H  \nATOM   2762  HH  TYR A 179      -4.272 -20.822  42.388  0.00 22.87           H  \nATOM   2763  N   ARG A 180       1.455 -24.212  38.636  1.00 12.18           N  \nATOM   2764  CA  ARG A 180       2.453 -24.205  39.682  1.00 11.90           C  \nATOM   2765  C   ARG A 180       2.510 -25.540  40.420  1.00 12.48           C  \nATOM   2766  O   ARG A 180       2.500 -25.600  41.655  1.00 13.52           O  \nATOM   2767  CB  ARG A 180       2.222 -23.020  40.631  1.00 12.07           C  \nATOM   2768  CG  ARG A 180       2.326 -21.704  39.882  1.00 12.17           C  \nATOM   2769  CD  ARG A 180       2.050 -20.542  40.809  1.00 10.98           C  \nATOM   2770  NE  ARG A 180       1.950 -19.257  40.116  1.00 10.88           N  \nATOM   2771  CZ  ARG A 180       1.388 -18.171  40.633  1.00 11.89           C  \nATOM   2772  NH1 ARG A 180       0.820 -18.216  41.829  1.00 12.47           N  \nATOM   2773  NH2 ARG A 180       1.426 -17.063  39.945  1.00 12.28           N  \nATOM   2774  H   ARG A 180       1.711 -23.958  37.798  1.00 11.99           H  \nATOM   2775  HA  ARG A 180       3.324 -24.071  39.246  1.00 12.10           H  \nATOM   2776  HB2 ARG A 180       1.332 -23.097  41.035  1.00 12.06           H  \nATOM   2777  HB3 ARG A 180       2.892 -23.042  41.345  1.00 12.06           H  \nATOM   2778  HG2 ARG A 180       3.226 -21.611  39.504  1.00 11.86           H  \nATOM   2779  HG3 ARG A 180       1.679 -21.690  39.145  1.00 11.86           H  \nATOM   2780  HD2 ARG A 180       1.211 -20.713  41.289  1.00 11.23           H  \nATOM   2781  HD3 ARG A 180       2.768 -20.489  41.476  1.00 11.22           H  \nATOM   2782  HE  ARG A 180       2.280 -19.201  39.310  1.00 11.14           H  \nATOM   2783 HH11 ARG A 180       0.817 -18.950  42.245  0.00 12.47           H  \nATOM   2784 HH12 ARG A 180       0.430 -17.477  42.115  0.00 12.47           H  \nATOM   2785 HH21 ARG A 180       0.860 -16.416  40.130  1.00 12.16           H  \nATOM   2786 HH22 ARG A 180       2.018 -16.962  39.304  1.00 12.15           H  \nATOM   2787  N   ALA A 181       2.586 -26.609  39.619  1.00 12.87           N  \nATOM   2788  CA  ALA A 181       2.914 -27.996  39.974  1.00 13.17           C  \nATOM   2789  C   ALA A 181       1.737 -28.724  40.606  1.00 15.04           C  \nATOM   2790  O   ALA A 181       1.522 -29.889  40.296  1.00 14.96           O  \nATOM   2791  CB  ALA A 181       4.128 -28.139  40.836  1.00 15.18           C  \nATOM   2792  H   ALA A 181       2.415 -26.453  38.735  1.00 12.84           H  \nATOM   2793  HA  ALA A 181       3.110 -28.461  39.127  1.00 13.82           H  \nATOM   2794  HB1 ALA A 181       4.395 -29.073  40.870  1.00 14.53           H  \nATOM   2795  HB2 ALA A 181       4.853 -27.609  40.465  1.00 14.54           H  \nATOM   2796  HB3 ALA A 181       3.928 -27.828  41.735  1.00 14.53           H  \nATOM   2797  N   LEU A 182       0.924 -28.045  41.429  1.00 17.64           N  \nATOM   2798  CA  LEU A 182      -0.065 -28.778  42.205  1.00 20.49           C  \nATOM   2799  C   LEU A 182      -1.440 -28.127  42.316  1.00 19.60           C  \nATOM   2800  O   LEU A 182      -2.348 -28.689  42.913  1.00 24.14           O  \nATOM   2801  CB  LEU A 182       0.529 -29.012  43.594  1.00 25.13           C  \nATOM   2802  CG  LEU A 182       0.843 -27.778  44.425  1.00 27.51           C  \nATOM   2803  CD1 LEU A 182      -0.419 -27.152  45.039  1.00 30.76           C  \nATOM   2804  CD2 LEU A 182       1.853 -28.148  45.514  1.00 34.27           C  \nATOM   2805  H   LEU A 182       0.996 -27.140  41.523  1.00 17.59           H  \nATOM   2806  HA  LEU A 182      -0.192 -29.654  41.774  1.00 20.50           H  \nATOM   2807  HB2 LEU A 182      -0.094 -29.572  44.099  1.00 24.48           H  \nATOM   2808  HB3 LEU A 182       1.354 -29.525  43.484  1.00 24.50           H  \nATOM   2809  HG  LEU A 182       1.263 -27.106  43.834  1.00 28.74           H  \nATOM   2810 HD11 LEU A 182      -0.196 -26.739  45.891  1.00 29.71           H  \nATOM   2811 HD12 LEU A 182      -1.091 -27.841  45.180  1.00 29.66           H  \nATOM   2812 HD13 LEU A 182      -0.770 -26.476  44.436  1.00 29.65           H  \nATOM   2813 HD21 LEU A 182       2.673 -28.470  45.101  1.00 32.00           H  \nATOM   2814 HD22 LEU A 182       1.480 -28.847  46.079  1.00 32.02           H  \nATOM   2815 HD23 LEU A 182       2.050 -27.364  46.055  1.00 32.01           H  \nATOM   2816  N   ASN A 183      -1.658 -26.949  41.770  1.00 15.35           N  \nATOM   2817  CA  ASN A 183      -2.907 -26.250  41.927  1.00 15.73           C  \nATOM   2818  C   ASN A 183      -3.852 -26.617  40.802  1.00 17.48           C  \nATOM   2819  O   ASN A 183      -3.532 -26.392  39.635  1.00 16.70           O  \nATOM   2820  CB  ASN A 183      -2.704 -24.721  41.952  1.00 18.00           C  \nATOM   2821  CG  ASN A 183      -1.716 -24.311  43.025  1.00 19.39           C  \nATOM   2822  OD1 ASN A 183      -0.502 -24.153  42.807  1.00 22.57           O  \nATOM   2823  ND2 ASN A 183      -2.195 -24.290  44.255  1.00 21.16           N  \nATOM   2824  H   ASN A 183      -1.003 -26.560  41.267  1.00 16.38           H  \nATOM   2825  HA  ASN A 183      -3.313 -26.514  42.786  1.00 16.42           H  \nATOM   2826  HB2 ASN A 183      -2.377 -24.425  41.078  1.00 17.73           H  \nATOM   2827  HB3 ASN A 183      -3.564 -24.286  42.121  1.00 17.74           H  \nATOM   2828 HD21 ASN A 183      -3.041 -24.408  44.407  0.00 20.49           H  \nATOM   2829 HD22 ASN A 183      -1.654 -24.016  44.921  0.00 20.49           H  \nATOM   2830  N   LEU A 184      -5.044 -27.140  41.142  1.00 18.58           N  \nATOM   2831  CA  LEU A 184      -6.057 -27.477  40.157  1.00 19.38           C  \nATOM   2832  C   LEU A 184      -6.756 -26.208  39.664  1.00 20.40           C  \nATOM   2833  O   LEU A 184      -7.246 -25.384  40.466  1.00 23.97           O  \nATOM   2834  CB  LEU A 184      -6.951 -28.485  40.889  1.00 23.20           C  \nATOM   2835  CG  LEU A 184      -7.960 -29.271  40.086  1.00 27.02           C  \nATOM   2836  CD1 LEU A 184      -7.255 -30.126  39.039  1.00 25.61           C  \nATOM   2837  CD2 LEU A 184      -8.784 -30.154  41.024  1.00 26.53           C  \nATOM   2838  H   LEU A 184      -5.228 -27.281  42.026  1.00 18.52           H  \nATOM   2839  HA  LEU A 184      -5.618 -27.918  39.395  1.00 20.08           H  \nATOM   2840  HB2 LEU A 184      -6.368 -29.126  41.343  1.00 23.03           H  \nATOM   2841  HB3 LEU A 184      -7.437 -27.999  41.585  1.00 23.04           H  \nATOM   2842  HG  LEU A 184      -8.566 -28.639  39.627  1.00 25.89           H  \nATOM   2843 HD11 LEU A 184      -7.833 -30.864  38.778  1.00 26.02           H  \nATOM   2844 HD12 LEU A 184      -6.427 -30.479  39.410  1.00 26.02           H  \nATOM   2845 HD13 LEU A 184      -7.053 -29.582  38.257  1.00 26.03           H  \nATOM   2846 HD21 LEU A 184      -9.160 -29.609  41.737  1.00 26.67           H  \nATOM   2847 HD22 LEU A 184      -8.212 -30.841  41.409  1.00 26.67           H  \nATOM   2848 HD23 LEU A 184      -9.504 -30.575  40.524  1.00 26.67           H  \nATOM   2849  N   GLU A 185      -6.783 -26.007  38.339  1.00 19.15           N  \nATOM   2850  CA  GLU A 185      -7.531 -24.921  37.728  1.00 20.30           C  \nATOM   2851  C   GLU A 185      -9.038 -25.067  37.988  1.00 20.79           C  \nATOM   2852  O   GLU A 185      -9.582 -26.157  37.948  1.00 20.51           O  \nATOM   2853  CB  GLU A 185      -7.221 -24.819  36.228  1.00 21.21           C  \nATOM   2854  CG  GLU A 185      -7.955 -23.730  35.497  1.00 22.57           C  \nATOM   2855  CD  GLU A 185      -7.666 -22.310  35.955  1.00 26.53           C  \nATOM   2856  OE1 GLU A 185      -6.973 -22.172  36.980  1.00 30.00           O  \nATOM   2857  OE2 GLU A 185      -8.031 -21.360  35.217  1.00 28.93           O  \nATOM   2858  H   GLU A 185      -6.314 -26.573  37.795  1.00 19.71           H  \nATOM   2859  HA  GLU A 185      -7.232 -24.089  38.157  1.00 20.39           H  \nATOM   2860  HB2 GLU A 185      -6.258 -24.672  36.122  1.00 21.30           H  \nATOM   2861  HB3 GLU A 185      -7.437 -25.678  35.810  1.00 21.30           H  \nATOM   2862  HG2 GLU A 185      -7.734 -23.792  34.543  1.00 23.11           H  \nATOM   2863  HG3 GLU A 185      -8.918 -23.888  35.581  1.00 23.10           H  \nATOM   2864  N   SER A 186      -9.711 -23.921  38.213  1.00 24.35           N  \nATOM   2865  CA  SER A 186     -11.164 -23.885  38.344  1.00 26.76           C  \nATOM   2866  C   SER A 186     -11.821 -24.566  37.161  1.00 24.70           C  \nATOM   2867  O   SER A 186     -11.443 -24.311  36.029  1.00 23.82           O  \nATOM   2868  CB  SER A 186     -11.671 -22.457  38.427  1.00 30.30           C  \nATOM   2869  OG  SER A 186     -11.273 -21.876  39.650  1.00 38.06           O  \nATOM   2870  H   SER A 186      -9.246 -23.138  38.286  1.00 23.98           H  \nATOM   2871  HA  SER A 186     -11.418 -24.368  39.177  1.00 26.51           H  \nATOM   2872  HB2 SER A 186     -11.307 -21.934  37.677  1.00 31.05           H  \nATOM   2873  HB3 SER A 186     -12.653 -22.451  38.362  1.00 31.04           H  \nATOM   2874  HG  SER A 186     -10.529 -21.599  39.725  0.00 39.00           H  \nATOM   2875  N   GLY A 187     -12.793 -25.450  37.453  1.00 26.50           N  \nATOM   2876  CA  GLY A 187     -13.538 -26.121  36.415  1.00 25.78           C  \nATOM   2877  C   GLY A 187     -12.905 -27.443  35.996  1.00 24.58           C  \nATOM   2878  O   GLY A 187     -13.500 -28.122  35.161  1.00 28.83           O  \nATOM   2879  H   GLY A 187     -12.990 -25.633  38.326  1.00 25.90           H  \nATOM   2880  HA2 GLY A 187     -14.457 -26.291  36.742  1.00 25.65           H  \nATOM   2881  HA3 GLY A 187     -13.603 -25.526  35.626  1.00 25.65           H  \nATOM   2882  N   ILE A 188     -11.732 -27.778  36.570  1.00 24.23           N  \nATOM   2883  CA  ILE A 188     -11.026 -29.003  36.216  1.00 22.75           C  \nATOM   2884  C   ILE A 188     -11.382 -30.056  37.265  1.00 23.78           C  \nATOM   2885  O   ILE A 188     -11.293 -29.784  38.456  1.00 24.23           O  \nATOM   2886  CB  ILE A 188      -9.499 -28.772  36.090  1.00 21.44           C  \nATOM   2887  CG1 ILE A 188      -9.181 -27.756  34.990  1.00 19.30           C  \nATOM   2888  CG2 ILE A 188      -8.731 -30.078  35.834  1.00 21.75           C  \nATOM   2889  CD1 ILE A 188      -9.762 -28.038  33.624  1.00 20.32           C  \nATOM   2890  H   ILE A 188     -11.366 -27.232  37.205  1.00 23.97           H  \nATOM   2891  HA  ILE A 188     -11.355 -29.303  35.355  1.00 22.97           H  \nATOM   2892  HB  ILE A 188      -9.180 -28.398  36.947  1.00 21.31           H  \nATOM   2893 HG12 ILE A 188      -9.504 -26.877  35.283  1.00 20.02           H  \nATOM   2894 HG13 ILE A 188      -8.208 -27.694  34.901  1.00 20.01           H  \nATOM   2895 HG21 ILE A 188      -9.145 -30.562  35.100  1.00 21.66           H  \nATOM   2896 HG22 ILE A 188      -8.748 -30.631  36.635  1.00 21.67           H  \nATOM   2897 HG23 ILE A 188      -7.807 -29.874  35.605  1.00 21.65           H  \nATOM   2898 HD11 ILE A 188     -10.591 -27.540  33.513  1.00 20.03           H  \nATOM   2899 HD12 ILE A 188      -9.942 -28.989  33.535  1.00 20.02           H  \nATOM   2900 HD13 ILE A 188      -9.127 -27.764  32.941  1.00 20.03           H  \nATOM   2901  N   PRO A 189     -11.762 -31.297  36.866  1.00 23.37           N  \nATOM   2902  CA  PRO A 189     -12.149 -32.327  37.839  1.00 23.87           C  \nATOM   2903  C   PRO A 189     -11.020 -32.788  38.745  1.00 23.97           C  \nATOM   2904  O   PRO A 189      -9.860 -32.822  38.365  1.00 21.13           O  \nATOM   2905  CB  PRO A 189     -12.638 -33.505  36.984  1.00 25.85           C  \nATOM   2906  CG  PRO A 189     -12.700 -33.001  35.567  1.00 27.61           C  \nATOM   2907  CD  PRO A 189     -11.968 -31.679  35.467  1.00 24.96           C  \nATOM   2908  HA  PRO A 189     -12.901 -31.987  38.386  1.00 24.16           H  \nATOM   2909  HB2 PRO A 189     -12.019 -34.261  37.051  1.00 25.75           H  \nATOM   2910  HB3 PRO A 189     -13.524 -33.802  37.282  1.00 25.77           H  \nATOM   2911  HG2 PRO A 189     -12.291 -33.654  34.963  1.00 26.55           H  \nATOM   2912  HG3 PRO A 189     -13.634 -32.884  35.296  1.00 26.55           H  \nATOM   2913  HD2 PRO A 189     -11.117 -31.783  35.003  1.00 25.16           H  \nATOM   2914  HD3 PRO A 189     -12.508 -31.017  34.998  1.00 25.15           H  \nATOM   2915  N   ALA A 190     -11.386 -33.197  39.961  1.00 24.32           N  \nATOM   2916  CA  ALA A 190     -10.451 -33.585  40.995  1.00 23.47           C  \nATOM   2917  C   ALA A 190      -9.684 -34.873  40.701  1.00 21.24           C  \nATOM   2918  O   ALA A 190      -8.703 -35.144  41.387  1.00 23.92           O  \nATOM   2919  CB  ALA A 190     -11.235 -33.745  42.284  1.00 32.30           C  \nATOM   2920  H   ALA A 190     -12.278 -33.238  40.153  1.00 24.05           H  \nATOM   2921  HA  ALA A 190      -9.798 -32.856  41.111  1.00 24.66           H  \nATOM   2922  HB1 ALA A 190     -10.626 -33.956  43.011  1.00 29.26           H  \nATOM   2923  HB2 ALA A 190     -11.702 -32.916  42.485  1.00 29.26           H  \nATOM   2924  HB3 ALA A 190     -11.881 -34.464  42.184  1.00 29.26           H  \nATOM   2925  N   ARG A 191     -10.057 -35.602  39.637  1.00 21.91           N  \nATOM   2926  CA  ARG A 191      -9.279 -36.746  39.200  1.00 19.63           C  \nATOM   2927  C   ARG A 191      -7.906 -36.368  38.641  1.00 17.22           C  \nATOM   2928  O   ARG A 191      -7.026 -37.223  38.569  1.00 17.72           O  \nATOM   2929  CB  ARG A 191      -9.998 -37.627  38.179  1.00 19.67           C  \nATOM   2930  CG  ARG A 191     -10.402 -36.995  36.869  1.00 20.22           C  \nATOM   2931  CD  ARG A 191     -11.119 -38.035  36.024  1.00 23.53           C  \nATOM   2932  NE  ARG A 191     -11.424 -37.555  34.706  1.00 22.32           N  \nATOM   2933  CZ  ARG A 191     -12.465 -36.784  34.427  1.00 23.97           C  \nATOM   2934  NH1 ARG A 191     -13.333 -36.491  35.380  1.00 27.81           N  \nATOM   2935  NH2 ARG A 191     -12.670 -36.367  33.198  1.00 25.78           N  \nATOM   2936  H   ARG A 191     -10.826 -35.394  39.189  1.00 21.21           H  \nATOM   2937  HA  ARG A 191      -9.121 -37.301  39.998  1.00 19.57           H  \nATOM   2938  HB2 ARG A 191      -9.416 -38.390  37.980  1.00 19.78           H  \nATOM   2939  HB3 ARG A 191     -10.805 -37.982  38.606  1.00 19.78           H  \nATOM   2940  HG2 ARG A 191     -10.999 -36.236  37.033  1.00 20.83           H  \nATOM   2941  HG3 ARG A 191      -9.608 -36.674  36.391  1.00 20.82           H  \nATOM   2942  HD2 ARG A 191     -10.551 -38.828  35.948  1.00 22.41           H  \nATOM   2943  HD3 ARG A 191     -11.949 -38.301  36.474  1.00 22.44           H  \nATOM   2944  HE  ARG A 191     -10.897 -37.783  34.054  1.00 22.99           H  \nATOM   2945 HH11 ARG A 191     -13.141 -36.792  36.167  0.00 27.06           H  \nATOM   2946 HH12 ARG A 191     -14.045 -36.059  35.176  0.00 27.06           H  \nATOM   2947 HH21 ARG A 191     -13.488 -36.326  32.880  1.00 25.21           H  \nATOM   2948 HH22 ARG A 191     -11.987 -36.131  32.696  1.00 25.20           H  \nATOM   2949  N   PHE A 192      -7.712 -35.077  38.321  1.00 17.68           N  \nATOM   2950  CA  PHE A 192      -6.394 -34.577  37.956  1.00 16.16           C  \nATOM   2951  C   PHE A 192      -5.640 -34.030  39.157  1.00 17.89           C  \nATOM   2952  O   PHE A 192      -6.188 -33.219  39.926  1.00 19.36           O  \nATOM   2953  CB  PHE A 192      -6.505 -33.422  36.949  1.00 15.69           C  \nATOM   2954  CG  PHE A 192      -7.082 -33.848  35.626  1.00 16.13           C  \nATOM   2955  CD1 PHE A 192      -6.251 -34.337  34.622  1.00 15.06           C  \nATOM   2956  CD2 PHE A 192      -8.451 -33.859  35.416  1.00 17.87           C  \nATOM   2957  CE1 PHE A 192      -6.796 -34.750  33.420  1.00 15.15           C  \nATOM   2958  CE2 PHE A 192      -8.986 -34.285  34.210  1.00 17.66           C  \nATOM   2959  CZ  PHE A 192      -8.155 -34.736  33.213  1.00 16.55           C  \nATOM   2960  H   PHE A 192      -8.420 -34.499  38.311  1.00 17.22           H  \nATOM   2961  HA  PHE A 192      -5.868 -35.311  37.549  1.00 16.68           H  \nATOM   2962  HB2 PHE A 192      -7.071 -32.721  37.335  1.00 15.90           H  \nATOM   2963  HB3 PHE A 192      -5.612 -33.045  36.802  1.00 15.91           H  \nATOM   2964  HD1 PHE A 192      -5.315 -34.350  34.748  1.00 15.44           H  \nATOM   2965  HD2 PHE A 192      -9.026 -33.550  36.096  1.00 17.28           H  \nATOM   2966  HE1 PHE A 192      -6.225 -35.061  32.735  1.00 15.54           H  \nATOM   2967  HE2 PHE A 192      -9.920 -34.265  34.075  1.00 17.30           H  \nATOM   2968  HZ  PHE A 192      -8.514 -35.023  32.388  1.00 16.44           H  \nATOM   2969  N   SER A 193      -4.391 -34.473  39.297  1.00 17.41           N  \nATOM   2970  CA  SER A 193      -3.542 -33.975  40.361  1.00 19.00           C  \nATOM   2971  C   SER A 193      -2.105 -33.859  39.897  1.00 18.63           C  \nATOM   2972  O   SER A 193      -1.642 -34.593  39.049  1.00 18.30           O  \nATOM   2973  CB  SER A 193      -3.657 -34.833  41.602  1.00 21.64           C  \nATOM   2974  OG  SER A 193      -3.168 -36.115  41.335  1.00 24.47           O  \nATOM   2975  H   SER A 193      -4.059 -35.100  38.721  1.00 17.88           H  \nATOM   2976  HA  SER A 193      -3.857 -33.060  40.593  1.00 19.07           H  \nATOM   2977  HB2 SER A 193      -3.140 -34.426  42.334  1.00 21.60           H  \nATOM   2978  HB3 SER A 193      -4.599 -34.887  41.881  1.00 21.60           H  \nATOM   2979  HG  SER A 193      -2.165 -36.281  41.350  0.00 16.74           H  \nATOM   2980  N   GLY A 194      -1.396 -32.859  40.400  1.00 16.72           N  \nATOM   2981  CA  GLY A 194      -0.007 -32.722  40.063  1.00 16.23           C  \nATOM   2982  C   GLY A 194       0.860 -32.769  41.304  1.00 16.35           C  \nATOM   2983  O   GLY A 194       0.417 -32.329  42.377  1.00 17.40           O  \nATOM   2984  H   GLY A 194      -1.784 -32.252  40.964  1.00 17.04           H  \nATOM   2985  HA2 GLY A 194       0.256 -33.450  39.450  1.00 16.37           H  \nATOM   2986  HA3 GLY A 194       0.129 -31.861  39.596  1.00 16.37           H  \nATOM   2987  N   SER A 195       2.091 -33.229  41.108  1.00 14.95           N  \nATOM   2988  CA  SER A 195       3.059 -33.302  42.175  1.00 16.16           C  \nATOM   2989  C   SER A 195       4.443 -33.014  41.635  1.00 16.26           C  \nATOM   2990  O   SER A 195       4.726 -33.005  40.420  1.00 13.89           O  \nATOM   2991  CB  SER A 195       2.993 -34.654  42.832  1.00 18.48           C  \nATOM   2992  OG  SER A 195       3.444 -35.664  41.927  1.00 27.15           O  \nATOM   2993  H   SER A 195       2.336 -33.510  40.274  1.00 15.56           H  \nATOM   2994  HA  SER A 195       2.836 -32.607  42.852  1.00 16.36           H  \nATOM   2995  HB2 SER A 195       3.558 -34.656  43.637  1.00 19.62           H  \nATOM   2996  HB3 SER A 195       2.067 -34.843  43.103  1.00 19.62           H  \nATOM   2997  HG  SER A 195       4.155 -35.802  41.934  0.00 27.99           H  \nATOM   2998  N   GLY A 196       5.382 -32.844  42.569  1.00 17.12           N  \nATOM   2999  CA  GLY A 196       6.772 -32.713  42.204  1.00 14.74           C  \nATOM   3000  C   GLY A 196       7.376 -31.404  42.637  1.00 15.40           C  \nATOM   3001  O   GLY A 196       6.674 -30.485  43.073  1.00 16.45           O  \nATOM   3002  H   GLY A 196       5.147 -32.810  43.451  1.00 16.33           H  \nATOM   3003  HA2 GLY A 196       7.280 -33.456  42.617  1.00 15.43           H  \nATOM   3004  HA3 GLY A 196       6.859 -32.798  41.225  1.00 15.42           H  \nATOM   3005  N   SER A 197       8.689 -31.325  42.432  1.00 15.27           N  \nATOM   3006  CA  SER A 197       9.430 -30.098  42.646  1.00 16.03           C  \nATOM   3007  C   SER A 197      10.762 -30.159  41.937  1.00 16.90           C  \nATOM   3008  O   SER A 197      11.218 -31.246  41.540  1.00 16.22           O  \nATOM   3009  CB  SER A 197       9.627 -29.810  44.119  1.00 18.14           C  \nATOM   3010  OG  SER A 197      10.457 -30.835  44.665  1.00 22.02           O  \nATOM   3011  H   SER A 197       9.142 -32.068  42.150  1.00 15.48           H  \nATOM   3012  HA  SER A 197       8.904 -29.352  42.250  1.00 16.43           H  \nATOM   3013  HB2 SER A 197      10.056 -28.932  44.235  1.00 18.46           H  \nATOM   3014  HB3 SER A 197       8.758 -29.799  44.580  1.00 18.44           H  \nATOM   3015  HG  SER A 197      11.374 -30.442  44.602  0.00 22.23           H  \nATOM   3016  N   ARG A 198      11.368 -28.974  41.795  1.00 18.01           N  \nATOM   3017  CA  ARG A 198      12.706 -28.793  41.234  1.00 17.83           C  \nATOM   3018  C   ARG A 198      12.780 -29.264  39.789  1.00 18.32           C  \nATOM   3019  O   ARG A 198      12.465 -28.471  38.903  1.00 16.27           O  \nATOM   3020  CB  ARG A 198      13.779 -29.408  42.136  1.00 19.99           C  \nATOM   3021  CG  ARG A 198      13.716 -28.752  43.507  1.00 22.66           C  \nATOM   3022  CD  ARG A 198      14.640 -29.340  44.552  1.00 28.63           C  \nATOM   3023  NE  ARG A 198      14.615 -28.436  45.714  1.00 32.59           N  \nATOM   3024  CZ  ARG A 198      13.610 -28.291  46.603  1.00 37.40           C  \nATOM   3025  NH1 ARG A 198      12.501 -29.017  46.536  1.00 37.12           N  \nATOM   3026  NH2 ARG A 198      13.750 -27.445  47.619  1.00 36.28           N  \nATOM   3027  H   ARG A 198      10.913 -28.229  42.061  1.00 17.70           H  \nATOM   3028  HA  ARG A 198      12.870 -27.822  41.221  1.00 18.34           H  \nATOM   3029  HB2 ARG A 198      13.624 -30.371  42.226  1.00 20.02           H  \nATOM   3030  HB3 ARG A 198      14.664 -29.265  41.740  1.00 20.04           H  \nATOM   3031  HG2 ARG A 198      13.929 -27.799  43.407  1.00 23.27           H  \nATOM   3032  HG3 ARG A 198      12.797 -28.811  43.842  1.00 23.28           H  \nATOM   3033  HD2 ARG A 198      14.335 -30.231  44.804  1.00 27.95           H  \nATOM   3034  HD3 ARG A 198      15.552 -29.406  44.196  1.00 27.93           H  \nATOM   3035  HE  ARG A 198      15.324 -27.944  45.843  1.00 32.67           H  \nATOM   3036 HH11 ARG A 198      12.650 -30.015  46.099  0.00 39.26           H  \nATOM   3037 HH12 ARG A 198      12.070 -29.407  47.375  0.00 39.26           H  \nATOM   3038 HH21 ARG A 198      13.239 -27.525  48.330  1.00 36.61           H  \nATOM   3039 HH22 ARG A 198      14.352 -26.805  47.578  1.00 36.60           H  \nATOM   3040  N   THR A 199      13.213 -30.510  39.548  1.00 17.59           N  \nATOM   3041  CA  THR A 199      13.363 -31.001  38.192  1.00 16.41           C  \nATOM   3042  C   THR A 199      12.404 -32.146  37.878  1.00 17.41           C  \nATOM   3043  O   THR A 199      12.458 -32.584  36.735  1.00 18.18           O  \nATOM   3044  CB  THR A 199      14.793 -31.466  37.862  1.00 18.47           C  \nATOM   3045  OG1 THR A 199      15.127 -32.608  38.649  1.00 21.03           O  \nATOM   3046  CG2 THR A 199      15.849 -30.404  38.098  1.00 19.84           C  \nATOM   3047  H   THR A 199      13.411 -31.068  40.243  1.00 17.49           H  \nATOM   3048  HA  THR A 199      13.146 -30.258  37.583  1.00 17.20           H  \nATOM   3049  HB  THR A 199      14.818 -31.723  36.911  1.00 18.76           H  \nATOM   3050  HG1 THR A 199      15.088 -32.424  39.468  0.00 21.81           H  \nATOM   3051 HG21 THR A 199      16.652 -30.625  37.593  1.00 19.40           H  \nATOM   3052 HG22 THR A 199      16.065 -30.364  39.047  1.00 19.41           H  \nATOM   3053 HG23 THR A 199      15.511 -29.540  37.807  1.00 19.39           H  \nATOM   3054  N   ASP A 200      11.563 -32.578  38.813  1.00 16.72           N  \nATOM   3055  CA  ASP A 200      10.835 -33.823  38.619  1.00 17.74           C  \nATOM   3056  C   ASP A 200       9.363 -33.639  38.974  1.00 14.78           C  \nATOM   3057  O   ASP A 200       9.037 -33.369  40.125  1.00 17.07           O  \nATOM   3058  CB  ASP A 200      11.432 -34.946  39.488  1.00 23.78           C  \nATOM   3059  CG  ASP A 200      10.734 -36.272  39.262  1.00 33.80           C  \nATOM   3060  OD1 ASP A 200      10.855 -36.816  38.138  1.00 46.19           O  \nATOM   3061  OD2 ASP A 200      10.059 -36.742  40.192  1.00 53.91           O  \nATOM   3062  H   ASP A 200      11.430 -32.095  39.577  1.00 17.12           H  \nATOM   3063  HA  ASP A 200      10.903 -34.088  37.673  1.00 17.90           H  \nATOM   3064  HB2 ASP A 200      12.382 -35.051  39.275  1.00 24.12           H  \nATOM   3065  HB3 ASP A 200      11.352 -34.704  40.434  1.00 24.15           H  \nATOM   3066  N   PHE A 201       8.460 -33.791  37.981  1.00 14.10           N  \nATOM   3067  CA  PHE A 201       7.054 -33.497  38.140  1.00 13.37           C  \nATOM   3068  C   PHE A 201       6.227 -34.629  37.567  1.00 12.74           C  \nATOM   3069  O   PHE A 201       6.648 -35.340  36.651  1.00 14.55           O  \nATOM   3070  CB  PHE A 201       6.620 -32.177  37.443  1.00 13.56           C  \nATOM   3071  CG  PHE A 201       7.430 -31.039  38.009  1.00 14.07           C  \nATOM   3072  CD1 PHE A 201       8.666 -30.701  37.519  1.00 14.44           C  \nATOM   3073  CD2 PHE A 201       6.915 -30.324  39.067  1.00 13.92           C  \nATOM   3074  CE1 PHE A 201       9.419 -29.687  38.113  1.00 14.36           C  \nATOM   3075  CE2 PHE A 201       7.655 -29.290  39.648  1.00 13.83           C  \nATOM   3076  CZ  PHE A 201       8.886 -28.990  39.166  1.00 13.41           C  \nATOM   3077  H   PHE A 201       8.748 -34.095  37.169  1.00 14.10           H  \nATOM   3078  HA  PHE A 201       6.854 -33.424  39.105  1.00 13.43           H  \nATOM   3079  HB2 PHE A 201       6.775 -32.251  36.478  1.00 13.63           H  \nATOM   3080  HB3 PHE A 201       5.666 -32.021  37.601  1.00 13.64           H  \nATOM   3081  HD1 PHE A 201       9.035 -31.201  36.809  1.00 14.29           H  \nATOM   3082  HD2 PHE A 201       6.068 -30.550  39.416  1.00 13.93           H  \nATOM   3083  HE1 PHE A 201      10.266 -29.458  37.766  1.00 14.13           H  \nATOM   3084  HE2 PHE A 201       7.299 -28.806  40.375  1.00 13.79           H  \nATOM   3085  HZ  PHE A 201       9.375 -28.278  39.547  1.00 13.75           H  \nATOM   3086  N   THR A 202       5.042 -34.756  38.123  1.00 13.08           N  \nATOM   3087  CA  THR A 202       4.122 -35.805  37.711  1.00 14.19           C  \nATOM   3088  C   THR A 202       2.716 -35.259  37.634  1.00 14.31           C  \nATOM   3089  O   THR A 202       2.245 -34.570  38.531  1.00 15.50           O  \nATOM   3090  CB  THR A 202       4.127 -36.991  38.687  1.00 16.66           C  \nATOM   3091  OG1 THR A 202       5.462 -37.466  38.825  1.00 22.02           O  \nATOM   3092  CG2 THR A 202       3.251 -38.117  38.207  1.00 18.89           C  \nATOM   3093  H   THR A 202       4.784 -34.176  38.781  1.00 13.26           H  \nATOM   3094  HA  THR A 202       4.391 -36.124  36.821  1.00 14.43           H  \nATOM   3095  HB  THR A 202       3.801 -36.679  39.564  1.00 17.47           H  \nATOM   3096  HG1 THR A 202       5.794 -37.678  38.017  0.00 21.90           H  \nATOM   3097 HG21 THR A 202       3.570 -38.958  38.580  1.00 18.19           H  \nATOM   3098 HG22 THR A 202       3.280 -38.162  37.235  1.00 18.16           H  \nATOM   3099 HG23 THR A 202       2.334 -37.963  38.497  1.00 18.17           H  \nATOM   3100  N   LEU A 203       1.999 -35.624  36.570  1.00 12.20           N  \nATOM   3101  CA  LEU A 203       0.575 -35.458  36.421  1.00 12.01           C  \nATOM   3102  C   LEU A 203      -0.071 -36.826  36.581  1.00 13.46           C  \nATOM   3103  O   LEU A 203       0.324 -37.758  35.862  1.00 14.61           O  \nATOM   3104  CB  LEU A 203       0.242 -34.901  35.025  1.00 12.84           C  \nATOM   3105  CG  LEU A 203      -1.243 -34.738  34.761  1.00 13.60           C  \nATOM   3106  CD1 LEU A 203      -1.861 -33.636  35.607  1.00 13.80           C  \nATOM   3107  CD2 LEU A 203      -1.519 -34.465  33.305  1.00 14.34           C  \nATOM   3108  H   LEU A 203       2.449 -36.010  35.874  1.00 12.63           H  \nATOM   3109  HA  LEU A 203       0.242 -34.849  37.120  1.00 12.50           H  \nATOM   3110  HB2 LEU A 203       0.674 -34.031  34.926  1.00 12.81           H  \nATOM   3111  HB3 LEU A 203       0.618 -35.501  34.351  1.00 12.82           H  \nATOM   3112  HG  LEU A 203      -1.690 -35.587  34.997  1.00 13.63           H  \nATOM   3113 HD11 LEU A 203      -2.771 -33.466  35.306  1.00 13.74           H  \nATOM   3114 HD12 LEU A 203      -1.334 -32.823  35.518  1.00 13.74           H  \nATOM   3115 HD13 LEU A 203      -1.876 -33.912  36.540  1.00 13.74           H  \nATOM   3116 HD21 LEU A 203      -1.121 -35.168  32.761  1.00 14.11           H  \nATOM   3117 HD22 LEU A 203      -1.135 -33.607  33.056  1.00 14.11           H  \nATOM   3118 HD23 LEU A 203      -2.480 -34.446  33.154  1.00 14.11           H  \nATOM   3119  N   THR A 204      -1.025 -36.932  37.503  1.00 14.82           N  \nATOM   3120  CA  THR A 204      -1.741 -38.173  37.733  1.00 15.71           C  \nATOM   3121  C   THR A 204      -3.195 -37.934  37.391  1.00 15.96           C  \nATOM   3122  O   THR A 204      -3.813 -36.978  37.832  1.00 17.39           O  \nATOM   3123  CB  THR A 204      -1.567 -38.646  39.179  1.00 17.21           C  \nATOM   3124  OG1 THR A 204      -0.190 -38.847  39.447  1.00 18.61           O  \nATOM   3125  CG2 THR A 204      -2.389 -39.907  39.428  1.00 20.16           C  \nATOM   3126  H   THR A 204      -1.240 -36.203  38.010  1.00 14.67           H  \nATOM   3127  HA  THR A 204      -1.378 -38.859  37.130  1.00 15.85           H  \nATOM   3128  HB  THR A 204      -1.902 -37.933  39.774  1.00 17.71           H  \nATOM   3129  HG1 THR A 204       0.163 -39.467  38.854  0.00 18.95           H  \nATOM   3130 HG21 THR A 204      -1.985 -40.421  40.149  1.00 19.20           H  \nATOM   3131 HG22 THR A 204      -2.410 -40.448  38.618  1.00 19.19           H  \nATOM   3132 HG23 THR A 204      -3.298 -39.661  39.676  1.00 19.20           H  \nATOM   3133  N   ILE A 205      -3.789 -38.849  36.589  1.00 14.40           N  \nATOM   3134  CA  ILE A 205      -5.184 -38.845  36.290  1.00 15.02           C  \nATOM   3135  C   ILE A 205      -5.736 -40.144  36.870  1.00 15.50           C  \nATOM   3136  O   ILE A 205      -5.298 -41.216  36.458  1.00 16.60           O  \nATOM   3137  CB  ILE A 205      -5.416 -38.769  34.778  1.00 15.40           C  \nATOM   3138  CG1 ILE A 205      -4.607 -37.665  34.083  1.00 17.01           C  \nATOM   3139  CG2 ILE A 205      -6.895 -38.625  34.479  1.00 16.79           C  \nATOM   3140  CD1 ILE A 205      -4.683 -37.786  32.559  1.00 19.09           C  \nATOM   3141  H   ILE A 205      -3.271 -39.505  36.217  1.00 14.90           H  \nATOM   3142  HA  ILE A 205      -5.610 -38.086  36.720  1.00 15.08           H  \nATOM   3143  HB  ILE A 205      -5.121 -39.631  34.394  1.00 15.89           H  \nATOM   3144 HG12 ILE A 205      -4.954 -36.790  34.356  1.00 17.07           H  \nATOM   3145 HG13 ILE A 205      -3.669 -37.725  34.363  1.00 17.07           H  \nATOM   3146 HG21 ILE A 205      -7.177 -37.712  34.663  1.00 16.35           H  \nATOM   3147 HG22 ILE A 205      -7.406 -39.237  35.036  1.00 16.35           H  \nATOM   3148 HG23 ILE A 205      -7.060 -38.830  33.542  1.00 16.36           H  \nATOM   3149 HD11 ILE A 205      -5.534 -37.432  32.246  1.00 18.42           H  \nATOM   3150 HD12 ILE A 205      -4.608 -38.721  32.301  1.00 18.41           H  \nATOM   3151 HD13 ILE A 205      -3.956 -37.280  32.155  1.00 18.42           H  \nATOM   3152  N   ASN A 206      -6.647 -40.034  37.812  1.00 16.85           N  \nATOM   3153  CA  ASN A 206      -7.125 -41.210  38.513  1.00 18.14           C  \nATOM   3154  C   ASN A 206      -8.508 -40.942  39.081  1.00 17.28           C  \nATOM   3155  O   ASN A 206      -8.689 -40.061  39.926  1.00 18.35           O  \nATOM   3156  CB  ASN A 206      -6.186 -41.579  39.650  1.00 18.81           C  \nATOM   3157  CG  ASN A 206      -6.564 -42.891  40.288  1.00 21.87           C  \nATOM   3158  OD1 ASN A 206      -7.213 -43.709  39.655  1.00 23.45           O  \nATOM   3159  ND2 ASN A 206      -6.059 -43.117  41.477  1.00 23.03           N  \nATOM   3160  H   ASN A 206      -6.985 -39.213  38.032  1.00 16.80           H  \nATOM   3161  HA  ASN A 206      -7.164 -41.963  37.884  1.00 17.85           H  \nATOM   3162  HB2 ASN A 206      -5.273 -41.642  39.303  1.00 19.32           H  \nATOM   3163  HB3 ASN A 206      -6.207 -40.872  40.327  1.00 19.32           H  \nATOM   3164 HD21 ASN A 206      -5.649 -42.532  41.937  0.00 23.09           H  \nATOM   3165 HD22 ASN A 206      -6.273 -43.929  41.889  0.00 23.09           H  \nATOM   3166  N   PRO A 207      -9.544 -41.719  38.694  1.00 17.20           N  \nATOM   3167  CA  PRO A 207      -9.526 -42.699  37.610  1.00 17.39           C  \nATOM   3168  C   PRO A 207      -9.734 -42.077  36.234  1.00 17.35           C  \nATOM   3169  O   PRO A 207     -10.493 -41.110  36.110  1.00 21.85           O  \nATOM   3170  CB  PRO A 207     -10.753 -43.585  37.915  1.00 17.76           C  \nATOM   3171  CG  PRO A 207     -11.652 -42.721  38.745  1.00 19.93           C  \nATOM   3172  CD  PRO A 207     -10.836 -41.612  39.352  1.00 17.22           C  \nATOM   3173  HA  PRO A 207      -8.699 -43.241  37.640  1.00 17.43           H  \nATOM   3174  HB2 PRO A 207     -11.203 -43.854  37.088  1.00 18.15           H  \nATOM   3175  HB3 PRO A 207     -10.490 -44.390  38.409  1.00 18.16           H  \nATOM   3176  HG2 PRO A 207     -12.364 -42.346  38.188  1.00 18.74           H  \nATOM   3177  HG3 PRO A 207     -12.069 -43.255  39.452  1.00 18.74           H  \nATOM   3178  HD2 PRO A 207     -11.243 -40.743  39.178  1.00 17.82           H  \nATOM   3179  HD3 PRO A 207     -10.746 -41.731  40.315  1.00 17.82           H  \nATOM   3180  N   VAL A 208      -9.138 -42.686  35.216  1.00 17.30           N  \nATOM   3181  CA  VAL A 208      -9.236 -42.195  33.851  1.00 16.51           C  \nATOM   3182  C   VAL A 208     -10.634 -42.508  33.313  1.00 19.31           C  \nATOM   3183  O   VAL A 208     -11.160 -43.615  33.494  1.00 18.30           O  \nATOM   3184  CB  VAL A 208      -8.142 -42.835  32.967  1.00 16.39           C  \nATOM   3185  CG1 VAL A 208      -8.423 -42.707  31.476  1.00 16.14           C  \nATOM   3186  CG2 VAL A 208      -6.763 -42.267  33.273  1.00 17.10           C  \nATOM   3187  H   VAL A 208      -8.649 -43.442  35.376  1.00 17.12           H  \nATOM   3188  HA  VAL A 208      -9.108 -41.229  33.858  1.00 17.15           H  \nATOM   3189  HB  VAL A 208      -8.120 -43.800  33.182  1.00 16.50           H  \nATOM   3190 HG11 VAL A 208      -8.792 -41.826  31.288  1.00 16.23           H  \nATOM   3191 HG12 VAL A 208      -9.060 -43.390  31.204  1.00 16.23           H  \nATOM   3192 HG13 VAL A 208      -7.595 -42.822  30.978  1.00 16.23           H  \nATOM   3193 HG21 VAL A 208      -6.084 -42.800  32.825  1.00 16.88           H  \nATOM   3194 HG22 VAL A 208      -6.609 -42.288  34.234  1.00 16.88           H  \nATOM   3195 HG23 VAL A 208      -6.713 -41.348  32.958  1.00 16.88           H  \nATOM   3196  N   GLU A 209     -11.203 -41.522  32.623  1.00 16.85           N  \nATOM   3197  CA  GLU A 209     -12.438 -41.590  31.845  1.00 19.31           C  \nATOM   3198  C   GLU A 209     -12.154 -41.358  30.360  1.00 20.25           C  \nATOM   3199  O   GLU A 209     -11.071 -40.929  29.947  1.00 18.11           O  \nATOM   3200  CB  GLU A 209     -13.428 -40.514  32.319  1.00 24.72           C  \nATOM   3201  CG  GLU A 209     -13.540 -40.400  33.826  1.00 30.09           C  \nATOM   3202  CD  GLU A 209     -14.676 -39.525  34.314  1.00 28.92           C  \nATOM   3203  OE1 GLU A 209     -15.011 -39.619  35.521  1.00 38.96           O  \nATOM   3204  OE2 GLU A 209     -15.226 -38.779  33.478  1.00 35.12           O  \nATOM   3205  H   GLU A 209     -10.775 -40.715  32.638  1.00 17.96           H  \nATOM   3206  HA  GLU A 209     -12.843 -42.479  31.961  1.00 19.90           H  \nATOM   3207  HB2 GLU A 209     -13.146 -39.648  31.957  1.00 24.42           H  \nATOM   3208  HB3 GLU A 209     -14.312 -40.721  31.949  1.00 24.44           H  \nATOM   3209  HG2 GLU A 209     -13.658 -41.298  34.203  1.00 28.45           H  \nATOM   3210  HG3 GLU A 209     -12.700 -40.040  34.180  1.00 28.43           H  \nATOM   3211  N   ALA A 210     -13.137 -41.670  29.513  1.00 20.56           N  \nATOM   3212  CA  ALA A 210     -12.946 -41.659  28.065  1.00 21.05           C  \nATOM   3213  C   ALA A 210     -12.708 -40.222  27.586  1.00 20.22           C  \nATOM   3214  O   ALA A 210     -12.008 -39.998  26.593  1.00 24.45           O  \nATOM   3215  CB  ALA A 210     -14.157 -42.260  27.387  1.00 20.83           C  \nATOM   3216  H   ALA A 210     -13.957 -41.898  29.846  1.00 20.60           H  \nATOM   3217  HA  ALA A 210     -12.153 -42.201  27.846  1.00 20.75           H  \nATOM   3218  HB1 ALA A 210     -14.026 -42.256  26.424  1.00 20.90           H  \nATOM   3219  HB2 ALA A 210     -14.279 -43.175  27.691  1.00 20.90           H  \nATOM   3220  HB3 ALA A 210     -14.947 -41.738  27.608  1.00 20.90           H  \nATOM   3221  N   ASP A 211     -13.246 -39.260  28.336  1.00 21.09           N  \nATOM   3222  CA  ASP A 211     -13.089 -37.833  28.047  1.00 24.02           C  \nATOM   3223  C   ASP A 211     -11.632 -37.390  28.228  1.00 20.36           C  \nATOM   3224  O   ASP A 211     -11.258 -36.281  27.814  1.00 20.79           O  \nATOM   3225  CB  ASP A 211     -13.982 -36.992  28.968  1.00 27.74           C  \nATOM   3226  CG  ASP A 211     -14.054 -35.519  28.586  1.00 35.95           C  \nATOM   3227  OD1 ASP A 211     -13.728 -34.656  29.445  1.00 39.04           O  \nATOM   3228  OD2 ASP A 211     -14.366 -35.254  27.411  1.00 44.18           O  \nATOM   3229  H   ASP A 211     -13.754 -39.497  29.057  1.00 21.53           H  \nATOM   3230  HA  ASP A 211     -13.357 -37.673  27.112  1.00 23.36           H  \nATOM   3231  HB2 ASP A 211     -14.890 -37.358  28.951  1.00 28.51           H  \nATOM   3232  HB3 ASP A 211     -13.644 -37.056  29.885  1.00 28.51           H  \nATOM   3233  N   ASP A 212     -10.782 -38.227  28.837  1.00 16.27           N  \nATOM   3234  CA  ASP A 212      -9.412 -37.794  29.136  1.00 14.78           C  \nATOM   3235  C   ASP A 212      -8.451 -38.147  28.015  1.00 15.73           C  \nATOM   3236  O   ASP A 212      -7.240 -37.858  28.072  1.00 15.16           O  \nATOM   3237  CB  ASP A 212      -8.946 -38.406  30.463  1.00 17.60           C  \nATOM   3238  CG  ASP A 212      -9.827 -38.043  31.642  1.00 20.29           C  \nATOM   3239  OD1 ASP A 212     -10.382 -36.938  31.625  1.00 20.67           O  \nATOM   3240  OD2 ASP A 212      -9.947 -38.901  32.606  1.00 20.14           O  \nATOM   3241  H   ASP A 212     -11.048 -39.065  29.084  1.00 16.81           H  \nATOM   3242  HA  ASP A 212      -9.415 -36.814  29.241  1.00 15.77           H  \nATOM   3243  HB2 ASP A 212      -8.929 -39.381  30.377  1.00 17.47           H  \nATOM   3244  HB3 ASP A 212      -8.035 -38.101  30.652  1.00 17.46           H  \nATOM   3245  N   VAL A 213      -8.935 -38.785  26.943  1.00 15.28           N  \nATOM   3246  CA  VAL A 213      -8.107 -39.072  25.791  1.00 16.40           C  \nATOM   3247  C   VAL A 213      -7.705 -37.741  25.149  1.00 16.42           C  \nATOM   3248  O   VAL A 213      -8.565 -36.952  24.751  1.00 19.31           O  \nATOM   3249  CB  VAL A 213      -8.874 -39.999  24.820  1.00 17.83           C  \nATOM   3250  CG1 VAL A 213      -8.173 -40.159  23.495  1.00 18.94           C  \nATOM   3251  CG2 VAL A 213      -9.119 -41.368  25.465  1.00 19.53           C  \nATOM   3252  H   VAL A 213      -9.810 -39.049  26.931  1.00 15.68           H  \nATOM   3253  HA  VAL A 213      -7.305 -39.532  26.094  1.00 16.43           H  \nATOM   3254  HB  VAL A 213      -9.755 -39.584  24.647  1.00 18.05           H  \nATOM   3255 HG11 VAL A 213      -7.211 -40.170  23.634  1.00 18.58           H  \nATOM   3256 HG12 VAL A 213      -8.407 -39.416  22.911  1.00 18.59           H  \nATOM   3257 HG13 VAL A 213      -8.449 -40.995  23.078  1.00 18.59           H  \nATOM   3258 HG21 VAL A 213      -9.613 -41.934  24.848  1.00 18.99           H  \nATOM   3259 HG22 VAL A 213      -9.633 -41.256  26.283  1.00 18.98           H  \nATOM   3260 HG23 VAL A 213      -8.265 -41.785  25.675  1.00 18.98           H  \nATOM   3261  N   ALA A 214      -6.399 -37.512  25.078  1.00 14.36           N  \nATOM   3262  CA  ALA A 214      -5.855 -36.202  24.757  1.00 14.73           C  \nATOM   3263  C   ALA A 214      -4.338 -36.308  24.765  1.00 13.50           C  \nATOM   3264  O   ALA A 214      -3.769 -37.343  25.137  1.00 13.34           O  \nATOM   3265  CB  ALA A 214      -6.295 -35.191  25.795  1.00 14.63           C  \nATOM   3266  H   ALA A 214      -5.818 -38.201  25.232  1.00 14.91           H  \nATOM   3267  HA  ALA A 214      -6.164 -35.924  23.863  1.00 14.40           H  \nATOM   3268  HB1 ALA A 214      -5.693 -34.427  25.784  1.00 14.66           H  \nATOM   3269  HB2 ALA A 214      -7.197 -34.891  25.595  1.00 14.67           H  \nATOM   3270  HB3 ALA A 214      -6.279 -35.601  26.676  1.00 14.66           H  \nATOM   3271  N   ILE A 215      -3.641 -35.219  24.384  1.00 13.78           N  \nATOM   3272  CA  ILE A 215      -2.218 -35.100  24.589  1.00 13.44           C  \nATOM   3273  C   ILE A 215      -2.019 -34.131  25.743  1.00 11.71           C  \nATOM   3274  O   ILE A 215      -2.670 -33.082  25.756  1.00 13.01           O  \nATOM   3275  CB  ILE A 215      -1.537 -34.581  23.311  1.00 14.16           C  \nATOM   3276  CG1 ILE A 215      -1.858 -35.477  22.101  1.00 18.34           C  \nATOM   3277  CG2 ILE A 215      -0.049 -34.463  23.530  1.00 14.68           C  \nATOM   3278  CD1 ILE A 215      -1.212 -35.015  20.806  1.00 25.28           C  \nATOM   3279  H   ILE A 215      -4.084 -34.528  23.981  1.00 13.63           H  \nATOM   3280  HA  ILE A 215      -1.848 -35.968  24.820  1.00 13.27           H  \nATOM   3281  HB  ILE A 215      -1.893 -33.677  23.123  1.00 14.83           H  \nATOM   3282 HG12 ILE A 215      -1.557 -36.390  22.295  1.00 18.62           H  \nATOM   3283 HG13 ILE A 215      -2.831 -35.502  21.974  1.00 18.61           H  \nATOM   3284 HG21 ILE A 215       0.321 -35.339  23.737  1.00 14.52           H  \nATOM   3285 HG22 ILE A 215       0.127 -33.857  24.270  1.00 14.51           H  \nATOM   3286 HG23 ILE A 215       0.376 -34.116  22.727  1.00 14.55           H  \nATOM   3287 HD11 ILE A 215      -0.498 -35.630  20.562  1.00 22.88           H  \nATOM   3288 HD12 ILE A 215      -0.843 -34.125  20.923  1.00 22.83           H  \nATOM   3289 HD13 ILE A 215      -1.879 -34.997  20.098  1.00 22.88           H  \nATOM   3290  N   TYR A 216      -1.085 -34.469  26.629  1.00 10.47           N  \nATOM   3291  CA  TYR A 216      -0.833 -33.691  27.832  1.00 10.27           C  \nATOM   3292  C   TYR A 216       0.578 -33.140  27.760  1.00 10.32           C  \nATOM   3293  O   TYR A 216       1.534 -33.813  27.358  1.00 11.56           O  \nATOM   3294  CB  TYR A 216      -0.975 -34.557  29.083  1.00 10.16           C  \nATOM   3295  CG  TYR A 216      -2.380 -35.006  29.292  1.00 11.99           C  \nATOM   3296  CD1 TYR A 216      -2.854 -36.149  28.664  1.00 11.54           C  \nATOM   3297  CD2 TYR A 216      -3.254 -34.270  30.069  1.00 11.49           C  \nATOM   3298  CE1 TYR A 216      -4.200 -36.474  28.757  1.00 11.72           C  \nATOM   3299  CE2 TYR A 216      -4.605 -34.571  30.156  1.00 12.90           C  \nATOM   3300  CZ  TYR A 216      -5.074 -35.703  29.517  1.00 13.09           C  \nATOM   3301  OH  TYR A 216      -6.388 -36.021  29.698  1.00 13.59           O  \nATOM   3302  H   TYR A 216      -0.579 -35.215  26.479  1.00 10.71           H  \nATOM   3303  HA  TYR A 216      -1.476 -32.942  27.879  1.00 10.30           H  \nATOM   3304  HB2 TYR A 216      -0.393 -35.340  28.994  1.00 10.59           H  \nATOM   3305  HB3 TYR A 216      -0.679 -34.041  29.862  1.00 10.59           H  \nATOM   3306  HD1 TYR A 216      -2.286 -36.651  28.105  1.00 11.72           H  \nATOM   3307  HD2 TYR A 216      -2.945 -33.471  30.463  1.00 11.93           H  \nATOM   3308  HE1 TYR A 216      -4.520 -37.252  28.332  1.00 12.00           H  \nATOM   3309  HE2 TYR A 216      -5.179 -34.057  30.700  1.00 12.54           H  \nATOM   3310  HH  TYR A 216      -6.706 -36.759  29.360  0.00 13.76           H  \nATOM   3311  N   TYR A 217       0.691 -31.856  28.107  1.00  9.28           N  \nATOM   3312  CA  TYR A 217       1.924 -31.087  28.023  1.00  9.62           C  \nATOM   3313  C   TYR A 217       2.282 -30.464  29.374  1.00 10.23           C  \nATOM   3314  O   TYR A 217       1.397 -29.911  29.996  1.00 10.95           O  \nATOM   3315  CB  TYR A 217       1.811 -29.959  27.008  1.00  9.99           C  \nATOM   3316  CG  TYR A 217       1.634 -30.380  25.577  1.00  9.35           C  \nATOM   3317  CD1 TYR A 217       2.753 -30.559  24.780  1.00  9.68           C  \nATOM   3318  CD2 TYR A 217       0.379 -30.500  25.015  1.00 10.74           C  \nATOM   3319  CE1 TYR A 217       2.636 -30.903  23.456  1.00 10.69           C  \nATOM   3320  CE2 TYR A 217       0.256 -30.901  23.705  1.00 11.69           C  \nATOM   3321  CZ  TYR A 217       1.376 -31.116  22.946  1.00 11.82           C  \nATOM   3322  OH  TYR A 217       1.250 -31.466  21.608  1.00 13.00           O  \nATOM   3323  H   TYR A 217      -0.053 -31.430  28.423  1.00  9.60           H  \nATOM   3324  HA  TYR A 217       2.658 -31.689  27.747  1.00  9.75           H  \nATOM   3325  HB2 TYR A 217       1.050 -29.396  27.263  1.00  9.75           H  \nATOM   3326  HB3 TYR A 217       2.619 -29.409  27.073  1.00  9.75           H  \nATOM   3327  HD1 TYR A 217       3.610 -30.443  25.153  1.00  9.94           H  \nATOM   3328  HD2 TYR A 217      -0.390 -30.396  25.551  1.00 10.63           H  \nATOM   3329  HE1 TYR A 217       3.402 -31.065  22.929  1.00 10.70           H  \nATOM   3330  HE2 TYR A 217      -0.600 -31.016  23.325  1.00 11.40           H  \nATOM   3331  HH  TYR A 217       1.936 -31.619  21.124  0.00 12.53           H  \nATOM   3332  N   CYS A 218       3.553 -30.577  29.741  1.00 10.45           N  \nATOM   3333  CA  CYS A 218       4.103 -29.761  30.807  1.00 10.79           C  \nATOM   3334  C   CYS A 218       4.712 -28.497  30.221  1.00 10.40           C  \nATOM   3335  O   CYS A 218       5.097 -28.456  29.045  1.00 10.78           O  \nATOM   3336  CB  CYS A 218       5.082 -30.523  31.672  1.00 12.47           C  \nATOM   3337  SG  CYS A 218       6.450 -31.212  30.722  1.00 15.62           S  \nATOM   3338  H   CYS A 218       4.097 -31.180  29.320  1.00 10.48           H  \nATOM   3339  HA  CYS A 218       3.354 -29.493  31.383  1.00 10.94           H  \nATOM   3340  HB2 CYS A 218       5.444 -29.924  32.355  1.00 12.71           H  \nATOM   3341  HB3 CYS A 218       4.615 -31.254  32.124  1.00 12.71           H  \nATOM   3342  N   GLN A 219       4.856 -27.460  31.092  1.00  9.61           N  \nATOM   3343  CA  GLN A 219       5.410 -26.181  30.681  1.00 10.04           C  \nATOM   3344  C   GLN A 219       6.011 -25.488  31.889  1.00  9.08           C  \nATOM   3345  O   GLN A 219       5.367 -25.426  32.934  1.00 11.78           O  \nATOM   3346  CB  GLN A 219       4.291 -25.292  30.132  1.00 10.50           C  \nATOM   3347  CG  GLN A 219       4.749 -23.928  29.622  1.00 10.39           C  \nATOM   3348  CD  GLN A 219       4.240 -22.752  30.415  1.00 11.66           C  \nATOM   3349  OE1 GLN A 219       5.043 -21.893  30.884  1.00 13.98           O  \nATOM   3350  NE2 GLN A 219       2.965 -22.679  30.624  1.00 11.13           N  \nATOM   3351  H   GLN A 219       4.596 -27.568  31.961  1.00  9.90           H  \nATOM   3352  HA  GLN A 219       6.098 -26.323  29.995  1.00  9.85           H  \nATOM   3353  HB2 GLN A 219       3.848 -25.771  29.401  1.00 10.36           H  \nATOM   3354  HB3 GLN A 219       3.630 -25.161  30.843  1.00 10.37           H  \nATOM   3355  HG2 GLN A 219       5.728 -23.903  29.623  1.00 10.70           H  \nATOM   3356  HG3 GLN A 219       4.457 -23.827  28.693  1.00 10.70           H  \nATOM   3357 HE21 GLN A 219       2.429 -23.331  30.283  0.00 22.56           H  \nATOM   3358 HE22 GLN A 219       2.622 -22.051  31.100  0.00 22.56           H  \nATOM   3359  N   GLN A 220       7.185 -24.882  31.712  1.00  9.08           N  \nATOM   3360  CA  GLN A 220       7.836 -24.133  32.792  1.00  9.71           C  \nATOM   3361  C   GLN A 220       7.782 -22.652  32.487  1.00  9.31           C  \nATOM   3362  O   GLN A 220       7.948 -22.255  31.350  1.00 10.15           O  \nATOM   3363  CB  GLN A 220       9.262 -24.622  33.040  1.00 10.71           C  \nATOM   3364  CG  GLN A 220      10.295 -24.312  31.983  1.00 10.27           C  \nATOM   3365  CD  GLN A 220      10.863 -22.904  31.979  1.00 10.98           C  \nATOM   3366  OE1 GLN A 220      10.829 -22.237  33.025  1.00 11.21           O  \nATOM   3367  NE2 GLN A 220      11.233 -22.405  30.805  1.00 10.50           N  \nATOM   3368  H   GLN A 220       7.598 -24.930  30.898  1.00  9.23           H  \nATOM   3369  HA  GLN A 220       7.321 -24.291  33.613  1.00  9.70           H  \nATOM   3370  HB2 GLN A 220       9.565 -24.240  33.890  1.00 10.36           H  \nATOM   3371  HB3 GLN A 220       9.226 -25.594  33.158  1.00 10.36           H  \nATOM   3372  HG2 GLN A 220      11.040 -24.942  32.087  1.00 10.54           H  \nATOM   3373  HG3 GLN A 220       9.896 -24.483  31.103  1.00 10.53           H  \nATOM   3374 HE21 GLN A 220      11.053 -22.950  29.875  0.00  9.95           H  \nATOM   3375 HE22 GLN A 220      11.651 -21.747  30.609  0.00  9.95           H  \nATOM   3376  N   SER A 221       7.596 -21.863  33.569  1.00 10.89           N  \nATOM   3377  CA  SER A 221       7.645 -20.408  33.498  1.00 10.51           C  \nATOM   3378  C   SER A 221       8.666 -19.835  34.479  1.00 11.49           C  \nATOM   3379  O   SER A 221       8.618 -18.640  34.749  1.00 12.41           O  \nATOM   3380  CB  SER A 221       6.302 -19.757  33.732  1.00 12.23           C  \nATOM   3381  OG  SER A 221       5.743 -20.049  34.962  1.00 12.71           O  \nATOM   3382  H   SER A 221       7.431 -22.253  34.379  1.00 10.41           H  \nATOM   3383  HA  SER A 221       7.943 -20.164  32.583  1.00 11.07           H  \nATOM   3384  HB2 SER A 221       6.403 -18.783  33.654  1.00 11.91           H  \nATOM   3385  HB3 SER A 221       5.683 -20.048  33.025  1.00 11.91           H  \nATOM   3386  HG  SER A 221       6.040 -19.568  35.687  0.00 12.60           H  \nATOM   3387  N   ASN A 222       9.604 -20.649  34.976  1.00 10.91           N  \nATOM   3388  CA  ASN A 222      10.606 -20.112  35.906  1.00 11.34           C  \nATOM   3389  C   ASN A 222      11.622 -19.231  35.187  1.00 11.51           C  \nATOM   3390  O   ASN A 222      12.218 -18.344  35.814  1.00 14.73           O  \nATOM   3391  CB  ASN A 222      11.356 -21.226  36.610  1.00 11.04           C  \nATOM   3392  CG  ASN A 222      12.068 -20.729  37.852  1.00 12.33           C  \nATOM   3393  OD1 ASN A 222      11.439 -20.115  38.694  1.00 14.87           O  \nATOM   3394  ND2 ASN A 222      13.314 -21.092  37.983  1.00 15.67           N  \nATOM   3395  H   ASN A 222       9.634 -21.531  34.739  1.00 11.14           H  \nATOM   3396  HA  ASN A 222      10.143 -19.573  36.590  1.00 11.24           H  \nATOM   3397  HB2 ASN A 222      10.722 -21.929  36.861  1.00 11.41           H  \nATOM   3398  HB3 ASN A 222      12.011 -21.612  35.993  1.00 11.43           H  \nATOM   3399 HD21 ASN A 222      13.690 -21.604  37.359  0.00 16.34           H  \nATOM   3400 HD22 ASN A 222      13.777 -20.856  38.692  0.00 16.34           H  \nATOM   3401  N   GLU A 223      11.880 -19.471  33.887  1.00 12.08           N  \nATOM   3402  CA  GLU A 223      12.774 -18.605  33.149  1.00 13.80           C  \nATOM   3403  C   GLU A 223      12.265 -18.428  31.734  1.00 13.78           C  \nATOM   3404  O   GLU A 223      11.502 -19.242  31.211  1.00 13.72           O  \nATOM   3405  CB  GLU A 223      14.221 -19.073  33.261  1.00 20.22           C  \nATOM   3406  CG  GLU A 223      14.584 -20.268  32.466  1.00 26.84           C  \nATOM   3407  CD  GLU A 223      16.109 -20.373  32.263  1.00 36.55           C  \nATOM   3408  OE1 GLU A 223      16.558 -20.702  31.122  1.00 45.92           O  \nATOM   3409  OE2 GLU A 223      16.845 -20.113  33.212  1.00 38.59           O  \nATOM   3410  H   GLU A 223      11.499 -20.187  33.465  1.00 12.32           H  \nATOM   3411  HA  GLU A 223      12.727 -17.719  33.577  1.00 14.47           H  \nATOM   3412  HB2 GLU A 223      14.801 -18.331  32.990  1.00 19.79           H  \nATOM   3413  HB3 GLU A 223      14.407 -19.261  34.205  1.00 19.75           H  \nATOM   3414  HG2 GLU A 223      14.265 -21.065  32.923  1.00 26.96           H  \nATOM   3415  HG3 GLU A 223      14.150 -20.227  31.588  1.00 26.96           H  \nATOM   3416  N   ASP A 224      12.718 -17.345  31.116  1.00 13.70           N  \nATOM   3417  CA  ASP A 224      12.435 -17.004  29.742  1.00 13.32           C  \nATOM   3418  C   ASP A 224      13.488 -17.614  28.839  1.00 14.53           C  \nATOM   3419  O   ASP A 224      14.684 -17.604  29.126  1.00 16.10           O  \nATOM   3420  CB  ASP A 224      12.401 -15.491  29.524  1.00 14.00           C  \nATOM   3421  CG  ASP A 224      11.188 -14.849  30.174  1.00 15.35           C  \nATOM   3422  OD1 ASP A 224      10.077 -15.387  29.983  1.00 16.84           O  \nATOM   3423  OD2 ASP A 224      11.349 -13.858  30.943  1.00 17.02           O  \nATOM   3424  H   ASP A 224      13.250 -16.774  31.592  1.00 13.63           H  \nATOM   3425  HA  ASP A 224      11.556 -17.372  29.507  1.00 13.74           H  \nATOM   3426  HB2 ASP A 224      13.210 -15.089  29.901  1.00 14.14           H  \nATOM   3427  HB3 ASP A 224      12.379 -15.299  28.565  1.00 14.14           H  \nATOM   3428  N   PRO A 225      13.123 -18.222  27.691  1.00 12.92           N  \nATOM   3429  CA  PRO A 225      11.741 -18.410  27.272  1.00 12.10           C  \nATOM   3430  C   PRO A 225      11.023 -19.508  28.064  1.00 10.85           C  \nATOM   3431  O   PRO A 225      11.607 -20.560  28.383  1.00 11.91           O  \nATOM   3432  CB  PRO A 225      11.892 -18.871  25.831  1.00 12.82           C  \nATOM   3433  CG  PRO A 225      13.211 -19.550  25.799  1.00 14.04           C  \nATOM   3434  CD  PRO A 225      14.114 -18.760  26.733  1.00 14.94           C  \nATOM   3435  HA  PRO A 225      11.247 -17.553  27.308  1.00 12.13           H  \nATOM   3436  HB2 PRO A 225      11.174 -19.493  25.583  1.00 12.92           H  \nATOM   3437  HB3 PRO A 225      11.877 -18.110  25.215  1.00 12.92           H  \nATOM   3438  HG2 PRO A 225      13.128 -20.476  26.103  1.00 13.95           H  \nATOM   3439  HG3 PRO A 225      13.574 -19.551  24.890  1.00 13.95           H  \nATOM   3440  HD2 PRO A 225      14.763 -19.337  27.175  1.00 14.23           H  \nATOM   3441  HD3 PRO A 225      14.580 -18.044  26.262  1.00 14.23           H  \nATOM   3442  N   TRP A 226       9.753 -19.299  28.288  1.00 10.51           N  \nATOM   3443  CA  TRP A 226       8.875 -20.373  28.724  1.00  9.78           C  \nATOM   3444  C   TRP A 226       8.954 -21.493  27.706  1.00 10.43           C  \nATOM   3445  O   TRP A 226       9.022 -21.236  26.505  1.00 10.87           O  \nATOM   3446  CB  TRP A 226       7.428 -19.933  28.882  1.00 10.85           C  \nATOM   3447  CG  TRP A 226       7.172 -18.890  29.922  1.00 10.10           C  \nATOM   3448  CD1 TRP A 226       8.096 -18.240  30.683  1.00 11.05           C  \nATOM   3449  CD2 TRP A 226       5.880 -18.387  30.291  1.00 10.41           C  \nATOM   3450  NE1 TRP A 226       7.456 -17.337  31.491  1.00 11.14           N  \nATOM   3451  CE2 TRP A 226       6.113 -17.435  31.317  1.00 10.46           C  \nATOM   3452  CE3 TRP A 226       4.572 -18.698  29.905  1.00 11.75           C  \nATOM   3453  CZ2 TRP A 226       5.059 -16.740  31.890  1.00 11.59           C  \nATOM   3454  CZ3 TRP A 226       3.523 -18.018  30.505  1.00 11.91           C  \nATOM   3455  CH2 TRP A 226       3.791 -17.063  31.478  1.00 12.42           C  \nATOM   3456  H   TRP A 226       9.404 -18.463  28.167  1.00 10.42           H  \nATOM   3457  HA  TRP A 226       9.198 -20.700  29.596  1.00 10.24           H  \nATOM   3458  HB2 TRP A 226       7.120 -19.595  28.017  1.00 10.41           H  \nATOM   3459  HB3 TRP A 226       6.894 -20.726  29.095  1.00 10.41           H  \nATOM   3460  HD1 TRP A 226       9.033 -18.345  30.626  1.00 10.90           H  \nATOM   3461  HE1 TRP A 226       7.855 -16.814  32.068  1.00 11.02           H  \nATOM   3462  HE3 TRP A 226       4.410 -19.342  29.235  1.00 11.42           H  \nATOM   3463  HZ2 TRP A 226       5.207 -16.099  32.565  1.00 11.46           H  \nATOM   3464  HZ3 TRP A 226       2.632 -18.191  30.244  1.00 11.87           H  \nATOM   3465  HH2 TRP A 226       3.066 -16.610  31.872  1.00 11.97           H  \nATOM   3466  N   THR A 227       9.038 -22.725  28.213  1.00  9.76           N  \nATOM   3467  CA  THR A 227       9.224 -23.897  27.376  1.00 10.32           C  \nATOM   3468  C   THR A 227       8.253 -24.989  27.805  1.00  9.65           C  \nATOM   3469  O   THR A 227       7.787 -25.071  28.923  1.00 10.05           O  \nATOM   3470  CB  THR A 227      10.666 -24.381  27.377  1.00 11.28           C  \nATOM   3471  OG1 THR A 227      11.102 -24.597  28.715  1.00 10.94           O  \nATOM   3472  CG2 THR A 227      11.619 -23.447  26.686  1.00 11.68           C  \nATOM   3473  H   THR A 227       8.974 -22.840  29.118  1.00 10.05           H  \nATOM   3474  HA  THR A 227       8.996 -23.639  26.453  1.00 10.26           H  \nATOM   3475  HB  THR A 227      10.690 -25.246  26.903  1.00 11.10           H  \nATOM   3476  HG1 THR A 227      11.077 -23.832  29.202  0.00 10.72           H  \nATOM   3477 HG21 THR A 227      12.518 -23.822  26.711  1.00 11.56           H  \nATOM   3478 HG22 THR A 227      11.616 -22.586  27.136  1.00 11.56           H  \nATOM   3479 HG23 THR A 227      11.345 -23.329  25.759  1.00 11.56           H  \nATOM   3480  N   PHE A 228       7.963 -25.839  26.811  1.00  9.67           N  \nATOM   3481  CA  PHE A 228       7.017 -26.945  26.896  1.00 10.28           C  \nATOM   3482  C   PHE A 228       7.716 -28.277  26.651  1.00 10.54           C  \nATOM   3483  O   PHE A 228       8.656 -28.344  25.875  1.00 12.10           O  \nATOM   3484  CB  PHE A 228       5.925 -26.825  25.839  1.00  9.97           C  \nATOM   3485  CG  PHE A 228       4.991 -25.647  25.978  1.00 10.15           C  \nATOM   3486  CD1 PHE A 228       3.731 -25.792  26.533  1.00 10.85           C  \nATOM   3487  CD2 PHE A 228       5.403 -24.392  25.541  1.00 10.94           C  \nATOM   3488  CE1 PHE A 228       2.859 -24.702  26.644  1.00 11.31           C  \nATOM   3489  CE2 PHE A 228       4.531 -23.320  25.638  1.00 11.16           C  \nATOM   3490  CZ  PHE A 228       3.267 -23.488  26.161  1.00  9.76           C  \nATOM   3491  H   PHE A 228       8.395 -25.719  26.015  1.00  9.81           H  \nATOM   3492  HA  PHE A 228       6.604 -26.954  27.793  1.00 10.15           H  \nATOM   3493  HB2 PHE A 228       6.357 -26.776  24.961  1.00 10.09           H  \nATOM   3494  HB3 PHE A 228       5.392 -27.647  25.858  1.00 10.09           H  \nATOM   3495  HD1 PHE A 228       3.452 -26.644  26.829  1.00 10.77           H  \nATOM   3496  HD2 PHE A 228       6.254 -24.282  25.148  1.00 10.78           H  \nATOM   3497  HE1 PHE A 228       1.995 -24.812  27.009  1.00 10.80           H  \nATOM   3498  HE2 PHE A 228       4.802 -22.469  25.331  1.00 10.75           H  \nATOM   3499  HZ  PHE A 228       2.699 -22.739  26.251  1.00 10.45           H  \nATOM   3500  N   GLY A 229       7.227 -29.303  27.343  1.00 11.04           N  \nATOM   3501  CA  GLY A 229       7.623 -30.668  26.972  1.00 11.49           C  \nATOM   3502  C   GLY A 229       6.937 -31.084  25.683  1.00 11.71           C  \nATOM   3503  O   GLY A 229       6.099 -30.383  25.118  1.00 11.81           O  \nATOM   3504  H   GLY A 229       6.652 -29.170  28.041  1.00 11.02           H  \nATOM   3505  HA2 GLY A 229       8.604 -30.704  26.853  1.00 11.44           H  \nATOM   3506  HA3 GLY A 229       7.371 -31.290  27.699  1.00 11.44           H  \nATOM   3507  N   GLY A 230       7.337 -32.256  25.132  1.00 12.53           N  \nATOM   3508  CA  GLY A 230       6.919 -32.670  23.793  1.00 12.96           C  \nATOM   3509  C   GLY A 230       5.550 -33.326  23.714  1.00 13.08           C  \nATOM   3510  O   GLY A 230       5.100 -33.676  22.617  1.00 14.13           O  \nATOM   3511  H   GLY A 230       7.886 -32.809  25.610  1.00 12.42           H  \nATOM   3512  HA2 GLY A 230       6.926 -31.876  23.202  1.00 12.88           H  \nATOM   3513  HA3 GLY A 230       7.594 -33.302  23.439  1.00 12.88           H  \nATOM   3514  N   GLY A 231       4.898 -33.530  24.861  1.00 13.11           N  \nATOM   3515  CA  GLY A 231       3.571 -34.088  24.879  1.00 12.54           C  \nATOM   3516  C   GLY A 231       3.610 -35.578  25.205  1.00 13.31           C  \nATOM   3517  O   GLY A 231       4.527 -36.284  24.807  1.00 15.23           O  \nATOM   3518  H   GLY A 231       5.298 -33.319  25.655  1.00 12.96           H  \nATOM   3519  HA2 GLY A 231       3.034 -33.613  25.558  1.00 12.85           H  \nATOM   3520  HA3 GLY A 231       3.144 -33.953  23.997  1.00 12.85           H  \nATOM   3521  N   THR A 232       2.525 -36.020  25.849  1.00 13.36           N  \nATOM   3522  CA  THR A 232       2.296 -37.432  26.095  1.00 13.05           C  \nATOM   3523  C   THR A 232       0.875 -37.707  25.645  1.00 12.39           C  \nATOM   3524  O   THR A 232      -0.065 -37.101  26.148  1.00 12.31           O  \nATOM   3525  CB  THR A 232       2.401 -37.773  27.580  1.00 13.96           C  \nATOM   3526  OG1 THR A 232       3.751 -37.585  28.015  1.00 15.38           O  \nATOM   3527  CG2 THR A 232       1.895 -39.185  27.875  1.00 15.40           C  \nATOM   3528  H   THR A 232       1.905 -35.418  26.148  1.00 13.27           H  \nATOM   3529  HA  THR A 232       2.932 -37.971  25.574  1.00 13.16           H  \nATOM   3530  HB  THR A 232       1.830 -37.136  28.071  1.00 14.30           H  \nATOM   3531  HG1 THR A 232       4.347 -38.118  27.540  0.00 15.19           H  \nATOM   3532 HG21 THR A 232       2.336 -39.527  28.673  1.00 14.94           H  \nATOM   3533 HG22 THR A 232       2.094 -39.769  27.121  1.00 14.93           H  \nATOM   3534 HG23 THR A 232       0.933 -39.163  28.023  1.00 14.94           H  \nATOM   3535  N   LYS A 233       0.713 -38.712  24.749  1.00 13.51           N  \nATOM   3536  CA  LYS A 233      -0.595 -39.103  24.262  1.00 15.09           C  \nATOM   3537  C   LYS A 233      -1.221 -40.156  25.175  1.00 14.06           C  \nATOM   3538  O   LYS A 233      -0.524 -41.090  25.526  1.00 14.75           O  \nATOM   3539  CB  LYS A 233      -0.426 -39.624  22.823  1.00 17.50           C  \nATOM   3540  CG  LYS A 233      -1.672 -40.155  22.163  1.00 22.80           C  \nATOM   3541  CD  LYS A 233      -1.333 -40.848  20.856  1.00 28.25           C  \nATOM   3542  CE  LYS A 233      -2.564 -41.157  20.037  1.00 34.63           C  \nATOM   3543  NZ  LYS A 233      -2.205 -42.074  18.939  1.00 39.90           N  \nATOM   3544  H   LYS A 233       1.448 -39.161  24.442  1.00 13.60           H  \nATOM   3545  HA  LYS A 233      -1.175 -38.308  24.245  1.00 14.97           H  \nATOM   3546  HB2 LYS A 233      -0.073 -38.894  22.273  1.00 17.99           H  \nATOM   3547  HB3 LYS A 233       0.245 -40.338  22.832  1.00 17.99           H  \nATOM   3548  HG2 LYS A 233      -2.115 -40.790  22.764  1.00 22.51           H  \nATOM   3549  HG3 LYS A 233      -2.289 -39.412  21.990  1.00 22.52           H  \nATOM   3550  HD2 LYS A 233      -0.737 -40.274  20.331  1.00 28.15           H  \nATOM   3551  HD3 LYS A 233      -0.859 -41.685  21.048  1.00 28.19           H  \nATOM   3552  HE2 LYS A 233      -3.243 -41.575  20.600  1.00 34.12           H  \nATOM   3553  HE3 LYS A 233      -2.934 -40.334  19.665  1.00 34.12           H  \nATOM   3554  HZ1 LYS A 233      -1.913 -42.866  19.283  1.00 38.14           H  \nATOM   3555  HZ2 LYS A 233      -1.543 -41.705  18.433  1.00 38.18           H  \nATOM   3556  HZ3 LYS A 233      -2.935 -42.224  18.416  1.00 38.18           H  \nATOM   3557  N   LEU A 234      -2.423 -39.907  25.632  1.00 12.57           N  \nATOM   3558  CA  LEU A 234      -3.163 -40.860  26.455  1.00 14.55           C  \nATOM   3559  C   LEU A 234      -4.276 -41.419  25.578  1.00 16.41           C  \nATOM   3560  O   LEU A 234      -5.107 -40.661  25.060  1.00 15.89           O  \nATOM   3561  CB  LEU A 234      -3.730 -40.184  27.701  1.00 14.60           C  \nATOM   3562  CG  LEU A 234      -4.569 -41.116  28.581  1.00 15.95           C  \nATOM   3563  CD1 LEU A 234      -3.740 -42.285  29.083  1.00 17.56           C  \nATOM   3564  CD2 LEU A 234      -5.137 -40.316  29.738  1.00 17.63           C  \nATOM   3565  H   LEU A 234      -2.818 -39.107  25.432  1.00 13.37           H  \nATOM   3566  HA  LEU A 234      -2.560 -41.589  26.718  1.00 14.53           H  \nATOM   3567  HB2 LEU A 234      -2.988 -39.831  28.232  1.00 14.89           H  \nATOM   3568  HB3 LEU A 234      -4.285 -39.428  27.422  1.00 14.89           H  \nATOM   3569  HG  LEU A 234      -5.321 -41.469  28.045  1.00 16.28           H  \nATOM   3570 HD11 LEU A 234      -4.190 -42.701  29.839  1.00 17.05           H  \nATOM   3571 HD12 LEU A 234      -2.865 -41.967  29.362  1.00 17.06           H  \nATOM   3572 HD13 LEU A 234      -3.635 -42.939  28.370  1.00 17.04           H  \nATOM   3573 HD21 LEU A 234      -5.627 -39.550  29.394  1.00 17.09           H  \nATOM   3574 HD22 LEU A 234      -4.410 -40.007  30.307  1.00 17.10           H  \nATOM   3575 HD23 LEU A 234      -5.738 -40.878  30.257  1.00 17.10           H  \nATOM   3576  N   GLU A 235      -4.354 -42.771  25.518  1.00 14.71           N  \nATOM   3577  CA  GLU A 235      -5.461 -43.462  24.849  1.00 16.34           C  \nATOM   3578  C   GLU A 235      -6.050 -44.507  25.791  1.00 16.00           C  \nATOM   3579  O   GLU A 235      -5.421 -44.877  26.793  1.00 15.63           O  \nATOM   3580  CB  GLU A 235      -5.046 -44.147  23.548  1.00 22.87           C  \nATOM   3581  CG  GLU A 235      -3.761 -44.931  23.602  1.00 28.77           C  \nATOM   3582  CD  GLU A 235      -3.088 -45.142  22.247  1.00 39.01           C  \nATOM   3583  OE1 GLU A 235      -2.053 -45.850  22.220  1.00 46.36           O  \nATOM   3584  OE2 GLU A 235      -3.579 -44.575  21.227  1.00 43.11           O  \nATOM   3585  H   GLU A 235      -3.703 -43.272  25.918  1.00 15.52           H  \nATOM   3586  HA  GLU A 235      -6.161 -42.800  24.644  1.00 17.02           H  \nATOM   3587  HB2 GLU A 235      -5.767 -44.751  23.273  1.00 22.27           H  \nATOM   3588  HB3 GLU A 235      -4.962 -43.457  22.859  1.00 22.35           H  \nATOM   3589  HG2 GLU A 235      -3.128 -44.471  24.189  1.00 29.14           H  \nATOM   3590  HG3 GLU A 235      -3.944 -45.809  23.995  1.00 29.16           H  \nATOM   3591  N   ILE A 236      -7.227 -45.021  25.399  1.00 17.84           N  \nATOM   3592  CA  ILE A 236      -7.857 -46.149  26.079  1.00 17.86           C  \nATOM   3593  C   ILE A 236      -7.426 -47.458  25.427  1.00 18.22           C  \nATOM   3594  O   ILE A 236      -7.459 -47.584  24.199  1.00 22.41           O  \nATOM   3595  CB  ILE A 236      -9.389 -46.008  26.138  1.00 18.36           C  \nATOM   3596  CG1 ILE A 236      -9.875 -44.691  26.734  1.00 19.68           C  \nATOM   3597  CG2 ILE A 236      -9.963 -47.178  26.932  1.00 20.13           C  \nATOM   3598  CD1 ILE A 236      -9.232 -44.320  28.071  1.00 22.12           C  \nATOM   3599  H   ILE A 236      -7.655 -44.651  24.681  1.00 17.41           H  \nATOM   3600  HA  ILE A 236      -7.533 -46.154  26.994  1.00 18.03           H  \nATOM   3601  HB  ILE A 236      -9.734 -46.067  25.213  1.00 18.83           H  \nATOM   3602 HG12 ILE A 236      -9.695 -43.974  26.092  1.00 19.90           H  \nATOM   3603 HG13 ILE A 236     -10.847 -44.742  26.858  1.00 19.90           H  \nATOM   3604 HG21 ILE A 236      -9.384 -47.376  27.688  1.00 19.56           H  \nATOM   3605 HG22 ILE A 236     -10.026 -47.962  26.360  1.00 19.56           H  \nATOM   3606 HG23 ILE A 236     -10.851 -46.948  27.259  1.00 19.56           H  \nATOM   3607 HD11 ILE A 236      -8.332 -43.984  27.918  1.00 21.33           H  \nATOM   3608 HD12 ILE A 236      -9.191 -45.105  28.644  1.00 21.33           H  \nATOM   3609 HD13 ILE A 236      -9.763 -43.631  28.507  1.00 21.33           H  \nATOM   3610  N   LYS A 237      -6.971 -48.408  26.244  1.00 20.38           N  \nATOM   3611  CA  LYS A 237      -6.740 -49.793  25.822  1.00 26.56           C  \nATOM   3612  C   LYS A 237      -8.070 -50.414  25.388  1.00 26.38           C  \nATOM   3613  O   LYS A 237      -8.111 -50.804  24.200  1.00 42.19           O  \nATOM   3614  CB  LYS A 237      -6.245 -50.683  26.966  1.00 29.68           C  \nATOM   3615  CG  LYS A 237      -4.763 -50.659  27.294  1.00 35.16           C  \nATOM   3616  CD  LYS A 237      -4.404 -51.621  28.422  1.00 38.25           C  \nATOM   3617  CE  LYS A 237      -3.067 -51.329  29.066  1.00 42.18           C  \nATOM   3618  NZ  LYS A 237      -1.980 -51.319  28.058  1.00 47.26           N  \nATOM   3619  H   LYS A 237      -6.790 -48.186  27.111  1.00 21.14           H  \nATOM   3620  HA  LYS A 237      -6.099 -49.812  25.074  1.00 25.78           H  \nATOM   3621  HB2 LYS A 237      -6.736 -50.431  27.776  1.00 30.06           H  \nATOM   3622  HB3 LYS A 237      -6.490 -51.608  26.756  1.00 30.08           H  \nATOM   3623  HG2 LYS A 237      -4.254 -50.900  26.490  1.00 34.45           H  \nATOM   3624  HG3 LYS A 237      -4.509 -49.749  27.553  1.00 34.41           H  \nATOM   3625  HD2 LYS A 237      -5.102 -51.577  29.111  1.00 38.35           H  \nATOM   3626  HD3 LYS A 237      -4.391 -52.536  28.068  1.00 38.37           H  \nATOM   3627  HE2 LYS A 237      -3.099 -50.462  29.511  1.00 42.27           H  \nATOM   3628  HE3 LYS A 237      -2.871 -52.008  29.738  1.00 42.32           H  \nATOM   3629  HZ1 LYS A 237      -1.936 -50.505  27.660  1.00 45.61           H  \nATOM   3630  HZ2 LYS A 237      -2.137 -51.951  27.427  1.00 45.61           H  \nATOM   3631  HZ3 LYS A 237      -1.186 -51.494  28.461  1.00 45.62           H  \nTER    3632      LYS A 237                                                       \nEND   \n",
                                                    12,
                                                    22
                                                ]
                                                ]
                                            }
                                            }
                                          },
               "proteinmpnn": {"data": {
                                        "inputs": [
                                            "ATOM      1  N   GLY A   1      -0.558  27.179  41.317  1.00 21.04           N  \nATOM      2  CA  GLY A   1      -1.509  26.374  40.524  1.00 16.68           C  \nATOM      3  C   GLY A   1      -1.526  24.960  41.092  1.00 15.87           C  \nATOM      4  O   GLY A   1      -0.500  24.432  41.547  1.00 16.66           O  \nATOM      5  N   GLY A   2      -2.668  24.299  40.975  1.00 12.31           N  \nATOM      6  CA  GLY A   2      -2.762  22.908  41.342  1.00 13.13           C  \nATOM      7  C   GLY A   2      -2.062  22.087  40.284  1.00 11.44           C  \nATOM      8  O   GLY A   2      -2.220  22.324  39.071  1.00 14.43           O  \nATOM      9  N   GLY A   3      -1.305  21.126  40.763  1.00 11.44           N  \nATOM     10  CA  GLY A   3      -0.644  20.160  39.867  1.00 14.12           C  \nATOM     11  C   GLY A   3      -0.873  18.747  40.390  1.00 12.08           C  \nATOM     12  O   GLY A   3      -0.726  18.496  41.568  1.00 12.29           O  \nATOM     13  N   GLY A   4      -1.204  17.847  39.460  1.00 10.44           N  \nATOM     14  CA  GLY A   4      -1.270  16.445  39.756  1.00  9.06           C  \nATOM     15  C   GLY A   4      -0.364  15.767  38.754  1.00  8.23           C  \nATOM     16  O   GLY A   4      -0.622  15.807  37.560  1.00  9.15           O  \nATOM     17  N   GLY A   5       0.681  15.126  39.283  1.00  7.33           N  \nATOM     18  CA  GLY A   5       1.737  14.593  38.429  1.00  7.82           C  \nATOM     19  C   GLY A   5       1.847  13.088  38.649  1.00  6.85           C  \nATOM     20  O   GLY A   5       2.246  12.621  39.718  1.00  7.36           O  \nATOM     21  N   GLY A   6       1.410  12.336  37.628  1.00  6.56           N  \nATOM     22  CA  GLY A   6       1.300  10.882  37.749  1.00  6.53           C  \nATOM     23  C   GLY A   6       2.559  10.183  37.289  1.00  6.46           C  \nATOM     24  O   GLY A   6       3.386  10.693  36.534  1.00  7.91           O  \nATOM     25  N   GLY A   7       2.717   8.971  37.780  1.00  6.17           N  \nATOM     26  CA  GLY A   7       3.846   8.108  37.435  1.00  6.74           C  \nATOM     27  C   GLY A   7       3.707   7.548  36.037  1.00  7.15           C  \nATOM     28  O   GLY A   7       2.673   7.645  35.379  1.00  7.47           O  \nATOM     29  N   GLY A   8       4.830   7.032  35.551  1.00  6.17           N  \nATOM     30  CA  GLY A   8       4.961   6.687  34.135  1.00  7.13           C  \nATOM     31  C   GLY A   8       4.239   5.436  33.661  1.00  6.52           C  \nATOM     32  O   GLY A   8       3.747   4.624  34.441  1.00  7.50           O  \nATOM     33  N   GLY A   9       4.231   5.309  32.354  1.00  7.11           N  \nATOM     34  CA  GLY A   9       3.583   4.195  31.729  1.00  7.44           C  \nATOM     35  C   GLY A   9       4.155   2.871  32.208  1.00  7.70           C  \nATOM     36  O   GLY A   9       5.365   2.701  32.437  1.00  9.91           O  \nATOM     37  N   GLY A  10       3.272   1.863  32.262  1.00  8.58           N  \nATOM     38  CA  GLY A  10       3.610   0.535  32.729  1.00 10.18           C  \nATOM     39  C   GLY A  10       3.244  -0.472  31.663  1.00  9.73           C  \nATOM     40  O   GLY A  10       2.166  -0.439  31.111  1.00 10.59           O  \nATOM     41  N   GLY A  11       4.130  -1.405  31.408  1.00 10.68           N  \nATOM     42  CA  GLY A  11       3.893  -2.535  30.520  1.00  9.86           C  \nATOM     43  C   GLY A  11       4.019  -3.762  31.399  1.00 11.70           C  \nATOM     44  O   GLY A  11       5.102  -4.026  31.944  1.00 14.32           O  \nATOM     45  N   GLY A  12       2.911  -4.451  31.615  1.00  9.82           N  \nATOM     46  CA  GLY A  12       2.820  -5.495  32.610  1.00 10.74           C  \nATOM     47  C   GLY A  12       2.163  -6.739  32.063  1.00 10.47           C  \nATOM     48  O   GLY A  12       1.542  -6.725  31.003  1.00 10.71           O  \nATOM     49  N   GLY A  13       2.340  -7.857  32.768  1.00 11.42           N  \nATOM     50  CA  GLY A  13       1.801  -9.150  32.283  1.00 10.62           C  \nATOM     51  C   GLY A  13       0.551  -9.552  33.054  1.00  8.72           C  \nATOM     52  O   GLY A  13       0.361  -9.138  34.191  1.00  9.35           O  \nATOM     53  N   GLY A  14      -0.300 -10.357  32.441  1.00  8.87           N  \nATOM     54  CA  GLY A  14      -1.487 -10.791  33.179  1.00  8.59           C  \nATOM     55  C   GLY A  14      -1.174 -11.505  34.488  1.00  8.25           C  \nATOM     56  O   GLY A  14      -0.210 -12.267  34.597  1.00  9.73           O  \nATOM     57  N   GLY A  15      -2.016 -11.279  35.493  1.00  7.52           N  \nATOM     58  CA  GLY A  15      -1.906 -12.019  36.721  1.00  8.86           C  \nATOM     59  C   GLY A  15      -1.057 -11.339  37.753  1.00  9.34           C  \nATOM     60  O   GLY A  15      -0.874 -11.867  38.821  1.00 16.65           O  \nATOM     61  N   GLY A  16      -0.524 -10.189  37.407  1.00  8.32           N  \nATOM     62  CA  GLY A  16       0.338  -9.440  38.278  1.00  7.67           C  \nATOM     63  C   GLY A  16      -0.430  -8.275  38.898  1.00  7.64           C  \nATOM     64  O   GLY A  16      -1.605  -8.077  38.615  1.00  8.06           O  \nATOM     65  N   GLY A  17       0.248  -7.519  39.762  1.00  7.77           N  \nATOM     66  CA  GLY A  17      -0.289  -6.270  40.299  1.00  8.58           C  \nATOM     67  C   GLY A  17       0.555  -5.108  39.832  1.00  8.49           C  \nATOM     68  O   GLY A  17       1.741  -5.291  39.525  1.00  9.51           O  \nATOM     69  N   GLY A  18      -0.032  -3.920  39.798  1.00  7.62           N  \nATOM     70  CA  GLY A  18       0.703  -2.709  39.577  1.00  8.00           C  \nATOM     71  C   GLY A  18       0.245  -1.619  40.535  1.00  7.37           C  \nATOM     72  O   GLY A  18      -0.929  -1.583  40.902  1.00  8.42           O  \nATOM     73  N   GLY A  19       1.163  -0.768  40.944  1.00  7.72           N  \nATOM     74  CA  GLY A  19       0.840   0.374  41.790  1.00  8.10           C  \nATOM     75  C   GLY A  19       1.362   1.652  41.183  1.00  7.95           C  \nATOM     76  O   GLY A  19       2.518   1.777  40.816  1.00 12.25           O  \nATOM     77  N   GLY A  20       0.473   2.610  40.997  1.00  7.37           N  \nATOM     78  CA  GLY A  20       0.757   3.880  40.307  1.00  7.26           C  \nATOM     79  C   GLY A  20       0.543   5.038  41.294  1.00  6.69           C  \nATOM     80  O   GLY A  20      -0.187   4.872  42.263  1.00  7.45           O  \nATOM     81  N   GLY A  21       1.165   6.158  41.008  1.00  6.98           N  \nATOM     82  C   GLY A  21       0.679   8.568  41.266  1.00  7.24           C  \nATOM     83  O   GLY A  21       0.748   8.756  40.032  1.00  7.54           O  \nATOM     84  N   GLY A  22       0.202   9.470  42.125  1.00  7.84           N  \nATOM     85  CA  GLY A  22      -0.345  10.750  41.739  1.00  7.34           C  \nATOM     86  C   GLY A  22       0.148  11.769  42.773  1.00  7.78           C  \nATOM     87  O   GLY A  22      -0.403  11.830  43.859  1.00  8.49           O  \nATOM     88  N   GLY A  23       1.208  12.499  42.409  1.00  7.86           N  \nATOM     89  CA  GLY A  23       1.825  13.462  43.323  1.00  8.19           C  \nATOM     90  C   GLY A  23       1.086  14.800  43.173  1.00 10.18           C  \nATOM     91  O   GLY A  23       0.983  15.368  42.077  1.00 10.35           O  \nATOM     92  N   GLY A  24       0.581  15.291  44.304  1.00  9.35           N  \nATOM     93  CA  GLY A  24      -0.184  16.521  44.346  1.00  9.63           C  \nATOM     94  C   GLY A  24       0.669  17.690  44.830  1.00 10.71           C  \nATOM     95  O   GLY A  24       1.512  17.555  45.732  1.00 14.20           O  \nATOM     96  N   GLY A  25       0.394  18.845  44.260  1.00 10.05           N  \nATOM     97  CA  GLY A  25       1.027  20.093  44.732  1.00 11.09           C  \nATOM     98  C   GLY A  25       0.104  21.283  44.451  1.00 11.45           C  \nATOM     99  O   GLY A  25      -0.815  21.234  43.611  1.00 11.10           O  \nATOM    100  N   GLY A  26       0.323  22.371  45.185  1.00 11.90           N  \nATOM    101  CA  GLY A  26      -0.324  23.644  44.887  1.00 12.71           C  \nATOM    102  C   GLY A  26      -1.635  23.883  45.619  1.00 11.35           C  \nATOM    103  O   GLY A  26      -2.340  24.830  45.307  1.00 13.59           O  \nATOM    104  N   GLY A  27      -1.933  23.016  46.574  1.00 11.17           N  \nATOM    105  CA  GLY A  27      -3.122  23.098  47.416  1.00 11.10           C  \nATOM    106  C   GLY A  27      -2.929  22.205  48.628  1.00 11.32           C  \nATOM    107  O   GLY A  27      -1.988  21.438  48.685  1.00 14.21           O  \nATOM    108  N   GLY A  28      -3.851  22.282  49.608  1.00 12.51           N  \nATOM    109  CA  GLY A  28      -3.762  21.480  50.816  1.00 12.80           C  \nATOM    110  C   GLY A  28      -4.247  20.057  50.549  1.00 12.05           C  \nATOM    111  O   GLY A  28      -5.463  19.774  50.394  1.00 11.84           O  \nATOM    112  N   GLY A  29      -3.277  19.137  50.480  1.00 12.66           N  \nATOM    113  CA  GLY A  29      -3.539  17.738  50.083  1.00 11.99           C  \nATOM    114  C   GLY A  29      -4.614  17.069  50.917  1.00 12.28           C  \nATOM    115  O   GLY A  29      -5.435  16.301  50.398  1.00 12.50           O  \nATOM    116  N   GLY A  30      -4.635  17.354  52.230  1.00 13.00           N  \nATOM    117  CA  GLY A  30      -5.524  16.648  53.122  1.00 14.66           C  \nATOM    118  C   GLY A  30      -6.921  17.244  53.145  1.00 13.16           C  \nATOM    119  O   GLY A  30      -7.774  16.729  53.883  1.00 17.26           O  \nATOM    120  N   GLY A  31      -7.188  18.263  52.327  1.00 11.96           N  \nATOM    121  CA  GLY A  31      -8.493  18.936  52.356  1.00 11.51           C  \nATOM    122  C   GLY A  31      -9.510  18.406  51.350  1.00 13.32           C  \nATOM    123  O   GLY A  31     -10.704  18.505  51.599  1.00 14.95           O  \nATOM    124  N   GLY A  32      -9.047  17.831  50.233  1.00 11.12           N  \nATOM    125  CA  GLY A  32      -9.920  17.481  49.095  1.00  9.84           C  \nATOM    126  C   GLY A  32      -9.854  16.003  48.731  1.00  8.77           C  \nATOM    127  O   GLY A  32      -8.768  15.379  48.722  1.00  9.77           O  \nATOM    128  N   GLY A  33     -11.003  15.453  48.340  1.00  8.69           N  \nATOM    129  CA  GLY A  33     -11.046  14.150  47.730  1.00  8.09           C  \nATOM    130  C   GLY A  33     -10.263  14.091  46.409  1.00  7.60           C  \nATOM    131  O   GLY A  33     -10.240  15.059  45.660  1.00  7.23           O  \nATOM    132  N   GLY A  34      -9.630  12.939  46.201  1.00  6.91           N  \nATOM    133  CA  GLY A  34      -9.013  12.594  44.923  1.00  6.86           C  \nATOM    134  C   GLY A  34      -9.787  11.397  44.373  1.00  6.43           C  \nATOM    135  O   GLY A  34      -9.946  10.372  45.024  1.00  7.25           O  \nATOM    136  N   GLY A  35     -10.273  11.571  43.132  1.00  6.11           N  \nATOM    137  CA  GLY A  35     -10.940  10.501  42.383  1.00  6.16           C  \nATOM    138  C   GLY A  35     -10.035   9.896  41.342  1.00  5.69           C  \nATOM    139  O   GLY A  35      -9.101  10.535  40.863  1.00  6.43           O  \nATOM    140  N   GLY A  36     -10.392   8.664  40.966  1.00  5.62           N  \nATOM    141  CA  GLY A  36      -9.637   7.938  39.958  1.00  5.26           C  \nATOM    142  C   GLY A  36     -10.568   7.472  38.878  1.00  5.43           C  \nATOM    143  O   GLY A  36     -11.655   6.951  39.159  1.00  5.56           O  \nATOM    144  N   GLY A  37     -10.114   7.647  37.620  1.00  5.50           N  \nATOM    145  CA  GLY A  37     -10.934   7.454  36.417  1.00  5.80           C  \nATOM    146  C   GLY A  37     -10.173   6.586  35.428  1.00  5.37           C  \nATOM    147  O   GLY A  37      -8.987   6.841  35.166  1.00  6.75           O  \nATOM    148  N   GLY A  38     -10.848   5.610  34.832  1.00  5.83           N  \nATOM    149  CA  GLY A  38     -10.303   4.691  33.848  1.00  5.62           C  \nATOM    150  C   GLY A  38     -10.775   5.049  32.439  1.00  6.24           C  \nATOM    151  O   GLY A  38     -11.991   5.164  32.215  1.00  6.50           O  \nATOM    152  N   GLY A  39      -9.844   5.176  31.497  1.00  6.01           N  \nATOM    153  CA  GLY A  39     -10.115   5.451  30.091  1.00  7.49           C  \nATOM    154  C   GLY A  39      -9.633   4.275  29.284  1.00  8.01           C  \nATOM    155  O   GLY A  39      -8.424   4.105  29.045  1.00  8.42           O  \nATOM    156  N   GLY A  40     -10.540   3.430  28.871  1.00 11.02           N  \nATOM    157  CA  GLY A  40     -10.206   2.222  28.172  1.00 13.24           C  \nATOM    158  C   GLY A  40     -10.806   2.399  26.777  1.00 14.15           C  \nATOM    159  O   GLY A  40     -11.991   2.807  26.585  1.00 13.28           O  \nATOM    160  N   GLY A  41     -10.008   2.093  25.752  1.00 22.54           N  \nATOM    161  CA  GLY A  41     -10.565   2.275  24.424  1.00 19.57           C  \nATOM    162  C   GLY A  41     -11.891   1.522  24.179  1.00 21.77           C  \nATOM    163  O   GLY A  41     -12.054   0.356  24.544  1.00 22.02           O  \nATOM    164  N   GLY A  42     -12.827   2.236  23.568  1.00 19.49           N  \nATOM    165  CA  GLY A  42     -14.215   1.774  23.378  1.00 21.34           C  \nATOM    166  C   GLY A  42     -15.138   1.792  24.585  1.00 18.54           C  \nATOM    167  O   GLY A  42     -16.305   1.406  24.499  1.00 22.42           O  \nATOM    168  N   GLY A  43     -14.610   2.224  25.740  1.00 14.70           N  \nATOM    169  CA  GLY A  43     -15.362   2.199  26.987  1.00 12.38           C  \nATOM    170  C   GLY A  43     -15.323   3.566  27.682  1.00 10.29           C  \nATOM    171  O   GLY A  43     -15.514   3.652  28.872  1.00 10.93           O  \nATOM    172  N   GLY A  44     -15.094   4.626  26.914  1.00 10.26           N  \nATOM    173  CA  GLY A  44     -15.255   5.985  27.425  1.00  9.57           C  \nATOM    174  C   GLY A  44     -14.448   6.258  28.679  1.00  8.89           C  \nATOM    175  O   GLY A  44     -13.255   5.950  28.721  1.00  8.68           O  \nATOM    176  N   GLY A  45     -15.135   6.772  29.690  1.00  7.74           N  \nATOM    177  CA  GLY A  45     -14.569   7.037  31.020  1.00  6.53           C  \nATOM    178  C   GLY A  45     -15.386   6.300  32.060  1.00  7.27           C  \nATOM    179  O   GLY A  45     -16.621   6.273  31.994  1.00  8.98           O  \nATOM    180  N   GLY A  46     -14.726   5.751  33.051  1.00  7.28           N  \nATOM    181  CA  GLY A  46     -15.347   4.931  34.109  1.00  7.01           C  \nATOM    182  C   GLY A  46     -14.751   5.334  35.436  1.00  7.02           C  \nATOM    183  O   GLY A  46     -13.537   5.485  35.531  1.00  8.33           O  \nATOM    184  N   GLY A  47     -15.601   5.599  36.430  1.00  6.32           N  \nATOM    185  CA  GLY A  47     -15.129   6.105  37.724  1.00  5.22           C  \nATOM    186  C   GLY A  47     -14.829   4.927  38.642  1.00  5.65           C  \nATOM    187  O   GLY A  47     -15.672   4.033  38.788  1.00  6.67           O  \nATOM    188  N   GLY A  48     -13.626   4.885  39.236  1.00  4.86           N  \nATOM    189  CA  GLY A  48     -13.177   3.791  40.050  1.00  5.64           C  \nATOM    190  C   GLY A  48     -13.520   3.980  41.522  1.00  5.18           C  \nATOM    191  O   GLY A  48     -14.001   3.049  42.189  1.00  6.46           O  \nATOM    192  N   GLY A  49     -13.226   5.154  42.047  1.00  5.76           N  \nATOM    193  CA  GLY A  49     -13.377   5.415  43.468  1.00  5.64           C  \nATOM    194  C   GLY A  49     -12.731   6.708  43.843  1.00  5.41           C  \nATOM    195  O   GLY A  49     -12.106   7.409  43.009  1.00  6.75           O  \nATOM    196  N   GLY A  50     -12.855   7.063  45.134  1.00  5.72           N  \nATOM    197  CA  GLY A  50     -12.191   8.257  45.630  1.00  6.17           C  \nATOM    198  C   GLY A  50     -11.667   8.054  47.035  1.00  6.96           C  \nATOM    199  O   GLY A  50     -12.040   7.103  47.714  1.00  8.10           O  \nATOM    200  N   GLY A  51     -10.764   8.948  47.409  1.00  6.63           N  \nATOM    201  CA  GLY A  51     -10.118   8.876  48.736  1.00  8.26           C  \nATOM    202  C   GLY A  51      -9.929  10.286  49.262  1.00  8.89           C  \nATOM    203  O   GLY A  51      -9.590  11.207  48.550  1.00  8.33           O  \nATOM    204  N   GLY A  52     -10.123  10.376  50.590  1.00  8.67           N  \nATOM    205  CA  GLY A  52      -9.894  11.632  51.276  1.00  9.29           C  \nATOM    206  C   GLY A  52      -8.561  11.497  52.025  1.00  8.83           C  \nATOM    207  O   GLY A  52      -8.459  10.666  52.952  1.00 10.20           O  \nATOM    208  N   GLY A  53      -7.543  12.273  51.626  1.00  9.72           N  \nATOM    209  CA  GLY A  53      -6.242  11.981  52.281  1.00 11.70           C  \nATOM    210  C   GLY A  53      -6.162  12.438  53.724  1.00 12.96           C  \nATOM    211  O   GLY A  53      -5.222  12.049  54.422  1.00 15.57           O  \nATOM    212  N   GLY A  54      -7.114  13.249  54.185  1.00 11.70           N  \nATOM    213  CA  GLY A  54      -7.184  13.620  55.603  1.00 13.27           C  \nATOM    214  C   GLY A  54      -7.631  12.465  56.480  1.00 14.91           C  \nATOM    215  O   GLY A  54      -6.964  12.148  57.465  1.00 18.64           O  \nATOM    216  N   GLY A  55      -8.754  11.850  56.158  1.00 12.54           N  \nATOM    217  CA  GLY A  55      -9.329  10.771  56.926  1.00 14.22           C  \nATOM    218  C   GLY A  55      -8.838   9.424  56.517  1.00 14.27           C  \nATOM    219  O   GLY A  55      -9.065   8.440  57.214  1.00 16.86           O  \nATOM    220  N   GLY A  56      -8.281   9.345  55.299  1.00 12.53           N  \nATOM    221  CA  GLY A  56      -7.917   8.077  54.666  1.00 14.17           C  \nATOM    222  C   GLY A  56      -9.094   7.225  54.245  1.00 14.10           C  \nATOM    223  O   GLY A  56      -8.932   6.081  53.865  1.00 18.29           O  \nATOM    224  N   GLY A  57     -10.292   7.777  54.315  1.00 12.96           N  \nATOM    225  CA  GLY A  57     -11.491   7.008  53.969  1.00 12.85           C  \nATOM    226  C   GLY A  57     -11.584   6.951  52.466  1.00 10.67           C  \nATOM    227  O   GLY A  57     -11.341   7.948  51.798  1.00 12.47           O  \nATOM    228  N   GLY A  58     -11.903   5.775  51.967  1.00 11.20           N  \nATOM    229  CA AGLY A  58     -12.080   5.569  50.533  0.50 11.43           C  \nATOM    230  CA BGLY A  58     -12.098   5.619  50.541  0.50 10.71           C  \nATOM    231  C   GLY A  58     -13.468   5.050  50.233  1.00 11.52           C  \nATOM    232  O   GLY A  58     -14.070   4.305  51.037  1.00 14.19           O  \nATOM    233  N   GLY A  59     -13.955   5.422  49.051  1.00  8.42           N  \nATOM    234  CA  GLY A  59     -15.202   4.886  48.519  1.00  9.54           C  \nATOM    235  C   GLY A  59     -14.957   4.287  47.142  1.00  8.37           C  \nATOM    236  O   GLY A  59     -14.300   4.946  46.329  1.00 10.65           O  \nATOM    237  N   GLY A  60     -15.548   3.137  46.826  1.00  8.31           N  \nATOM    238  CA  GLY A  60     -15.259   2.464  45.575  1.00  8.39           C  \nATOM    239  C   GLY A  60     -16.557   2.251  44.799  1.00  7.69           C  \nATOM    240  O   GLY A  60     -17.604   1.945  45.392  1.00  8.74           O  \nATOM    241  N   GLY A  61     -16.479   2.371  43.483  1.00  6.40           N  \nATOM    242  CA  GLY A  61     -17.435   1.727  42.601  1.00  7.44           C  \nATOM    243  C   GLY A  61     -17.405   0.247  42.905  1.00  7.70           C  \nATOM    244  O   GLY A  61     -16.325  -0.374  42.950  1.00  7.64           O  \nATOM    245  N   GLY A  62     -18.556  -0.345  43.143  1.00  7.42           N  \nATOM    246  CA  GLY A  62     -18.623  -1.769  43.434  1.00  9.48           C  \nATOM    247  C   GLY A  62     -17.922  -2.601  42.411  1.00  9.04           C  \nATOM    248  O   GLY A  62     -17.365  -3.674  42.740  1.00 10.36           O  \nATOM    249  N   GLY A  63     -17.939  -2.142  41.175  1.00  8.51           N  \nATOM    250  CA  GLY A  63     -17.263  -2.873  40.080  1.00  9.91           C  \nATOM    251  C   GLY A  63     -15.757  -3.046  40.291  1.00  9.77           C  \nATOM    252  O   GLY A  63     -15.146  -3.923  39.674  1.00 13.21           O  \nATOM    253  N   GLY A  64     -15.188  -2.245  41.176  1.00  7.61           N  \nATOM    254  CA  GLY A  64     -13.740  -2.276  41.462  1.00  7.18           C  \nATOM    255  C   GLY A  64     -13.384  -2.743  42.855  1.00  7.66           C  \nATOM    256  O   GLY A  64     -12.187  -2.759  43.204  1.00  7.83           O  \nATOM    257  N   GLY A  65     -14.383  -3.183  43.627  1.00  7.92           N  \nATOM    258  CA  GLY A  65     -14.102  -3.699  44.983  1.00  7.94           C  \nATOM    259  C   GLY A  65     -13.513  -5.079  44.872  1.00  8.35           C  \nATOM    260  O   GLY A  65     -14.188  -6.026  44.470  1.00  9.40           O  \nATOM    261  N   GLY A  66     -12.226  -5.163  45.189  1.00  8.61           N  \nATOM    262  CA  GLY A  66     -11.407  -6.335  44.929  1.00  9.83           C  \nATOM    263  C   GLY A  66     -10.475  -6.264  43.743  1.00  8.83           C  \nATOM    264  O   GLY A  66      -9.729  -7.227  43.504  1.00  9.77           O  \nATOM    265  N   GLY A  67     -10.485  -5.151  43.022  1.00  8.77           N  \nATOM    266  CA  GLY A  67      -9.603  -4.875  41.894  1.00  8.36           C  \nATOM    267  C   GLY A  67      -8.657  -3.750  42.249  1.00  7.25           C  \nATOM    268  O   GLY A  67      -7.455  -3.872  42.012  1.00  8.62           O  \nATOM    269  N   GLY A  68      -9.188  -2.617  42.707  1.00  7.30           N  \nATOM    270  CA  GLY A  68      -8.432  -1.419  42.957  1.00  7.34           C  \nATOM    271  C   GLY A  68      -8.353  -1.130  44.441  1.00  7.95           C  \nATOM    272  O   GLY A  68      -9.336  -1.247  45.173  1.00  9.50           O  \nATOM    273  N   GLY A  69      -7.193  -0.661  44.886  1.00  7.86           N  \nATOM    274  CA  GLY A  69      -6.995  -0.185  46.259  1.00  7.70           C  \nATOM    275  C   GLY A  69      -6.405   1.192  46.238  1.00  7.90           C  \nATOM    276  O   GLY A  69      -5.352   1.393  45.592  1.00  7.39           O  \nATOM    277  N   GLY A  70      -7.068   2.161  46.869  1.00  7.72           N  \nATOM    278  CA  GLY A  70      -6.621   3.537  46.945  1.00  7.66           C  \nATOM    279  C   GLY A  70      -5.983   3.797  48.290  1.00  9.11           C  \nATOM    280  O   GLY A  70      -6.570   3.399  49.333  1.00 11.07           O  \nATOM    281  N   GLY A  71      -4.814   4.419  48.277  1.00  8.41           N  \nATOM    282  CA  GLY A  71      -4.126   4.843  49.499  1.00  8.94           C  \nATOM    283  C   GLY A  71      -3.569   6.257  49.290  1.00  8.26           C  \nATOM    284  O   GLY A  71      -3.588   6.819  48.169  1.00  8.87           O  \nATOM    285  N   GLY A  72      -3.092   6.860  50.369  1.00  9.97           N  \nATOM    286  CA  GLY A  72      -2.479   8.152  50.347  1.00 11.68           C  \nATOM    287  C   GLY A  72      -1.351   8.222  51.356  1.00 13.21           C  \nATOM    288  O   GLY A  72      -1.424   7.611  52.424  1.00 16.20           O  \nATOM    289  N   GLY A  73      -0.326   8.973  50.995  1.00 11.90           N  \nATOM    290  CA  GLY A  73       0.803   9.270  51.906  1.00 13.04           C  \nATOM    291  C   GLY A  73       0.839  10.778  52.086  1.00 12.73           C  \nATOM    292  O   GLY A  73       1.137  11.549  51.163  1.00 13.33           O  \nATOM    293  N   GLY A  74       0.512  11.205  53.300  1.00 14.85           N  \nATOM    294  CA  GLY A  74       0.481  12.607  53.573  1.00 15.70           C  \nATOM    295  C   GLY A  74       1.893  13.215  53.666  1.00 17.11           C  \nATOM    296  O   GLY A  74       2.038  14.429  53.473  1.00 23.42           O  \nATOM    297  N   GLY A  75       2.932  12.410  53.926  1.00 14.92           N  \nATOM    298  CA  GLY A  75       4.294  12.972  54.004  1.00 16.94           C  \nATOM    299  C   GLY A  75       4.792  13.447  52.636  1.00 18.84           C  \nATOM    300  O   GLY A  75       5.442  14.480  52.513  1.00 23.54           O  \nATOM    301  N   GLY A  76       4.369  12.757  51.587  1.00 14.74           N  \nATOM    302  CA  GLY A  76       4.852  13.083  50.239  1.00 16.47           C  \nATOM    303  C   GLY A  76       3.727  13.674  49.388  1.00 13.78           C  \nATOM    304  O   GLY A  76       3.912  13.879  48.195  1.00 15.65           O  \nATOM    305  N   GLY A  77       2.562  13.937  49.977  1.00 13.08           N  \nATOM    306  CA  GLY A  77       1.382  14.420  49.219  1.00 12.07           C  \nATOM    307  C   GLY A  77       1.139  13.607  47.943  1.00 10.50           C  \nATOM    308  O   GLY A  77       0.966  14.173  46.869  1.00 11.43           O  \nATOM    309  N   GLY A  78       1.124  12.286  48.127  1.00  9.62           N  \nATOM    310  CA  GLY A  78       0.922  11.365  47.005  1.00  8.98           C  \nATOM    311  C   GLY A  78      -0.220  10.416  47.273  1.00  9.47           C  \nATOM    312  O   GLY A  78      -0.349   9.814  48.344  1.00 10.22           O  \nATOM    313  N   GLY A  79      -1.074  10.294  46.240  1.00  7.74           N  \nATOM    314  CA  GLY A  79      -2.142   9.299  46.201  1.00  7.68           C  \nATOM    315  C   GLY A  79      -1.696   8.145  45.345  1.00  6.89           C  \nATOM    316  O   GLY A  79      -1.029   8.353  44.337  1.00  8.86           O  \nATOM    317  N   GLY A  80      -2.094   6.960  45.717  1.00  6.94           N  \nATOM    318  CA  GLY A  80      -1.729   5.744  44.998  1.00  6.81           C  \nATOM    319  C   GLY A  80      -2.922   4.920  44.637  1.00  6.92           C  \nATOM    320  O   GLY A  80      -3.912   4.887  45.366  1.00  7.53           O  \nATOM    321  N   GLY A  81      -2.816   4.162  43.548  1.00  6.90           N  \nATOM    322  CA  GLY A  81      -3.800   3.150  43.217  1.00  6.78           C  \nATOM    323  C   GLY A  81      -3.077   1.883  42.826  1.00  6.45           C  \nATOM    324  O   GLY A  81      -2.165   1.908  41.957  1.00  8.04           O  \nATOM    325  N   GLY A  82      -3.483   0.796  43.443  1.00  7.19           N  \nATOM    326  CA  GLY A  82      -2.931  -0.535  43.146  1.00  7.64           C  \nATOM    327  C   GLY A  82      -4.028  -1.368  42.500  1.00  7.25           C  \nATOM    328  O   GLY A  82      -5.160  -1.397  42.997  1.00  8.63           O  \nATOM    329  N   GLY A  83      -3.714  -2.036  41.409  1.00  6.98           N  \nATOM    330  CA  GLY A  83      -4.607  -2.945  40.712  1.00  7.17           C  \nATOM    331  C   GLY A  83      -4.037  -4.345  40.848  1.00  8.34           C  \nATOM    332  O   GLY A  83      -2.868  -4.563  40.553  1.00  9.16           O  \nATOM    333  N   GLY A  84      -4.889  -5.305  41.176  1.00  7.75           N  \nATOM    334  CA  GLY A  84      -4.482  -6.706  41.325  1.00  8.68           C  \nATOM    335  C   GLY A  84      -5.050  -7.548  40.220  1.00  7.38           C  \nATOM    336  O   GLY A  84      -5.949  -7.131  39.499  1.00  8.16           O  \nATOM    337  N   GLY A  85      -4.523  -8.756  40.076  1.00  6.91           N  \nATOM    338  CA  GLY A  85      -5.040  -9.726  39.089  1.00  6.94           C  \nATOM    339  C   GLY A  85      -5.241  -9.133  37.726  1.00  6.73           C  \nATOM    340  O   GLY A  85      -6.311  -9.195  37.101  1.00  7.61           O  \nATOM    341  N   GLY A  86      -4.132  -8.583  37.177  1.00  6.84           N  \nATOM    342  CA  GLY A  86      -4.262  -7.829  35.939  1.00  6.58           C  \nATOM    343  C   GLY A  86      -4.718  -8.696  34.775  1.00  6.56           C  \nATOM    344  O   GLY A  86      -4.193  -9.798  34.582  1.00  7.60           O  \nATOM    345  N   GLY A  87      -5.614  -8.138  33.962  1.00  7.05           N  \nATOM    346  CA  GLY A  87      -6.143  -8.799  32.806  1.00  6.81           C  \nATOM    347  C   GLY A  87      -6.160  -7.802  31.650  1.00  7.55           C  \nATOM    348  O   GLY A  87      -5.941  -6.597  31.806  1.00  7.59           O  \nATOM    349  N   GLY A  88      -6.477  -8.294  30.461  1.00  8.50           N  \nATOM    350  CA  GLY A  88      -6.573  -7.391  29.291  1.00  9.38           C  \nATOM    351  C   GLY A  88      -7.561  -6.273  29.501  1.00  8.61           C  \nATOM    352  O   GLY A  88      -7.374  -5.184  28.964  1.00 10.63           O  \nATOM    353  N   GLY A  89      -8.586  -6.507  30.320  1.00  9.73           N  \nATOM    354  CA AGLY A  89      -9.657  -5.532  30.627  0.50 10.55           C  \nATOM    355  CA BGLY A  89      -9.598  -5.475  30.497  0.50  9.80           C  \nATOM    356  C   GLY A  89      -9.077  -4.335  31.379  1.00  9.72           C  \nATOM    357  O   GLY A  89      -9.725  -3.254  31.449  1.00  8.91           O  \nATOM    358  N   GLY A  90      -7.925  -4.548  32.026  1.00  8.39           N  \nATOM    359  CA  GLY A  90      -7.283  -3.485  32.803  1.00  7.57           C  \nATOM    360  C   GLY A  90      -6.346  -2.576  31.964  1.00  7.88           C  \nATOM    361  O   GLY A  90      -5.808  -1.580  32.471  1.00  7.99           O  \nATOM    362  N  AGLY A  91      -6.092  -2.920  30.701  0.50  7.11           N  \nATOM    363  N  BGLY A  91      -6.121  -2.915  30.717  0.50  7.14           N  \nATOM    364  CA AGLY A  91      -5.301  -2.053  29.823  0.50  7.59           C  \nATOM    365  CA BGLY A  91      -5.251  -2.163  29.855  0.50  7.89           C  \nATOM    366  C  AGLY A  91      -6.097  -0.800  29.578  0.50  7.67           C  \nATOM    367  C  BGLY A  91      -5.968  -0.848  29.447  0.50  7.80           C  \nATOM    368  O  AGLY A  91      -7.240  -0.813  29.110  0.50  8.00           O  \nATOM    369  O  BGLY A  91      -6.923  -0.889  28.679  0.50  8.97           O  \nATOM    370  N   GLY A  92      -5.510   0.295  29.970  1.00  7.36           N  \nATOM    371  CA  GLY A  92      -6.253   1.573  29.999  1.00  6.73           C  \nATOM    372  C   GLY A  92      -5.325   2.674  30.428  1.00  6.76           C  \nATOM    373  O   GLY A  92      -4.223   2.386  30.937  1.00  8.35           O  \nATOM    374  N   GLY A  93      -5.769   3.912  30.263  1.00  6.97           N  \nATOM    375  CA  GLY A  93      -5.121   5.063  30.888  1.00  6.88           C  \nATOM    376  C   GLY A  93      -5.928   5.381  32.135  1.00  6.70           C  \nATOM    377  O   GLY A  93      -7.164   5.470  32.133  1.00  7.41           O  \nATOM    378  N   GLY A  94      -5.214   5.607  33.231  1.00  5.76           N  \nATOM    379  CA  GLY A  94      -5.824   5.912  34.540  1.00  5.40           C  \nATOM    380  C   GLY A  94      -5.460   7.296  35.015  1.00  5.39           C  \nATOM    381  O   GLY A  94      -4.287   7.684  34.968  1.00  6.76           O  \nATOM    382  N   GLY A  95      -6.462   8.094  35.374  1.00  5.43           N  \nATOM    383  CA  GLY A  95      -6.290   9.477  35.811  1.00  5.14           C  \nATOM    384  C   GLY A  95      -6.621   9.610  37.273  1.00  5.82           C  \nATOM    385  O   GLY A  95      -7.575   9.012  37.758  1.00  7.17           O  \nATOM    386  N   GLY A  96      -5.881  10.466  37.947  1.00  6.10           N  \nATOM    387  CA  GLY A  96      -6.331  11.047  39.204  1.00  6.11           C  \nATOM    388  C   GLY A  96      -6.870  12.435  38.957  1.00  6.60           C  \nATOM    389  O   GLY A  96      -6.448  13.116  38.039  1.00  7.59           O  \nATOM    390  N   GLY A  97      -7.812  12.850  39.806  1.00  5.89           N  \nATOM    391  CA  GLY A  97      -8.417  14.178  39.667  1.00  6.86           C  \nATOM    392  C   GLY A  97      -8.783  14.693  41.036  1.00  6.85           C  \nATOM    393  O   GLY A  97      -9.269  13.934  41.882  1.00  7.75           O  \nATOM    394  N   GLY A  98      -8.555  15.986  41.243  1.00  6.38           N  \nATOM    395  CA  GLY A  98      -8.952  16.606  42.518  1.00  6.81           C  \nATOM    396  C   GLY A  98     -10.429  17.043  42.454  1.00  6.35           C  \nATOM    397  O   GLY A  98     -10.824  17.729  41.540  1.00  7.13           O  \nATOM    398  N   GLY A  99     -11.172  16.659  43.491  1.00  6.79           N  \nATOM    399  CA  GLY A  99     -12.602  17.025  43.639  1.00  6.36           C  \nATOM    400  C   GLY A  99     -12.688  18.348  44.402  1.00  6.92           C  \nATOM    401  O   GLY A  99     -12.380  18.409  45.605  1.00  8.28           O  \nATOM    402  N   GLY A 100     -13.057  19.415  43.703  1.00  6.90           N  \nATOM    403  CA  GLY A 100     -13.187  20.733  44.294  1.00  8.20           C  \nATOM    404  C   GLY A 100     -14.670  21.018  44.418  1.00  7.97           C  \nATOM    405  O   GLY A 100     -15.327  21.465  43.477  1.00  7.48           O  \nATOM    406  N   GLY A 101     -15.249  20.722  45.586  1.00  8.43           N  \nATOM    407  CA  GLY A 101     -16.680  20.967  45.796  1.00  7.99           C  \nATOM    408  C   GLY A 101     -17.595  20.377  44.742  1.00  7.40           C  \nATOM    409  O   GLY A 101     -18.551  21.015  44.301  1.00  9.03           O  \nATOM    410  N   GLY A 102     -17.249  19.151  44.325  1.00  6.65           N  \nATOM    411  CA  GLY A 102     -17.880  18.302  43.322  1.00  7.00           C  \nATOM    412  C   GLY A 102     -17.372  18.500  41.897  1.00  5.77           C  \nATOM    413  O   GLY A 102     -17.698  17.681  41.023  1.00  7.22           O  \nATOM    414  N   GLY A 103     -16.503  19.497  41.685  1.00  7.09           N  \nATOM    415  CA  GLY A 103     -15.904  19.726  40.390  1.00  6.81           C  \nATOM    416  C   GLY A 103     -14.606  18.959  40.218  1.00  6.82           C  \nATOM    417  O   GLY A 103     -13.646  19.271  40.928  1.00  8.10           O  \nATOM    418  N   GLY A 104     -14.549  18.067  39.240  1.00  6.89           N  \nATOM    419  CA  GLY A 104     -13.256  17.466  38.862  1.00  6.93           C  \nATOM    420  C   GLY A 104     -12.567  18.451  37.900  1.00  5.93           C  \nATOM    421  O   GLY A 104     -12.614  18.297  36.670  1.00  6.61           O  \nATOM    422  N   GLY A 105     -11.975  19.528  38.455  1.00  6.89           N  \nATOM    423  CA  GLY A 105     -11.364  20.553  37.635  1.00  7.22           C  \nATOM    424  C   GLY A 105      -9.882  20.404  37.420  1.00  8.13           C  \nATOM    425  O   GLY A 105      -9.380  21.038  36.509  1.00 13.23           O  \nATOM    426  N   GLY A 106      -9.210  19.601  38.220  1.00  6.58           N  \nATOM    427  CA  GLY A 106      -7.775  19.393  38.084  1.00  7.33           C  \nATOM    428  C   GLY A 106      -7.547  17.927  37.854  1.00  7.37           C  \nATOM    429  O   GLY A 106      -7.999  17.115  38.648  1.00  8.27           O  \nATOM    430  N   GLY A 107      -6.844  17.610  36.778  1.00  6.89           N  \nATOM    431  CA  GLY A 107      -6.607  16.226  36.382  1.00  6.73           C  \nATOM    432  C   GLY A 107      -5.096  15.995  36.246  1.00  6.65           C  \nATOM    433  O   GLY A 107      -4.364  16.861  35.739  1.00  7.75           O  \nATOM    434  N   GLY A 108      -4.628  14.821  36.625  1.00  5.92           N  \nATOM    435  CA  GLY A 108      -3.286  14.341  36.258  1.00  6.48           C  \nATOM    436  C   GLY A 108      -3.214  14.119  34.767  1.00  7.30           C  \nATOM    437  O   GLY A 108      -4.231  14.161  34.047  1.00  6.86           O  \nATOM    438  N   GLY A 109      -1.997  13.830  34.295  1.00  6.30           N  \nATOM    439  CA  GLY A 109      -1.798  13.598  32.880  1.00  7.31           C  \nATOM    440  C   GLY A 109      -2.087  12.167  32.457  1.00  6.85           C  \nATOM    441  O   GLY A 109      -2.063  11.858  31.260  1.00  8.42           O  \nATOM    442  N   GLY A 110      -2.382  11.291  33.405  1.00  6.25           N  \nATOM    443  CA  GLY A 110      -2.728   9.896  33.163  1.00  6.37           C  \nATOM    444  C   GLY A 110      -1.528   9.010  33.251  1.00  6.65           C  \nATOM    445  O   GLY A 110      -0.402   9.452  32.954  1.00  7.98           O  \nATOM    446  N   GLY A 111      -1.779   7.776  33.662  1.00  6.29           N  \nATOM    447  CA  GLY A 111      -0.798   6.700  33.591  1.00  5.68           C  \nATOM    448  C   GLY A 111      -1.350   5.609  32.690  1.00  5.88           C  \nATOM    449  O   GLY A 111      -2.382   5.034  32.987  1.00  6.55           O  \nATOM    450  N   GLY A 112      -0.640   5.287  31.615  1.00  6.47           N  \nATOM    451  CA  GLY A 112      -1.006   4.189  30.742  1.00  6.89           C  \nATOM    452  C   GLY A 112      -0.565   2.890  31.354  1.00  7.62           C  \nATOM    453  O   GLY A 112       0.618   2.717  31.691  1.00  9.40           O  \nATOM    454  N   GLY A 113      -1.453   1.910  31.442  1.00  7.19           N  \nATOM    455  CA  GLY A 113      -1.112   0.536  31.840  1.00  7.29           C  \nATOM    456  C   GLY A 113      -1.447  -0.314  30.633  1.00  7.25           C  \nATOM    457  O   GLY A 113      -2.596  -0.323  30.162  1.00  8.28           O  \nATOM    458  N   GLY A 114      -0.452  -1.026  30.108  1.00  6.56           N  \nATOM    459  CA  GLY A 114      -0.650  -1.967  28.997  1.00  6.78           C  \nATOM    460  C   GLY A 114      -0.431  -3.359  29.583  1.00  8.09           C  \nATOM    461  O   GLY A 114       0.658  -3.674  30.052  1.00  8.38           O  \nATOM    462  N   GLY A 115      -1.470  -4.199  29.512  1.00  8.55           N  \nATOM    463  CA  GLY A 115      -1.395  -5.589  29.952  1.00  8.62           C  \nATOM    464  C   GLY A 115      -1.187  -6.435  28.696  1.00 11.28           C  \nATOM    465  O   GLY A 115      -2.038  -6.446  27.815  1.00 17.59           O  \nATOM    466  N   GLY A 116      -0.036  -7.076  28.578  1.00 13.10           N  \nATOM    467  CA  GLY A 116       0.314  -7.812  27.342  1.00 19.07           C  \nATOM    468  C   GLY A 116      -0.589  -9.028  27.076  1.00 30.32           C  \nATOM    469  O   GLY A 116      -1.182  -9.598  28.019  1.00 31.24           O  \nATOM    470  N   GLY A 117     -24.023  -1.474  38.559  1.00 43.33           N  \nATOM    471  CA  GLY A 117     -25.106  -1.720  39.513  1.00 37.93           C  \nATOM    472  C   GLY A 117     -26.471  -1.362  38.952  1.00 36.26           C  \nATOM    473  O   GLY A 117     -26.616  -0.389  38.209  1.00 25.27           O  \nATOM    474  N   GLY A 118     -27.488  -2.145  39.318  1.00 35.38           N  \nATOM    475  CA  GLY A 118     -28.854  -1.898  38.865  1.00 33.61           C  \nATOM    476  C   GLY A 118     -29.418  -0.543  39.236  1.00 30.10           C  \nATOM    477  O   GLY A 118     -30.219   0.022  38.488  1.00 30.64           O  \nATOM    478  N   GLY A 119     -29.008  -0.022  40.389  1.00 24.59           N  \nATOM    479  CA  GLY A 119     -29.466   1.286  40.835  1.00 23.40           C  \nATOM    480  C   GLY A 119     -28.508   2.387  40.403  1.00 17.38           C  \nATOM    481  O   GLY A 119     -28.743   3.545  40.756  1.00 19.20           O  \nATOM    482  N   GLY A 120     -27.427   2.056  39.665  1.00 14.11           N  \nATOM    483  CA  GLY A 120     -26.503   3.121  39.273  1.00 11.69           C  \nATOM    484  C   GLY A 120     -27.187   4.038  38.293  1.00  9.61           C  \nATOM    485  O   GLY A 120     -28.065   3.617  37.548  1.00 12.43           O  \nATOM    486  N   GLY A 121     -26.809   5.319  38.330  1.00  7.96           N  \nATOM    487  CA  GLY A 121     -27.393   6.291  37.387  1.00  8.24           C  \nATOM    488  C   GLY A 121     -26.766   6.084  36.028  1.00  7.64           C  \nATOM    489  O   GLY A 121     -25.509   6.054  35.890  1.00  9.75           O  \nATOM    490  N   GLY A 122     -27.568   5.870  35.013  1.00  7.35           N  \nATOM    491  CA  GLY A 122     -27.100   5.617  33.625  1.00  8.09           C  \nATOM    492  C   GLY A 122     -27.142   6.936  32.874  1.00  7.43           C  \nATOM    493  O   GLY A 122     -28.193   7.597  32.813  1.00  7.51           O  \nATOM    494  N   GLY A 123     -25.994   7.323  32.342  1.00  7.20           N  \nATOM    495  CA  GLY A 123     -25.853   8.534  31.579  1.00  6.87           C  \nATOM    496  C   GLY A 123     -25.729   8.141  30.114  1.00  8.02           C  \nATOM    497  O   GLY A 123     -24.878   7.299  29.779  1.00  9.90           O  \nATOM    498  N   GLY A 124     -26.570   8.707  29.270  1.00  7.25           N  \nATOM    499  CA  GLY A 124     -26.557   8.319  27.856  1.00  9.07           C  \nATOM    500  C   GLY A 124     -26.319   9.537  26.984  1.00  8.16           C  \nATOM    501  O   GLY A 124     -26.737  10.640  27.294  1.00 11.69           O  \nATOM    502  N   GLY A 125     -25.580   9.335  25.905  1.00  7.31           N  \nATOM    503  CA  GLY A 125     -25.257  10.334  24.890  1.00  6.71           C  \nATOM    504  C   GLY A 125     -25.602   9.671  23.567  1.00  8.16           C  \nATOM    505  O   GLY A 125     -24.869   8.801  23.100  1.00  9.95           O  \nATOM    506  N   GLY A 126     -26.730  10.061  22.999  1.00  8.35           N  \nATOM    507  CA  GLY A 126     -27.227   9.342  21.817  1.00  8.87           C  \nATOM    508  C   GLY A 126     -26.324   9.436  20.602  1.00  8.31           C  \nATOM    509  O   GLY A 126     -26.351   8.539  19.750  1.00 10.92           O  \nATOM    510  N   GLY A 127     -25.648  10.573  20.419  1.00  7.50           N  \nATOM    511  CA  GLY A 127     -24.789  10.755  19.226  1.00  8.18           C  \nATOM    512  C   GLY A 127     -23.388  10.276  19.490  1.00  9.11           C  \nATOM    513  O   GLY A 127     -22.718  10.862  20.378  1.00  9.37           O  \nATOM    514  N   GLY A 128     -22.885   9.317  18.714  1.00  9.11           N  \nATOM    515  CA  GLY A 128     -21.474   8.957  18.913  1.00  9.99           C  \nATOM    516  C   GLY A 128     -20.527  10.067  18.463  1.00  9.51           C  \nATOM    517  O   GLY A 128     -19.421  10.187  18.949  1.00 10.17           O  \nATOM    518  N   GLY A 129     -20.956  10.885  17.491  1.00  8.86           N  \nATOM    519  CA  GLY A 129     -20.115  11.938  16.962  1.00  9.12           C  \nATOM    520  C   GLY A 129     -20.900  13.144  16.474  1.00  9.54           C  \nATOM    521  O   GLY A 129     -22.136  13.020  16.185  1.00 10.67           O  \nATOM    522  N   GLY A 130     -20.223  14.274  16.393  1.00 10.10           N  \nATOM    523  CA  GLY A 130     -20.715  15.430  15.660  1.00 10.57           C  \nATOM    524  C   GLY A 130     -19.603  15.908  14.728  1.00 13.05           C  \nATOM    525  O   GLY A 130     -18.441  15.925  15.102  1.00 13.27           O  \nATOM    526  N   GLY A 131     -19.984  16.251  13.492  1.00 14.16           N  \nATOM    527  CA  GLY A 131     -19.042  16.598  12.410  1.00 17.26           C  \nATOM    528  C   GLY A 131     -19.437  17.994  11.997  1.00 16.51           C  \nATOM    529  O   GLY A 131     -20.587  18.203  11.574  1.00 19.33           O  \nATOM    530  N   GLY A 132     -18.547  18.965  12.158  1.00 17.22           N  \nATOM    531  CA  GLY A 132     -18.872  20.334  11.966  1.00 18.85           C  \nATOM    532  C   GLY A 132     -17.783  21.090  11.192  1.00 17.40           C  \nATOM    533  O   GLY A 132     -16.698  20.569  10.956  1.00 20.05           O  \nATOM    534  N   GLY A 133     -18.098  22.275  10.710  1.00 22.16           N  \nATOM    535  CA  GLY A 133     -17.079  23.087  10.047  1.00 19.79           C  \nATOM    536  C   GLY A 133     -16.773  24.233  10.989  1.00 20.82           C  \nATOM    537  O   GLY A 133     -17.638  24.700  11.729  1.00 19.48           O  \nATOM    538  N   GLY A 134     -15.536  24.715  10.972  1.00 21.36           N  \nATOM    539  CA  GLY A 134     -15.210  25.820  11.858  1.00 18.31           C  \nATOM    540  C   GLY A 134     -16.118  27.022  11.716  1.00 21.43           C  \nATOM    541  O   GLY A 134     -16.495  27.388  10.603  1.00 23.24           O  \nATOM    542  N   GLY A 135     -16.510  27.595  12.845  1.00 19.39           N  \nATOM    543  CA  GLY A 135     -17.420  28.709  12.913  1.00 20.50           C  \nATOM    544  C   GLY A 135     -18.873  28.295  13.103  1.00 19.42           C  \nATOM    545  O   GLY A 135     -19.663  29.135  13.474  1.00 24.49           O  \nATOM    546  N   GLY A 136     -19.217  27.020  12.824  1.00 18.20           N  \nATOM    547  CA  GLY A 136     -20.574  26.508  13.072  1.00 18.01           C  \nATOM    548  C   GLY A 136     -20.789  26.491  14.606  1.00 16.96           C  \nATOM    549  O   GLY A 136     -19.839  26.407  15.402  1.00 16.23           O  \nATOM    550  N   GLY A 137     -22.056  26.522  14.996  1.00 14.21           N  \nATOM    551  CA  GLY A 137     -22.477  26.182  16.345  1.00 14.12           C  \nATOM    552  C   GLY A 137     -22.938  24.748  16.377  1.00 12.67           C  \nATOM    553  O   GLY A 137     -23.372  24.207  15.374  1.00 14.31           O  \nATOM    554  N   GLY A 138     -22.894  24.111  17.561  1.00  9.68           N  \nATOM    555  CA  GLY A 138     -23.293  22.712  17.717  1.00 10.20           C  \nATOM    556  C   GLY A 138     -23.915  22.476  19.079  1.00  9.51           C  \nATOM    557  O   GLY A 138     -23.546  23.110  20.055  1.00 10.92           O  \nATOM    558  N   GLY A 139     -24.839  21.528  19.104  1.00  9.48           N  \nATOM    559  CA  GLY A 139     -25.472  21.068  20.351  1.00 10.62           C  \nATOM    560  C   GLY A 139     -25.154  19.616  20.552  1.00 10.29           C  \nATOM    561  O   GLY A 139     -25.194  18.824  19.580  1.00 12.76           O  \nATOM    562  N   GLY A 140     -24.848  19.255  21.807  1.00  8.40           N  \nATOM    563  CA  GLY A 140     -24.560  17.866  22.181  1.00  8.75           C  \nATOM    564  C   GLY A 140     -25.495  17.487  23.336  1.00  7.70           C  \nATOM    565  O   GLY A 140     -25.525  18.210  24.341  1.00  8.42           O  \nATOM    566  N   GLY A 141     -26.211  16.377  23.173  1.00  7.40           N  \nATOM    567  CA AGLY A 141     -27.176  15.933  24.143  0.50  8.00           C  \nATOM    568  CA BGLY A 141     -27.192  15.956  24.153  0.50  8.46           C  \nATOM    569  C   GLY A 141     -26.675  14.854  25.092  1.00  7.17           C  \nATOM    570  O   GLY A 141     -25.808  14.039  24.756  1.00  9.06           O  \nATOM    571  N   GLY A 142     -27.280  14.857  26.284  1.00  6.36           N  \nATOM    572  CA  GLY A 142     -27.049  13.838  27.272  1.00  7.52           C  \nATOM    573  C   GLY A 142     -28.361  13.609  28.048  1.00  7.88           C  \nATOM    574  O   GLY A 142     -29.154  14.551  28.231  1.00  9.04           O  \nATOM    575  N   GLY A 143     -28.561  12.407  28.567  1.00  7.48           N  \nATOM    576  CA AGLY A 143     -29.699  12.129  29.441  0.50  7.49           C  \nATOM    577  CA BGLY A 143     -29.713  12.118  29.414  0.50  7.80           C  \nATOM    578  C   GLY A 143     -29.239  11.335  30.637  1.00  7.52           C  \nATOM    579  O   GLY A 143     -28.261  10.526  30.575  1.00  9.52           O  \nATOM    580  N   GLY A 144     -29.919  11.538  31.768  1.00  6.81           N  \nATOM    581  CA  GLY A 144     -29.657  10.777  32.991  1.00  5.96           C  \nATOM    582  C   GLY A 144     -30.865   9.935  33.320  1.00  6.55           C  \nATOM    583  O   GLY A 144     -32.002  10.359  33.109  1.00  8.34           O  \nATOM    584  N   GLY A 145     -30.637   8.752  33.912  1.00  6.46           N  \nATOM    585  CA  GLY A 145     -31.748   7.851  34.213  1.00  7.55           C  \nATOM    586  C   GLY A 145     -32.549   8.260  35.425  1.00  6.33           C  \nATOM    587  O   GLY A 145     -33.644   7.752  35.619  1.00  7.67           O  \nATOM    588  N   GLY A 146     -32.066   9.195  36.225  1.00  6.57           N  \nATOM    589  CA  GLY A 146     -32.864   9.792  37.273  1.00  8.68           C  \nATOM    590  C   GLY A 146     -32.475  11.257  37.367  1.00  7.00           C  \nATOM    591  O   GLY A 146     -31.439  11.711  36.842  1.00  7.14           O  \nATOM    592  N   GLY A 147     -33.357  12.027  37.974  1.00  6.52           N  \nATOM    593  CA  GLY A 147     -33.086  13.445  38.190  1.00  6.01           C  \nATOM    594  C   GLY A 147     -31.743  13.643  38.884  1.00  5.60           C  \nATOM    595  O   GLY A 147     -31.439  12.990  39.872  1.00  5.93           O  \nATOM    596  N   GLY A 148     -30.988  14.640  38.401  1.00  5.41           N  \nATOM    597  CA  GLY A 148     -29.730  15.033  39.039  1.00  6.31           C  \nATOM    598  C   GLY A 148     -29.898  16.240  39.937  1.00  5.68           C  \nATOM    599  O   GLY A 148     -28.912  16.774  40.484  1.00  5.75           O  \nATOM    600  N   GLY A 149     -31.163  16.652  40.187  1.00  5.66           N  \nATOM    601  CA  GLY A 149     -31.457  17.780  41.095  1.00  6.03           C  \nATOM    602  C   GLY A 149     -31.371  17.328  42.534  1.00  6.46           C  \nATOM    603  O   GLY A 149     -32.015  16.346  42.927  1.00  8.19           O  \nATOM    604  N   GLY A 150     -30.607  18.032  43.327  1.00  6.38           N  \nATOM    605  CA  GLY A 150     -30.395  17.778  44.754  1.00  6.80           C  \nATOM    606  C   GLY A 150     -31.319  18.672  45.575  1.00  7.31           C  \nATOM    607  O   GLY A 150     -31.736  19.745  45.129  1.00  7.68           O  \nATOM    608  N   GLY A 151     -31.528  18.268  46.807  1.00  8.28           N  \nATOM    609  CA  GLY A 151     -32.297  19.067  47.742  1.00  9.28           C  \nATOM    610  C   GLY A 151     -31.737  20.488  47.928  1.00  8.49           C  \nATOM    611  O   GLY A 151     -32.496  21.459  48.221  1.00 10.82           O  \nATOM    612  N   GLY A 152     -30.439  20.644  47.749  1.00  7.49           N  \nATOM    613  CA  GLY A 152     -29.832  21.966  47.898  1.00  7.54           C  \nATOM    614  C   GLY A 152     -30.119  22.933  46.760  1.00  6.84           C  \nATOM    615  O   GLY A 152     -29.820  24.112  46.899  1.00  7.89           O  \nATOM    616  N   GLY A 153     -30.707  22.434  45.685  1.00  6.82           N  \nATOM    617  CA  GLY A 153     -31.110  23.265  44.558  1.00  6.98           C  \nATOM    618  C   GLY A 153     -30.220  23.179  43.346  1.00  5.97           C  \nATOM    619  O   GLY A 153     -30.578  23.643  42.260  1.00  6.17           O  \nATOM    620  N   GLY A 154     -29.047  22.580  43.514  1.00  6.24           N  \nATOM    621  CA  GLY A 154     -28.117  22.349  42.427  1.00  5.91           C  \nATOM    622  C   GLY A 154     -28.403  21.082  41.659  1.00  5.56           C  \nATOM    623  O   GLY A 154     -28.912  20.095  42.241  1.00  6.51           O  \nATOM    624  N   GLY A 155     -28.128  21.084  40.358  1.00  5.33           N  \nATOM    625  CA  GLY A 155     -28.256  19.911  39.506  1.00  5.73           C  \nATOM    626  C   GLY A 155     -26.858  19.416  39.145  1.00  5.56           C  \nATOM    627  O   GLY A 155     -26.092  20.150  38.503  1.00  6.54           O  \nATOM    628  N   GLY A 156     -26.516  18.196  39.595  1.00  5.29           N  \nATOM    629  CA  GLY A 156     -25.132  17.732  39.597  1.00  5.59           C  \nATOM    630  C   GLY A 156     -24.731  17.029  38.295  1.00  4.89           C  \nATOM    631  O   GLY A 156     -24.331  15.847  38.280  1.00  5.54           O  \nATOM    632  N   GLY A 157     -24.838  17.795  37.223  1.00  5.22           N  \nATOM    633  CA  GLY A 157     -24.461  17.368  35.876  1.00  5.59           C  \nATOM    634  C   GLY A 157     -23.219  18.171  35.474  1.00  5.66           C  \nATOM    635  O   GLY A 157     -23.140  19.386  35.708  1.00  6.06           O  \nATOM    636  N   GLY A 158     -22.241  17.490  34.850  1.00  5.04           N  \nATOM    637  CA  GLY A 158     -20.950  18.071  34.473  1.00  5.54           C  \nATOM    638  C   GLY A 158     -20.651  17.701  33.036  1.00  5.65           C  \nATOM    639  O   GLY A 158     -20.992  16.626  32.589  1.00  6.10           O  \nATOM    640  N   GLY A 159     -19.994  18.627  32.328  1.00  5.39           N  \nATOM    641  CA  GLY A 159     -19.509  18.376  30.973  1.00  5.75           C  \nATOM    642  C   GLY A 159     -18.014  18.509  30.945  1.00  6.08           C  \nATOM    643  O   GLY A 159     -17.460  19.482  31.469  1.00  6.65           O  \nATOM    644  N   GLY A 160     -17.375  17.535  30.273  1.00  6.10           N  \nATOM    645  CA  GLY A 160     -15.934  17.511  30.082  1.00  6.36           C  \nATOM    646  C   GLY A 160     -15.629  17.410  28.581  1.00  6.42           C  \nATOM    647  O   GLY A 160     -16.361  16.775  27.813  1.00  7.53           O  \nATOM    648  N   GLY A 161     -14.478  17.964  28.196  1.00  6.98           N  \nATOM    649  CA  GLY A 161     -13.859  17.756  26.866  1.00  8.22           C  \nATOM    650  C   GLY A 161     -12.546  17.059  27.054  1.00  8.06           C  \nATOM    651  O   GLY A 161     -11.716  17.542  27.857  1.00  8.92           O  \nATOM    652  N   GLY A 162     -12.338  15.949  26.347  1.00  8.50           N  \nATOM    653  CA  GLY A 162     -11.024  15.315  26.330  1.00  8.37           C  \nATOM    654  C   GLY A 162     -10.458  15.516  24.899  1.00  9.32           C  \nATOM    655  O   GLY A 162     -10.899  14.846  23.961  1.00  9.86           O  \nATOM    656  N   GLY A 163      -9.567  16.500  24.768  1.00  9.32           N  \nATOM    657  CA AGLY A 163      -8.864  16.756  23.504  0.50 11.01           C  \nATOM    658  CA BGLY A 163      -8.858  16.738  23.503  0.50 11.12           C  \nATOM    659  C   GLY A 163      -7.929  15.601  23.164  1.00 12.93           C  \nATOM    660  O   GLY A 163      -7.536  14.832  24.037  1.00 12.34           O  \nATOM    661  N   GLY A 164      -7.619  15.460  21.862  1.00 13.51           N  \nATOM    662  CA  GLY A 164      -6.791  14.321  21.484  1.00 15.44           C  \nATOM    663  C   GLY A 164      -5.398  14.333  22.203  1.00 12.26           C  \nATOM    664  O   GLY A 164      -4.779  15.372  22.285  1.00 18.21           O  \nATOM    665  N   GLY A 165      -5.107  13.214  22.842  1.00 16.65           N  \nATOM    666  CA  GLY A 165      -3.850  12.997  23.567  1.00 20.49           C  \nATOM    667  C   GLY A 165      -3.761  13.745  24.873  1.00 18.98           C  \nATOM    668  O   GLY A 165      -2.667  13.859  25.430  1.00 23.62           O  \nATOM    669  N   GLY A 166      -4.902  14.256  25.374  1.00 15.19           N  \nATOM    670  CA  GLY A 166      -4.886  15.083  26.579  1.00 13.43           C  \nATOM    671  C   GLY A 166      -5.787  14.427  27.622  1.00 11.48           C  \nATOM    672  O   GLY A 166      -6.565  13.528  27.335  1.00 11.24           O  \nATOM    673  N   GLY A 167      -5.680  14.917  28.855  1.00 10.86           N  \nATOM    674  CA  GLY A 167      -6.593  14.555  29.911  1.00 10.17           C  \nATOM    675  C   GLY A 167      -7.924  15.279  29.737  1.00  9.31           C  \nATOM    676  O   GLY A 167      -8.006  16.332  29.112  1.00  9.56           O  \nATOM    677  N   GLY A 168      -8.989  14.734  30.327  1.00  8.99           N  \nATOM    678  CA  GLY A 168     -10.238  15.481  30.332  1.00  8.54           C  \nATOM    679  C   GLY A 168     -10.101  16.854  30.992  1.00  7.75           C  \nATOM    680  O   GLY A 168      -9.285  17.008  31.900  1.00 10.29           O  \nATOM    681  N   GLY A 169     -10.879  17.811  30.503  1.00  8.40           N  \nATOM    682  CA AGLY A 169     -10.991  19.091  31.139  0.50  8.44           C  \nATOM    683  CA BGLY A 169     -10.988  19.189  30.995  0.50  8.04           C  \nATOM    684  C   GLY A 169     -12.437  19.457  31.395  1.00  7.32           C  \nATOM    685  O   GLY A 169     -13.322  19.205  30.573  1.00  7.78           O  \nATOM    686  N   GLY A 170     -12.674  20.009  32.572  1.00  7.43           N  \nATOM    687  CA  GLY A 170     -14.022  20.448  32.937  1.00  6.34           C  \nATOM    688  C   GLY A 170     -14.446  21.680  32.176  1.00  7.49           C  \nATOM    689  O   GLY A 170     -13.721  22.652  32.128  1.00  9.54           O  \nATOM    690  N   GLY A 171     -15.632  21.643  31.557  1.00  7.02           N  \nATOM    691  CA  GLY A 171     -16.190  22.774  30.847  1.00  7.10           C  \nATOM    692  C   GLY A 171     -17.330  23.449  31.599  1.00  5.98           C  \nATOM    693  O   GLY A 171     -17.355  24.668  31.730  1.00  6.71           O  \nATOM    694  N   GLY A 172     -18.262  22.626  32.121  1.00  6.02           N  \nATOM    695  CA  GLY A 172     -19.505  23.134  32.701  1.00  6.07           C  \nATOM    696  C   GLY A 172     -19.750  22.338  33.961  1.00  6.17           C  \nATOM    697  O   GLY A 172     -19.655  21.102  33.911  1.00  5.97           O  \nATOM    698  N   GLY A 173     -20.040  23.004  35.056  1.00  5.79           N  \nATOM    699  CA  GLY A 173     -20.297  22.327  36.336  1.00  6.13           C  \nATOM    700  C   GLY A 173     -21.688  22.657  36.853  1.00  5.75           C  \nATOM    701  O   GLY A 173     -22.204  23.746  36.619  1.00  5.58           O  \nATOM    702  N   GLY A 174     -22.299  21.693  37.538  1.00  5.62           N  \nATOM    703  CA  GLY A 174     -23.668  21.877  38.068  1.00  6.32           C  \nATOM    704  C   GLY A 174     -24.601  22.430  36.980  1.00  6.15           C  \nATOM    705  O   GLY A 174     -25.287  23.438  37.147  1.00  6.53           O  \nATOM    706  N   GLY A 175     -24.612  21.707  35.857  1.00  5.60           N  \nATOM    707  CA  GLY A 175     -25.522  21.873  34.722  1.00  6.01           C  \nATOM    708  C   GLY A 175     -25.275  23.068  33.821  1.00  5.16           C  \nATOM    709  O   GLY A 175     -25.313  22.915  32.608  1.00  5.76           O  \nATOM    710  N   GLY A 176     -25.017  24.238  34.399  1.00  5.95           N  \nATOM    711  CA  GLY A 176     -24.944  25.462  33.605  1.00  6.18           C  \nATOM    712  C   GLY A 176     -23.747  26.359  33.913  1.00  6.21           C  \nATOM    713  O   GLY A 176     -23.569  27.341  33.205  1.00  7.20           O  \nATOM    714  N   GLY A 177     -23.001  26.079  34.945  1.00  6.22           N  \nATOM    715  CA  GLY A 177     -21.956  27.027  35.344  1.00  6.34           C  \nATOM    716  C   GLY A 177     -20.678  26.847  34.521  1.00  7.50           C  \nATOM    717  O   GLY A 177     -20.176  25.740  34.392  1.00  7.03           O  \nATOM    718  N   GLY A 178     -20.115  27.956  34.047  1.00  7.17           N  \nATOM    719  CA  GLY A 178     -18.952  27.895  33.202  1.00  7.59           C  \nATOM    720  C   GLY A 178     -17.723  27.749  34.084  1.00  8.04           C  \nATOM    721  O   GLY A 178     -17.488  28.549  35.007  1.00  8.99           O  \nATOM    722  N   GLY A 179     -16.893  26.767  33.789  1.00  8.23           N  \nATOM    723  CA  GLY A 179     -15.668  26.545  34.566  1.00  9.84           C  \nATOM    724  C   GLY A 179     -14.593  27.571  34.256  1.00 10.92           C  \nATOM    725  O   GLY A 179     -14.544  28.157  33.179  1.00 10.71           O  \nATOM    726  N   GLY A 180     -13.726  27.777  35.234  1.00 16.30           N  \nATOM    727  CA  GLY A 180     -12.637  28.758  35.086  1.00 19.26           C  \nATOM    728  C   GLY A 180     -11.773  28.405  33.895  1.00 16.83           C  \nATOM    729  O   GLY A 180     -11.424  27.228  33.666  1.00 19.34           O  \nATOM    730  N   GLY A 181     -11.438  29.426  33.120  1.00 19.58           N  \nATOM    731  CA  GLY A 181     -10.540  29.230  31.994  1.00 20.26           C  \nATOM    732  C   GLY A 181     -11.190  28.737  30.724  1.00 21.93           C  \nATOM    733  O   GLY A 181     -10.518  28.533  29.723  1.00 25.76           O  \nATOM    734  N   GLY A 182     -12.514  28.520  30.756  1.00 14.42           N  \nATOM    735  CA  GLY A 182     -13.194  27.968  29.609  1.00 12.93           C  \nATOM    736  C   GLY A 182     -13.891  29.127  28.884  1.00 12.13           C  \nATOM    737  O   GLY A 182     -14.498  29.990  29.534  1.00 13.66           O  \nATOM    738  N   GLY A 183     -13.787  29.191  27.548  1.00 13.66           N  \nATOM    739  CA  GLY A 183     -14.408  30.346  26.880  1.00 14.35           C  \nATOM    740  C   GLY A 183     -15.937  30.342  26.972  1.00 15.38           C  \nATOM    741  O   GLY A 183     -16.570  29.272  27.035  1.00 12.22           O  \nATOM    742  N   GLY A 184     -16.549  31.511  26.927  1.00 16.02           N  \nATOM    743  CA  GLY A 184     -18.012  31.547  27.109  1.00 16.48           C  \nATOM    744  C   GLY A 184     -18.775  31.202  25.831  1.00 14.09           C  \nATOM    745  O   GLY A 184     -19.989  31.216  25.842  1.00 15.26           O  \nATOM    746  N   GLY A 185     -18.075  30.759  24.784  1.00 12.97           N  \nATOM    747  CA  GLY A 185     -18.671  30.017  23.691  1.00 13.45           C  \nATOM    748  C   GLY A 185     -19.327  28.712  24.139  1.00 10.83           C  \nATOM    749  O   GLY A 185     -20.165  28.185  23.418  1.00 11.51           O  \nATOM    750  N   GLY A 186     -18.913  28.181  25.290  1.00  9.72           N  \nATOM    751  CA  GLY A 186     -19.512  26.950  25.866  1.00  8.52           C  \nATOM    752  C   GLY A 186     -20.613  27.326  26.824  1.00  9.37           C  \nATOM    753  O   GLY A 186     -20.408  28.215  27.663  1.00 10.70           O  \nATOM    754  N   GLY A 187     -21.741  26.646  26.723  1.00  8.36           N  \nATOM    755  CA  GLY A 187     -22.821  26.834  27.693  1.00  8.28           C  \nATOM    756  C   GLY A 187     -23.572  25.537  27.880  1.00  7.79           C  \nATOM    757  O   GLY A 187     -23.541  24.630  27.058  1.00  9.04           O  \nATOM    758  N   GLY A 188     -24.250  25.466  29.014  1.00  7.37           N  \nATOM    759  CA  GLY A 188     -25.014  24.269  29.353  1.00  7.51           C  \nATOM    760  C   GLY A 188     -26.393  24.573  29.863  1.00  7.29           C  \nATOM    761  O   GLY A 188     -26.580  25.588  30.562  1.00  7.63           O  \nATOM    762  N   GLY A 189     -27.342  23.707  29.478  1.00  6.88           N  \nATOM    763  CA  GLY A 189     -28.707  23.861  29.955  1.00  7.31           C  \nATOM    764  C   GLY A 189     -29.259  22.472  30.206  1.00  6.58           C  \nATOM    765  O   GLY A 189     -28.666  21.456  29.820  1.00  6.53           O  \nATOM    766  N   GLY A 190     -30.441  22.412  30.824  1.00  6.61           N  \nATOM    767  CA  GLY A 190     -31.071  21.124  31.005  1.00  6.87           C  \nATOM    768  C   GLY A 190     -32.309  21.149  31.847  1.00  7.46           C  \nATOM    769  O   GLY A 190     -32.654  22.174  32.424  1.00  8.57           O  \nATOM    770  N   GLY A 191     -32.946  19.972  31.937  1.00  7.04           N  \nATOM    771  CA  GLY A 191     -34.207  19.797  32.662  1.00  6.92           C  \nATOM    772  C   GLY A 191     -34.056  19.043  33.949  1.00  7.29           C  \nATOM    773  O   GLY A 191     -35.030  18.790  34.635  1.00  8.32           O  \nATOM    774  N   GLY A 192     -32.828  18.621  34.268  1.00  7.36           N  \nATOM    775  CA  GLY A 192     -32.537  17.703  35.381  1.00  7.23           C  \nATOM    776  C   GLY A 192     -32.418  16.245  34.992  1.00  6.60           C  \nATOM    777  O   GLY A 192     -31.823  15.451  35.741  1.00  6.79           O  \nATOM    778  N   GLY A 193     -32.989  15.884  33.843  1.00  6.82           N  \nATOM    779  CA  GLY A 193     -32.854  14.548  33.263  1.00  7.19           C  \nATOM    780  C   GLY A 193     -32.403  14.538  31.825  1.00  6.33           C  \nATOM    781  O   GLY A 193     -32.008  13.479  31.315  1.00  8.31           O  \nATOM    782  N   GLY A 194     -32.508  15.678  31.148  1.00  6.64           N  \nATOM    783  CA  GLY A 194     -31.957  15.833  29.770  1.00  7.45           C  \nATOM    784  C   GLY A 194     -31.187  17.130  29.720  1.00  6.91           C  \nATOM    785  O   GLY A 194     -31.610  18.115  30.336  1.00  7.50           O  \nATOM    786  N   GLY A 195     -30.062  17.090  29.008  1.00  5.98           N  \nATOM    787  CA  GLY A 195     -29.089  18.172  29.074  1.00  6.21           C  \nATOM    788  C   GLY A 195     -28.535  18.456  27.698  1.00  6.74           C  \nATOM    789  O   GLY A 195     -28.409  17.567  26.875  1.00  6.66           O  \nATOM    790  N   GLY A 196     -28.098  19.704  27.518  1.00  6.22           N  \nATOM    791  CA  GLY A 196     -27.520  20.137  26.242  1.00  6.54           C  \nATOM    792  C   GLY A 196     -26.286  21.000  26.488  1.00  6.37           C  \nATOM    793  O   GLY A 196     -26.360  22.020  27.211  1.00  7.92           O  \nATOM    794  N   GLY A 197     -25.161  20.578  25.897  1.00  6.85           N  \nATOM    795  CA  GLY A 197     -23.971  21.425  25.813  1.00  6.66           C  \nATOM    796  C   GLY A 197     -24.048  22.144  24.462  1.00  7.41           C  \nATOM    797  O   GLY A 197     -24.222  21.490  23.437  1.00  8.09           O  \nATOM    798  N   GLY A 198     -23.945  23.480  24.477  1.00  8.12           N  \nATOM    799  CA AGLY A 198     -23.902  24.249  23.250  0.50  8.98           C  \nATOM    800  CA BGLY A 198     -23.915  24.275  23.243  0.50  8.89           C  \nATOM    801  C   GLY A 198     -22.542  24.890  23.090  1.00  9.33           C  \nATOM    802  O   GLY A 198     -22.000  25.442  24.041  1.00  9.72           O  \nATOM    803  N   GLY A 199     -22.003  24.766  21.885  1.00  9.49           N  \nATOM    804  CA  GLY A 199     -20.705  25.346  21.562  1.00 10.98           C  \nATOM    805  C   GLY A 199     -20.920  26.284  20.387  1.00  9.91           C  \nATOM    806  O   GLY A 199     -21.278  25.822  19.312  1.00 11.42           O  \nATOM    807  N   GLY A 200     -20.734  27.579  20.598  1.00 11.45           N  \nATOM    808  CA AGLY A 200     -20.761  28.517  19.476  0.50 14.17           C  \nATOM    809  CA BGLY A 200     -20.743  28.556  19.508  0.50 14.51           C  \nATOM    810  C   GLY A 200     -19.392  28.638  18.830  1.00 15.75           C  \nATOM    811  O   GLY A 200     -18.359  28.433  19.470  1.00 14.75           O  \nATOM    812  N   GLY A 201     -19.407  29.000  17.549  1.00 17.25           N  \nATOM    813  CA  GLY A 201     -18.190  29.378  16.840  1.00 19.29           C  \nATOM    814  C   GLY A 201     -17.068  28.370  17.050  1.00 17.45           C  \nATOM    815  O   GLY A 201     -15.977  28.694  17.509  1.00 17.97           O  \nATOM    816  N   GLY A 202     -17.367  27.128  16.698  1.00 14.03           N  \nATOM    817  CA  GLY A 202     -16.451  26.012  16.905  1.00 14.44           C  \nATOM    818  C   GLY A 202     -15.121  26.272  16.225  1.00 16.19           C  \nATOM    819  O   GLY A 202     -15.064  26.816  15.117  1.00 18.80           O  \nATOM    820  N   GLY A 203     -14.064  25.871  16.896  1.00 17.50           N  \nATOM    821  CA  GLY A 203     -12.720  26.034  16.377  1.00 18.04           C  \nATOM    822  C   GLY A 203     -11.933  24.790  16.612  1.00 16.80           C  \nATOM    823  O   GLY A 203     -12.367  23.862  17.309  1.00 17.30           O  \nATOM    824  N   GLY A 204     -10.767  24.725  15.989  1.00 17.77           N  \nATOM    825  CA  GLY A 204      -9.950  23.512  16.044  1.00 20.97           C  \nATOM    826  C   GLY A 204      -9.630  23.021  17.444  1.00 17.65           C  \nATOM    827  O   GLY A 204      -9.572  21.810  17.644  1.00 19.95           O  \nATOM    828  N   GLY A 205      -9.445  23.938  18.403  1.00 18.83           N  \nATOM    829  CA  GLY A 205      -9.163  23.576  19.795  1.00 19.59           C  \nATOM    830  C   GLY A 205     -10.322  22.823  20.463  1.00 18.23           C  \nATOM    831  O   GLY A 205     -10.141  22.294  21.554  1.00 18.69           O  \nATOM    832  N   GLY A 206     -11.520  22.867  19.863  1.00 15.06           N  \nATOM    833  CA  GLY A 206     -12.707  22.197  20.414  1.00 13.11           C  \nATOM    834  C   GLY A 206     -12.840  20.738  20.024  1.00 11.29           C  \nATOM    835  O   GLY A 206     -13.749  20.000  20.465  1.00 12.01           O  \nATOM    836  N   GLY A 207     -11.969  20.288  19.133  1.00 11.09           N  \nATOM    837  CA  GLY A 207     -11.978  18.926  18.708  1.00 10.78           C  \nATOM    838  C   GLY A 207     -11.626  17.978  19.858  1.00 11.11           C  \nATOM    839  O   GLY A 207     -10.703  18.232  20.608  1.00 13.11           O  \nATOM    840  N   GLY A 208     -12.375  16.881  19.972  1.00  9.81           N  \nATOM    841  CA  GLY A 208     -12.154  15.859  21.008  1.00  9.92           C  \nATOM    842  C   GLY A 208     -13.429  15.205  21.413  1.00 10.96           C  \nATOM    843  O   GLY A 208     -14.427  15.384  20.725  1.00 12.85           O  \nATOM    844  N   GLY A 209     -13.424  14.488  22.530  1.00  8.06           N  \nATOM    845  CA  GLY A 209     -14.597  13.769  22.969  1.00  9.09           C  \nATOM    846  C   GLY A 209     -15.254  14.505  24.135  1.00  8.32           C  \nATOM    847  O   GLY A 209     -14.617  14.826  25.125  1.00  7.90           O  \nATOM    848  N   GLY A 210     -16.554  14.775  24.004  1.00  7.17           N  \nATOM    849  CA  GLY A 210     -17.349  15.428  25.055  1.00  7.08           C  \nATOM    850  C   GLY A 210     -18.091  14.374  25.867  1.00  6.91           C  \nATOM    851  O   GLY A 210     -18.807  13.528  25.320  1.00  7.43           O  \nATOM    852  N   GLY A 211     -17.924  14.431  27.177  1.00  6.72           N  \nATOM    853  CA  GLY A 211     -18.594  13.521  28.089  1.00  6.54           C  \nATOM    854  C   GLY A 211     -19.443  14.300  29.081  1.00  6.45           C  \nATOM    855  O   GLY A 211     -18.985  15.330  29.648  1.00  7.29           O  \nATOM    856  N   GLY A 212     -20.647  13.810  29.316  1.00  6.23           N  \nATOM    857  CA  GLY A 212     -21.382  14.257  30.502  1.00  6.63           C  \nATOM    858  C   GLY A 212     -21.069  13.310  31.662  1.00  6.29           C  \nATOM    859  O   GLY A 212     -20.618  12.186  31.475  1.00  7.77           O  \nATOM    860  N   GLY A 213     -21.322  13.777  32.893  1.00  6.25           N  \nATOM    861  CA AGLY A 213     -21.033  12.982  34.115  0.50  5.74           C  \nATOM    862  CA BGLY A 213     -21.058  13.005  34.052  0.50  6.13           C  \nATOM    863  C   GLY A 213     -21.963  13.492  35.174  1.00  6.35           C  \nATOM    864  O   GLY A 213     -22.220  14.698  35.268  1.00  6.45           O  \nATOM    865  N   GLY A 214     -22.396  12.579  36.003  1.00  5.84           N  \nATOM    866  CA  GLY A 214     -23.147  12.915  37.228  1.00  5.42           C  \nATOM    867  C   GLY A 214     -22.271  12.771  38.447  1.00  5.48           C  \nATOM    868  O   GLY A 214     -21.546  11.773  38.583  1.00  5.81           O  \nATOM    869  N   GLY A 215     -22.374  13.750  39.350  1.00  5.79           N  \nATOM    870  CA  GLY A 215     -21.714  13.711  40.669  1.00  5.70           C  \nATOM    871  C   GLY A 215     -22.744  13.871  41.784  1.00  6.91           C  \nATOM    872  O   GLY A 215     -22.484  14.480  42.811  1.00  7.70           O  \nATOM    873  N   GLY A 216     -23.926  13.319  41.530  1.00  6.30           N  \nATOM    874  CA  GLY A 216     -24.966  13.331  42.550  1.00  6.98           C  \nATOM    875  C   GLY A 216     -24.713  12.251  43.594  1.00  7.48           C  \nATOM    876  O   GLY A 216     -24.791  12.501  44.814  1.00  9.20           O  \nATOM    877  N   GLY A 217     -24.438  11.041  43.127  1.00  7.61           N  \nATOM    878  CA  GLY A 217     -24.324   9.941  44.047  1.00  8.22           C  \nATOM    879  C   GLY A 217     -23.366   8.866  43.518  1.00  8.23           C  \nATOM    880  O   GLY A 217     -23.216   8.662  42.327  1.00  7.92           O  \nATOM    881  N   GLY A 218     -22.702   8.214  44.453  1.00  8.40           N  \nATOM    882  CA  GLY A 218     -21.825   7.133  44.126  1.00  7.73           C  \nATOM    883  C   GLY A 218     -22.585   5.970  43.535  1.00  7.25           C  \nATOM    884  O   GLY A 218     -23.731   5.682  43.922  1.00  9.07           O  \nATOM    885  N   GLY A 219     -22.008   5.245  42.596  1.00  7.34           N  \nATOM    886  CA  GLY A 219     -20.750   5.581  41.912  1.00  6.89           C  \nATOM    887  C   GLY A 219     -20.980   6.666  40.888  1.00  6.26           C  \nATOM    888  O   GLY A 219     -22.018   6.719  40.211  1.00  6.41           O  \nATOM    889  N   GLY A 220     -20.047   7.598  40.781  1.00  5.72           N  \nATOM    890  CA  GLY A 220     -20.155   8.593  39.723  1.00  5.50           C  \nATOM    891  C   GLY A 220     -20.084   7.928  38.373  1.00  5.48           C  \nATOM    892  O   GLY A 220     -19.402   6.913  38.211  1.00  6.29           O  \nATOM    893  N   GLY A 221     -20.823   8.448  37.381  1.00  5.76           N  \nATOM    894  CA  GLY A 221     -21.007   7.808  36.096  1.00  5.88           C  \nATOM    895  C   GLY A 221     -20.924   8.838  34.974  1.00  5.34           C  \nATOM    896  O   GLY A 221     -21.367   9.984  35.126  1.00  6.23           O  \nATOM    897  N   GLY A 222     -20.363   8.385  33.865  1.00  6.51           N  \nATOM    898  CA  GLY A 222     -20.153   9.202  32.657  1.00  6.75           C  \nATOM    899  C   GLY A 222     -21.013   8.669  31.516  1.00  6.26           C  \nATOM    900  O   GLY A 222     -21.317   7.475  31.441  1.00  7.09           O  \nATOM    901  N   GLY A 223     -21.418   9.567  30.634  1.00  5.92           N  \nATOM    902  CA  GLY A 223     -21.983   9.184  29.367  1.00  6.79           C  \nATOM    903  C   GLY A 223     -20.965   8.517  28.476  1.00  7.82           C  \nATOM    904  O   GLY A 223     -19.768   8.515  28.766  1.00  7.63           O  \nATOM    905  N   GLY A 224     -21.437   7.999  27.349  1.00  8.75           N  \nATOM    906  CA  GLY A 224     -20.557   7.244  26.457  1.00  9.92           C  \nATOM    907  C   GLY A 224     -19.669   8.108  25.565  1.00  9.93           C  \nATOM    908  O   GLY A 224     -18.748   7.582  24.923  1.00 11.22           O  \nATOM    909  N   GLY A 225     -19.910   9.415  25.565  1.00  7.76           N  \nATOM    910  CA  GLY A 225     -19.092  10.363  24.843  1.00  8.82           C  \nATOM    911  C   GLY A 225     -19.620  10.679  23.455  1.00  8.12           C  \nATOM    912  O   GLY A 225     -20.135   9.785  22.782  1.00 10.37           O  \nATOM    913  N   GLY A 226     -19.395  11.920  23.032  1.00  7.66           N  \nATOM    914  CA  GLY A 226     -19.687  12.346  21.687  1.00  8.22           C  \nATOM    915  C   GLY A 226     -18.429  12.998  21.123  1.00  8.16           C  \nATOM    916  O   GLY A 226     -18.000  14.025  21.652  1.00  8.75           O  \nATOM    917  N   GLY A 227     -17.858  12.406  20.068  1.00  9.41           N  \nATOM    918  CA  GLY A 227     -16.645  12.900  19.433  1.00 10.79           C  \nATOM    919  C   GLY A 227     -16.957  14.033  18.512  1.00  9.61           C  \nATOM    920  O   GLY A 227     -17.792  13.837  17.599  1.00 14.59           O  \nATOM    921  N   GLY A 228     -16.353  15.193  18.698  1.00  9.93           N  \nATOM    922  CA  GLY A 228     -16.484  16.310  17.779  1.00 10.49           C  \nATOM    923  C   GLY A 228     -15.269  16.317  16.850  1.00 12.73           C  \nATOM    924  O   GLY A 228     -14.120  16.285  17.315  1.00 12.77           O  \nATOM    925  N   GLY A 229     -15.542  16.368  15.543  1.00 13.88           N  \nATOM    926  CA AGLY A 229     -14.480  16.468  14.545  0.50 16.39           C  \nATOM    927  CA BGLY A 229     -14.468  16.496  14.550  0.50 15.87           C  \nATOM    928  C   GLY A 229     -14.835  17.523  13.514  1.00 17.57           C  \nATOM    929  O   GLY A 229     -16.008  17.812  13.338  1.00 19.35           O  \nATOM    930  N   GLY A 230     -13.813  18.080  12.846  1.00 19.26           N  \nATOM    931  CA  GLY A 230     -14.030  19.013  11.733  1.00 19.72           C  \nATOM    932  C   GLY A 230     -14.237  18.255  10.403  1.00 26.79           C  \nATOM    933  O   GLY A 230     -13.614  17.238  10.118  1.00 31.32           O  \nATOM    934  N   GLY A 231     -15.191  18.709   9.616  1.00 27.15           N  \nATOM    935  CA  GLY A 231     -15.359  18.162   8.247  1.00 28.70           C  \nATOM    936  C   GLY A 231     -14.137  18.379   7.347  1.00 28.35           C  \nATOM    937  O   GLY A 231     -13.501  19.450   7.473  1.00 29.27           O  \nTER     938      GLY A 231                      \nEND   \n"
                                        ]
                                        }
                               },
               
                "scimilarity_gene_order": {"data":{
                                                    "dataframe_split": {
                                                        "columns": [
                                                        "input"
                                                        ],
                                                        "data": [
                                                        [
                                                            "get_gene_order"
                                                        ]
                                                        ]
                                                    }
                                                    }
                                        },
               "scimilarity_get_embedding": {"data":{
                                                "dataframe_split": {
                                                            "columns": [
                                                            "celltype_sample",
                                                            "celltype_sample_obs"
                                                            ],
                                                            "data": [
                                                            [
                                                                "{\"columns\":[\"celltype_subsample\"],\"index\":[\"124332\"],\"data\":[[[0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.7197530222,0.0,1.0139774031,1.0139774031,0.7197530222,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,1.6516739778,1.7168584863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.0,0.0,2.7571553781,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,1.2410258927,0.0,0.3010734745,0.3010734745,1.2410258927,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.5321666255,0.0,3.2284417293,0.0,0.0,2.2060850259,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,1.2410258927,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,1.0139774031,0.0,0.7197530222,0.0,0.5321666255,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,1.5069812313,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,1.8357172412,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,2.0379505562,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.0,1.4259422959,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,1.1339317312,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,2.2440452892,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,1.2410258927,0.5321666255,0.7197530222,0.0,0.3010734745,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.8902370509,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.8776474034,0.5321666255,0.0,0.7197530222,0.5321666255,0.5321666255,0.8776474034,1.0139774031,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.8776474034,0.0,0.7197530222,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,1.7168584863,1.5069812313,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.2410258927,0.3010734745,0.5321666255,0.0,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,1.2410258927,0.0,0.0,2.0827077633,0.3010734745,0.3010734745,0.0,1.5069812313,0.0,0.0,0.5321666255,1.6516739778,1.2410258927,1.1339317312,0.0,0.0,0.3010734745,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.7197530222,0.5321666255,0.5321666255,0.0,0.5321666255,0.0,0.0,1.3377522779,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,2.2060850259,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,1.5819423237,0.3010734745,0.5321666255,0.3010734745,0.7197530222,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.8776474034,0.0,1.991095899,0.0,0.5321666255,0.8776474034,0.0,0.3010734745,0.7197530222,0.5321666255,0.7197530222,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.8776474034,0.7197530222,0.0,0.0,1.2410258927,0.3010734745,0.0,1.3377522779,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.8776474034,0.0,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,1.0139774031,0.0,0.5321666255,0.0,0.3010734745,0.0,0.8776474034,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,1.6516739778,0.5321666255,0.7197530222,0.0,0.0,0.0,1.2410258927,0.0,1.3377522779,1.7780527507,0.7197530222,0.0,0.5321666255,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.3377522779,0.3010734745,0.5321666255,1.4259422959,0.5321666255,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,2.1255472866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,2.1666267281,0.0,0.8776474034,0.3010734745,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.5819423237,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,1.8357172412,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.5321666255,1.0139774031,1.1339317312,0.7197530222,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.7197530222,0.0,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.8776474034,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.0,0.0,1.2410258927,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.8776474034,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.3010734745,0.8776474034,1.1339317312,0.0,1.1339317312,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,1.3377522779,0.0,0.5321666255,0.0,0.7197530222,1.5819423237,0.7197530222,0.0,1.0139774031,0.0,0.0,0.5321666255,0.0,0.7197530222,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,1.5069812313,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,2.2806171158,0.0,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,3.662056192,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,1.4259422959,1.3377522779,0.0,0.5321666255,0.5321666255,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.8776474034,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,1.5819423237,1.1339317312,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,1.8902370509,0.0,0.7197530222,0.7197530222,0.0,1.1339317312,0.0,0.8776474034,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.8776474034,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.8902370509,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,2.9400472367,0.0,0.0,0.0,0.0,1.8902370509,0.0,0.7197530222,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.7197530222,1.2410258927,0.0,0.0,0.3010734745,0.0,0.0,2.0379505562,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,1.4259422959,0.0,0.0,2.0827077633,0.5321666255,0.0,1.8902370509,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,1.5819423237,0.8776474034,0.0,2.0379505562,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0379505562,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,3.5773766084,0.0,0.3010734745,0.0,1.8902370509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.4259422959,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.7197530222,0.7197530222,0.3010734745,0.0,1.1339317312,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,1.1339317312,0.0,0.7197530222,0.0,0.3010734745,0.0,0.5321666255,0.7197530222,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,1.3377522779,0.3010734745,0.0,2.0827077633,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3499774106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.6516739778,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.3377522779,0.0,0.0,1.1339317312,0.0,1.1339317312,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,1.7780527507,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,1.0139774031,0.3010734745,0.3010734745,1.1339317312,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.7197530222,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,1.2410258927,0.7197530222,1.991095899,0.0,0.0,0.0,0.0,1.4259422959,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,2.5606704651,1.2410258927,0.5321666255,0.5321666255,0.0,0.0,0.5321666255,0.0,0.7197530222,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.2410258927,0.0,0.5321666255,0.7197530222,0.0,0.0,0.7197530222,0.3010734745,0.0,0.3010734745,1.1339317312,0.0,0.3010734745,2.3829331223,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.8776474034,0.0,0.0,0.0,0.5321666255,0.5321666255,0.5321666255,0.0,0.7197530222,1.7780527507,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.3010734745,0.3010734745,1.5819423237,1.3377522779,0.7197530222,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.7197530222,1.1339317312,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8902370509,0.0,1.6516739778,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,1.9419374998,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.7168584863,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,2.2440452892,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,1.4259422959,1.6516739778,0.0,0.0,0.5321666255,0.3010734745,0.7197530222,0.0,1.4259422959,0.7197530222,0.3010734745,0.0,0.7197530222,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.8776474034,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,1.0139774031,0.8776474034,1.0139774031,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,1.8902370509,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,1.0139774031,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9419374998,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,1.0139774031,0.5321666255,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,1.991095899,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,1.7168584863,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1255472866,0.3010734745,0.5321666255,0.0,0.7197530222,0.0,0.7197530222,0.3010734745,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.0,0.0,1.4259422959,0.5321666255,1.0139774031,0.3010734745,1.3377522779,0.0,0.0,0.0,1.8357172412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,1.6516739778,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.3010734745,0.7197530222,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,2.4457550249,0.0,0.0,1.7780527507,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.0,0.0,4.0281238103,3.5474779724,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,3.7809948133,0.8776474034,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.8357172412,0.0,2.2440452892,0.0,2.1666267281,2.4148373165,2.8827211208,0.0,0.0,0.7197530222,2.4457550249,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,1.3377522779,0.8776474034,0.0,0.3010734745,0.0,0.3010734745,0.0,0.8776474034,0.8776474034,0.0,0.3010734745,2.2806171158,0.5321666255,0.0,1.6516739778,0.3010734745,0.3010734745,1.1339317312,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,1.0139774031,1.3377522779,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.0139774031,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,1.0139774031,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.5321666255,0.3010734745,0.3010734745,0.5321666255,0.0,1.5069812313,0.0,1.1339317312,1.2410258927,1.1339317312,0.0,1.2410258927,0.0,1.991095899,1.1339317312,1.5069812313,1.0139774031,0.5321666255,0.0,1.2410258927,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.7197530222,1.1339317312,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,1.0139774031,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.1339317312,0.3010734745,0.3010734745,1.8902370509,0.0,0.0,0.0,1.2410258927,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,3.3086487696,0.8776474034,0.0,0.0,0.3010734745,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,0.0,0.7197530222,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.7197530222,0.3010734745,1.1339317312,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,1.8902370509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,1.2410258927,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,2.3829331223,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.8776474034,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.7197530222,0.3010734745,0.0,0.8776474034,0.0,1.2410258927,2.4457550249,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.8776474034,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,1.2410258927,0.3010734745,0.0,0.0,0.0,3.2826213916,0.3010734745,0.3010734745,1.2410258927,0.3010734745,0.3010734745,1.1339317312,2.1255472866,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.5069812313,1.1339317312,0.0,0.0,0.0,0.0,1.5819423237,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.7197530222,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.8776474034,0.3010734745,1.1339317312,0.0,0.0,0.3010734745,0.7197530222,0.7197530222,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,2.5606704651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,1.3377522779,0.0,0.0,1.0139774031,0.0,0.3010734745,0.3010734745,1.1339317312,1.0139774031,1.0139774031,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.3010734745,0.5321666255,0.8776474034,0.0,0.7197530222,0.0,1.8357172412,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,1.5069812313,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.5321666255,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.5321666255,0.5321666255,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,2.4457550249,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.7197530222,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,1.5069812313,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.2410258927,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.3010734745,0.0,0.0,1.0139774031,0.0,0.5321666255,1.991095899,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,1.0139774031,0.3010734745,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.8776474034,0.3010734745,0.3010734745,0.0,0.0,0.8776474034,0.3010734745,0.3010734745,1.5069812313,0.3010734745,0.8776474034,0.7197530222,0.0,0.0,1.1339317312,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.5321666255,2.0379505562,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,1.3377522779,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.2410258927,0.0,1.1339317312,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.0,1.1339317312,0.0,0.0,0.0,1.3377522779,0.3010734745,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.7197530222,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.3010734745,1.1339317312,0.0,0.3010734745,0.0,1.5069812313,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.7197530222,0.3010734745,0.0,0.0,1.1339317312,0.0,0.7197530222,0.7197530222,0.3010734745,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9419374998,2.3499774106,0.0,1.1339317312,0.0,0.0,0.0,1.1339317312,0.3010734745,0.8776474034,0.3010734745,1.9419374998,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,3.0117027811,0.0,0.8776474034,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.5321666255,0.5321666255,0.5321666255,0.5321666255,0.3010734745,1.0139774031,0.7197530222,1.4259422959,0.0,0.7197530222,0.0,0.5321666255,0.7197530222,0.7197530222,0.5321666255,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,1.0139774031,0.3010734745,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.2410258927,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,2.5606704651,0.0,0.3010734745,0.0,0.0,1.5069812313,0.0,0.0,0.3010734745,2.4457550249,2.315898499,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.8776474034,0.0,0.0,2.2440452892,0.8776474034,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,1.9419374998,0.0,0.3010734745,0.0,2.2806171158,0.0,1.1339317312,0.0,0.0,0.0,0.8776474034,0.0,0.8776474034,0.5321666255,0.8776474034,0.0,0.0,0.8776474034,0.5321666255,0.3010734745,0.0,0.7197530222,0.5321666255,0.8776474034,0.8776474034,0.0,0.0,0.0,1.0139774031,1.3377522779,0.8776474034,0.7197530222,0.8776474034,0.5321666255,1.1339317312,1.0139774031,1.0139774031,0.5321666255,0.0,1.2410258927,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,1.1339317312,0.5321666255,0.0,0.8776474034,1.3377522779,0.0,0.5321666255,0.8776474034,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,1.991095899,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,1.1339317312,1.4259422959,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,2.5606704651,0.3010734745,0.3010734745,0.0,1.6516739778,1.3377522779,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,1.2410258927,0.0,0.0,0.3010734745,0.5321666255,0.8776474034,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.8776474034,0.3010734745,0.3010734745,0.0,1.0139774031,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,1.5069812313,0.0,0.3010734745,1.9419374998,0.8776474034,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.2410258927,0.7197530222,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,1.1339317312,1.1339317312,0.3010734745,0.0,0.8776474034,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.1339317312,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,1.0139774031,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,1.2410258927,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,1.3377522779,0.0,0.0,0.0,1.0139774031,0.5321666255,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.8776474034,1.0139774031,0.8776474034,0.0,0.0,0.0,0.0,0.7197530222,0.0,1.6516739778,0.0,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.3010734745,0.3010734745,1.0139774031,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,1.0139774031,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.5321666255,0.3010734745,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.8776474034,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,1.0139774031,0.0,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,2.1255472866,0.3010734745,0.3010734745,0.0,0.5321666255,0.8776474034,0.0,1.3377522779,0.0,0.3010734745,0.3010734745,1.3377522779,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,1.7168584863,0.3010734745,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.991095899,0.8776474034,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.3010734745,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,1.6516739778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0379505562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.3377522779,0.5321666255,1.6516739778,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.1339317312,0.3010734745,0.0,2.4148373165,0.0,0.3010734745,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,4.1348306493,0.0,0.0,1.2410258927,0.3010734745,1.0139774031,0.0,0.0,0.0,1.0139774031,1.8902370509,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,3.5373097548,1.7780527507,0.8776474034,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,2.2440452892,0.0,0.0,0.0,2.6389444026,1.7168584863,0.0,0.0,2.8628581139,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.6516739778,0.3010734745,0.0,0.0,0.0,0.0,0.0,3.0456927814,1.1339317312,0.3010734745,0.3010734745,1.1339317312,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.8776474034,0.0,1.3377522779,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.3010734745,0.3010734745,0.7197530222,0.8776474034,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.0,1.991095899,0.3010734745,1.2410258927,0.0,0.0,0.0,0.8776474034,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,2.2806171158,1.5819423237,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,1.9419374998,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,1.0139774031,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,2.1666267281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0379505562,0.0,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,1.2410258927,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.8776474034,0.7197530222,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,1.5819423237,0.0,0.0,1.1339317312,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.2410258927,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.8776474034,1.2410258927,0.0,0.0,0.8776474034,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.7197530222,0.7197530222,0.3010734745,0.0,1.2410258927,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.3010734745,0.0,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,1.2410258927,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,1.4259422959,0.5321666255,0.5321666255,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,1.4259422959,0.0,0.8776474034,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,1.2410258927,0.0,0.0,1.5819423237,0.0,0.0,0.3010734745,0.7197530222,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,1.7780527507,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.5321666255,0.0,0.8776474034,0.0,1.5819423237,0.3010734745,0.0,0.0,0.7197530222,0.0,0.7197530222,0.5321666255,0.0,0.7197530222,0.7197530222,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.0,0.7197530222,1.1339317312,0.0,0.3010734745,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.5321666255,0.0,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.0,1.2410258927,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.3010734745,0.7197530222,0.5321666255,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,1.0139774031,0.0,0.0,1.3377522779,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,1.6516739778,0.3010734745,0.0,1.1339317312,0.0,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,1.6516739778,1.8902370509,1.991095899,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.5321666255,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.7197530222,1.1339317312,0.0,0.3010734745,0.0,0.0,0.0,2.0379505562,0.8776474034,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.8776474034,0.7197530222,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,1.3377522779,1.0139774031,0.5321666255,1.5069812313,0.0,1.1339317312,1.0139774031,0.7197530222,1.2410258927,0.0,0.3010734745,0.7197530222,0.0,0.8776474034,1.3377522779,1.0139774031,1.0139774031,0.8776474034,0.0,0.8776474034,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.7197530222,0.5321666255,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.5321666255,0.5321666255,0.5321666255,0.0,0.0,0.8776474034,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4259422959,1.6516739778,1.6516739778,1.0139774031,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.8776474034,0.0,1.3377522779,0.0,1.0139774031,1.0139774031,2.2440452892,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,1.4259422959,0.0,1.3377522779,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.8776474034,0.0,2.5606704651,0.0,0.5321666255,0.3010734745,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,2.4757454273,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,1.1339317312,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7168584863,2.1666267281,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,1.3377522779,2.2806171158,1.3377522779,3.8580469251,1.5819423237,2.9400472367,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.3377522779,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,3.2558984451,0.3010734745,1.2410258927,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.0,0.3010734745,0.3010734745,0.7197530222,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,1.7168584863,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,1.5069812313,0.7197530222,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,0.7197530222,0.5321666255,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.9419374998,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.5321666255,0.0,0.5321666255,1.7780527507,1.2410258927,1.2410258927,0.0,0.0,0.5321666255,0.5321666255,0.5321666255,0.0,0.3010734745,0.7197530222,1.991095899,0.0,0.5321666255,0.0,0.0,1.0139774031,0.0,1.6516739778,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.3010734745,0.0,0.3010734745,2.0379505562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.0,1.7780527507,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.5321666255,0.0,1.3377522779,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.7197530222,0.3010734745,0.7197530222,0.0,1.1339317312,0.0,0.0,0.0,0.3010734745,0.3010734745,2.1255472866,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,1.5069812313,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,1.0139774031,2.1666267281,3.2422643182,1.5069812313,0.5321666255,0.0,0.0,0.5321666255,1.5819423237,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,1.2410258927,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.1339317312,0.0,0.7197530222,0.0,0.8776474034,0.0,0.5321666255,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,2.2060850259,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,1.1339317312,0.3010734745,0.0,1.8357172412,1.2410258927,0.3010734745,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.0,1.1339317312,0.0,1.5069812313,0.7197530222,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,2.5874483359,0.3010734745,0.0,1.1339317312,0.5321666255,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.8776474034,0.0,0.3010734745,0.0,0.0,1.1339317312,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,1.8357172412,0.0,0.7197530222,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,1.3377522779,0.3010734745,0.5321666255,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,3.1711576312,0.0,1.1339317312,0.0,0.0,0.7197530222,1.1339317312,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.7197530222,0.0,1.0139774031,0.0,0.7197530222,0.8776474034,0.0,0.3010734745,1.1339317312,0.0,0.0,0.5321666255,0.0,0.0,1.0139774031,1.0139774031,0.0,1.1339317312,0.0,0.5321666255,1.0139774031,0.5321666255,1.0139774031,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.6516739778,0.0,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.3010734745,1.0139774031,2.0827077633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,1.4259422959,0.8776474034,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,3.8654354024,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,1.3377522779,0.3010734745,0.0,0.0,0.8776474034,0.5321666255,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.0,0.0,2.8628581139,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,2.711533792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.8776474034,0.0,0.3010734745,0.3010734745,1.0139774031,0.3010734745,1.3377522779,0.5321666255,0.0,0.0,0.8776474034,0.0,0.0,2.5874483359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,1.3377522779,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.0,0.0,0.0,0.8776474034,1.7780527507,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.8776474034,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,1.0139774031,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,1.8357172412,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,1.2410258927,0.0,0.3010734745,0.3010734745,0.7197530222,0.8776474034,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5048625323,1.3377522779,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.7197530222,0.8776474034,0.0,2.2060850259,0.0,0.3010734745,0.3010734745,1.5819423237,1.5819423237,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,1.2410258927,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,1.2410258927,0.0,2.1666267281,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,5.1682108819,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.7197530222,0.0,0.0,1.6516739778,1.0139774031,0.0,0.0,0.8776474034,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,1.0139774031,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,1.0139774031,0.0,0.0,0.3010734745,0.3010734745,1.3377522779,0.0,0.5321666255,0.5321666255,0.5321666255,0.3010734745,1.7168584863,0.0,0.3010734745,1.2410258927,1.0139774031,1.0139774031,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,1.1339317312,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.5321666255,0.0,0.5321666255,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,1.2410258927,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.0,0.8776474034,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,1.4259422959,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.3010734745,0.3010734745,1.6516739778,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,1.0139774031,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,1.2410258927,0.0,0.7197530222,0.7197530222,1.1339317312,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.7197530222,0.5321666255,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,1.4259422959,1.7780527507,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.8776474034,0.5321666255,0.3010734745,0.0,0.5321666255,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.7197530222,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.4259422959,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.5819423237,0.0,0.3010734745,1.5819423237,0.0,0.5321666255,1.1339317312,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,1.5069812313,0.3010734745,0.0,1.5819423237,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,1.3377522779,0.5321666255,0.0,0.0,0.0,0.5321666255,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.3010734745,1.0139774031,0.3010734745,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.8902370509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.4259422959,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,3.3086487696,0.0,0.0,1.4259422959,0.0,0.0,0.3010734745,1.1339317312,2.6389444026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,1.2410258927,1.1339317312,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,1.4259422959,0.5321666255,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,1.0139774031,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.7197530222,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,1.1339317312,0.8776474034,0.5321666255,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,1.2410258927,0.5321666255,0.3010734745,0.8776474034,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.0139774031,0.3010734745,0.5321666255,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.7197530222,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.5321666255,0.3010734745,0.7197530222,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,3.4630810996,2.9400472367,3.7648454391,3.2957197563,0.0,3.2558984451,2.8628581139,3.2144253954,2.8425925586,0.0,3.3214127549,1.991095899,1.1339317312,0.0,0.0,1.7168584863,0.5321666255,0.0,0.0,0.0,0.3010734745,2.2806171158,3.4408199922,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.0139774031,0.5321666255,0.0,0.0,1.1339317312,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,1.1339317312,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.3010734745,0.5321666255,0.5321666255,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,1.3377522779,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,1.4259422959,0.7197530222,1.3377522779,1.2410258927,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.0,0.0,1.6516739778,1.2410258927,0.0,0.0,0.0,0.3010734745,2.7571553781,0.0,0.0,2.2060850259,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,1.1339317312,0.7197530222,0.0,0.3010734745,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,1.2410258927,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.9419374998,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.2410258927,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,1.0139774031,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,1.2410258927,1.4259422959,1.1339317312,1.2410258927,0.3010734745,1.1339317312,0.0,0.0,1.1339317312,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,1.1339317312,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,1.0139774031,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.0,0.3010734745,1.2410258927,0.3010734745,0.8776474034,1.8357172412,0.0,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,1.6516739778,0.7197530222,0.7197530222,1.1339317312,0.0,0.8776474034,1.2410258927,0.5321666255,0.5321666255,1.0139774031,0.0,0.7197530222,0.5321666255,0.0,1.1339317312,0.5321666255,0.0,0.7197530222,1.1339317312,1.5069812313,1.1339317312,0.0,0.5321666255,0.5321666255,0.3010734745,0.3010734745,0.3010734745,2.8219078006,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.3377522779,1.2410258927,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.8776474034,0.3010734745,0.3010734745,0.7197530222,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,1.3377522779,0.7197530222,1.7168584863,0.0,0.5321666255,0.3010734745,0.0,0.0,1.1339317312,0.0,0.0,0.3010734745,0.0,1.3377522779,1.4259422959,0.5321666255,0.8776474034,0.3010734745,0.0,0.8776474034,0.5321666255,0.0,0.0,0.7197530222,1.0139774031,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,1.991095899,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.5321666255,0.0,0.3010734745,0.8776474034,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.7197530222,0.8776474034,0.7197530222,0.5321666255,0.3010734745,1.5069812313,1.5819423237,1.7168584863,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,1.7168584863,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,1.4259422959,0.3010734745,1.0139774031,1.2410258927,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,1.2410258927,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.5321666255,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.7197530222,0.0,0.8776474034,1.6516739778,0.5321666255,0.3010734745,0.5321666255,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,1.1339317312,0.0,0.0,0.3010734745,1.0139774031,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.0,1.5069812313,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.7168584863,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,1.7168584863,0.0,0.0,0.8776474034,0.0,0.0,0.8776474034,0.0,0.0,1.0139774031,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.8776474034,1.5819423237,1.1339317312,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.5321666255,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.8776474034,0.5321666255,0.5321666255,0.0,0.0,0.0,0.8776474034,0.0,0.7197530222,0.5321666255,1.5069812313,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.3010734745,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.8776474034,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,2.4457550249,0.3010734745,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,1.1339317312,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.3010734745,1.2410258927,0.0,0.0,0.8776474034,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.3010734745,0.0,1.2410258927,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,0.8776474034,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.5069812313,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.5321666255,0.5321666255,0.3010734745,0.8776474034,0.0,0.3010734745,0.5321666255,0.0,0.0,1.991095899,0.0,0.0,0.0,0.0,0.0,0.0,1.8902370509,0.0,0.0,1.6516739778,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.5321666255,0.0,1.0139774031,0.0,0.0,0.0,0.7197530222,0.5321666255,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.8776474034,0.3010734745,0.3010734745,0.0,0.7197530222,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.0,0.5321666255,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.0,0.5321666255,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,1.0139774031,0.3010734745,1.1339317312,1.2410258927,0.3010734745,1.3377522779,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.7197530222,1.1339317312,0.3010734745,0.5321666255,0.0,1.5819423237,0.3010734745,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,2.1666267281,0.0,0.0,0.3010734745,0.7197530222,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,1.2410258927,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,1.9419374998,0.3010734745,0.0,0.0,1.5819423237,0.0,0.3010734745,0.0,0.3010734745,1.6516739778,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.3010734745,0.3010734745,1.0139774031,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.0139774031,0.3010734745,0.3010734745,1.0139774031,0.3010734745,1.1339317312,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.7197530222,0.7197530222,0.3010734745,1.0139774031,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.8776474034,0.5321666255,0.3010734745,0.3010734745,0.0,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,1.2410258927,0.0,0.0,0.7197530222,0.8776474034,1.4259422959,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.8776474034,0.0,0.0,0.8776474034,0.5321666255,0.0,0.0,1.1339317312,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.5321666255,1.7168584863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.7197530222,0.0,0.0,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,1.2410258927,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.0,0.8776474034,0.0,0.3010734745,0.0,0.7197530222,1.6516739778,0.5321666255,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,1.0139774031,0.3010734745,0.0,2.1255472866,1.0139774031,1.7168584863,0.0,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,1.1339317312,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.0,0.7197530222,2.2806171158,0.3010734745,2.0827077633,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,1.5819423237,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.5321666255,1.7168584863,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.7197530222,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.8776474034,0.0,1.1339317312,0.8776474034,1.5069812313,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,3.3214127549,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.8776474034,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,2.1255472866,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.8776474034,0.0,1.1339317312,1.3377522779,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.3010734745,0.3010734745,0.7197530222,0.3010734745,0.0,0.3010734745,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,0.8776474034,0.3010734745,0.0,1.4259422959,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,2.6389444026,1.5069812313,0.3010734745,0.3010734745,1.0139774031,0.3010734745,0.5321666255,0.3010734745,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,1.2410258927,0.0,0.3010734745,1.0139774031,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,1.1339317312,0.0,0.3010734745,0.8776474034,0.0,0.5321666255,0.0,0.0,0.7197530222,0.0,0.7197530222,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,1.2410258927,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.8776474034,0.0,0.5321666255,1.5069812313,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,1.5069812313,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.7197530222,0.3010734745,0.8776474034,0.0,0.0,0.8776474034,0.0,0.7197530222,0.0,2.711533792,0.0,0.0,0.7197530222,1.2410258927,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.8776474034,0.3010734745,0.0,1.0139774031,0.0,0.8776474034,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.8776474034,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,1.4259422959,0.0,1.1339317312,0.0,0.0,0.0,0.7197530222,1.0139774031,1.0139774031,0.7197530222,0.8776474034,0.3010734745,0.0,0.5321666255,0.0,1.1339317312,0.7197530222,0.0,0.7197530222,0.5321666255,0.7197530222,0.7197530222,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.8776474034,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.5321666255,0.7197530222,0.5321666255,0.3010734745,1.2410258927,0.3010734745,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.7197530222,0.5321666255,1.0139774031,0.3010734745,1.2410258927,0.0,0.3010734745,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,2.7346047287,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,1.2410258927,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.5819423237,0.3010734745,0.0,0.8776474034,3.3214127549,1.2410258927,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.5321666255,0.8776474034,0.0,2.315898499,0.0,0.0,0.8776474034,1.0139774031,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,1.3377522779,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,1.7168584863,0.0,0.3010734745,0.0,1.3377522779,1.0139774031,0.0,0.0,0.0,0.0,0.0,1.5069812313,1.1339317312,0.5321666255,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.7197530222,0.8776474034,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.4259422959,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.5321666255,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.3010734745,1.0139774031,0.7197530222,0.3010734745,0.0,0.3010734745,0.3010734745,1.991095899,0.0,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.5321666255,0.0,0.0,2.2806171158,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.5321666255,0.7197530222,0.7197530222,0.0,0.7197530222,0.0,0.0,1.8357172412,0.0,0.0,0.3010734745,0.0,0.3010734745,1.4259422959,0.3010734745,0.3010734745,0.0,0.3010734745,1.3377522779,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,1.7780527507,0.3010734745,0.0,0.0,2.5874483359,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.8776474034,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.7197530222,0.0,1.0139774031,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,1.1339317312,0.7197530222,0.0,1.0139774031,0.8776474034,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.7197530222,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,1.5819423237,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.5321666255,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.3377522779,0.0,0.0,0.0,0.0,0.5321666255,1.0139774031,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,3.0456927814,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.3377522779,0.3010734745,0.0,0.0,1.4259422959,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,1.5819423237,0.0,0.5321666255,1.4259422959,0.3010734745,1.5069812313,0.0,1.1339317312,0.0,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,1.5819423237,0.0,1.5069812313,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,1.7168584863,0.7197530222,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,1.6516739778,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.8776474034,1.1339317312,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,1.2410258927,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5606704651,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,0.7197530222,0.0,0.5321666255,0.0,0.3010734745,0.0,1.0139774031,0.5321666255,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.7168584863,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,1.6516739778,1.7168584863,0.3010734745,0.0,1.5069812313,1.3377522779,0.0,0.0,0.3010734745,0.5321666255,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.9419374998,0.5321666255,0.8776474034,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.8776474034,1.0139774031,0.0,0.0,0.0,0.0,1.7780527507,0.0,1.0139774031,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.7197530222,0.0,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,1.1339317312,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.8776474034,0.7197530222,0.0,0.5321666255,0.0,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,1.1339317312,2.6389444026,0.5321666255,0.0,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,1.991095899,1.7780527507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.123521286,2.80078613,0.0,2.9400472367,2.8425925586,3.3464621256,3.3708993286,2.3499774106,2.9021972606,0.0,2.5606704651,2.4148373165,2.80078613,2.687917992,3.0456927814,1.9419374998,0.8776474034,2.0379505562,2.1255472866,2.6389444026,0.0,0.7197530222,2.0827077633,2.8628581139,3.0946050759,2.3499774106,2.9942644834,1.3377522779,2.5048625323,3.1259327692,3.3708993286,0.0,2.5331557639,2.687917992,2.5606704651,1.9419374998,0.3010734745,1.2410258927,1.8357172412,2.80078613,2.2060850259,2.9584482049,0.0,0.0,2.1255472866,2.0827077633,2.4757454273,2.5606704651,2.9942644834,2.6389444026,0.5321666255,2.5874483359,2.6637309689,2.2440452892,3.3340158721,2.5606704651,0.5321666255,1.2410258927,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,2.4148373165,2.4148373165,2.8219078006,2.5048625323,3.2144253954,3.1103915956,2.80078613,2.2806171158,2.5331557639,3.3947535722,2.7571553781,0.5321666255,3.1563087767,2.4757454273,1.991095899,1.2410258927,2.8628581139,2.2806171158,2.3829331223,3.4180520231,2.9213013155,1.2410258927,2.711533792,2.6389444026,2.9584482049,2.8425925586,3.2144253954,0.0,0.0,1.991095899,3.3587553723,0.3010734745,0.3010734745,0.0,0.8776474034,0.0,0.5321666255,0.0,0.3010734745,0.0,0.5321666255,0.0,2.315898499,2.9765166863,2.9021972606,2.1255472866,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,1.3377522779,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.7197530222,0.7197530222,0.5321666255,1.1339317312,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,1.0139774031,1.0139774031,1.2410258927,0.8776474034,0.0,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,1.7168584863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.8776474034,1.2410258927,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,2.4148373165,1.3377522779,1.0139774031,0.0,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,1.6516739778,0.3010734745,0.0,1.7780527507,1.9419374998,0.0,1.3377522779,0.0,0.5321666255,0.3010734745,0.0,2.5874483359,0.0,3.4520124892,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,1.3377522779,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,1.5819423237,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.7197530222,0.5321666255,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,2.2806171158,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.3010734745,1.2410258927,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,1.3377522779,0.7197530222,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.8776474034,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.5321666255,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,1.0139774031,0.0,0.3010734745,0.0,0.5321666255,0.7197530222,0.0,0.3010734745,0.8776474034,1.6516739778,0.0,0.0,0.0,1.5819423237,1.1339317312,1.7168584863,0.0,0.8776474034,0.7197530222,0.8776474034,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.7197530222,0.0,0.0,0.8776474034,1.5069812313,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,1.3377522779,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.8776474034,1.0139774031,0.5321666255,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,2.5606704651,0.3010734745,0.0,0.5321666255,0.3010734745,0.7197530222,0.0,1.2410258927,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,2.2440452892,0.0,0.7197530222,1.4259422959,0.0,0.0,1.0139774031,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,1.5819423237,0.3010734745,0.0,0.0,1.1339317312,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.0,0.7197530222,0.0,0.0,0.5321666255,0.3010734745,0.8776474034,0.0,0.3010734745,0.5321666255,1.1339317312,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.9419374998,0.0,0.3010734745,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.8776474034,0.5321666255,0.3010734745,1.8357172412,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.4259422959,0.0,1.1339317312,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,1.2410258927,0.0,0.5321666255,0.0,0.0,0.8776474034,0.3010734745,0.3010734745,1.6516739778,0.0,1.7168584863,0.0,0.0,1.6516739778,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7168584863,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,1.7168584863,1.991095899,0.0,1.0139774031,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.4259422959,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,1.2410258927,0.0,1.3377522779,0.0,0.0,0.0,0.0,1.6516739778,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.8776474034,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,1.3377522779,0.0,0.0,0.5321666255,0.0,0.3010734745,0.7197530222,0.5321666255,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.8776474034,0.0,0.0,0.3010734745,1.5069812313,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,1.9419374998,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,1.4259422959,0.5321666255,0.0,0.0,0.0,1.2410258927,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.8776474034,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,1.0139774031,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.8776474034,0.0,1.2410258927,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,1.1339317312,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,1.0139774031,0.0,1.0139774031,0.8776474034,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.3010734745,1.2410258927,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,1.4259422959,0.8776474034,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.0,0.5321666255,0.3010734745,1.2410258927,1.0139774031,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.3010734745,0.8776474034,0.3010734745,0.0,1.3377522779,0.5321666255,0.7197530222,0.8776474034,0.8776474034,1.3377522779,0.7197530222,0.5321666255,0.5321666255,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.8776474034,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.7197530222,0.3010734745,0.8776474034,1.7168584863,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.5321666255,1.991095899,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,2.1666267281,0.0,0.0,0.0,0.0,1.0139774031,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,1.3377522779,1.1339317312,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8357172412,2.1666267281,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9942644834,1.0139774031,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,1.7168584863,0.3010734745,0.0,1.1339317312,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,1.0139774031,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,1.4259422959,0.3010734745,1.2410258927,0.8776474034,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.0,0.8776474034,0.0,0.3010734745,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,1.4259422959,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.3010734745,1.4259422959,1.4259422959,0.0,0.5321666255,1.2410258927,0.7197530222,0.0,0.0,0.0,1.7168584863,0.0,0.0,0.3010734745,0.0,2.1255472866,0.0,0.0,0.5321666255,0.0,0.5321666255,0.7197530222,0.5321666255,0.5321666255,0.0,0.5321666255,0.3010734745,0.0,1.4259422959,0.0,0.0,1.1339317312,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.7197530222,0.3010734745,0.7197530222,0.8776474034,1.3377522779,1.5069812313,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.5321666255,0.7197530222,0.8776474034,1.2410258927,0.7197530222,0.0,0.5321666255,0.0,0.3010734745,0.0,0.8776474034,0.7197530222,0.0,0.0,0.7197530222,0.3010734745,0.0,0.7197530222,1.5069812313,1.1339317312,1.5819423237,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,0.3010734745,1.2410258927,0.7197530222,0.3010734745,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.7197530222,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.8776474034,1.9419374998,1.1339317312,0.3010734745,0.3010734745,0.5321666255,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,3.2558984451,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,1.1339317312,0.3010734745,0.5321666255,0.0,0.3010734745,0.3010734745,1.5069812313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,1.1339317312,0.3010734745,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.7197530222,0.0,0.7197530222,0.7197530222,0.3010734745,0.0,0.8776474034,0.5321666255,1.0139774031,0.0,0.0,0.7197530222,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.8776474034,0.3010734745,0.5321666255,0.3010734745,2.5874483359,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,1.1339317312,0.0,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,0.5321666255,0.0,0.8776474034,0.0,0.5321666255,0.5321666255,0.0,0.7197530222,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.3010734745,0.3010734745,2.1666267281,0.5321666255,0.0,0.0,0.5321666255,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7780527507,0.0,0.7197530222,0.3010734745,0.0,1.5069812313,0.0,1.0139774031,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.5321666255,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,1.5819423237,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,2.2060850259,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.991095899,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.8776474034,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,1.6516739778,0.7197530222,0.0,0.0,0.8776474034,1.1339317312,0.0,1.0139774031,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.7197530222,0.3010734745,0.0,1.3377522779,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,1.0139774031,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,1.4259422959,0.8776474034,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,1.3377522779,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,1.0139774031,1.7168584863,0.0,0.0,0.3010734745,0.0,0.0,2.2806171158,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,1.5069812313,0.3010734745,2.3829331223,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.5321666255,0.0,0.7197530222,1.1339317312,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.8776474034,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.5321666255,0.8776474034,0.3010734745,0.0,0.3010734745,0.3010734745,0.7197530222,0.3010734745,0.7197530222,0.0,0.3010734745,0.8776474034,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,2.1666267281,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,1.3377522779,0.3010734745,0.0,0.0,0.3010734745,1.991095899,0.0,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.5321666255,0.0,0.0,0.7197530222,0.3010734745,0.7197530222,4.1005114671,1.9419374998,3.6346187504,0.0,0.0,0.0,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.5321666255,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.8776474034,0.3010734745,0.3010734745,0.8776474034,1.0139774031,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.8776474034,0.0,0.0,0.0,0.3010734745,0.5321666255,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,1.0139774031,0.3010734745,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,1.1339317312,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,1.2410258927,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.3010734745,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2002098077,0.7197530222,0.0,3.8944552231,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.5321666255,0.7197530222,1.1339317312,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3377522779,0.5321666255,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,1.2410258927,0.3010734745,1.7780527507,1.3377522779,0.0,0.0,0.5321666255,0.0,0.0,0.8776474034,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.7197530222,0.0,0.8776474034,0.0,0.0,0.0,0.0,1.5069812313,0.5321666255,0.0,0.0,0.8776474034,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.7197530222,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.8776474034,0.3010734745,0.7197530222,2.1255472866,0.3010734745,1.6516739778,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,1.5819423237,1.0139774031,2.4148373165,0.3010734745,0.0,0.0,0.0,0.0,1.5819423237,1.5069812313,0.0,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,1.0139774031,0.5321666255,0.3010734745,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,1.6516739778,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.3010734745,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,1.0139774031,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.7197530222,0.0,1.5069812313,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.5321666255,0.0,1.6516739778,0.0,0.7197530222,2.6135278133,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.7197530222,0.0,0.0,0.0,0.0,0.3010734745,2.4457550249,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.7197530222,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,1.6516739778,0.0,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.5321666255,0.0,0.0,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.3010734745,0.0,1.2410258927,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.8776474034,0.0,0.0,1.1339317312,0.3010734745,0.3010734745,1.0139774031,0.0,0.7197530222,0.5321666255,0.7197530222,0.0,0.0,0.0,0.8776474034,1.5069812313,0.3010734745,0.0,0.0,2.0827077633,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,0.5321666255,0.5321666255,0.5321666255,0.0,1.1339317312,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.5321666255,0.5321666255,0.5321666255,0.3010734745,2.1255472866,0.3010734745,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,1.0139774031,0.0,1.3377522779,0.0,0.0,0.5321666255,0.5321666255,1.7780527507,0.0,0.7197530222,1.5069812313,0.0,0.0,1.2410258927,0.7197530222,0.0,0.5321666255,0.0,1.1339317312,0.0,0.7197530222,0.0,0.0,0.7197530222,0.7197530222,1.0139774031,0.5321666255,0.3010734745,0.7197530222,0.8776474034,1.0139774031,0.3010734745,0.0,0.7197530222,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,1.1339317312,0.3010734745,0.0,1.1339317312,0.0,0.0,0.8776474034,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,1.1339317312,1.0139774031,0.5321666255,1.3377522779,0.0,0.0,0.0,0.7197530222,0.0,0.8776474034,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,1.0139774031,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.5321666255,0.3010734745,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.0,1.1339317312,0.5321666255,0.5321666255,0.0,0.3010734745,1.2410258927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.0,1.5819423237,0.3010734745,0.0,0.3010734745,1.0139774031,0.0,1.3377522779,0.0,0.0,0.5321666255,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,0.0,1.0139774031,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.5069812313,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.0,0.5321666255,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.8776474034,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.7197530222,0.0,0.0,0.7197530222,0.8776474034,1.3377522779,0.0,0.8776474034,0.0,0.0,0.0,1.0139774031,0.7197530222,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,1.1339317312,0.8776474034,0.3010734745,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.5321666255,0.5321666255,0.0,0.7197530222,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,1.4259422959,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5819423237,0.0,1.3377522779,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.5321666255,0.0,0.3010734745,0.3010734745,2.6389444026,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,1.5069812313,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.8776474034,0.0,1.1339317312,0.7197530222,0.3010734745,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.8776474034,0.3010734745,0.0,0.0,0.3010734745,1.2410258927,0.0,1.2410258927,0.3010734745,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,1.4259422959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,1.2410258927,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,1.2410258927,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.3010734745,0.0,0.0,0.8776474034,0.0,0.5321666255,0.7197530222,1.1339317312,0.0,0.0,0.0,0.7197530222,0.8776474034,0.3010734745,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.3010734745,0.8776474034,0.8776474034,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,1.0139774031,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.3377522779,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,1.3377522779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.7197530222,0.0,1.2410258927,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.0,0.3010734745,0.0,1.8357172412,0.0,0.5321666255,0.0,1.1339317312,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.7197530222,0.3010734745,0.3010734745,0.0,0.3010734745,1.0139774031,0.5321666255,0.7197530222,1.0139774031,0.5321666255,0.5321666255,0.0,0.0,1.1339317312,0.3010734745,0.0,0.0,1.0139774031,0.0,0.5321666255,0.3010734745,1.7168584863,0.0,1.0139774031,0.0,0.7197530222,0.0,0.3010734745,0.3010734745,0.8776474034,0.0,0.0,0.3010734745,1.0139774031,0.8776474034,0.0,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.7197530222,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.1339317312,1.4259422959,0.5321666255,0.8776474034,1.5069812313,1.5069812313,0.7197530222,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.8776474034,0.3010734745,0.0,0.0,0.0,0.0,1.5819423237,0.5321666255,0.0,0.3010734745,2.1666267281,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,1.2410258927,0.0,0.0,0.5321666255,0.0,0.5321666255,0.0,0.0,0.0,0.7197530222,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.7197530222,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.3377522779,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.5321666255,0.5321666255,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,1.1339317312,0.5321666255,0.3010734745,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.5321666255,0.3010734745,2.7792086905,0.0,1.6516739778,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,2.0827077633,0.0,1.3377522779,1.0139774031,1.1339317312,0.0,0.0,0.3010734745,0.3010734745,1.5069812313,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.3010734745,2.2806171158,1.1339317312,2.0379505562,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.4259422959,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.7197530222,0.3010734745,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.3010734745,1.3377522779,0.0,0.8776474034,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.8776474034,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.7197530222,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.7197530222,0.0,0.0,0.0,0.3010734745,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.7197530222,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,1.7780527507,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.3010734745,0.5321666255,1.4259422959,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.3010734745,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,1.0139774031,0.5321666255,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.5321666255,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.3010734745,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,1.0139774031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.3010734745,0.3010734745,0.5321666255,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5321666255,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.5321666255,0.0,0.0,0.7197530222,0.0,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,1.0139774031,0.0,0.3010734745,0.3010734745,0.0,0.0,0.7197530222,0.8776474034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1339317312,0.0,0.3010734745,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8357172412,0.0,0.5321666255,0.0,0.0,0.0,0.0,0.3010734745,0.0,0.0,0.0,0.0,0.7197530222,0.0,0.3010734745,1.0139774031,0.3010734745,0.3010734745]]]}",
                                                                "{\"columns\":[\"celltype_raw\",\"celltype_id\",\"sample\",\"study\",\"n_counts\",\"n_genes\",\"celltype_name\",\"doublet_score\",\"pred_dbl\",\"Disease\"],\"index\":[\"124332\"],\"data\":[[\"Myofibroblast\",\"CL:0000186\",\"DS000011735-GSM4058950\",\"DS000011735\",4685.1401367188,7575,\"myofibroblast cell\",0.0894005299,false,\"IPF\"]]}"
                                                            ],
                                                            [
                                                                "{\"columns\":[\"celltype_subsample\"],\"index\":[\"123942\",\"126138\"],\"data\":[[[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3951271217,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,4.2230753855,3.3951271217,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,3.7893505142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,1.7583877715,2.734966155,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0713639799,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,3.2194904294,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7893505142,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,3.3951271217,2.734966155,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,1.7583877715,1.7583877715,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,3.7893505142,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5444753897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,3.0062911256,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.3951271217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,3.6743897774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6743897774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,3.2194904294,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5983187765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,1.7583877715,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.734966155,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,2.3614338268,3.0062911256,3.0062911256,3.3951271217,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,2.3614338268,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,3.3951271217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5444753897,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,1.7583877715,2.734966155,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,2.734966155,3.3951271217,2.734966155,2.3614338268,3.2194904294,2.3614338268,0.0,1.7583877715,2.3614338268,2.3614338268,2.734966155,2.3614338268,2.734966155,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,2.3614338268,3.2194904294,2.734966155,2.734966155,0.0,0.0,3.5444753897,2.3614338268,0.0,3.0062911256,0.0,2.734966155,1.7583877715,0.0,0.0,2.734966155,2.3614338268,1.7583877715,2.734966155,0.0,0.0,1.7583877715,0.0,2.3614338268,2.734966155,2.3614338268,2.3614338268,0.0,1.7583877715,1.7583877715,2.3614338268,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.734966155,0.0,3.2194904294,2.734966155,2.3614338268,0.0,2.734966155,3.0062911256,2.734966155,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,2.3614338268,2.3614338268,4.1500939716,3.7893505142,0.0,1.7583877715,3.0062911256,2.3614338268,2.3614338268,2.734966155,0.0,0.0,2.3614338268,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,3.0062911256,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,2.3614338268,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6743897774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.734966155,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,3.6743897774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5444753897,2.734966155,3.6743897774,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,3.3951271217,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,2.3614338268,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,2.734966155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,3.2194904294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.734966155,0.0,0.0,1.7583877715,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,1.7583877715,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,2.3614338268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,1.7583877715,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0062911256,0.0,1.7583877715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7583877715,0.0,0.0]],[[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2813952948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.4829838364,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,3.4102186481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5243244893,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.719622284,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3546379513,3.6267334826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5243244893,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.1184177064,0.0,2.4829838364,2.4829838364,2.9598415386,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.719622284,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,1.5387971416,3.5243244893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,2.1184177064,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.8046113573,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,3.2813952948,3.133487718,0.0,0.0,3.6267334826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,1.5387971416,0.0,2.7495715485,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,2.1184177064,0.0,1.5387971416,2.1184177064,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,2.1184177064,3.8046113573,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.4829838364,3.2813952948,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.4829838364,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.4829838364,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.7495715485,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,2.9598415386,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.9598415386,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.8849375588,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,2.4829838364,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,3.2813952948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.4829838364,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,1.5387971416,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,1.5387971416,2.1184177064,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.4829838364,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,2.9598415386,1.5387971416,0.0,0.0,2.1184177064,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.9598415386,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,1.5387971416,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,1.5387971416,0.0,2.4829838364,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,2.1184177064,2.4829838364,2.9598415386,2.1184177064,2.4829838364,2.7495715485,0.0,2.7495715485,2.1184177064,2.9598415386,2.9598415386,2.4829838364,2.4829838364,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.4829838364,1.5387971416,2.4829838364,2.4829838364,2.7495715485,0.0,0.0,2.4829838364,2.4829838364,0.0,2.4829838364,1.5387971416,1.5387971416,1.5387971416,1.5387971416,0.0,2.9598415386,2.7495715485,2.1184177064,0.0,0.0,0.0,2.4829838364,2.1184177064,2.1184177064,2.1184177064,2.7495715485,2.7495715485,0.0,1.5387971416,1.5387971416,0.0,3.4102186481,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.7495715485,0.0,2.9598415386,0.0,2.4829838364,3.2813952948,0.0,3.133487718,2.4829838364,0.0,2.7495715485,2.4829838364,2.1184177064,0.0,2.1184177064,1.5387971416,0.0,2.9598415386,2.7495715485,0.0,1.5387971416,1.5387971416,2.7495715485,2.4829838364,2.1184177064,0.0,0.0,2.1184177064,2.4829838364,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.9598415386,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,3.6267334826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,2.4829838364,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,2.4829838364,0.0,2.4829838364,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.133487718,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.4829838364,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,2.1184177064,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.1184177064,2.1184177064,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,3.133487718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,2.4829838364,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,3.5243244893,1.5387971416,3.9555758452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,1.5387971416,3.2813952948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9598415386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,2.1184177064,0.0,2.7495715485,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,3.8829396847,0.0,2.4829838364,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,2.1184177064,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5387971416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7495715485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]]]}",
                                                                "{\"columns\":[\"celltype_raw\",\"celltype_id\",\"sample\",\"study\",\"n_counts\",\"n_genes\",\"celltype_name\",\"doublet_score\",\"pred_dbl\",\"Disease\"],\"index\":[\"123942\",\"126138\"],\"data\":[[\"Myofibroblast\",\"CL:0000186\",\"DS000011735-GSM4058950\",\"DS000011735\",2814.7053222656,1464,\"myofibroblast cell\",0.0825588703,false,\"IPF\"],[\"Myofibroblast\",\"CL:0000186\",\"DS000011735-GSM4058950\",\"DS000011735\",3068.9243164062,1762,\"myofibroblast cell\",0.0894005299,false,\"IPF\"]]}"
                                                            ]
                                                            ]
                                                        }                                                    
                    }
                                        },
               "scimilarity_search_nearest": {"data":{
                                                "dataframe_split": {
                                                    "columns": [
                                                    "embedding",
                                                    "params"
                                                    ],
                                                    "data": [
                                                    [
                                                        [
                                                        0.15414905548095703,
                                                        0.008094907738268375,
                                                        0.007071635685861111,
                                                        0.11320032179355621,
                                                        -0.026421435177326202,
                                                        -0.017847076058387756,
                                                        -0.05076976120471954,
                                                        0.0011923465644940734,
                                                        0.07798487693071365,
                                                        -0.003710819873958826,
                                                        -0.17938536405563354,
                                                        -0.0004844943177886307,
                                                        0.008351462893188,
                                                        0.004367381799966097,
                                                        0.004908163100481033,
                                                        -0.040169887244701385,
                                                        -0.011811240576207638,
                                                        0.01156391017138958,
                                                        -0.08166100084781647,
                                                        0.0016855495050549507,
                                                        0.015247325412929058,
                                                        0.015074350871145725,
                                                        -0.028049131855368614,
                                                        -0.005492010153830051,
                                                        -0.1235520988702774,
                                                        -0.003601083531975746,
                                                        0.047126784920692444,
                                                        0.006535789463669062,
                                                        -0.144910529255867,
                                                        -0.011237619444727898,
                                                        0.0017997614340856671,
                                                        0.004101550672203302,
                                                        -0.009247836656868458,
                                                        0.013190013356506824,
                                                        0.04192541539669037,
                                                        0.2575031816959381,
                                                        -0.0009331764304079115,
                                                        -0.39358916878700256,
                                                        -0.001351714483462274,
                                                        0.005353550426661968,
                                                        0.08716313540935516,
                                                        -0.0016448496608063579,
                                                        -0.08964385092258453,
                                                        -0.0020443156827241182,
                                                        0.04125257208943367,
                                                        0.019619246944785118,
                                                        -0.07807464152574539,
                                                        -0.003015730297192931,
                                                        -0.03305833414196968,
                                                        -0.01275385357439518,
                                                        0.0009606768726371229,
                                                        0.00738324411213398,
                                                        -0.008370054885745049,
                                                        -0.005742344539612532,
                                                        0.008405598811805248,
                                                        0.008876178413629532,
                                                        0.021073367446660995,
                                                        0.006765063852071762,
                                                        0.005318769253790379,
                                                        0.0002230188256362453,
                                                        0.48463231325149536,
                                                        0.005790474358946085,
                                                        -0.07760076224803925,
                                                        0.006920183543115854,
                                                        0.008430127054452896,
                                                        0.0013455882435664535,
                                                        0.007747098803520203,
                                                        0.4135277569293976,
                                                        -0.006457436364144087,
                                                        -0.0014662533067166805,
                                                        0.0041463314555585384,
                                                        -0.0016529172426089644,
                                                        -0.01140100508928299,
                                                        0.003924323245882988,
                                                        0.00287616322748363,
                                                        0.03074786067008972,
                                                        0.06868357211351395,
                                                        0.01342939306050539,
                                                        0.11059071868658066,
                                                        -0.0033012893982231617,
                                                        0.00782158225774765,
                                                        -0.012643266469240189,
                                                        0.0006877025589346886,
                                                        -0.018326250836253166,
                                                        0.017164604738354683,
                                                        0.008796787820756435,
                                                        -0.29134342074394226,
                                                        0.03959014639258385,
                                                        0.011093121021986008,
                                                        0.0011855673510581255,
                                                        0.003105160780251026,
                                                        0.17362509667873383,
                                                        0.001208254136145115,
                                                        -0.0011200892040506005,
                                                        -0.01247179601341486,
                                                        -0.012863626703619957,
                                                        0.005789272487163544,
                                                        -0.030835511162877083,
                                                        -0.013879022561013699,
                                                        -0.010378285311162472,
                                                        0.06195586919784546,
                                                        -0.11314792186021805,
                                                        -0.0007368993829004467,
                                                        -0.03829242289066315,
                                                        -0.000894454657100141,
                                                        0.09568963199853897,
                                                        -0.07871697098016739,
                                                        0.007398643996566534,
                                                        -0.005940796807408333,
                                                        0.011188283562660217,
                                                        0.0020390392746776342,
                                                        -0.15420489013195038,
                                                        0.00025915916194207966,
                                                        0.01163828931748867,
                                                        -0.001341450959444046,
                                                        0.005878814030438662,
                                                        0.0029710964299738407,
                                                        0.0021886832546442747,
                                                        -0.0017645113402977586,
                                                        0.007092277519404888,
                                                        -0.014363319613039494,
                                                        0.007801887579262257,
                                                        -0.012165925465524197,
                                                        0.0010459859622642398,
                                                        0.013607311062514782,
                                                        -0.0015619386686012149,
                                                        0.016956765204668045,
                                                        -0.12390629947185516
                                                        ],
                                                        "{\"k\": 10}"
                                                    ]
                                                    ]
                                                }
                                            }
                            },
               
               }               

In [0]:
endpoints_to_keep_alive_with_prefixes = [ f"{prefix}{e}{postfix}" for e in endpoints_to_keep_alive]

In [0]:
endpoints_to_keep_alive_with_prefixes

In [0]:
request_dict_with_prefix = { f"{prefix}{key}{postfix}" : request_dict[key] for key in request_dict.keys() }

In [0]:
request_dict_with_prefix.keys()

In [0]:
wait_until_endpoints_ready(db_host, endpoints_to_keep_alive_with_prefixes, db_token, check_interval=60)

In [0]:
num_hours_to_keep_alive = 3
num_mins_to_wait_for_wake_up = 30 
num_mins_to_ping = 15

#first wake everybody up
print("Everybody wakeup 📣!!!")
send_inference_inputs_parallel(db_host, request_dict_with_prefix, db_token)

print(f"Waiting for {num_mins_to_wait_for_wake_up} minutes for endpoints to scale up")
print("----------------------")
time.sleep(num_mins_to_wait_for_wake_up * 60)

for i in range(num_hours_to_keep_alive * 60 // num_mins_to_ping):
    send_inference_inputs_parallel(db_host, request_dict_with_prefix, db_token)
    print(f"sleeping for {num_mins_to_ping} minutes")
    print("----------------------")
    time.sleep(num_mins_to_ping * 60)